In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import logging
from typing import Dict, List, Tuple, Any, Optional
#import mlflow
from datetime import datetime
import os
import numpy as np
from tqdm import tqdm
import time
import gc

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('erp_bot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [3]:
class CBv2Dataset(Dataset):
    """Dataset class for BERT intent classification with optimized encoding"""
    
    def __init__(self, texts: List[str], main_intents: List[str], 
                 sub_intents: List[str], tokenizer: Any, max_len: int = 128,
                 precompute_encodings: bool = True):
        self.texts = texts
        self.main_intents = main_intents
        self.sub_intents = sub_intents
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.encodings = None
        
        # Pre-compute encodings for all texts in batches (much faster)
        if precompute_encodings:
            logger.info("Pre-computing encodings for all texts (this will speed up training)")
            self._precompute_encodings()
    
    def _precompute_encodings(self):
        """Pre-compute encodings in batches to save time during training"""
        batch_size = 256  # Process this many texts at once
        batches = [self.texts[i:i + batch_size] for i in range(0, len(self.texts), batch_size)]
        
        # Prepare to store the encodings
        self.encodings = {
            'input_ids': [],
            'attention_mask': []
        }
        
        # Process each batch with tqdm for progress
        logger.info(f"Processing {len(batches)} batches of encodings")
        for i, batch_texts in enumerate(tqdm(batches, desc="Encoding batches")):
            # Encode the batch
            batch_encodings = self.tokenizer.batch_encode_plus(
                batch_texts,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            
            # Store the encodings
            self.encodings['input_ids'].append(batch_encodings['input_ids'])
            self.encodings['attention_mask'].append(batch_encodings['attention_mask'])
            
            # Clear GPU cache if using CUDA
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # Concatenate all batches
        self.encodings['input_ids'] = torch.cat(self.encodings['input_ids'], dim=0)
        self.encodings['attention_mask'] = torch.cat(self.encodings['attention_mask'], dim=0)
        
        logger.info(f"Completed encoding {len(self.texts)} texts")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        try:
            text = str(self.texts[idx])
            main_intent = self.main_intents[idx]
            sub_intent = self.sub_intents[idx]
            
            # Use pre-computed encodings if available
            if self.encodings is not None:
                return {
                    'text': text,
                    'input_ids': self.encodings['input_ids'][idx],
                    'attention_mask': self.encodings['attention_mask'][idx],
                    'main_intent': torch.tensor(main_intent, dtype=torch.long),
                    'sub_intent': torch.tensor(sub_intent, dtype=torch.long)
                }
            
            # Fallback to on-the-fly encoding (slower)
            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            
            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'main_intent': torch.tensor(main_intent, dtype=torch.long),
                'sub_intent': torch.tensor(sub_intent, dtype=torch.long)
            }
        except Exception as e:
            logger.error(f"Error processing dataset item at index {idx}: {str(e)}")
            raise


In [4]:
class HierarchicalIntentClassifier(nn.Module):
    """Hierarchical BERT classifier for main and sub-intents"""
    
    def __init__(self, intent_structure: Dict[str, Dict[str, List[str]]]):
        super().__init__()
        try:
            self.bert = BertModel.from_pretrained('bert-large-uncased') #V1.8Extended (Large 340M Model)
            self.drop = nn.Dropout(0.3)
            
            # Main intent classifier
            self.n_main_intents = len(intent_structure)
            self.main_classifier = nn.Linear(1024, self.n_main_intents)
            
            # Create sub-classifiers for each main intent
            self.sub_classifiers = nn.ModuleDict({
                main_intent: nn.Linear(1024, len(sub_intents))
                for main_intent, sub_intents in intent_structure.items()
            })
            
            # Store the structure for reference
            self.intent_structure = intent_structure
            
        except Exception as e:
            logger.error(f"Error initializing hierarchical classifier: {str(e)}")
            raise Exception(f"Failed to initialize model: {str(e)}")

    def forward(self, input_ids, attention_mask, main_intent=None):
        try:
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs[1]
            dropped = self.drop(pooled_output)
            
            # Get main intent prediction
            main_output = self.main_classifier(dropped)
            
            if main_intent is None:
                main_intent = torch.argmax(main_output, dim=1)
            
            # Get sub-intent prediction for the specific main intent
            batch_size = input_ids.size(0)
            sub_outputs = torch.zeros(
                batch_size, 
                max(len(subs) for subs in self.intent_structure.values())
            ).to(input_ids.device)
            
            for i in range(batch_size):
                intent_name = list(self.intent_structure.keys())[main_intent[i]]
                sub_classifier = self.sub_classifiers[intent_name]
                sub_output = sub_classifier(dropped[i].unsqueeze(0))
                sub_outputs[i, :sub_output.size(1)] = sub_output
                
            return main_output, sub_outputs
            
        except Exception as e:
            logger.error(f"Error in forward pass: {str(e)}")
            raise Exception(f"Forward pass failed: {str(e)}")

In [5]:
class IntentClassifier:
    """Main interface for intent classification"""
    
    def __init__(self, model_path: Optional[str] = None, confidence_threshold: float = 0.7):
        """
        Initialize the intent classifier.
        
        Args:
            model_path: Path to the saved model file
            confidence_threshold: Threshold for confidence score (0.0 to 1.0)
        """
        if not 0 <= confidence_threshold <= 1:
            raise ValueError("Confidence threshold must be between 0 and 1")
            
        # Set device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
        self.model = None
        self.intent_structure = {}
        self.main_intent_to_id = {}
        self.sub_intent_to_id = {}
        self.confidence_threshold = confidence_threshold
        
        if model_path:
            self.load_model(model_path)

    def load_model(self, path: str):
        """
        Load the model from a saved checkpoint.
        
        Args:
            path: Path to the model checkpoint
        """
        try:
            checkpoint = torch.load(path, map_location=self.device)
            self.intent_structure = checkpoint['intent_structure']
            self.model = HierarchicalIntentClassifier(self.intent_structure)
            self.model.load_state_dict(checkpoint['model_state'])
            self.main_intent_to_id = checkpoint['main_intent_to_id']
            self.sub_intent_to_id = checkpoint['sub_intent_to_id']
            self.model.to(self.device)
            logger.info(f"Model successfully loaded from {path}")
            
        except Exception as e:
            logger.error(f"Error loading model from {path}: {str(e)}")
            raise Exception(f"Failed to load model from {path}: {str(e)}")

    def save_model(self, path: str):
        """
        Save the model to a checkpoint file.
        
        Args:
            path: Path where to save the model
        """
        if self.model is None:
            raise ValueError("No model to save")
            
        # Make sure the directory exists
        os.makedirs(os.path.dirname(path), exist_ok=True)
        
        torch.save({
            'model_state': self.model.state_dict(),
            'intent_structure': self.intent_structure,
            'main_intent_to_id': self.main_intent_to_id,
            'sub_intent_to_id': self.sub_intent_to_id
        }, path)
        logger.info(f"Model saved to {path}")

    def prepare_training_data(self, intent_data: Dict[str, Dict[str, List[str]]]):
        """
        Prepare data for training.
        
        Args:
            intent_data: Dictionary of intent structure with examples
            
        Returns:
            Prepared training data (texts, main intents, sub intents)
        """
        self.intent_structure = intent_data
        texts, main_intents, sub_intents = [], [], []
        
        # Create mappings
        self.main_intent_to_id = {intent: idx for idx, intent in enumerate(intent_data.keys())}
        
        # Create sub-intent mappings for each main intent
        self.sub_intent_to_id = {}
        for main_intent, sub_dict in intent_data.items():
            self.sub_intent_to_id[main_intent] = {
                sub_intent: idx for idx, sub_intent in enumerate(sub_dict.keys())
            }
        
        # Prepare training data
        for main_intent, sub_dict in intent_data.items():
            main_id = self.main_intent_to_id[main_intent]
            
            for sub_intent, examples in sub_dict.items():
                sub_id = self.sub_intent_to_id[main_intent][sub_intent]
                texts.extend(examples)
                main_intents.extend([main_id] * len(examples))
                sub_intents.extend([sub_id] * len(examples))
        
        return texts, main_intents, sub_intents

    def train(self, intent_data: Dict[str, Dict[str, List[str]]], 
             epochs: int = 40, 
             batch_size: int = 32,
             gradient_accumulation_steps: int = 4,
             learning_rate: float = 2e-5,
             weight_decay: float = 0.01,
             warmup_steps: int = 500,
             fp16: bool = True,
             num_workers: int = 4,
             eval_split: float = 0.1):
        """
        Train the model with optimized performance.
        
        Args:
            intent_data: Dictionary of intent structure with examples
            epochs: Number of training epochs
            batch_size: Batch size for training
            gradient_accumulation_steps: Number of steps to accumulate gradients
            learning_rate: Learning rate for optimizer
            weight_decay: Weight decay for regularization
            warmup_steps: Number of warmup steps for scheduler
            fp16: Whether to use mixed precision training
            num_workers: Number of workers for data loading
            eval_split: Fraction of data to use for evaluation
        """
        logger.info(f"Starting training on {self.device}")
        logger.info(f"Dataset size: {sum(len(examples) for intent in intent_data.values() for examples in intent.values())} examples")
        
        # Prepare data
        start_time = time.time()
        texts, main_intents, sub_intents = self.prepare_training_data(intent_data)
        logger.info(f"Data preparation took {time.time() - start_time:.2f} seconds")
        
        # Split data for training and evaluation
        indices = np.arange(len(texts))
        np.random.shuffle(indices)
        split_idx = int(len(indices) * (1 - eval_split))
        train_indices = indices[:split_idx]
        eval_indices = indices[split_idx:]
        
        # Create train and eval subsets
        train_texts = [texts[i] for i in train_indices]
        train_main_intents = [main_intents[i] for i in train_indices]
        train_sub_intents = [sub_intents[i] for i in train_indices]
        
        eval_texts = [texts[i] for i in eval_indices]
        eval_main_intents = [main_intents[i] for i in eval_indices]
        eval_sub_intents = [sub_intents[i] for i in eval_indices]
        
        # Create datasets
        logger.info("Creating train and eval datasets")
        train_dataset = CBv2Dataset(train_texts, train_main_intents, train_sub_intents, self.tokenizer)
        eval_dataset = CBv2Dataset(eval_texts, eval_main_intents, eval_sub_intents, self.tokenizer)
        
        # Create data loaders with optimal settings
        train_dataloader = DataLoader(
            train_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            num_workers=num_workers,
            pin_memory=True
        )
        
        eval_dataloader = DataLoader(
            eval_dataset, 
            batch_size=batch_size*2,  # Larger batch size for evaluation
            shuffle=False, 
            num_workers=num_workers,
            pin_memory=True
        )
        
        # Initialize model
        self.model = HierarchicalIntentClassifier(self.intent_structure)
        self.model.to(self.device)
        
        # Prepare optimizer and scheduler
        optimizer = torch.optim.AdamW(
            self.model.parameters(), 
            lr=learning_rate, 
            weight_decay=weight_decay
        )
        
        total_steps = len(train_dataloader) * epochs // gradient_accumulation_steps
        scheduler = get_linear_schedule_with_warmup(
            optimizer, 
            num_warmup_steps=warmup_steps, 
            num_training_steps=total_steps
        )
        
        criterion = nn.CrossEntropyLoss()
        
        # Setup mixed precision training
        scaler = torch.cuda.amp.GradScaler() if fp16 and torch.cuda.is_available() else None
        
        # Start MLflow run
        #mlflow.start_run()
        
        # Training loop
        logger.info(f"Starting training for {epochs} epochs")
        best_eval_loss = float('inf')
        
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0
            progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
            
            for step, batch in enumerate(progress_bar):
                # Move batch to device
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                main_labels = batch['main_intent'].to(self.device)
                sub_labels = batch['sub_intent'].to(self.device)
                
                # Forward and backward pass with mixed precision
                if scaler:
                    with torch.cuda.amp.autocast():
                        main_outputs, sub_outputs = self.model(input_ids, attention_mask, main_labels)
                        main_loss = criterion(main_outputs, main_labels)
                        sub_loss = criterion(sub_outputs, sub_labels)
                        loss = main_loss + sub_loss
                        loss = loss / gradient_accumulation_steps
                    
                    # Backward pass with scaling
                    scaler.scale(loss).backward()
                    
                    # Gradient accumulation
                    if (step + 1) % gradient_accumulation_steps == 0:
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                        scaler.step(optimizer)
                        scaler.update()
                        scheduler.step()
                        optimizer.zero_grad()
                    
                else:
                    # Standard training without mixed precision
                    main_outputs, sub_outputs = self.model(input_ids, attention_mask, main_labels)
                    main_loss = criterion(main_outputs, main_labels)
                    sub_loss = criterion(sub_outputs, sub_labels)
                    loss = main_loss + sub_loss
                    loss = loss / gradient_accumulation_steps
                    
                    loss.backward()
                    
                    # Gradient accumulation
                    if (step + 1) % gradient_accumulation_steps == 0:
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                        optimizer.step()
                        scheduler.step()
                        optimizer.zero_grad()
                
                train_loss += loss.item() * gradient_accumulation_steps
                progress_bar.set_postfix({'loss': train_loss / (step + 1)})
            
            avg_train_loss = train_loss / len(train_dataloader)
            
            # Evaluation phase
            if eval_split > 0:
                self.model.eval()
                eval_loss = 0
                progress_bar = tqdm(eval_dataloader, desc=f"Epoch {epoch+1}/{epochs} [Eval]")
                
                with torch.no_grad():
                    for step, batch in enumerate(progress_bar):
                        input_ids = batch['input_ids'].to(self.device)
                        attention_mask = batch['attention_mask'].to(self.device)
                        main_labels = batch['main_intent'].to(self.device)
                        sub_labels = batch['sub_intent'].to(self.device)
                        
                        main_outputs, sub_outputs = self.model(input_ids, attention_mask, main_labels)
                        main_loss = criterion(main_outputs, main_labels)
                        sub_loss = criterion(sub_outputs, sub_labels)
                        loss = main_loss + sub_loss
                        
                        eval_loss += loss.item()
                        progress_bar.set_postfix({'loss': eval_loss / (step + 1)})
                
                avg_eval_loss = eval_loss / len(eval_dataloader)
                
                # Save best model
                if avg_eval_loss < best_eval_loss:
                    best_eval_loss = avg_eval_loss
                    # Save model checkpoint
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    self.save_model(f"models/erp_bot_best_updx.pt")
                    logger.info(f"New best model saved with eval loss: {best_eval_loss:.4f}")
                
                logger.info(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Eval Loss: {avg_eval_loss:.4f}")
                # mlflow.log_metrics({
                #     "train_loss": avg_train_loss,
                #     "eval_loss": avg_eval_loss
                # }, step=epoch)
            else:
                logger.info(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}")
                # mlflow.log_metrics({"train_loss": avg_train_loss}, step=epoch)
            
            # Memory cleanup
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
        
        # mlflow.end_run()
        
        # Save final model
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.save_model(f"models/erp_bot_final_{timestamp}.pt")
        logger.info("Training completed")

    def predict(self, text: str) -> Dict[str, Any]:
        """
        Predict intents for given text.
        
        Args:
            text: Input text for prediction
            
        Returns:
            Dictionary containing prediction results
        """
        if not self.model:
            raise ValueError("Model not loaded")
            
        self.model.eval()
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        with torch.no_grad():
            main_outputs, _ = self.model(input_ids, attention_mask)
            main_probs = torch.softmax(main_outputs, dim=1)
            main_confidence, main_predicted = torch.max(main_probs, 1)
            
            # Check confidence threshold
            if main_confidence.item() < self.confidence_threshold:
                return {
                    'status': 'llm_fallback',
                    'confidence': {
                        'main': main_confidence.item(),
                        'sub': 0.0
                    },
                    'original_query': text
                }
            
            # Get main intent prediction
            main_id_to_intent = {v: k for k, v in self.main_intent_to_id.items()}
            predicted_main_intent = main_id_to_intent[main_predicted.item()]
            
            # Get sub-intent prediction
            dropped = self.model.drop(self.model.bert(input_ids, attention_mask)[1])
            sub_classifier = self.model.sub_classifiers[predicted_main_intent]
            sub_outputs = sub_classifier(dropped)
            sub_probs = torch.softmax(sub_outputs, dim=1)
            sub_confidence, sub_predicted = torch.max(sub_probs, 1)
            
            sub_id_to_intent = {v: k for k, v in self.sub_intent_to_id[predicted_main_intent].items()}
            predicted_sub_intent = sub_id_to_intent[sub_predicted.item()]
            
            return {
                'status': 'success',
                'main_intent': predicted_main_intent,
                'sub_intent': predicted_sub_intent,
                'confidence': {
                    'main': main_confidence.item(),
                    'sub': sub_confidence.item()
                }
            }

In [6]:
intent_samples = {
    "front_desk" : {
    "student_directory": [
            "Show me the student directory",
            "Access list of students",
            "Open student records",
            "Where is the student directory ",
            "List all students",
            "Search student names",
            "Can I see all students ",
            "View student list",
            "Pull up student info",
            "Get student data",
            "Find a student in the directory",
            "How to access student directory ",
            "Student records please",
            "Student details required",
            "Directory of all students",
            "Show student profiles",
            "Open students database",
            "Lookup student directory",
            "I want student directory acAcess",
            "Give me student information",
            "Student contact list",
            "Student roll list",
            "Browse students",
            "Student listing please",
            "I need the student directory",
            "Fetch student details",
            "Show me enrolled students",
            "View academic students",
            "Student profiles list",
            "Can I browse student directory ",
            "Access student contact book",
            "Show names of students",
            "Retrieve student records",
            "View school students",
            "Display student data",
            "Get student registry",
            "Where can I find student info ",
            "List current students",
            "Display names of students",
            "Open classwise student list",
            "All students list please",
            "How can I open student directory ",
            "Student contact details",
            "Student ID records",
            "Roll numbers list",
            "Find student contact",
            "Students enrolled list",
            "I need student info",
            "Student directory screen",
            "Students overview"
          ],
          "staff_directory": [
            "Open the staff directory",
            "List of all staff members",
            "Show teacher directory",
            "Staff contact list",
            "Find a teacher",
            "Where is the staff directory ",
            "Lookup staff members",
            "Staff members list",
            "Access school staff info",
            "Directory of staff",
            "Show me staff names",
            "Search for faculty members",
            "View staff profiles",
            "Staff listing please",
            "Employee directory screen",
            "Get details of staff",
            "Teacher info required",
            "Open staff contact book",
            "Retrieve staff details",
            "Display teacher names",
            "List teaching and non-teaching staff",
            "Staff records please",
            "Who are the staff members ",
            "Staff information screen",
            "Faculty contact directory",
            "I need the staff directory",
            "Find a staff member",
            "Teacher directory access",
            "Get staff profiles",
            "View academic staff",
            "Search in staff list",
            "Show me employees list",
            "Access employee directory",
            "All staff details",
            "Can I view the staff directory ",
            "Staff registry view",
            "Open employee contact list",
            "Faculty members list",
            "Employee contact info",
            "Non-teaching staff listing",
            "Teaching staff directory",
            "Check staff records",
            "Staff database open",
            "List all employees",
            "Directory of school personnel",
            "Show admin staff",
            "Contact staff details",
            "Display employee data",
            "Staff login list",
            "Staff role details"
          ],
          "visitor_management": [
            "Open visitor management",
            "Log a visitor entry",
            "Show visitor records",
            "Visitor management system",
            "Where is visitor data ",
            "Visitor tracking please",
            "Visitor list access",
            "View current visitors",
            "Log visitor details",
            "Can I register a visitor ",
            "Open visitors list",
            "List of visitors today",
            "Visitor check-in form",
            "Manage visitor data",
            "Show who visited today",
            "Track guest entries",
            "Visitor book access",
            "Retrieve visitor logs",
            "Visitors logbook open",
            "Access guest history",
            "I want to manage visitors",
            "Get visitor summary",
            "Search for visitor",
            "Who came today ",
            "Past visitor records",
            "Visitor log management",
            "Visitor database open",
            "New visitor registration",
            "Open visitor check-in",
            "List external visitors",
            "Visitor module access",
            "Visitor form entry",
            "Guest log view",
            "Today's visitor list",
            "Visitor purpose log",
            "Visitor approval records",
            "Manage guest entries",
            "Visitor pass list",
            "Contact details of visitor",
            "Visitor in and out log",
            "View guestbook",
            "School guest entry log",
            "Add new visitor",
            "Track visitor timing",
            "Log a parent visit",
            "I want visitor info",
            "Visitor appointment records",
            "External entry records",
            "Security visitor log",
            "Fetch visitor entries"
          ],
          "complaint_book": [
            "Open the complaint book",
            "Log a new complaint",
            "Where can I register a complaint ",
            "Show me all complaints",
            "Access complaint records",
            "File a complaint",
            "Complaint logbook please",
            "How to submit a complaint ",
            "Complaint registration screen",
            "I want to make a complaint",
            "Record a complaint",
            "Open complaint management",
            "View filed complaints",
            "Complaint list please",
            "Who handles complaints ",
            "Submit my complaint",
            "Where to file complaint ",
            "Open complaints dashboard",
            "Complaint logbook access",
            "Can I see complaint history ",
            "Manage student complaints",
            "Manage staff complaints",
            "Register feedback or complaint",
            "Complaint register form",
            "Complaint management system",
            "Track submitted complaints",
            "Complaint form open",
            "Complaint submission area",
            "Post a complaint",
            "How can I complain ",
            "Open issue log",
            "Complaint details required",
            "Enter new complaint",
            "Show resolved complaints",
            "What complaints are pending ",
            "Log complaint now",
            "Report an incident",
            "Complaint tracking dashboard",
            "Where can I check complaint ",
            "Raise a grievance",
            "Add to complaint book",
            "Complaint support portal",
            "System for registering complaint",
            "School complaint form",
            "File concern or complaint",
            "Send issue report",
            "Complaint and grievance module",
            "Look up past complaints",
            "I want to raise a concern",
            "Complaint records history"
          ],
          "postal_send_or_receive": [
              "Open postal send and receive module",
              "Show all postal entries",
              "Log a postal item",
              "Where can I track postal records ",
              "Manage postal dispatch and receive",
              "Access incoming and outgoing mails",
              "Postal records please",
              "Record a received post",
              "Log an outgoing post",
              "Show received postal items",
              "Track school postal history",
              "How to send a post ",
              "Enter a new postal dispatch",
              "Add incoming letter",
              "Add outgoing courier",
              "Who received the last parcel ",
              "Postal system open",
              "Can I send a postal item ",
              "Register new mail",
              "Post register access",
              "Postal in and out entries",
              "Where is the mail log ",
              "Track courier sent from school",
              "Manage school postal service",
              "Postal dashboard access",
              "Received courier list",
              "Outgoing letter records",
              "View all postal transactions",
              "Postal log open",
              "Send a parcel via school",
              "School dispatch record",
              "Postal item tracking",
              "Mail received entry",
              "Mail sent entry",
              "Register postal packet",
              "Incoming parcel details",
              "Courier dispatch system",
              "How to enter postal data ",
              "Mail management tool",
              "Who received the letter ",
              "Postal record history",
              "New postal transaction",
              "Mailbook management",
              "Parcel tracking system",
              "Log new postal mail",
              "Postal registry open",
              "Courier sent confirmation",
              "Receive post entry",
              "School postal record view",
              "Postal transaction summary"
            ],
            "phone_call_log": [
              "Open phone call log",
              "Show incoming and outgoing calls",
              "Log a new phone call",
              "Access call history",
              "View today's call log",
              "Record a phone call entry",
              "Call register open",
              "Where can I find phone records ",
              "Phone call dashboard",
              "Track incoming calls",
              "Track outgoing calls",
              "Call logbook please",
              "Manage phone call records",
              "Log missed calls",
              "Search call logs",
              "View last phone call",
              "Enter call details",
              "Check school phone logs",
              "School phone usage log",
              "Incoming call register",
              "Outgoing call register",
              "Show recent calls",
              "Open communication log",
              "Phone interaction records",
              "Monitor school calls",
              "Open the call book",
              "Track received phone calls",
              "Track dialed numbers",
              "Add new phone record",
              "Call duration records",
              "Phone call system",
              "Phone call records please",
              "Manage telephone logs",
              "Access call registry",
              "Communication log dashboard",
              "Call report for today",
              "Phone call history view",
              "View past calls",
              "View missed calls",
              "Phone log access",
              "Communication records tool",
              "Add to phone call log",
              "View recorded phone conversations",
              "Who made the last call ",
              "Show phone communication list",
              "Call management screen",
              "Check who called school",
              "Call record update",
              "Log telephonic conversation",
              "Search phone call entry"
        ],
        'postal_report': [
            "Show me the postal report for this week ",
            "Can I see the list of all sent posts ",
            "I need a report of received letters today ",
            "What posts were sent out yesterday ",
            "Give me a summary of incoming and outgoing posts ",
            "Where can I find the postal records ",
            "Download the latest postal report ",
            "Has the post for today been recorded ",
            "Show postal activity for the last month ",
            "Generate a detailed postal report ",
            "List all received courier items ",
            "List all sent parcels this month ",
            "Postal report of the last week, please ",
            "I want to download the last 7 days’ postal report ",
            "How many posts were received this week ",
            "Give me data on all outgoing mail ",
            "Can I filter the postal report by date ",
            "Was any post sent to ABC school today ",
            "Show me undelivered posts in the report ",
            "Generate a report for this month’s postal activity ",
            "Who received the last courier ",
            "List posts received from the education department ",
            "Summarize the posts sent today ",
            "Can I see only the incoming post records ",
            "Do we have any high-priority post records ",
            "How many letters were received today ",
            "Create a PDF of the latest postal summary ",
            "Which branch sent the most letters this week ",
            "Were there any courier delays recorded ",
            "Give me a report of registered posts "
        ],
        'visitors_report': [
            "Show me today's visitor report ",
            "How many visitors came yesterday ",
            "Give me a list of all visitors this week ",
            "Generate a report of all visitors ",
            "Who visited the school today ",
            "I need details of yesterday’s visitors ",
            "How many parents visited in the last month ",
            "Can I see the list of visitors from last Friday ",
            "Download the latest visitor log ",
            "Give me the visitor summary for today ",
            "Who was the last visitor today ",
            "Are there any visitors currently in school ",
            "Show me the visitor records for last week ",
            "How many visitors came in March ",
            "Print today's visitor report ",
            "Which class had the most visitors ",
            "Were there any VIPs visiting today ",
            "Show entries made in the visitor register ",
            "Do we have a log of alumni visits ",
            "Summarize all visitor entries from the last 7 days ",
            "List visitors who came for admission inquiries ",
            "Any guests scheduled for today ",
            "Visitor report of the academic fair day ",
            "How many visits were logged this month ",
            "Filter the visitor report by purpose ",
            "Can I check if John Doe visited today ",
            "Did any government official visit this week ",
            "Download visitors report for the last quarter ",
            "Get the record of unapproved visitors ",
            "List all parent visits from the past two weeks "
        ],
        'complaints_report': [
            "Show me the complaint report for this week ",
            "How many complaints were received today ",
            "List all unresolved complaints ",
            "Download the complaints report ",
            "Give me a summary of student complaints ",
            "Generate a full report of complaints logged this month ",
            "What complaints were made yesterday ",
            "Who submitted the most recent complaint ",
            "Give me complaints related to hostel facilities ",
            "Are there any complaints pending resolution ",
            "Show complaints filed by staff members ",
            "Filter complaints by category ",
            "List complaints about classroom cleanliness ",
            "View the complaint history for grade 10 ",
            "Show all complaints lodged this semester ",
            "Were any complaints resolved today ",
            "Do we have any safety-related complaints ",
            "Can I get the complaints raised last Friday ",
            "Show me complaints regarding transport ",
            "Print the latest complaint report ",
            "How many parent complaints were filed this week ",
            "Summarize complaints received in April ",
            "Any complaints marked urgent ",
            "What are the most frequent complaint categories ",
            "Who handles student complaints ",
            "Report on complaints closed within 24 hours ",
            "List complaints received via email ",
            "Get a PDF of the complaints registered last week ",
            "Which department has the most complaints ",
            "Are anonymous complaints included in the report "
        ]
    },
        'voice_call': {
        'call_report': [
            "Show me today's call report ",
            "How many calls were made today ",
            "Generate a report of all voice calls ",
            "Download the latest call report ",
            "Give me details of outgoing calls this week ",
            "I want the report of missed calls ",
            "List all voice calls from yesterday ",
            "What calls were made in the last 7 days ",
            "Can I get the call logs for March ",
            "Print today's voice call report ",
            "Show call duration statistics ",
            "Report on failed call attempts ",
            "Which user made the most calls today ",
            "Give me a list of all received calls ",
            "How many calls were made during school hours ",
            "Filter the call report by user group ",
            "Any emergency calls made today ",
            "Get the call report for the last month ",
            "Were there any calls made after 5 PM ",
            "List all calls made to parents ",
            "Download call activity for this semester ",
            "How many calls were answered today ",
            "Show the average call duration report ",
            "Give me data on high-priority calls ",
            "Call logs of transport department please ",
            "Did any calls fail to connect today ",
            "Summarize the call logs from last Friday ",
            "Can I get a CSV of the latest call report ",
            "Which branch made the highest number of calls ",
            "Generate a voice call performance report "
        ],
        'group_manager': [
            "Open the group manager settings ",
            "How can I manage voice call groups ",
            "Show me the group manager dashboard ",
            "List all voice call groups ",
            "I want to edit an existing group ",
            "Can I add a new group in group manager ",
            "Who is the manager of the teachers' call group ",
            "Change the name of a call group ",
            "Add members to the academic group ",
            "How to remove a user from a call group ",
            "Create a new call group for grade 9 ",
            "Edit group members in group manager ",
            "Assign a new manager to this call group ",
            "Update the group settings ",
            "Can I view all active groups ",
            "Rename the admin call group ",
            "How do I delete an existing call group ",
            "Is there a limit to group members ",
            "I need access to the group manager ",
            "Remove duplicate members from a call group ",
            "Make someone else the group manager ",
            "Which group has the most members ",
            "List group managers for each group ",
            "Can I export all group details ",
            "Set rules for a call group in group manager ",
            "Check if a group exists in the manager ",
            "Deactivate a call group ",
            "Reassign group manager role ",
            "How to make a group private ",
            "Enable notifications for group manager changes "
        ],
        'default_group': [
            "What is the default call group ",
            "Set the default group for voice calls ",
            "How can I change the default call group ",
            "Assign default group for new users ",
            "Is there a default group for staff ",
            "Show the current default group setting ",
            "Can I reset the default group ",
            "What's the difference between default and custom groups ",
            "How do I view users in the default group ",
            "Who is part of the default call group ",
            "How many members are in the default group ",
            "Enable notifications for the default group ",
            "Make admin group the default group ",
            "What happens if no default group is set ",
            "Assign default group to new admissions ",
            "Set class teachers group as default ",
            "Remove someone from the default group ",
            "View all calls made by the default group ",
            "Get default group activity report ",
            "Can default group be renamed ",
            "What’s the default group ID ",
            "Show the group name set as default ",
            "Edit the current default group ",
            "How to restrict access to default group ",
            "Is default group active for all branches ",
            "List all users not in default group ",
            "Default group members are missing, why ",
            "Reassign default group for transport staff ",
            "Show call logs for default group only ",
            "How do I configure the default voice group "
        ],
        'manage_group': [
            "How do I manage call groups ",
            "Open group management panel ",
            "Add new users to an existing group ",
            "Edit the name of a call group ",
            "Delete an unwanted call group ",
            "How can I view all created groups ",
            "Update group member list ",
            "Assign permissions for group management ",
            "Move a user to a different group ",
            "List users in a particular group ",
            "Block a user from a group ",
            "Change group settings for voice calls ",
            "Restrict calls to group members only ",
            "Enable call logging for specific groups ",
            "Merge two call groups together ",
            "Split a large group into smaller ones ",
            "Add a teacher to multiple groups ",
            "Transfer group ownership ",
            "Who can manage call groups ",
            "Set notification preferences per group ",
            "How do I archive a group ",
            "Reinstate a deleted group ",
            "Export group details to Excel ",
            "Import group settings from another account ",
            "Assign backup manager to a group ",
            "Disable group temporarily ",
            "How to sort groups by size ",
            "Manage group access by department ",
            "Limit calls to specific groups only ",
            "Check if group has any recent activity "
        ],
        'registered_user': [
            "Show me all registered users for voice calls ",
            "Who are the registered users in the system ",
            "List registered users in the call system ",
            "How do I register a new user for voice calls ",
            "Check if this user is registered for calls ",
            "View user details in the voice call registry ",
            "Can I unregister a user from voice calls ",
            "How many users are currently registered ",
            "Who registered recently for calls ",
            "Add student as a registered user ",
            "Update details of a registered user ",
            "Remove a user from registered list ",
            "What’s the status of a user in call system ",
            "Get report of registered users this week ",
            "Who manages the registration of users ",
            "Is this teacher already registered ",
            "Batch register users for voice calls ",
            "Are parents registered for voice calls ",
            "Show all inactive registered users ",
            "Can a user register multiple times ",
            "Generate list of registered users with IDs ",
            "Export registered user list ",
            "Filter registered users by department ",
            "Send alert to all registered users ",
            "Verify the user’s registration status ",
            "What permissions do registered users have ",
            "Mark this user as registered manually ",
            "List all staff who are registered ",
            "Sync registration data with user database ",
            "How do I deactivate a registered user "
        ]
    },

        'enquiry_admission': {
        'process_stages': [
            "What are the stages in the admission process ",
            "Explain the complete enquiry to admission process ",
            "How many steps are involved in admission ",
            "List the process stages for student admission ",
            "What is the first stage in the admission process ",
            "Is counselling a part of the admission process ",
            "How long does each stage in admission take ",
            "After enquiry, what's the next step ",
            "Where does document verification happen ",
            "What stage comes after fee payment ",
            "Is there an interview in the admission process ",
            "How to track the stage of an enquiry ",
            "Do parents need to be present in all stages ",
            "How many follow-ups are done in enquiry process ",
            "Can a student skip stages if already enrolled before ",
            "What’s the final step in the admission process ",
            "Is the entrance test part of the process ",
            "How do I move a student to the next stage ",
            "Show admission process for new applicants ",
            "What’s the difference between enquiry and registration ",
            "Who handles each admission stage ",
            "Is there an automated stage tracking ",
            "Explain the shortlisting stage ",
            "How to know the current stage of an applicant ",
            "Can we edit stages in the admission pipeline ",
            "Do we notify students after every stage ",
            "How to mark a stage as completed ",
            "Are all stages mandatory ",
            "Show completed and pending stages for an enquiry ",
            "Is parent verification part of any stage "
        ],
        'enquiry_dashboard': [
            "Open the enquiry dashboard ",
            "Show me today's enquiries ",
            "What’s the total number of active enquiries ",
            "Filter enquiries by status on the dashboard ",
            "How do I view follow-ups from the dashboard ",
            "Show enquiries by source on dashboard ",
            "How many enquiries were received this week ",
            "Search a specific enquiry on the dashboard ",
            "Can I see pending enquiries on the dashboard ",
            "Where do I find completed enquiries ",
            "Display new enquiries added today ",
            "How do I view enquiries by counselor ",
            "Which class has the most enquiries ",
            "Sort enquiries by date of enquiry ",
            "Can I customize the dashboard view ",
            "Add a widget to the enquiry dashboard ",
            "What’s the enquiry conversion rate ",
            "Show enquiries converted to admission ",
            "Can I download the dashboard data ",
            "Give me a summary of enquiry activities ",
            "Highlight follow-ups due today ",
            "Display enquiries with no follow-up ",
            "Which staff handled the most enquiries ",
            "Are any enquiries marked as high priority ",
            "Show sources of enquiry like walk-in or online ",
            "Compare enquiries from this month and last month ",
            "List enquiries that need urgent action ",
            "Show enquiries with missing details ",
            "Enquiries categorized by interest area ",
            "Show rejected enquiries on the dashboard "
        ],
        'add_new_enquiry': [
            "How do I add a new enquiry ",
            "Enter a fresh student enquiry ",
            "Add a parent's contact for new enquiry ",
            "Register a new admission enquiry ",
            "Where to fill details for new enquiry ",
            "Create enquiry form for new student ",
            "Can I log a new enquiry from the dashboard ",
            "Start a new enquiry record ",
            "Input student details for enquiry ",
            "Open form to add new enquiry ",
            "Add enquiry with class preference ",
            "What fields are needed for new enquiry ",
            "Assign new enquiry to a staff member ",
            "How do I save a new enquiry entry ",
            "Is parent info mandatory for enquiry ",
            "What’s the format for enquiry entry ",
            "Submit a walk-in enquiry ",
            "Log an enquiry from phone call ",
            "Add enquiry received from online form ",
            "Start new enquiry from ERP system ",
            "Enter sibling information in enquiry ",
            "Set a follow-up date while adding enquiry ",
            "How to tag enquiry source ",
            "Add remarks to a new enquiry ",
            "Attach documents in a new enquiry ",
            "Save enquiry with incomplete data ",
            "Link enquiry to previous communication ",
            "Set preferred timing in new enquiry form ",
            "Record special notes in enquiry ",
            "Auto-fill enquiry with existing student data "
        ],
        'report': [
            "Generate enquiry report for this month ",
            "Show all enquiries in a report format ",
            "Download admission enquiry report ",
            "View report by enquiry source ",
            "Filter report by enquiry status ",
            "Create custom enquiry report ",
            "Report of enquiries handled by staff ",
            "How many enquiries converted to admissions ",
            "Report on daily enquiry activity ",
            "Weekly report of new enquiries ",
            "Report of cancelled enquiries ",
            "Show class-wise enquiry statistics ",
            "Generate report for follow-ups missed ",
            "Which staff had most enquiries ",
            "Graphical report of enquiry trends ",
            "Can I export enquiry report to Excel ",
            "Compare enquiry data over months ",
            "Department-wise enquiry report ",
            "Report on enquiries with pending follow-up ",
            "Monthly summary of enquiries ",
            "Show percentage of enquiries converted ",
            "Generate detailed report with parent info ",
            "Report of enquiries with no response ",
            "Show all sources in enquiry report ",
            "Get enquiry status distribution report ",
            "Generate enquiry report for academic year ",
            "Report of online vs offline enquiries ",
            "Enquiry performance report by counselor ",
            "Report showing rejected enquiries ",
            "Custom date range enquiry report "
        ]
    },

         'student_manager': {
         'student_managers_masters': [
            "Where can I access student manager master settings ",
            "Edit master details in student manager ",
            "Add a new category in student manager masters ",
            "Update class master details ",
            "Is there a list of master categories for students ",
            "What are the master records in student manager ",
            "Can I create new masters for student data ",
            "How to modify existing student master entries ",
            "Show me the master configuration for students ",
            "What is included in the student masters list ",
            "Are optional subjects part of the student master ",
            "How do I link a student to a master group ",
            "Can I delete a student manager master ",
            "Is there a setting to restore deleted master data ",
            "Where do I add student types in masters ",
            "Can I view the history of student master updates ",
            "How can I manage class sections in masters ",
            "What is the process to manage house assignments ",
            "Where do I manage bus routes in student masters ",
            "Add a new caste or religion in student master ",
            "Can I define custom student attributes ",
            "Do masters control dropdown fields in forms ",
            "Assign blood groups via master settings ",
            "Set up fee categories in student master ",
            "Enable hostel options in master list ",
            "Configure nationality in student manager masters ",
            "What is the function of the master setup tab ",
            "How do I create a club or activity master ",
            "Manage medical info fields in student masters ",
            "Create a guardian relation type in masters ",
            "Edit existing class teacher allocation master ",
            "How do I manage academic years in masters ",
            "Update the sports categories in student master ",
            "How do I categorize admission types ",
            "Is there a way to track changes in master setup ",
            "Add languages spoken as a master field ",
            "Create house colors in master list ",
            "Edit master for student uniform sizes ",
            "Define transport zones under student masters ",
            "Add special education categories in master list ",
            "Include disciplinary actions in student masters ",
            "Manage library categories in master section ",
            "Set term types for students in masters ",
            "What kind of student filters are set in masters ",
            "Manage document types required from students ",
            "Add previous school categories in student masters ",
            "Can I include alumni statuses in masters ",
            "Add birth certificate status in master records ",
            "Can I automate roll number series in masters ",
            "How to validate master entries for consistency "
        ],
            'community': [
            "How do I add a new community ",
            "Where can I manage community records ",
            "Can I delete a specific community entry ",
            "How many communities are currently listed ",
            "What is the purpose of the community field ",
            "Can students belong to multiple communities ",
            "Is it mandatory to fill the community field ",
            "How to edit an existing community record ",
            "What should I enter under community for a new student ",
            "How do I update the list of communities ",
            "Where do I find community settings in the ERP ",
            "Can I search students by their community ",
            "Is community data linked to any reports ",
            "Can I export the list of communities ",
            "How do I categorize communities ",
            "Can I bulk import community data ",
            "Who has permission to update community data ",
            "Are community details visible to students ",
            "Is there a predefined list of communities ",
            "How do I set a default community for new entries ",
            "What happens if community is left blank ",
            "Can I add regional communities ",
            "Is it possible to restrict access to community info ",
            "Do I need to update community records annually ",
            "How do I assign communities in bulk ",
            "Can I sort students by community ",
            "Are there guidelines for creating community entries ",
            "Can communities be renamed after creation ",
            "How do I merge duplicate community entries ",
            "Where is the community field in the student form ",
            "Can I mark a community as inactive ",
            "What is the community dropdown based on ",
            "Can I lock a community entry to prevent edits ",
            "Can I use custom names for communities ",
            "Is there a community audit log ",
            "How do I check when a community was added ",
            "Are community names case sensitive ",
            "Can students update their community info ",
            "How do I validate community entries ",
            "How are community codes managed ",
            "Can I assign community tags ",
            "Is the community list shared across campuses ",
            "How do I remove an old community ",
            "Can I link community to scholarship criteria ",
            "Can I assign priorities to communities ",
            "Where is the community module located ",
            "What community was assigned by default ",
            "Can I duplicate a community setup ",
            "How do I get a report of students by community ",
            "Can I restore deleted community records "
        ],
        'student_register': [
            "How can I register a new student ",
            "Where is the student registration form ",
            "Can I update student details after registration ",
            "Is photo upload mandatory for student registration ",
            "How do I assign a class during registration ",
            "Can I register students in bulk ",
            "What fields are required in the registration form ",
            "Is there a way to register students using Excel ",
            "How to validate entries during registration ",
            "Can I set custom fields for student registration ",
            "What’s the process to register a transfer student ",
            "Can I assign a roll number during registration ",
            "Where do I enter parent details during registration ",
            "How to register a student with special needs ",
            "What happens if I leave a mandatory field blank ",
            "Can I register alumni students again ",
            "How to check if a student is already registered ",
            "Can I preview the registration before saving ",
            "How do I assign a house during registration ",
            "Is guardian information compulsory ",
            "Where to enter address details in registration ",
            "Can I save a draft of a registration form ",
            "How do I upload documents during registration ",
            "What if two students have the same name ",
            "Can I use an admission number during registration ",
            "How to manage sibling details during registration ",
            "Can I import data from a previous school ",
            "Do I need a guardian ID during registration ",
            "How to add previous academic records ",
            "Can I assign a bus route while registering ",
            "Is religion a required field in the form ",
            "What does the KYS section mean in registration ",
            "Can I set a student ID manually ",
            "How to mark a student as part-time ",
            "What status should be used for temporary admission ",
            "Can I register a student under a scholarship ",
            "Is there an approval workflow after registration ",
            "How to add multiple guardians during registration ",
            "Can I assign a student to multiple activities ",
            "What’s the limit on uploading registration documents ",
            "How do I handle incomplete registrations ",
            "Can I register a student without an email ID ",
            "Can I use a mobile number as login for a student ",
            "How to assign login credentials at the time of registration ",
            "Can I view all registered students in a list ",
            "How to generate a report of newly registered students ",
            "What is the difference between registration and admission ",
            "Can I cancel a registration ",
            "How to assign a unique registration number ",
            "Can I mark a student as already enrolled while registering ",
            "Where do I save emergency contacts in registration "
        ],
        'religion_master': [
            "How can I add a new religion ",
            "Where do I manage the religion master ",
            "Can I edit an existing religion ",
            "Is it possible to delete a religion entry ",
            "How do I view the list of all religions ",
            "Is religion mandatory in the student form ",
            "Can a student be assigned more than one religion ",
            "How do I bulk update religion data for students ",
            "What is the purpose of the religion master ",
            "Where can I find religion master settings ",
            "Can I create a custom religion entry ",
            "Is there a default list of religions provided ",
            "Can I search students by religion ",
            "How do I enable religion field in student form ",
            "Who has permission to manage religion data ",
            "Is there any report based on religion data ",
            "Can I restrict access to religion information ",
            "How is religion linked to other student data ",
            "Can I set a default religion for admissions ",
            "Can students update their religion info ",
            "Is religion used in any analytics or filtering ",
            "Can I deactivate a religion entry ",
            "What happens if religion is not selected ",
            "How do I import religion data from a file ",
            "Can I export religion data for all students ",
            "Are religion names case-sensitive ",
            "Is there a log for changes in religion data ",
            "Can I filter religion data by date added ",
            "Can I group students by religion in reports ",
            "Can I add regional religious groups ",
            "What are the allowed characters in religion names ",
            "How do I merge duplicate religion entries ",
            "Can I tag religions with categories ",
            "Is religion data shared across departments ",
            "Can I undo changes in religion master ",
            "Can I hide certain religions from dropdowns ",
            "What should I do if a religion is missing ",
            "How do I standardize religion names ",
            "Can I add foreign religious denominations ",
            "Is there an approval process for adding religions ",
            "Where is the religion master module located ",
            "How can I see when a religion was last edited ",
            "Can I clone an existing religion entry ",
            "Is the religion list visible to parents ",
            "Can I archive old religion entries ",
            "Do I need to manually refresh the religion list ",
            "Can I see religion-wise admission trends ",
            "Is religion data linked to caste or community ",
            "Can I get an alert for missing religion info ",
            "Can I print a religion-wise student report ",
            "How do I reset the religion master to default "
        ],
        'caste': [
            "How do I add a new caste to the system ",
            "Can I edit an existing caste entry ",
            "Where is the caste master section located ",
            "How do I manage caste information for students ",
            "Is caste mandatory in student registration ",
            "Can I remove a caste from the list ",
            "Can I bulk upload caste details ",
            "Is it possible to filter students by caste ",
            "How does caste relate to student category ",
            "Is there a caste-based report available ",
            "Can I import caste data from an Excel sheet ",
            "What’s the process for updating caste entries ",
            "Can a student’s caste be changed later ",
            "Is caste data visible to all staff ",
            "Can I assign a default caste for admissions ",
            "Who can modify caste information ",
            "How many castes can be listed ",
            "Is caste data shared with other modules ",
            "Can I hide certain castes from dropdowns ",
            "What format should caste names follow ",
            "Is there an audit log for caste changes ",
            "Can I group castes under broader categories ",
            "Can I search for caste-wise student data ",
            "How do I view all registered castes ",
            "Can caste be left blank during registration ",
            "How is caste data used in reporting ",
            "Can I generate caste-wise admission stats ",
            "Is there an option to deactivate a caste ",
            "How can I export caste details ",
            "Can I merge duplicate caste entries ",
            "Is there a restriction on caste name length ",
            "How to remove an incorrectly added caste ",
            "Can I mark a caste as obsolete ",
            "Do caste changes reflect across modules ",
            "How do I check who edited a caste entry ",
            "Is caste master available to department heads ",
            "How do I reset caste master settings ",
            "What happens if caste is not selected ",
            "Can caste be auto-filled based on category ",
            "Can I add regional caste names ",
            "Are caste names case-sensitive ",
            "Can students view their caste info ",
            "How do I restrict editing of caste fields ",
            "Is there a report of students without caste info ",
            "Can caste be linked to scholarships ",
            "Is caste information available in ID cards ",
            "How can I standardize caste naming ",
            "What happens when caste data is archived ",
            "Is caste data backed up regularly ",
            "Can caste be used in search filters "
        ],
        'student_category': [
            "How can I add a new student category ",
            "Where do I edit an existing student category ",
            "How are student categories used in admissions ",
            "Can I delete a student category ",
            "Is it possible to assign a default student category ",
            "How do I bulk update student categories ",
            "Can I create custom student categories ",
            "Is student category mandatory during registration ",
            "Can I restrict access to student categories ",
            "Can student category be changed later ",
            "How do I view all available student categories ",
            "Can student categories be linked to fee structures ",
            "Is there a report by student category ",
            "Can I import student categories from a file ",
            "What is the maximum number of student categories allowed ",
            "Can I duplicate a student category ",
            "Are student categories visible to students ",
            "Who can modify student categories ",
            "Can I group student categories for reports ",
            "Can I filter students by category ",
            "How do I remove an unwanted student category ",
            "Is there a history of changes to student categories ",
            "Can student categories be color-coded ",
            "How to standardize student category names ",
            "Is student category used in scholarship applications ",
            "Can I set category-based admission criteria ",
            "How do I export student category data ",
            "Can I deactivate a student category without deleting ",
            "Can I merge two student categories ",
            "Is there an option for sub-categories under a category ",
            "Can student category be used in certificates ",
            "Is student category required for each grade ",
            "Can I assign different categories for siblings ",
            "What happens if no category is selected for a student ",
            "Is there a default student category setting ",
            "Can I create category-based student groups ",
            "How do I audit changes to student category settings ",
            "Can student categories be sorted alphabetically ",
            "Can students be reassigned to a different category in bulk ",
            "How to ensure student categories are consistent ",
            "Can student category impact exam reports ",
            "Is student category linked to hostel allocation ",
            "Can I archive old student categories ",
            "How does category selection affect fee discounts ",
            "How can I restrict edits to student categories ",
            "Can I create temporary student categories ",
            "Can student categories be department-specific ",
            "Is category selection mandatory for alumni data ",
            "How do I map student categories to academic years ",
            "Can I run a category-wise student strength report "
        ],
        'document_master': [
            "How do I add a new document type in the system ",
            "Can I edit an existing document type ",
            "Where do I manage student-related documents ",
            "Is it possible to delete a document master entry ",
            "How do I set mandatory documents for admission ",
            "Can I upload document templates in document master ",
            "How to categorize documents under document master ",
            "Who has access to modify the document master ",
            "Can I archive outdated document types ",
            "Is there a limit on the number of documents in master ",
            "Can document master entries be customized per school ",
            "How do I assign documents to student categories ",
            "Can I map documents to specific courses ",
            "Can I set expiry dates for document master records ",
            "How can I export document master data ",
            "Is document master linked to the admission process ",
            "Can I have different document masters for different branches ",
            "Can I bulk update the document master ",
            "Can I restore deleted document master entries ",
            "Is it possible to duplicate document master templates ",
            "How to mark certain documents as optional ",
            "How to enforce document submission deadlines ",
            "Is document master visible to students or only admins ",
            "Can document master affect exam eligibility ",
            "How do I enable notifications for missing documents ",
            "Can document types be sorted alphabetically ",
            "How do I make some documents visible only to admins ",
            "Can students upload documents against document master entries ",
            "Is there a document upload size limit ",
            "Can I integrate third-party storage with document master ",
            "Can I view document master changes history ",
            "How do I deactivate a document without deleting it ",
            "Is there a report for document collection status ",
            "How to map documents to events like graduation ",
            "How do I enforce document re-verification yearly ",
            "Can different classes have different document requirements ",
            "Can I create document master templates for next year ",
            "How to manage nationality-specific documents ",
            "How can I add multiple formats for the same document ",
            "Can I assign documents based on academic year ",
            "Is digital signing possible through document master ",
            "Can documents be auto-tagged to students ",
            "Can we make some documents confidential ",
            "How do I set reminders for document submission ",
            "Can parents view required documents via portal ",
            "Is document master integrated with student KYC ",
            "Can alumni documents be managed under document master ",
            "Can we automate document expiry alerts ",
            "How do I ensure mandatory documents are uploaded before exam registration ",
            "Can document master integrate with fee waivers based on document verification "
        ],
        'student_kys': [
            "What does KYS stand for in student data ",
            "How can I update the KYS information for a student ",
            "Where can I find the KYS section in student profile ",
            "Can I modify KYS data after student registration ",
            "How to track KYS status for students ",
            "What information is required in KYS for students ",
            "Can KYS data be linked to student academic records ",
            "Where do I update emergency contact details in KYS ",
            "How to add parent details to KYS ",
            "What documents are needed for KYS validation ",
            "How do I update a student's KYS form ",
            "Can I attach scanned copies of documents in KYS ",
            "What happens if KYS details are incomplete ",
            "Is KYS mandatory for all students ",
            "Where to view the status of KYS for students ",
            "How can I delete outdated KYS information ",
            "Can I submit KYS data online ",
            "What are the consequences of not filling KYS ",
            "How do I send a reminder for incomplete KYS data ",
            "Is KYS information shared with other departments ",
            "What is the process for verifying KYS data ",
            "How do I mark KYS as completed for a student ",
            "Can KYS data be updated after the student's admission ",
            "Is there a deadline for completing KYS ",
            "Can I print or download KYS for a student ",
            "What happens if KYS is rejected ",
            "How do I notify parents about missing KYS data ",
            "Can KYS include medical information ",
            "Where to find a KYS report for all students ",
            "How to track students with incomplete KYS ",
            "Is there an approval process for KYS ",
            "How do I update KYS status in the student portal ",
            "Can KYS data be linked to the student's attendance ",
            "Where do I update the student's address in KYS ",
            "Can I add a student's financial details in KYS ",
            "How do I request KYS data from students ",
            "Can KYS include information about a student's health ",
            "How do I remove old KYS forms from the system ",
            "Can I set up a KYS reminder system ",
            "Is there an option to lock KYS after approval ",
            "What are the security measures for KYS data ",
            "How do I track the KYS verification process ",
            "Can I update KYS in bulk for multiple students ",
            "What should I do if a student refuses to fill KYS ",
            "Can KYS be shared with government authorities ",
            "How do I assign KYS-related tasks to staff ",
            "What is the KYS approval workflow ",
            "How to generate a KYS report for audit purposes "
        ],
        'student_bulk_update': [
            "How can I update multiple students at once ",
            "Where can I access the bulk update option ",
            "What fields can I update in bulk for students ",
            "Is there a way to bulk update student addresses ",
            "Can I update student fees in bulk ",
            "How do I modify multiple student records in one go ",
            "Can I use Excel to bulk update student data ",
            "How to perform a bulk update for class assignments ",
            "Can I bulk update student contact details ",
            "Is it possible to update student status in bulk ",
            "How do I perform bulk updates for student grades ",
            "What is the limit on the number of students in a bulk update ",
            "Can I update student health information in bulk ",
            "How to bulk update student extracurricular activities ",
            "Can I apply bulk updates to student enrollment status ",
            "Can I bulk update student guardianship details ",
            "How do I verify bulk updates before applying them ",
            "Is there an approval process for bulk updates ",
            "How to bulk update student timetable data ",
            "Can I bulk update student roles in the system ",
            "What should I do if a bulk update fails ",
            "How can I track the success of a bulk update ",
            "Can I undo a bulk update ",
            "How do I handle conflicting data during a bulk update ",
            "Is it possible to bulk update student photos ",
            "Can I bulk update student transport details ",
            "How to ensure data accuracy in bulk updates ",
            "Can I assign bulk update tasks to staff members ",
            "What file formats are supported for bulk update ",
            "Can I set filters to update specific groups of students ",
            "Is there a log for tracking bulk updates ",
            "How do I schedule regular bulk updates ",
            "Can I update student health status in bulk ",
            "What are the requirements for bulk updating grades ",
            "How to bulk update student course enrollments ",
            "Can I bulk update student scholarship details ",
            "What happens if I mistakenly update the wrong data in bulk ",
            "How do I import bulk update data from another system ",
            "Is there a notification for successful bulk updates ",
            "Can I update student behavior records in bulk ",
            "How to bulk update student uniform sizes ",
            "Can I bulk update student club memberships ",
            "How do I assign bulk update permissions to admins ",
            "What safeguards are in place for bulk updates ",
            "Can I review changes before completing a bulk update ",
            "How to resolve errors after performing a bulk update ",
            "Is there a template for bulk updating student records ",
            "Can I use bulk update for transferring students between classes ",
            "How do I handle duplicate entries during bulk updates ",
            "What is the process for bulk updating graduation years "
        ],
        'student_roll_no_generator': [
            "How do I generate roll numbers for students ",
            "Can I generate roll numbers automatically ",
            "What format should the roll number follow ",
            "Can I generate roll numbers for all students at once ",
            "How do I assign unique roll numbers to students ",
            "Can I customize the roll number format ",
            "What happens if two students have the same roll number ",
            "How do I regenerate roll numbers for a batch of students ",
            "Can I set a specific range for roll numbers ",
            "Is there a way to batch generate roll numbers based on class ",
            "How do I assign roll numbers for newly admitted students ",
            "Can I include the student’s department in their roll number ",
            "What is the best practice for generating roll numbers ",
            "Can I manually override the automatically generated roll number ",
            "How do I reset roll numbers for students ",
            "Can I assign roll numbers to students already enrolled ",
            "Is there an option to exclude certain students from roll number generation ",
            "How do I handle roll number generation for transferred students ",
            "Can I integrate roll number generation with student registration ",
            "How do I link roll numbers to student records ",
            "Can I generate roll numbers in bulk for different classes ",
            "What should I do if the roll number generator gives an error ",
            "Can I change the roll number format after it's generated ",
            "How to ensure roll numbers are unique across multiple years ",
            "Can I update roll numbers once they are assigned ",
            "Is there a way to verify roll numbers before finalizing ",
            "How do I delete or clear existing roll numbers ",
            "Can I generate roll numbers based on alphabetical order ",
            "How do I include student’s batch information in the roll number ",
            "Can I generate roll numbers based on student admission date ",
            "Can I assign roll numbers manually for specific students ",
            "How do I reassign roll numbers if there is a mistake ",
            "What if a student’s roll number conflicts with another ",
            "How can I ensure roll numbers remain consistent in the system ",
            "How to export generated roll numbers for reporting purposes ",
            "Is there an option to generate roll numbers with specific prefixes ",
            "How to generate roll numbers for different sections of the same class ",
            "Can I create a roll number format that includes the academic year ",
            "How can I generate roll numbers for international students ",
            "Can I track roll number generation history ",
            "How do I assign roll numbers for multiple programs simultaneously ",
            "Can I allow students to choose their roll numbers ",
            "Is there an option to generate roll numbers with a checksum ",
            "How do I generate roll numbers for students across different campuses ",
            "Can I bulk assign roll numbers to students based on their grades ",
            "How do I ensure that roll numbers are assigned in sequence ",
            "How to resolve discrepancies in roll number assignments ",
            "Can I implement a custom rule for roll number generation ",
            "How do I validate roll numbers for students before issuing them "
        ],
        'student_optional_subject': [
            "How do I assign optional subjects to students ",
            "Can students choose their optional subjects ",
            "How do I view a student’s chosen optional subjects ",
            "What is the process for adding optional subjects ",
            "Can I change a student's optional subject after selection ",
            "How do I ensure students select their optional subjects on time ",
            "Is there a limit to the number of optional subjects a student can choose ",
            "How do I update the available list of optional subjects ",
            "Can I set prerequisites for optional subject selection ",
            "How do I check if a student has opted for an optional subject ",
            "Can optional subjects be added after the admission process ",
            "What should I do if a student wants to change their optional subject ",
            "How do I allocate optional subjects to students in bulk ",
            "Can I assign optional subjects based on student preferences ",
            "How do I generate a report for optional subjects chosen by students ",
            "Can I allow students to modify their optional subject choices ",
            "How do I limit optional subject selection to certain students ",
            "Can I assign optional subjects automatically based on certain criteria ",
            "How can I ensure that students select from a list of approved optional subjects ",
            "Is there an option to set a deadline for optional subject selection ",
            "Can I track the status of optional subject assignments for students ",
            "How do I remove an optional subject from a student’s list ",
            "Can I provide students with a list of available optional subjects to choose from ",
            "What happens if a student doesn’t select an optional subject ",
            "How do I manage optional subject assignments for different years ",
            "Can I allow students to select more than one optional subject ",
            "How do I handle students who want to take more than the allowed number of optional subjects ",
            "How do I validate if a student meets the requirements for an optional subject ",
            "Can I assign optional subjects based on student performance ",
            "How do I track which students have selected which optional subjects ",
            "Can I send reminders to students to select their optional subjects ",
            "Can I set up a waiting list for optional subjects ",
            "How do I handle students who missed the deadline for selecting optional subjects ",
            "Can optional subjects be associated with specific teachers ",
            "How do I assign optional subjects based on class schedules ",
            "Is there a way to allow students to switch optional subjects after the term starts ",
            "Can I generate a list of students who selected each optional subject ",
            "How do I ensure students are aware of the optional subject choices ",
            "Can I create a rule to assign optional subjects based on student interest ",
            "How do I allocate optional subjects based on class size ",
            "Can I bulk assign optional subjects to students based on their major ",
            "How do I track the success rate of students in optional subjects ",
            "Is there an option to review optional subject selection before finalizing ",
            "Can I add new optional subjects to the system after the term begins ",
            "How do I assign optional subjects to students in different campuses ",
            "Can I set a limit on the number of students in each optional subject ",
            "How do I handle optional subject assignments when a student changes their course "
        ],
        'student_promote_detain': [
            "How do I promote students to the next grade ",
            "What criteria are used to promote a student ",
            "Can I detain a student for not passing the exams ",
            "How do I check if a student is eligible for promotion ",
            "Can I promote students based on their performance ",
            "What is the process for detaining students ",
            "How do I handle students who do not meet the promotion criteria ",
            "Can I set promotion requirements based on grades ",
            "How do I notify students about their promotion status ",
            "What happens if a student does not qualify for promotion ",
            "Can I promote students without exam results ",
            "How do I decide which students should be detained ",
            "Can I promote students if they have failed in only one subject ",
            "How do I update student records after promotion ",
            "Can I review the promotion results before finalizing them ",
            "How do I generate a list of promoted and detained students ",
            "What actions do I need to take after detaining a student ",
            "Can a student appeal if they are detained ",
            "How do I notify parents about their child's detention or promotion ",
            "Can I change a student's promotion status after it's been decided ",
            "How do I set promotion criteria for different classes ",
            "Can I automatically promote students who meet certain criteria ",
            "What happens if a student refuses to accept their detention ",
            "How can I track students who are eligible for promotion ",
            "Can I bulk promote students based on their grades ",
            "How do I handle students who do not attend the promotion exam ",
            "How do I review the promotion process for fairness ",
            "Can I detain a student based on behavior instead of grades ",
            "How do I assign a student to a detention class ",
            "What happens if a student is promoted but does not attend the next grade ",
            "Can I promote students based on teacher recommendations ",
            "How do I handle students who are promoted but fail in the new grade ",
            "How do I ensure all students are fairly considered for promotion ",
            "Can I detain students with incomplete attendance records ",
            "How do I keep track of students who are promoted or detained over time ",
            "What should I do if a student meets the criteria for promotion but refuses it ",
            "Can I detain a student even if they meet the academic requirements ",
            "How do I set up notifications for promotion results ",
            "What are the legal implications of detaining a student ",
            "How do I manage promotion and detention records for multiple years ",
            "Can I promote students based on extracurricular achievements ",
            "How do I handle students who want to repeat a grade voluntarily ",
            "Can I promote students with conditional requirements ",
            "How do I inform students about their promotion or detention status ",
            "Can I detain a student for not meeting social or behavioral expectations ",
            "How do I monitor and evaluate the promotion and detention system "
        ],
        'left_student': [
            "How do I mark a student as left the school ",
            "What is the process to handle students who leave the school ",
            "Can I update a student's record if they leave mid-year ",
            "How do I track students who leave the school ",
            "Can I record the reason why a student left the school ",
            "How do I handle students who leave before completing their course ",
            "Can I flag students who have left the school in the system ",
            "How do I notify faculty about a student leaving ",
            "How do I process a student who has left the school voluntarily ",
            "Can I update student status if they transfer to another school ",
            "How do I deal with students who leave due to medical reasons ",
            "What should I do if a student leaves the school before graduation ",
            "Can I issue a certificate for students who leave the school ",
            "How do I ensure a student's record is properly updated when they leave ",
            "Can I track students who leave the school due to disciplinary reasons ",
            "How do I handle students who leave the school for personal reasons ",
            "Can I re-enroll a student who left and later wishes to return ",
            "How do I process the departure of a student who has left voluntarily ",
            "Is there a way to mark students as left in the system automatically ",
            "How do I transfer a student’s records to their new school ",
            "How do I manage refunds for students who leave the school mid-session ",
            "Can I prevent a student from rejoining if they left for disciplinary reasons ",
            "How do I delete or archive records for students who have left ",
            "Can I set a deadline for students to report their decision to leave ",
            "How do I generate a report of students who have left the school ",
            "Can I keep records of students who leave for future reference ",
            "How do I handle the academic records of students who left early ",
            "Can I provide a transfer certificate for students leaving the school ",
            "How do I mark a student as left when they transfer to a different branch ",
            "What happens if a student leaves without informing the school ",
            "How do I process the withdrawal of students from extracurricular activities ",
            "Can I issue a refund for tuition if a student leaves before the semester ends ",
            "How do I update the attendance records for students who leave the school ",
            "Can I track students who leave for employment reasons ",
            "How do I handle students who leave because of financial difficulties ",
            "What should I do if a student leaves the school but owes fees ",
            "How do I ensure all documents are returned when a student leaves ",
            "Can I allow students who left to re-enroll after a certain period ",
            "How do I mark a student's final exam results after they leave ",
            "Can I update the status of students who have transferred to online courses ",
            "How do I monitor students who leave the school at different times during the year ",
            "What happens if a student leaves the school but doesn't complete their assignments ",
            "Can I track a student's history even after they have left the school ",
            "How do I notify parents when their child leaves the school ",
            "Can I maintain a waiting list for students who left but might return ",
            "How do I handle the alumni records of students who have left the school ",
            "Can I automatically remove students who have left from all school activities "
        ],
        'meritorious_student': [
            "How do I identify meritorious students ",
            "What criteria are used to determine meritorious students ",
            "Can I award certificates to meritorious students ",
            "How do I create a list of meritorious students ",
            "Can I mark a student as meritorious based on their academic performance ",
            "How do I track meritorious students over the years ",
            "Can I give special recognition to meritorious students ",
            "How do I generate a report for meritorious students ",
            "Can I create a scholarship program for meritorious students ",
            "How do I set the minimum grade for being considered meritorious ",
            "Can I assign special privileges to meritorious students ",
            "How do I handle the academic records of meritorious students ",
            "Can I give meritorious students priority in extracurricular activities ",
            "How do I celebrate the achievements of meritorious students ",
            "Can I create a hall of fame for meritorious students ",
            "How do I evaluate students for merit-based awards ",
            "Can I provide additional opportunities for meritorious students ",
            "How do I organize an award ceremony for meritorious students ",
            "Can I publish the names of meritorious students on the school website ",
            "How do I verify the eligibility of students for meritorious recognition ",
            "Can I assign specific subjects for meritorious students to excel in ",
            "How do I handle the nomination process for meritorious students ",
            "Can I create an honor roll for meritorious students ",
            "How do I announce the names of meritorious students ",
            "Can I include non-academic achievements in meritorious student criteria ",
            "How do I track the progress of meritorious students over the years ",
            "Can I create a mentorship program for meritorious students ",
            "How do I ensure that the selection process for meritorious students is fair ",
            "Can I allow students to nominate others for meritorious recognition ",
            "How do I celebrate meritorious students during school events ",
            "Can I give meritorious students additional marks for achievements outside school ",
            "How do I maintain records of students who were previously meritorious ",
            "Can I have a special ceremony for meritorious students at the end of the year ",
            "How do I ensure that meritorious students are recognized in front of their peers ",
            "Can I provide a badge or pin for meritorious students ",
            "How do I encourage other students to strive for merit-based achievements ",
            "Can I allow students to apply for meritorious status based on their achievements ",
            "How do I handle cases where a meritorious student is not performing well academically ",
            "Can I track the impact of meritorious student programs on school performance ",
            "How do I review and update the criteria for meritorious students every year ",
            "Can I create a leadership program for meritorious students ",
            "How do I handle situations where a student with excellent marks is not recognized as meritorious ",
            "Can I establish a program to support meritorious students with college applications ",
            "How do I ensure equal opportunities for all students to be recognized as meritorious ",
            "Can I create a mentor-mentee relationship between meritorious students and senior leaders ",
            "How do I highlight the achievements of meritorious students in school newsletters ",
            "Can I grant special privileges to meritorious students during school trips ",
            "How do I encourage meritorious students to engage in community service "
        ],
        'student_sub_group': [
            "How do I create a sub-group for students ",
            "Can I assign students to specific sub-groups ",
            "How do I manage sub-groups within a class ",
            "What is the process to change a student’s sub-group ",
            "How do I track students in different sub-groups ",
            "Can I add or remove students from a sub-group ",
            "How do I view all the sub-groups in the system ",
            "Can I assign a teacher to a specific sub-group ",
            "How do I allocate resources for each sub-group ",
            "Can I generate reports for each sub-group ",
            "How do I label sub-groups based on student performance ",
            "Can I split a class into multiple sub-groups ",
            "How do I ensure that all sub-groups are balanced in size ",
            "Can I assign activities to specific sub-groups ",
            "How do I ensure that no student is left out of any sub-group ",
            "Can I have a sub-group for students with special needs ",
            "How do I track sub-group progress over time ",
            "Can I set different goals for each sub-group ",
            "How do I communicate with sub-groups individually ",
            "Can I organize events specifically for a sub-group ",
            "How do I manage assessments for each sub-group ",
            "Can I create sub-groups based on student interests ",
            "How do I handle sub-groups for different subjects ",
            "Can I change a student's sub-group after the semester starts ",
            "How do I ensure that sub-groups are homogeneous in terms of ability ",
            "Can I combine two sub-groups for special activities ",
            "How do I track attendance for students in sub-groups ",
            "Can I allocate different teaching assistants to sub-groups ",
            "How do I monitor the performance of students in each sub-group ",
            "Can I create sub-groups for co-curricular activities ",
            "How do I manage communication between multiple sub-groups ",
            "Can I assign specific tasks to students within a sub-group ",
            "How do I organize parent-teacher meetings for sub-groups ",
            "Can I assign a student leader to each sub-group ",
            "How do I manage group projects within a sub-group ",
            "Can I have a sub-group for high achievers ",
            "How do I view a list of all students in a particular sub-group ",
            "Can I merge two sub-groups based on certain criteria ",
            "How do I handle conflicts between students in the same sub-group ",
            "Can I create sub-groups for extra-curricular activities like sports ",
            "How do I ensure that students in sub-groups are equally distributed ",
            "Can I track a student's movement between sub-groups ",
            "How do I create sub-groups based on student preferences ",
            "Can I assign projects specifically for certain sub-groups ",
            "How do I maintain records for students in different sub-groups ",
            "Can I ensure that sub-groups are diverse in terms of skill set ",
            "How do I handle changes in sub-group assignments over time ",
            "Can I give performance feedback for students in each sub-group "
        ],
        'deleted_student_log': [
            "How can I view the deleted student log ",
            "What information is available in the deleted student log ",
            "Can I retrieve details of deleted students from the log ",
            "How do I track why a student was deleted ",
            "Can I search the deleted student log by student name ",
            "How can I restore a deleted student’s record ",
            "What happens when a student is deleted from the system ",
            "Can I see when a student was deleted ",
            "How do I filter the deleted student log by date ",
            "Can I generate a report for deleted students ",
            "How do I ensure no deleted records are lost ",
            "Can I find out if a student was deleted by mistake ",
            "How do I handle cases of accidentally deleted students ",
            "Can I check if a student was deleted due to disciplinary reasons ",
            "How do I monitor the frequency of student deletions ",
            "Can I export the deleted student log to a file ",
            "How do I prevent accidental deletion of student records ",
            "Can I restore a student to their original class after deletion ",
            "How do I track which admin deleted a student’s record ",
            "Can I set permissions for who can delete student records ",
            "How do I handle deleted student data in the archive ",
            "Can I analyze patterns in student deletions ",
            "How do I ensure deleted student information is securely stored ",
            "Can I delete a student’s record permanently from the system ",
            "How do I review past deletions of student records ",
            "Can I automatically flag certain records for deletion ",
            "How do I prevent deleted student records from affecting reports ",
            "Can I search for specific reasons for student deletions ",
            "How do I audit deleted student records for compliance ",
            "Can I recover deleted student data after a certain period ",
            "How do I add notes about a student’s deletion to the log ",
            "Can I create alerts for deleted student records ",
            "How do I verify that a student’s deletion was authorized ",
            "Can I set up notifications for deleted student records ",
            "How do I handle the deletion of a student with ongoing exams ",
            "Can I track the deletion of students in real-time ",
            "How do I manage deleted student records for future reference ",
            "Can I flag suspicious deletions in the student log ",
            "How do I search the deleted student log by reason for deletion ",
            "Can I track deleted students by the class they were in ",
            "How do I prevent unauthorized deletion of student records ",
            "Can I recover partially deleted student records ",
            "How do I create a backup of the deleted student log ",
            "Can I automate the deletion process for old student records ",
            "How do I secure the deleted student log against tampering ",
            "Can I request a report on student deletions for auditing purposes ",
            "How do I ensure the deleted student log complies with data privacy laws "
        ]
    },

        'employee_module': {
        'employee_master': [
            "How do I add a new employee to the system ",
            "Can I edit an employee's details ",
            "How do I remove an employee from the system ",
            "What fields are required to register an employee ",
            "Can I upload an employee's documents ",
            "How do I assign an employee to a department ",
            "Can I assign an employee to multiple roles ",
            "How do I link an employee to a manager ",
            "Can I update an employee’s salary details ",
            "How do I track an employee's work history ",
            "Can I add an employee's emergency contact information ",
            "How do I assign an employee to a specific project ",
            "Can I set an employee's leave balance in the system ",
            "How do I add an employee's performance reviews ",
            "Can I assign an employee to a training program ",
            "How do I assign an employee to a specific location ",
            "Can I change an employee's work shift ",
            "How do I view an employee’s attendance history ",
            "Can I add an employee’s health insurance details ",
            "How do I create a profile for a new employee ",
            "Can I mark an employee's position as vacant ",
            "How do I create employee benefits for an employee ",
            "Can I assign an employee a work phone number ",
            "How do I manage the employee's direct deposit details ",
            "Can I associate an employee with a specific project team ",
            "How do I track an employee’s skill set ",
            "Can I create custom fields for employee data ",
            "How do I monitor the status of an employee’s probation period ",
            "Can I add a new employee with bulk registration ",
            "How do I manage the probation period of employees ",
            "Can I track an employee’s promotion history ",
            "How do I set up an employee's role for the payroll system ",
            "Can I create an employee's career development plan ",
            "How do I ensure employee data is updated in real time ",
            "Can I add work-related certifications for an employee ",
            "How do I assign an employee a mentor ",
            "Can I update an employee’s work location details ",
            "How do I track an employee’s completed training courses ",
            "Can I set an employee's leave entitlement ",
            "How do I keep track of an employee's bonus payments ",
            "Can I assign different roles to an employee in various departments ",
            "How do I add an employee’s bank details to the system ",
            "Can I verify an employee's identity in the system ",
            "How do I track employee appraisals ",
            "Can I create an employee contract within the system ",
            "How do I mark an employee as terminated ",
            "Can I maintain a history of employee bonuses and raises "
        ],
        'designation': [
            "How do I add a new designation ",
            "Can I edit an existing designation ",
            "Show me the list of designations ",
            "What is the process to delete a designation ",
            "Help me update the designation name ",
            "How to search for a designation ",
            "Where can I manage employee designations ",
            "Add a new designation for teachers ",
            "Add a designation for office staff ",
            "Create a new role under designations ",
            "Remove a designation that's no longer needed ",
            "How do I assign designations to employees ",
            "Which module has the designation list ",
            "How can I download the list of designations ",
            "Can I duplicate an existing designation ",
            "I want to modify a designation title ",
            "Is there a way to archive designations ",
            "How to view active designations ",
            "How to view inactive designations ",
            "Help me find a specific designation ",
            "How do I add multiple designations quickly ",
            "Bulk upload designations possible ",
            "Where is the designation master located ",
            "How to approve newly added designations ",
            "Can I set hierarchy in designations ",
            "Set reporting structure using designations ",
            "What's the best way to organize designations ",
            "Link designations with departments ",
            "Is designation mandatory for employees ",
            "How to mark a designation as senior level ",
            "How to create designations for temporary staff ",
            "How do I transfer employees between designations ",
            "How many designations are allowed ",
            "What's the maximum character limit for a designation name ",
            "Can I assign the same designation to different employees ",
            "Create academic designations ",
            "Create administrative designations ",
            "Create technical staff designations ",
            "Restrict access based on designation ",
            "Can I set permissions based on designation ",
            "Where to update the designation descriptions ",
            "How to categorize designations ",
            "How to assign designations while onboarding employees ",
            "Designations for contract employees ",
            "Designations for permanent employees ",
            "View designation-wise employee count ",
            "Generate designation-wise employee reports ",
            "Search employee by designation ",
            "What happens if a designation is deleted ",
            "Is it possible to merge two designations "
        ],
        'deaprtment': [
            "How do I create a new department ",
            "Where can I view all departments ",
            "How to edit a department name ",
            "Can I delete a department ",
            "How to assign employees to a department ",
            "List all available departments ",
            "Show me the department master ",
            "Add a department for academic staff ",
            "Add a department for administrative staff ",
            "Create a technical department ",
            "How to manage departments ",
            "Is there a search option in departments ",
            "How to deactivate a department ",
            "How to reactivate a department ",
            "Bulk upload department names possible ",
            "Can I assign multiple employees to a department ",
            "Assign a head to a department ",
            "How do I set reporting hierarchy in a department ",
            "Departments related to academics ",
            "Departments related to support staff ",
            "Set department roles ",
            "Set department-specific permissions ",
            "What is the process for merging departments ",
            "Transfer employees between departments ",
            "How to create sub-departments ",
            "View department-wise employee list ",
            "Department names for the school ERP ",
            "Department master import/export option ",
            "Can we have the same name for two departments ",
            "Generate department-wise reports ",
            "Can I archive a department ",
            "Restore an archived department ",
            "Department master settings ",
            "Set default departments for new employees ",
            "Departments needed for new academic year ",
            "Map employees to new departments ",
            "Department-wise workload management ",
            "Separate teaching and non-teaching departments ",
            "Department structure examples ",
            "Where to manage department heads ",
            "Search department by head name ",
            "Can a department have multiple heads ",
            "Delete a department with no employees ",
            "What happens when a department is deleted ",
            "Update department description ",
            "Set department priority levels ",
            "Assign department heads automatically ",
            "Create departments during school setup ",
            "Standard departments for ERP setup ",
            "Assign department for staff onboarding "
        ],
        'grade_master': [
              "How do I add a new grade ",
              "Where can I edit a grade record ",
              "List all grades available ",
              "Can I delete a grade from the master ",
              "How to update an employee's grade ",
              "Assign grades to new employees ",
              "Set rules for grade allocation ",
              "View grade master settings ",
              "Difference between grades and designations ",
              "Grade vs Pay Level settings ",
              "Import grades in bulk ",
              "Export grade master data ",
              "Search a grade by name ",
              "Can grades have grade codes ",
              "Assign benefits based on grade ",
              "Manage grade hierarchy ",
              "Can I deactivate a grade ",
              "How to reactivate a grade ",
              "Map departments to grades ",
              "Grade settings for teaching staff ",
              "Grade settings for non-teaching staff ",
              "Assign grade during employee onboarding ",
              "Upgrade grade for promotion ",
              "Downgrade grade due to demotion ",
              "Create probationary grades ",
              "Special grades for contract employees ",
              "View employee count per grade ",
              "Configure grade wise leave policies ",
              "Grade wise salary structure setup ",
              "Are grades customizable ",
              "Set retirement age by grade ",
              "Grade criteria for bonus eligibility ",
              "Create grade groups ",
              "Assign different shifts for each grade ",
              "Create temporary grades ",
              "Set grade codes automatically ",
              "Lock grades once assigned ",
              "Create grade bands for reporting ",
              "How to merge two grades ",
              "Can a grade be renamed later ",
              "Set grade based on qualification ",
              "Grade transfer policy ",
              "Default grade for new hires ",
              "Grade mapping with performance appraisal ",
              "Configure grade-specific benefits ",
              "Mark a grade as obsolete ",
              "Set grade-specific allowances ",
              "Grade definitions for audit ",
              "How to handle missing grades ",
              "Bulk grade update option "
            ],
            'employee_group': [
                'Tell me about employee groups',
                'How do I create an employee group ',
                'Steps to manage employee groups',
                'List of employee groups',
                'Edit an employee group',
                'Delete an employee group',
                'Assign employees to a group',
                'View all employee groups',
                'Set up employee groups',
                'Employee grouping method',
                'Manage groups for employees',
                'Employee group configuration',
                'How to update employee groups',
                'Employee group entry process',
                'Show employee group details',
                'Can I merge employee groups ',
                'View employees in a group',
                'Add members to employee group',
                'Modify existing employee groups',
                'How to find employee groups',
                'Access employee group section',
                'Employee group settings guide',
                'Handling employee groups',
                'Bulk update employee groups',
                'Group assignment options',
                'Department vs employee group',
                'Employee group vs team',
                'Can I have multiple employee groups ',
                'Common employee groups setup',
                'Default employee groups list',
                'Special employee groups creation',
                'Employee group categorization',
                'Importance of employee groups',
                'Group employees based on department',
                'Managing employee groups effectively',
                'Employee group master data',
                'Employee group addition process',
                'Assign policies by employee group',
                'Separate employees into groups',
                'Grouping strategy for employees',
                'Examples of employee groups',
                'Where to create employee groups',
                'Link employee group with department',
                'Employee group vs designation',
                'Search employee groups',
                'New employee group creation',
                'Purpose of employee groups',
                'Why use employee groups ',
                'Help with employee group settings',
                'Accessing employee group master'
            ],
            'company_master': [
                'Tell me about company master',
                'What is company master in ERP ',
                'How to create a company record ',
                'Edit details in company master',
                'Delete a company from master',
                'Update company master info',
                'List of companies in master',
                'Add new company to ERP',
                'Manage company details',
                'Where to find company master',
                'Access company master settings',
                'Company master overview',
                'Master record for companies',
                'Assigning details in company master',
                'How to modify company info ',
                'Can I add multiple companies ',
                'Setting up company master',
                'Update address in company master',
                'Is GST required in company master ',
                'Upload logo in company master',
                'Configure branches in company master',
                'Difference between company master and branch',
                'Multi-company setup guide',
                'Managing multiple companies',
                'Show company master data',
                'Steps to create a company master',
                'Can we import company master ',
                'Company master help guide',
                'All fields in company master',
                'Mandatory fields in company master',
                'Company master entries',
                'Sample company master entry',
                'Edit company tax details',
                'Update company contacts',
                'Where to list companies ',
                'Support for company master module',
                'Find company master',
                'Master setup for companies',
                'Procedure for new company',
                'Can I delete a company record ',
                'Field mapping for company master',
                'ERP company master explained',
                'Admin access to company master',
                'Edit name in company master',
                'Bulk upload company master',
                'Company master portal',
                'Create master record for company',
                'Company profile update steps',
                'Company master maintenance'
        ],
        'qualification_master': [
                'Tell me about qualification master',
                'How to add a new qualification ',
                'Steps to edit qualification master',
                'Where can I find qualification master ',
                'View all qualifications',
                'Update qualification information',
                'Qualification master settings',
                'Delete a qualification entry',
                'Create qualification master record',
                'Can I import qualifications ',
                'Edit qualification details',
                'Modify existing qualifications',
                'Qualification entry guide',
                'How do I manage qualifications ',
                'Access qualification master module',
                'What is qualification master used for ',
                'Add qualifications to the system',
                'Can I categorize qualifications ',
                'Search qualifications in master',
                'Upload qualification data',
                'Required fields in qualification master',
                'Qualification list maintenance',
                'Link employee with qualification',
                'Where to enter qualifications ',
                'Qualification data entry form',
                'Add higher education qualifications',
                'Degree vs diploma in qualification master',
                'Insert qualification record',
                'Qualification master help',
                'Why use qualification master ',
                'Steps to update qualification info',
                'HR module qualification master',
                'Qualification categories in ERP',
                'How to assign qualifications ',
                'Master setup for qualifications',
                'Qualification code entry',
                'Setting up qualification types',
                'Qualification description field',
                'Qualification level configuration',
                'Manage education records',
                'Qualification master screen',
                'Edit or remove qualification',
                'Education master module',
                'Record employee qualifications',
                'Where to update qualifications ',
                'How to view qualification master ',
                'Qualification approval required ',
                'Qualification master in HRMS',
                'Qualification upload format',
                'Import qualification via Excel'
            ],
            'document_list': [
                'Tell me about document list',
                'How to add a document type ',
                'Document list configuration guide',
                'Delete a document type',
                'Update existing document details',
                'Document type master in ERP',
                'Where is the document list section ',
                'Steps to manage document list',
                'Document list for HR',
                'Can I edit document list entries ',
                'How to use document list ',
                'Add new document to list',
                'Required documents for employees',
                'Document list maintenance',
                'Edit document types',
                'Upload document types',
                'Create document list entry',
                'Document types master data',
                'Document categories list',
                'List of accepted documents',
                'Manage employee documents',
                'Employee document requirements',
                'Document type validation',
                'Link documents with employee',
                'Update employee document list',
                'Document type format',
                'Mandatory employee documents',
                'Document master data setup',
                'Track employee document list',
                'View document type list',
                'List of document names',
                'Help with document list section',
                'Document approval flow',
                'Access document list module',
                'Add ID proof documents',
                'Create master list of documents',
                'Document list form fields',
                'List medical documents',
                'Insurance document entry',
                'Where to manage document list ',
                'Update or delete document entries',
                'HR document types',
                'Upload new document templates',
                'Document master record',
                'Document settings for employees',
                'Document entry options',
                'Configuration for document names',
                'Import document types',
                'Bulk document upload'
            ],
            'appointment_type': [
                'What is appointment type ',
                'Add new appointment type',
                'Edit appointment types',
                'How to delete an appointment type ',
                'List all appointment types',
                'Appointment type setup guide',
                'Configure appointment types',
                'Manage types of appointment',
                'Appointment category master',
                'Where is appointment type module ',
                'Create appointment type',
                'Modify appointment information',
                'How to update appointment types ',
                'Types of employee appointments',
                'Can I view appointment types ',
                'Appointment duration settings',
                'Appointment type entry form',
                'Assign appointment type to employee',
                'Appointment vs contract type',
                'Full-time vs part-time type',
                'List of job appointments',
                'Appointment type configuration',
                'HR appointment type setup',
                'Master setup for appointments',
                'Temporary vs permanent type',
                'Show me appointment types',
                'Appointment record type',
                'Track appointment types',
                'Maintain appointment types',
                'How do I update appointment types ',
                'Create contract appointment type',
                'Probation appointment type entry',
                'Edit appointment categories',
                'Document appointment types',
                'Available appointment types',
                'Appointment settings in ERP',
                'Set default appointment type',
                'What are default appointment types ',
                'Employee status by appointment type',
                'Appointment master form',
                'Appointment type dropdown values',
                'Update type of appointment',
                'Appointment master section',
                'Help with appointment type module',
                'Delete appointment categories',
                'Can I add more appointment types ',
                'HRMS appointment classification',
                'Contractual appointment entry',
                'Appointment labels in the system',
                'Appointment records for HR'
            ],
             'shift_master': [
                'What is shift master ',
                'Add a new shift',
                'How do I define shifts ',
                'Update existing shift details',
                'Delete a shift entry',
                'Shift timing configuration',
                'Shift master module',
                'List all available shifts',
                'Assign shift to employee',
                'Where is shift master located ',
                'How to manage shift patterns ',
                'Shift rotation guide',
                'Edit shift information',
                'Define multiple shifts',
                'Add night shift in master',
                'Setup morning and evening shifts',
                'Can I customize shift timings ',
                'What are default shifts ',
                'Configure employee work shifts',
                'Shift scheduling in ERP',
                'Create shift time table',
                'Assign shift groups',
                'View shift list',
                'Download shift list as Excel',
                'Shift planning guide',
                'Shift category master',
                'Can I add lunch breaks in shift ',
                'How to link shift with attendance ',
                'Shift master help page',
                'Set flexible shift timings',
                'Weekend shift settings',
                'Can I copy shift data ',
                'Manage overtime shifts',
                'Employee shift roster',
                'Shift master for HRMS',
                'How to view shift details ',
                'Shift assignment history',
                'Edit or remove shift entry',
                'Bulk upload shift master',
                'Night shift vs day shift settings',
                'Multiple shift configuration guide',
                'Where to enter shift data ',
                'Shift wise attendance tracking',
                'Shift details entry form',
                'Shift duration and rules',
                'How to create shift templates ',
                'Shift overlap warning',
                'Leave and shift sync',
                'Assign default shift to new employee'
            ],
            'bank_master': [
                'Tell me about bank master',
                'Add a new bank',
                'Edit bank details',
                'Delete a bank entry',
                'Bank list for payroll',
                'How do I manage bank master ',
                'Configure banks in ERP',
                'View all banks',
                'Assign bank to employee',
                'Bank master setup steps',
                'Import bank list',
                'Bank master module guide',
                'Create bank record',
                'Edit IFSC codes in master',
                'Add bank branch information',
                'Bank master in payroll module',
                'Link employee with bank account',
                'How to change employee bank ',
                'What are default banks ',
                'Setup bank master for HRMS',
                'Manage employee bank details',
                'Payroll bank settings',
                'Bank code entry form',
                'Help with bank master',
                'List of employee banks',
                'Where to find bank master ',
                'Employee payment bank settings',
                'Update bank names and details',
                'Assign multiple banks to departments',
                'Bank configuration sheet',
                'Bank master upload format',
                'Bank and branch name entry',
                'Add new bank via form',
                'Which banks are supported ',
                'Bank account integration',
                'Bank master screen',
                'Export bank list as Excel',
                'Edit or remove banks',
                'Setup new payroll bank',
                'IFSC and MICR management',
                'Bank master validation',
                'Bank master best practices',
                'Assign bank for salary processing',
                'Default bank assignment',
                'Manage HR bank list',
                'What details are needed for bank master ',
                'Bank list overview',
                'Can I categorize banks ',
                'Configure payment bank rules'
            ],
        'my_salary_slip' : [
            "How can I view my salary slip ",
            "Can I download my previous salary slip ",
            "Where can I check my bonus details on the salary slip ",
            "How do I get details of my tax deductions ",
            "How do I view the breakdown of my salary ",
            "Can I get a digital copy of my salary slip ",
            "How can I check the amount paid for overtime in my salary slip ",
            "Can I request a hard copy of my salary slip ",
            "Where can I find the details of my allowances in the salary slip ",
            "How do I know if my salary slip has been updated ",
            "Can I view my salary slip for the last year ",
            "How do I download salary slips for tax filing purposes ",
            "Where can I check my net salary amount in the slip ",
            "How do I find out if any deductions have been made to my salary ",
            "Can I compare my salary slip across different months ",
            "How do I check if the salary slip includes my bonuses ",
            "Can I get a salary slip for a specific period ",
            "How do I print my salary slip ",
            "Can I download my salary slip from the portal ",
            "How do I check if my salary slip includes medical benefits ",
            "Can I view the total earnings in my salary slip ",
            "How can I know my Provident Fund (PF) details from the salary slip ",
            "Where can I find my HRA (House Rent Allowance) details ",
            "How do I check my pension contributions on the salary slip ",
            "Can I see my insurance deductions on my salary slip ",
            "How do I confirm if the tax calculations are accurate ",
            "Where can I find the employee benefits listed in the salary slip ",
            "Can I check the status of my salary payments ",
            "How do I see deductions for loan repayments on my salary slip ",
            "Can I get a detailed version of my salary slip ",
            "Where can I find information about salary adjustments ",
            "How do I track changes in my salary over time ",
            "Can I check if my salary slip includes my travel allowance ",
            "How can I find out about performance-based incentives in my salary slip ",
            "How do I track my paid leave days in the salary slip ",
            "Can I get salary slip notifications for new updates ",
            "How do I ensure my salary slip is accurate "
        ],
            
        'employee_profile' : [
            "How do I update my employee profile ",
            "Can I view my personal details in the profile ",
            "How do I change my contact information in the profile ",
            "Can I upload my profile picture ",
            "Where can I update my emergency contact information ",
            "How do I add my educational qualifications to my profile ",
            "Can I add my certifications to my employee profile ",
            "How do I check my job role and department in the profile ",
            "Can I view my work history in the profile ",
            "How do I update my work experience on the profile ",
            "Can I change my job title in the profile ",
            "How do I add my skills and expertise to the profile ",
            "Can I track my performance reviews in my employee profile ",
            "How do I update my bank details in the profile ",
            "Can I add my spouse or dependent information to the profile ",
            "How do I update my address details in the employee profile ",
            "Can I change my work location in the profile ",
            "How do I view my training history in my employee profile ",
            "Can I add my professional memberships to the profile ",
            "How do I track my achievements in the employee profile ",
            "Can I update my employment status in my profile ",
            "How do I view my role history and progression ",
            "Can I add my language proficiency details to my profile ",
            "How do I edit my salary details in the employee profile ",
            "Can I add my benefits and allowances information ",
            "How do I see my health insurance details in the profile ",
            "Can I view my performance goals and targets in my profile ",
            "How do I update my family details in the employee profile ",
            "Can I track my attendance history in my profile ",
            "How do I add my work achievements to my employee profile ",
            "Can I update my previous employers and references ",
            "How do I change my designation in the profile ",
            "Can I view the training programs I've attended in my profile ",
            "How do I manage my work schedule and shifts in the profile ",
            "Can I update my personal email address in the profile ",
            "How do I delete outdated information from my employee profile ",
            "Can I add my performance feedback to my profile ",
            "How do I ensure my employee profile is up to date ",
            "Can I add my personal projects and contributions ",
            "How do I view my career progression in the profile ",
            "Can I track my work anniversary and milestones ",
            "How do I ensure the privacy of my personal data in the profile "
        ],
         'employee_register': [
            'How do I access the employee register ',
            'Where can I view the full employee register ',
            'How do I search for an employee in the register ',
            'Can I filter employees by department in the register ',
            'How to download the employee register ',
            'Can I print the employee register ',
            'What is the difference between employee register and employee master ',
            'Can I export employee register to Excel ',
            'Is the employee register updated in real-time ',
            'How do I view only active employees in the register ',
            'Can I view contact details in the employee register ',
            'How to generate ID cards from the employee register ',
            'Can I sort employees by joining date in the register ',
            'How do I check the attendance from the employee register ',
            'Can I add new employees from the employee register page ',
            'Is it possible to remove someone from the employee register ',
            'How to view resigned employees in the register ',
            'Can I edit details directly from the employee register ',
            'How often is the employee register synced with HR data ',
            'Can I assign roles from the employee register ',
            'Is employee photo shown in the register ',
            'Can I customize fields in the employee register ',
            'How do I mark an employee as inactive in the register ',
            'Can I use filters like gender and category in the register ',
            'Is the employee register searchable by ID number ',
            'Can I bulk update records in the employee register ',
            'Can I sort employees alphabetically in the register ',
            'Is employee address visible in the register ',
            'Can I track promotion history through the register ',
            'How do I update designation from the employee register ',
            'Can I assign departments in the register ',
            'How is the employee register linked to payroll ',
            'Can I view emergency contacts in the register ',
            'Can I schedule emails from the employee register ',
            'How do I mark someone as left from the register ',
            'Can I add bank details in the employee register ',
            'How do I find teaching staff from the register ',
            'Is there a dashboard view for employee register ',
            'How do I sort by date of birth in the register ',
            'Can I enable two-factor login for register users ',
            'Can I use the register for employee verification ',
            'Can I link biometric data in the register ',
            'How to check who updated the employee register last ',
            'Can I download the employee register as PDF ',
            'Can I view contract status in the employee register ',
            'Can I integrate the register with payroll software ',
            'How to fetch only left employees from the register ',
            'Is there a mobile view of the employee register ',
            'Can I track employee location through the register ',
            'How do I change email ID in the employee register ',
            'Is there any backup of the employee register '
        ],
        'leave_management': [
            'How do I apply for leave ',
            'Where can I manage employee leave records ',
            'Can I cancel a leave request after submission ',
            'How to approve or reject a leave request ',
            'What types of leave can be applied ',
            'Can I view my leave balance ',
            'Is there a limit on the number of leaves per month ',
            'How do I assign leave types to employees ',
            'Where can I see pending leave requests ',
            'How do I check leave history ',
            'Can I edit a submitted leave request ',
            'How do I view department-wise leave summary ',
            'How to check leave status of an employee ',
            'Can leave requests be auto-approved ',
            'Can I export the leave management report ',
            'How is leave balance calculated ',
            'Can I assign comp-off through leave management ',
            'Is there an option for half-day leave ',
            'Can I apply for leave on behalf of someone ',
            'Can I view holidays in the leave calendar ',
            'What happens if an employee exceeds leave limit ',
            'How are weekends treated in leave calculation ',
            'Can I set carry forward rules for leaves ',
            'Is leave approval based on hierarchy ',
            'Can HR override leave approvals ',
            'How can I set a default leave policy ',
            'Can I get notifications for leave requests ',
            'Is there a mobile option to manage leave ',
            'Can I mark mass leave for all staff ',
            'Can employees see their total leave taken ',
            'How do I generate a leave ledger ',
            'Is leave encashment supported in this module ',
            'Can I block specific dates from leave application ',
            'Is it possible to import leave data from Excel ',
            'How do I view year-wise leave summary ',
            'How do I check leave for multiple employees together ',
            'Can I approve leaves in bulk ',
            'Is sick leave and casual leave treated separately ',
            'How do I update leave taken manually ',
            'Can I remove an approved leave ',
            'Can I get department-wise leave stats ',
            'How do I enable email alerts for leave actions ',
            'What permissions are required to approve leave ',
            'How do I define new leave types ',
            'Can I link leave with attendance module ',
            'What is the role of admin in leave approval ',
            'Can I set different rules for teaching and non-teaching staff ',
            'Is leave calendar integrated with public holidays ',
            'Can employees attach documents for sick leave ',
            'Can I generate monthly leave reports for payroll '
        ],
        'mark_attendance': [
            'How do I mark employee attendance ',
            'Can I mark attendance for all employees at once ',
            'How to mark half-day attendance ',
            'Is there an option to mark absent manually ',
            'Can I mark attendance using biometric ',
            'Where can I view the attendance marking page ',
            'Can I mark attendance for previous dates ',
            'How to mark late entry for an employee ',
            'Can I import attendance from Excel ',
            'How to update attendance if marked wrongly ',
            'Can attendance be marked automatically ',
            'How to mark attendance for teaching staff only ',
            'Is it possible to bulk mark attendance as present ',
            'Can I filter employees by department before marking ',
            'How to enable auto-marking of attendance ',
            'How do I see who marked attendance today ',
            'Can I mark attendance through mobile ',
            'How to notify employees about attendance status ',
            'Can leave be linked with attendance ',
            'Is approval required to mark attendance ',
            'Can I generate a report from attendance marked ',
            'Is it possible to mark attendance via QR code ',
            'Can I set default present for all and edit exceptions ',
            'How do I mark night shift attendance ',
            'How to mark multiple shifts for one employee ',
            'How can I view attendance history while marking ',
            'Can I change marked attendance status later ',
            'How to mark attendance if biometric fails ',
            'Can I set holiday while marking attendance ',
            'Can I assign attendance roles to other users ',
            'How to know who is absent today ',
            'Can I mark weekend attendance ',
            'How to view only pending attendance entries ',
            'Is there a shortcut for daily attendance marking ',
            'Can I copy yesterday’s attendance ',
            'How to save draft while marking attendance ',
            'Can attendance be marked offline ',
            'How to exclude certain employees while marking ',
            'Can attendance remarks be added ',
            'Is it possible to lock marked attendance ',
            'Can I schedule auto-attendance marking ',
            'How to change an employee’s shift while marking ',
            'How to mark training day instead of present/absent ',
            'Can I set reminders for attendance marking ',
            'How to send SMS after marking attendance ',
            'Can I add reason while marking someone absent ',
            'Can I mark attendance in different time slots ',
            'How to undo attendance marking for the day ',
            'Can I mark on-duty attendance ',
            'How to view attendance summary instantly ',
            'Can I sync attendance with payroll '
        ],
        'leave_dashboard': [
            'Where can I see the leave dashboard ',
            'How do I access the leave summary for all employees ',
            'Can I view department-wise leave data ',
            'What does the leave dashboard show ',
            'Can I check pending leave approvals from the dashboard ',
            'Is there a chart view in the leave dashboard ',
            'How do I track leave trends over months ',
            'Can I filter the leave dashboard by employee role ',
            'How to customize the leave dashboard view ',
            'Can I export leave dashboard data ',
            'Is real-time data shown in the leave dashboard ',
            'How to see employees on leave today ',
            'Does the leave dashboard show leave balance ',
            'How do I check most availed leave type ',
            'Can I compare leave taken by different employees ',
            'How do I view approved and rejected leaves ',
            'Can I search leave dashboard by employee name ',
            'Can I see leave statistics on the dashboard ',
            'Is there a date range filter on the leave dashboard ',
            'How to print leave dashboard reports ',
            'Can I get a weekly view of leaves ',
            'Is monthly summary available on the dashboard ',
            'Can I access team-wise leave status ',
            'How do I know who is on long leave ',
            'Can I get graphical view of leave consumption ',
            'Is there a pie chart for leave types ',
            'How to access previous year leave dashboard ',
            'Can I view dashboard by individual employee ',
            'Does the dashboard update automatically ',
            'Is there a shortcut to the leave dashboard ',
            'Can I bookmark the leave dashboard ',
            'Can I view absence trends in dashboard ',
            'Does the dashboard show upcoming approved leaves ',
            'How to find leave conflicts from dashboard ',
            'Can I view consolidated leave reports here ',
            'Is there a dashboard for only teaching staff ',
            'Can I customize widgets in leave dashboard ',
            'Can I get a downloadable version of the dashboard ',
            'How to view total leave used this month ',
            'Can I access dashboard from mobile ',
            'Is the leave dashboard role-restricted ',
            'Can I switch between table and graph view ',
            'How to schedule automatic reports from dashboard ',
            'Can I set alerts based on dashboard metrics ',
            'Is there a notification panel in the dashboard ',
            'Can the dashboard show upcoming holidays ',
            'How do I view leave approvals pending by manager ',
            'Does the dashboard show leave requests per month ',
            'Can I compare multiple years in the dashboard ',
            'How do I refresh the leave dashboard '
        ],
        'left_employee': [
            'How can I view the list of left employees ',
            'Where can I check employee exit records ',
            'Can I see the resignation date of an employee ',
            'How to get the exit details of a staff member ',
            'Is there a report of employees who have left ',
            'How do I update an employee as left ',
            'Can I restore a wrongly marked left employee ',
            'Where can I find resignation history ',
            'How to search by reason for leaving ',
            'Can I export the list of left employees ',
            'Does the system keep data of resigned employees ',
            'How can I archive a left employee’s profile ',
            'Can I generate a report of employees who left last month ',
            'How to add a new exit record ',
            'Can I filter employees by last working date ',
            'Is exit feedback stored in the system ',
            'How to categorize exits (resignation, termination, retirement) ',
            'Where can I find exit interviews ',
            'Can I sort left employees by department ',
            'How to remove access for a left employee ',
            'Is salary processed for left employees ',
            'How can I generate experience letters ',
            'Can I check if final settlement is completed ',
            'How to get a certificate for a left staff ',
            'Can I view exit remarks in profile ',
            'Is there a dashboard for employee exits ',
            'How to set exit status automatically ',
            'Can I track employees who left mid-year ',
            'How to manage resignation approvals ',
            'Is there a list of employees scheduled to leave ',
            'Can I view exit trends by year ',
            'How to delete a resignation entry ',
            'Can I change exit date after submission ',
            'How to mark retirement in the system ',
            'How to download a summary of left employees ',
            'Can I tag a replacement for a left employee ',
            'Is there an alert before an employee leaves ',
            'How to analyze attrition rate ',
            'Can I link resignation reason to HR actions ',
            'How to assign exit tasks to departments ',
            'Can I see tenure before employee left ',
            'Is there a bulk exit option ',
            'How to update left employee’s documents ',
            'Can I see which manager approved the exit ',
            'Is final working day and last day different ',
            'Can I merge duplicate exit records ',
            'Is there a filter for resignation vs termination ',
            'How to notify finance about exit ',
            'Can I check resigned employee’s leave balance ',
            'How to store handover details for left employees '
        ]
    },
        'fee_module': {
        'fee_module_master': [
            'What is the fee module master ',
            'How do I configure the fee module ',
            'Can I manage fee details in the fee module master ',
            'Where do I add new fee types ',
            'Is there a way to set up fee heads in the fee module master ',
            'How can I view the list of available fee categories ',
            'Can I update existing fee categories ',
            'How do I define fee structures ',
            'What fields are required in the fee module master ',
            'Can I delete fee records from the fee module master ',
            'Is there a limit to the number of fee heads I can add ',
            'How to configure payment terms in fee module master ',
            'What are the options for fee type configuration ',
            'How do I create a new fee head ',
            'Can I modify an existing fee head ',
            'How to manage discounts for each fee category ',
            'Can I link fee heads to specific departments ',
            'Where do I assign fee heads to different courses ',
            'Can I define different fee amounts based on grade levels ',
            'Can I set up fee due dates for each category ',
            'How do I assign fee heads to individual students ',
            'Can I configure fee collection frequency in the master module ',
            'Is there an option for fee collection in installments ',
            'How to set up refund rules for fee payments ',
            'Can I track fee changes in the fee module master ',
            'Can I assign payment modes (like cash, card) to fee categories ',
            'How to add penalty fees for late payments ',
            'How do I configure discounts based on early payments ',
            'Can I set up automatic fee reminders ',
            'How can I enable tax calculation for each fee ',
            'How to manage fee adjustments in the master module ',
            'Can I change the status of a fee head from active to inactive ',
            'Is there a provision to generate fee invoices in the module master ',
            'How to link fee heads to specific financial years ',
            'Can I integrate fee details with accounting systems ',
            'How do I handle multiple payment modes for fee transactions ',
            'Can I create and manage fee rules based on student categories ',
            'Is there an option to add additional charges to the fee ',
            'How do I assign fee heads to specific batches or groups ',
            'Can I create a fee report template in this module ',
            'How to assign discounts for siblings or staff members ',
            'Can I modify the fee structure during the academic year ',
            'How to delete a fee head from the system ',
            'What happens if I change the fee amount mid-year ',
            'Can I set up automated reminders for fee payments ',
            'How do I change the due date for fee payments ',
            'Can I track fee adjustments in the fee module master ',
            'How do I apply bulk fee adjustments '
        ],
        'account_master': [
			"Show me the account master settings ",
			"Where can I edit the account master ",
			"What is stored in the account master ",
			"Can I create a new account master ",
			"List all account master entries ",
			"I want to update account master details ",
			"Is the account master editable ",
			"Show account master configuration options ",
			"How do I access the account master section ",
			"Search account master entries for this term ",
			"Add a new account in the master ",
			"Remove an entry from account master ",
			"Filter account master by type ",
			"Sort account master by date added ",
			"Show latest entries in account master ",
			"Account master not loading, help ",
			"Download account master report ",
			"Print account master summary ",
			"Can I export account master to Excel ",
			"What is the use of account master ",
			"Find inactive accounts in account master ",
			"Is there a way to archive account master ",
			"Search for a specific account master ID ",
			"Who updated the account master last ",
			"Check if account master has duplicates ",
			"How to link parties to account master ",
			"Show changes in account master this week ",
			"Any recent updates in account master ",
			"Which user has access to account master ",
			"How many entries in account master "
		],
		'party_master': [
			"Open the party master module ",
			"Show all party master records ",
			"Create new party master entry ",
			"How to edit party master details ",
			"Is party master required for fee setup ",
			"Can I search in party master ",
			"List all parties in the system ",
			"View inactive entries in party master ",
			"Party master showing error, fix it ",
			"Who manages the party master ",
			"Steps to add vendor in party master ",
			"Difference between party and account master ",
			"Show vendors in party master ",
			"Add a student party entry ",
			"View party transaction history ",
			"Check party balance in master ",
			"Link fee head to party master ",
			"Can I import party master data ",
			"Download party master as PDF ",
			"Party master validation rules ",
			"Is party master connected to fees ",
			"Track dues via party master ",
			"Remove outdated party master records ",
			"Party master list for year 2024 ",
			"Filter party by region in master ",
			"Is GST info stored in party master ",
			"Get contact info from party master ",
			"Who added this party to master ",
			"Generate party report from master ",
			"Export party master to Excel "
		],
		'fee_head_groups': [
			"Define new fee head group ",
			"What is a fee head group ",
			"Show me all fee head groups ",
			"Add hostel fee group ",
			"Edit tuition fee head group ",
			"Group similar fee heads together ",
			"Fee head group list please ",
			"Filter fee head groups by category ",
			"Update fee head group structure ",
			"How to link fee heads to groups ",
			"Show fee head groups by type ",
			"Is fee head grouping mandatory ",
			"Assign students by fee group ",
			"Download fee head group summary ",
			"Create transport fee group ",
			"Merge two fee head groups ",
			"Rename fee head group ",
			"Delete unused fee head group ",
			"Fee head group usage report ",
			"How to search in fee head groups ",
			"Fee group for 2025 session ",
			"Show default fee head group ",
			"Fee head group setup process ",
			"Can I duplicate a fee head group ",
			"Who created the fee head group ",
			"Change sequence of fee head groups ",
			"Export fee head group table ",
			"Group fee heads by academic level ",
			"Enable or disable fee head group ",
			"List students by fee head group "
		],
		'fee_head_master': [
			"Open fee head master settings ",
			"Create new fee head ",
			"Add late fee in fee head master ",
			"Can I remove a fee head ",
			"Edit existing fee head info ",
			"View all fee head types ",
			"Fee head for sports activities ",
			"List fee heads for grade 10 ",
			"Sort fee heads by amount ",
			"Default fee heads setup ",
			"Tuition fee head configuration ",
			"Fee head master table view ",
			"Search for specific fee head ",
			"Assign fee head to students ",
			"Group fee heads by type ",
			"Check active fee heads ",
			"Download fee head master list ",
			"Export fee head master to Excel ",
			"Print all fee heads summary ",
			"Delete obsolete fee head ",
			"View archived fee heads ",
			"Link account to fee head ",
			"Enable GST in fee head ",
			"Fee head code management ",
			"Fee head master audit log ",
			"Who added this fee head ",
			"Check fee head usage history ",
			"Show inactive fee heads ",
			"Create category-based fee heads ",
			"Enable recurring fee head "
		],
		'receipt_master': [
			"Access receipt master module ",
			"Generate a new fee receipt ",
			"Edit existing receipt template ",
			"View all receipt formats ",
			"Assign template to receipt master ",
			"Print a sample fee receipt ",
			"Customize receipt layout ",
			"Receipt master not updating ",
			"Save default receipt format ",
			"Is logo available on receipt ",
			"Enable digital signature in receipt ",
			"Fee receipt settings please ",
			"Where to configure receipt master ",
			"Update footer on receipt ",
			"Show header in receipt format ",
			"Can we change receipt number prefix ",
			"List receipt master templates ",
			"How to print bulk receipts ",
			"Receipt master for hostel fees ",
			"Add QR code to receipt ",
			"Student copy receipt format ",
			"Receipt master usage logs ",
			"Import receipt format ",
			"Backup receipt master data ",
			"Restore receipt master layout ",
			"Who modified the receipt master ",
			"Is barcode included in receipt ",
			"Upload custom receipt design ",
			"Show deleted receipt formats ",
			"Lock receipt master editing "
		],
        "optional_fee": [
        	"Can I assign optional fees to a student ",
        	"How do I configure optional fee ",
        	"List all optional fees available ",
        	"Add a new optional fee entry ",
        	"Is there an option for optional fee ",
        	"Show students with optional fee assigned ",
        	"Enable optional fee for a group ",
        	"Optional fee setup instructions please ",
        	"What is the purpose of optional fee ",
        	"Can students choose their optional fees ",
        	"Where do I find optional fee settings ",
        	"List optional fee options for grade 8 ",
        	"Show me how to apply optional fee ",
        	"Remove optional fee from student ",
        	"Is optional fee different from regular fee ",
        	"Add transport as an optional fee ",
        	"Can optional fees be assigned automatically ",
        	"Optional fee entry not saving, help ",
        	"Can we customize optional fee labels ",
        	"Generate report for optional fees ",
        	"Print optional fee list ",
        	"Is hostel considered optional fee ",
        	"Add meal plan as optional fee ",
        	"Who can view optional fee details ",
        	"Optional fee setup for academic year 2025 ",
        	"Add optional fee for a single student ",
        	"Assign optional fee to all grade 10 students ",
        	"Optional fee not showing in receipt ",
        	"Why is optional fee not visible ",
        	"Edit existing optional fee entry ",
        	"Check optional fee for a specific student ",
        	"Search optional fee entries by category ",
        	"Optional fee options not loading ",
        	"Can I deactivate an optional fee ",
        	"How do I update an optional fee ",
        	"Enable optional fee for a class ",
        	"Optional fee form is blank ",
        	"Help me with optional fee configuration ",
        	"Show history of optional fee changes ",
        	"How many optional fees are active ",
        	"Can I import optional fee data ",
        	"Export optional fee list to Excel ",
        	"Optional fee based on student category ",
        	"Are optional fees refundable ",
        	"Optional fee not reflecting in dashboard ",
        	"Send optional fee SMS notification ",
        	"Add optional fee during admission ",
        	"Assign optional fees in bulk ",
        	"Can optional fee be tax exempted ",
        	"Do optional fees appear in fee reports "
        ],
        'configuration': [
            'How can I configure the fee settings ',
            'Where do I set up the fee structure for courses ',
            'Can I configure different fee types for students ',
            'How do I set the default fee for each student ',
            'Is it possible to configure fee payment terms ',
            'How do I enable fee reminders in the system ',
            'Can I change the payment methods in fee configuration ',
            'Where do I configure penalty charges for late payments ',
            'Can I set up discounts for early fee payments ',
            'How can I configure fee adjustments for students ',
            'Where can I set the payment due dates ',
            'Is there a way to configure tax rates for fee payments ',
            'How do I configure installment options for fee payments ',
            'Can I set up refund policies in the fee module configuration ',
            'How do I enable fee reminders via SMS or email ',
            'Can I configure fee heads for different departments ',
            'Where can I define course-specific fee structures ',
            'How do I configure fee exemptions for certain students ',
            'Can I set up fee payment schedules for students ',
            'How do I manage multiple payment modes in the configuration ',
            'Can I configure automatic fee receipt generation ',
            'How do I enable partial payments for fees ',
            'Can I configure fee collection for siblings or staff discounts ',
            'Where do I configure fee payment integration with banks ',
            'How do I manage fee records for different academic years ',
            'Can I set up fee category filters for reports ',
            'How do I configure the fee head approval process ',
            'Can I configure fee payment reminders based on due dates ',
            'Where do I set the grace period for fee payments ',
            'How do I configure the fee head hierarchy in the system ',
            'Can I allow online fee payments through the configuration ',
            'How do I enable fee waivers for students in certain categories ',
            'Where can I configure auto-generation of fee invoices ',
            'Can I set different fee rates for local and international students ',
            'How can I configure fee receipt templates ',
            'Can I set up monthly, quarterly, or yearly fee collection ',
            'How do I link fee heads to specific student groups ',
            'Can I enable fee tracking and history for students ',
            'Where do I manage overdue fee reports ',
            'Can I configure settings for fee refunds in the system ',
            'How can I set up fee reporting filters based on departments ',
            'Can I customize the fee payment notification messages ',
            'How do I enable fee reports by academic session ',
            'Can I configure the payment gateway for fee transactions ',
            'Where can I manage currency settings for fee payments ',
            'How do I configure fee statistics for reports ',
            'Can I customize the fee structure for each grade level ',
            'How do I integrate fee configuration with the student portal ',
            'Can I enable automatic fee generation for new students ',
            'Where can I set the currency for fee transactions ',
            'How can I enable recurring fee payments for students '
        ],
        'accept_fee': [
            'How can I accept fee payments ',
            'What is the process to accept fee payments from students ',
            'Can I accept fee payments through multiple methods ',
            'Where do I record fee payments made by students ',
            'Can I accept online fee payments ',
            'How do I manually accept fee payments ',
            'Is there a way to track fee payments in real-time ',
            'Can I generate a receipt for fee payments ',
            'How can I accept partial payments for fees ',
            'Can I accept fee payments through credit cards ',
            'How do I accept fee payments via bank transfer ',
            'Can I process fee payments from parents ',
            'How do I link a payment to a specific student ',
            'Can I accept fee payments via mobile payments ',
            'How do I mark a fee payment as paid ',
            'Can I accept fee payments in different currencies ',
            'Where can I accept fee payments in the student portal ',
            'How do I accept fee payments for a particular course ',
            'Can I accept fee payments through cheque ',
            'How do I update the fee payment status for a student ',
            'Can I accept fee payments for both tuition and extracurricular activities ',
            'How do I confirm that a fee payment has been received ',
            'Can I create fee payment categories for students ',
            'Where do I check the pending fee payments ',
            'How do I generate a report for accepted fee payments ',
            'Can I accept fee payments at the school office ',
            'How do I apply discounts when accepting fee payments ',
            'Can I accept fee payments from international students ',
            'How do I process fee payments in installments ',
            'Can I accept fee payments in advance for the year ',
            'How do I refund a fee payment ',
            'Can I accept fee payments for extra-curricular activities ',
            'Where can I review past fee payment records ',
            'Can I accept fee payments for multiple students at once ',
            'How do I accept a fee payment after the due date ',
            'Can I accept fee payments via mobile wallets ',
            'How do I accept emergency fee payments ',
            'Can I split fee payments between parents or guardians ',
            'How do I apply a late fee after accepting payment ',
            'Where can I configure automatic fee payment acceptance ',
            'How do I set up fee payment reminders before the due date ',
            'Can I accept payments through an automated system ',
            'How do I accept fee payments for new students ',
            'Can I check the payment status before accepting the fee ',
            'How can I confirm that a fee payment has been processed ',
            'Can I accept fee payments and track them simultaneously ',
            'How do I verify the accuracy of fee payments ',
            'Can I accept fee payments for scholarship students ',
            'Where do I add additional charges when accepting payments ',
            'How do I reconcile fee payments with the bank account ',
            'Can I accept fee payments through direct bank transfer '
        ],
        'fee_transaction_list': [
            'How can I view the fee transaction list ',
            'Where can I find a list of all fee transactions ',
            'Can I filter the fee transaction list by date ',
            'How do I search for a specific fee transaction ',
            'Can I see the fee transactions for a particular student ',
            'How can I check the status of a fee payment ',
            'Where can I view the transaction history for fee payments ',
            'Can I view the fee transaction list for this semester ',
            'How do I check fee payments made through online methods ',
            'Can I sort the fee transaction list by amount ',
            'How do I export the fee transaction list ',
            'Where can I find transactions for the past month ',
            'Can I see the fee transactions for a specific course ',
            'How can I check fee payments from parents ',
            'Can I see the transaction details for a student’s fee ',
            'Where can I view the total fee transactions for the academic year ',
            'How do I filter fee transactions by payment method ',
            'Can I track the status of pending fee transactions ',
            'Where can I view fee transactions that are pending approval ',
            'Can I get a list of cancelled fee transactions ',
            'How do I view fee transaction history for a specific student ',
            'Can I check fee transactions for a group of students ',
            'Where do I find all transactions related to fee refunds ',
            'Can I filter the fee transaction list by student group ',
            'How can I view transactions made by a specific department ',
            'Can I get a summary of all fee transactions ',
            'How do I view the transaction history for specific fee heads ',
            'Can I check the details of a specific fee payment ',
            'Where can I view fee payments made in installments ',
            'How do I see a breakdown of each fee transaction ',
            'Can I track multiple fee transactions at the same time ',
            'How do I search for transactions related to scholarships ',
            'Can I view the status of fee transactions in real-time ',
            'How do I check if a fee payment has been processed ',
            'Where can I see transactions for students who paid after the deadline ',
            'Can I filter the fee transactions list by fee head ',
            'How do I find transactions that are overpaid ',
            'Can I check transactions related to fee discounts ',
            'Where can I view the fee payment details for a particular department ',
            'How can I track all transactions related to a specific course ',
            'Can I see the transactions for students who paid in installments ',
            'How do I view the pending fee transaction list ',
            'Where can I find a list of all fee transactions for a semester ',
            'How can I check if a payment was made through the bank ',
            'Can I find details of the transactions made via mobile wallets ',
            'How do I view the total fee collection for the year ',
            'Can I search for a particular fee transaction by reference number ',
            'Where can I find the list of fee payments for the past week ',
            'How do I track fee transactions for different payment methods ',
            'Can I see a report on all fee payments made by cheque ',
            'How can I view a list of all cancelled fee transactions ',
            'Can I find the transaction details for a fee refund request '
        ],
        'fee_transaction_detail': [
            'How can I view the details of a fee transaction ',
            'Where can I find detailed information about a fee payment ',
            'Can I get the breakdown of a specific fee transaction ',
            'How do I see the details of a fee payment for a student ',
            'Where can I view the exact amount paid in a fee transaction ',
            'Can I see the payment method used for a specific fee transaction ',
            'How do I find the transaction date for a fee payment ',
            'Can I check the fee head related to a particular payment ',
            'Where can I find the receipt number for a fee transaction ',
            'Can I see the details of a fee transaction for a specific course ',
            'How do I view a fee transaction that was refunded ',
            'Can I view the fee transaction history for a student ',
            'Where can I see the fee payment reference number ',
            'Can I check the fee balance after a payment is made ',
            'How do I view the discount applied to a fee transaction ',
            'Can I check if a fee payment was partial or full ',
            'Where can I see the tax applied to a fee transaction ',
            'Can I view the fee transaction for a particular fee head ',
            'How can I check the payment status for a particular fee transaction ',
            'Where can I view the mode of payment for a fee transaction ',
            'Can I see the transaction details for a fee refund ',
            'How do I find the status of a pending fee transaction ',
            'Can I view the payment confirmation for a fee transaction ',
            'Where can I find the transaction date and time for a fee payment ',
            'How can I verify if a fee transaction was successful ',
            'Can I get the student details linked to a fee transaction ',
            'How do I see the total amount paid in a fee transaction ',
            'Can I check the payment method (e g , cheque, bank transfer) for a fee transaction ',
            'How do I get the complete details of a fee transaction in one place ',
            'Can I check the student’s outstanding balance after the fee payment ',
            'Where can I see a list of all transactions made for a particular fee head ',
            'Can I verify if a fee transaction has been processed or not ',
            'How do I view the transaction status for a fee payment ',
            'Can I get the details of a cancelled fee transaction ',
            'Where can I view the approval status for a fee transaction ',
            'Can I see the comments or notes related to a fee transaction ',
            'How can I find out if a fee transaction was linked to a scholarship ',
            'Where can I view the fee transaction details for multiple students ',
            'Can I check if there were any penalties applied during a fee transaction ',
            'How do I view a fee transaction that is marked as pending ',
            'Can I view fee transaction details for a specific semester ',
            'Where can I find the transaction amount before any deductions ',
            'Can I see the breakdown of the payment (tuition fee, extra charges) ',
            'How do I view the transactions for a student who paid late ',
            'Can I check if a student has made multiple fee payments ',
            'Where can I find the approval or rejection status for a fee transaction ',
            'Can I track the exact date when a fee payment was received ',
            'How do I view the fee payment details by reference number ',
            'Can I get the history of all fee transactions made by a student ',
            'Where can I find out if a fee transaction was processed on time ',
            'How do I confirm that a fee transaction has been successfully logged '
        ],
        'outstanding_report': [
            'How can I view the outstanding fee report ',
            'Where can I find the list of outstanding fee payments ',
            'Can I check which students have outstanding fee payments ',
            'How do I generate a report for outstanding fees ',
            'Where can I see the total amount of outstanding fees ',
            'Can I get a summary of all outstanding fee balances ',
            'How do I filter the outstanding fee report by student ',
            'Can I see the overdue fee amounts for students ',
            'How do I check the status of pending fee payments ',
            'Can I view the list of students with unpaid fees ',
            'Where can I see the outstanding fees for this semester ',
            'How can I filter the outstanding fee report by course ',
            'Can I generate a report for unpaid fees by department ',
            'How do I track unpaid fee transactions for students ',
            'Can I see a breakdown of outstanding fee balances by student ',
            'Where can I view students with the highest outstanding fees ',
            'Can I track overdue fee payments and penalties ',
            'How do I generate a report for outstanding fees with due dates ',
            'Can I get a detailed list of unpaid fees by payment method ',
            'Where can I check the overdue fees for the past month ',
            'Can I see the total outstanding balance for all students ',
            'How do I get a report on outstanding fees for specific students ',
            'Where can I view all students who haven’t paid their fees ',
            'Can I generate a report of fees due for the next semester ',
            'How do I see the outstanding fees by grade level ',
            'Can I filter outstanding fees by payment method ',
            'Where can I view the students with the most overdue fees ',
            'How do I check which students are eligible for fee discounts ',
            'Can I see a report of outstanding fees that have been partially paid ',
            'Where can I view overdue fees for a specific student group ',
            'Can I check outstanding fees for students who have applied for scholarships ',
            'How do I view the list of students with long overdue fees ',
            'Can I track outstanding fee payments by course group ',
            'How can I generate an overdue fee report for all students ',
            'Where can I find a detailed list of unpaid fees by department ',
            'Can I get the outstanding fee details for a particular class ',
            'How do I generate a report for all outstanding fees with fine details ',
            'Can I view the overdue fees for students who have left ',
            'Where can I find out the total unpaid fees for a semester ',
            'Can I get the outstanding fee report for students with fee discounts ',
            'How do I track overdue payments for fee refunds ',
            'Can I filter the outstanding fee report by payment status ',
            'Where can I check if any students have outstanding fees after a deadline ',
            'How can I see the outstanding fee amounts for each student ',
            'Can I generate a report for students who have not paid their fees for a particular term ',
            'How do I track pending fee payments after the deadline ',
            'Can I view a report showing students with pending fees and their due dates ',
            'Where can I find students who have not paid their fees in full ',
            'Can I generate a report of overdue fees for students in a specific course ',
            'How do I filter the outstanding fee report by fee head ',
            'Can I generate a list of students with outstanding fees and penalties ',
            'How do I get a report showing overdue fees and payment statuses for students ',
            'Where can I view outstanding fees with payment due dates for each student '
        ],
        'cancelled_payment': [
            'How can I view a report of cancelled fee payments ',
            'Where can I find the details of cancelled fee transactions ',
            'Can I get a list of all cancelled fee payments ',
            'How do I check the reason for a cancelled fee payment ',
            'Can I see a breakdown of each cancelled fee payment ',
            'Where can I track the status of cancelled fee payments ',
            'How do I find out why a specific fee payment was cancelled ',
            'Can I get the details of a cancelled fee payment by student ',
            'Where can I view a cancelled payment report by fee head ',
            'How do I find out if a cancelled payment has been refunded ',
            'Can I check if a cancelled fee payment is pending review ',
            'Where can I view the cancelled payment details for this semester ',
            'Can I see if any cancelled fee payments have been processed successfully ',
            'How do I find the outstanding balance after a cancelled fee payment ',
            'Can I check if a cancelled fee payment is linked to a dispute ',
            'How can I track cancelled fee payments for a specific department ',
            'Where can I see the transaction date and time for cancelled payments ',
            'Can I get the status of a cancelled fee payment (pending, processed) ',
            'How do I check if a cancelled payment is due to insufficient funds ',
            'Where can I find the payment method used for a cancelled fee transaction ',
            'Can I view the list of students who had their fee payments cancelled ',
            'How can I track the cancelled fee payments by date ',
            'Where can I find the total amount of cancelled payments for the year ',
            'Can I see the canceled payments for a particular fee head ',
            'How do I check if a student’s cancelled payment has been refunded ',
            'Can I generate a list of cancelled payments for a particular course ',
            'How do I view cancelled payments for specific payment methods ',
            'Can I find out if a cancelled payment was made via a bank transfer ',
            'Where can I see which fee payments were cancelled due to a mistake ',
            'How do I track if a cancelled fee payment has been reprocessed ',
            'Can I get a report of cancelled payments for a particular student ',
            'Where can I see if a cancelled payment was made before the deadline ',
            'Can I view cancelled fee payments that were marked as invalid ',
            'How do I view cancelled payments and their corresponding reason ',
            'Can I track if a cancelled payment has been reissued or refunded ',
            'Where can I see the list of cancelled payments due to payment gateway issues ',
            'How do I check the status of a cancelled payment that is under review ',
            'Can I see if a cancelled payment is related to a specific issue ',
            'Where can I view cancelled fee payments made by cheque ',
            'How do I find out which cancelled payments are pending approval ',
            'Can I generate a list of cancelled payments and their statuses ',
            'Where can I check if a cancelled fee payment was cancelled by mistake ',
            'How can I view cancelled payments that were flagged as fraudulent ',
            'Can I see the details of cancelled fee payments for a specific date range ',
            'How do I check if a cancelled payment was reversed or refunded ',
            'Can I get a report on cancelled payments for a specific fee head ',
            'Where can I find cancelled payment transactions related to overdue fees ',
            'Can I see the cancelled payments by student group or batch ',
            'How do I check if the cancelled payment is linked to a refund request ',
            'Where can I view a list of cancelled payments for students who were charged incorrectly ',
            'Can I get the reason for a cancelled payment in the report ',
            'How do I track cancelled payments for a specific academic year ',
            'Can I see if a cancelled payment resulted in a fee waiver or discount ',
            'Where can I find the cancellation date and time for a fee payment '
        ],
        'collection_summary': [
            'Can I get a summary of the fee collection for this month ',
            'Where can I find the total fee collection for the semester ',
            'How do I generate a report for the total fee collected this year ',
            'Can I view the collection summary for all fee heads ',
            'Where can I check the collection summary by class or grade ',
            'How do I track the fee collection per department ',
            'Can I see a breakdown of the fee collected by student category ',
            'Where can I find a detailed collection summary by fee type ',
            'How do I view the total fee collected for each course ',
            'Can I generate a report showing the fee collection summary by date ',
            'Where can I view the total amount collected for each student ',
            'How do I check the percentage of fee collected compared to the target ',
            'Can I get a summary of fee collection for a specific academic session ',
            'Where can I see if there are any discrepancies in the fee collection ',
            'How do I track the fee collection by payment method ',
            'Can I generate a fee collection summary for a specific student group ',
            'Where can I find the collection summary for students who paid late ',
            'How do I track the fee collection progress for the entire school ',
            'Can I check the total fee collection for each payment cycle ',
            'Where can I view a detailed breakdown of the fee collected per batch ',
            'How do I get a summary of fee collection for a specific date range ',
            'Can I see the fee collection summary for students with discounts ',
            'Where can I track the status of fee collection for outstanding payments ',
            'How do I check the fee collection status for new students ',
            'Can I view the total fee collected for each fee head (tuition, transportation, etc ) ',
            'Where can I find the summary of fee collection by region ',
            'How do I track the payment status for outstanding fee amounts ',
            'Can I get a report on the fee collection summary by gender ',
            'Where can I view a summary of fee collected from international students ',
            'How do I check if the fee collection for the month is on track ',
            'Can I generate a report showing the difference between expected and actual fee collection ',
            'Where can I find the collection summary for cancelled payments ',
            'How do I view the total amount collected for miscellaneous fees ',
            'Can I see the fee collection summary by payment gateway ',
            'Where can I track the total fee collected for each semester ',
            'How do I get a report showing fee collection by individual fee heads ',
            'Can I track the fee collection progress for each department ',
            'Where can I check if any students have paid their fees partially ',
            'How do I track the total fee collection for a specific fee structure ',
            'Can I see a comparison of fee collection between two periods ',
            'Where can I view the total fee collection for students with exemptions ',
            'How do I find out if the fee collection target has been met ',
            'Can I generate a report that shows fee collection status for each student ',
            'Where can I view a summary of fee collection for students on installments ',
            'How do I track the fee collection for students who have received scholarships ',
            'Can I see the percentage of students who have paid their fees so far ',
            'Where can I find the summary of the fee collected via online payment methods ',
            'How do I get the fee collection summary for a particular faculty ',
            'Can I check the collection status for unpaid or overdue fees ',
            'Where can I find the detailed summary of the total fee collected in the past month '
        ],
        'fee_head_report': [
            'Can I get a report on the fee collection for each fee head ',
            'Where can I view the fee collection summary for each fee head ',
            'How do I generate a report showing the fee collected for tuition ',
            'Can I track the fee collected for transportation in a separate report ',
            'Where can I see the collection report for miscellaneous fees ',
            'How do I find out the total fee collected for library services ',
            'Can I get a fee head report for each course offered ',
            'Where can I find the fee collection summary for hostel charges ',
            'How do I track the fee collected for sports facilities ',
            'Can I generate a report that shows the fee collection for extra-curricular activities ',
            'Where can I view the fee report for students who opted for optional subjects ',
            'How do I check the fee collected for examination charges ',
            'Can I see a detailed report of fee collected for the computer lab ',
            'Where can I view the fee collection for the medical center ',
            'How do I check the collection of fees for online resources ',
            'Can I generate a fee head report for uniform and books ',
            'Where can I find a report for collected fees for the annual function ',
            'How do I check the fee collection for the student council ',
            'Can I track the fee collection for activities and clubs ',
            'Where can I find the fee collection for special programs ',
            'How do I generate a report on the total fee collected for all students ',
            'Can I get a breakdown of the fee collected for each category ',
            'Where can I see the fee collection for optional programs ',
            'How do I track the fee head report for library and reading material ',
            'Can I get a report on fee collection for students with late payment ',
            'Where can I find the collection for specific student categories like scholarships ',
            'How do I track the fee collected for workshops and training programs ',
            'Can I generate a fee report based on the different fee structures ',
            'Where can I view the total fee collection for academic year-wise breakdown ',
            'How do I track the fee collection for each batch of students ',
            'Can I see a summary of the fee collected for early bird discount students ',
            'Where can I check the fee collection report for students who opted out of certain charges ',
            'How do I view a detailed report on fee collection for late payers ',
            'Can I get a fee head report based on department ',
            'Where can I find the fee collection report for students who paid partially ',
            'How do I check the fee collected for courses with specific discount rates ',
            'Can I see the breakdown of collected fees by student categories like residential and non-residential ',
            'Where can I view the fee collection summary for online students ',
            'How do I generate a report for fee collection by payment method ',
            'Can I track the fee collected for students who received financial aid ',
            'Where can I check the fee collection for students with installment plans ',
            'How do I find out the fee collected for compulsory and optional items separately ',
            'Can I get a fee head report for each term or semester ',
            'Where can I track the fee collection for students from different regions ',
            'How do I view the report for the fee collected from international students ',
            'Can I generate a report to compare fee collection between two academic years ',
            'Where can I find the fee collection data for various fee categories over time ',
            'How do I get the fee head report for students who applied for fee concessions ',
            'Can I see the fee collection for each grade or year ',
            'Where can I view a report on the collected fees for non-tuition related activities ',
            'How do I track the fee collected for students with specific exemptions ',
            'Can I see a breakdown of the fee collected for specific academic programs ',
            'Where can I view the fee collection summary for the entire institution ',
            'How do I generate a fee head report by student type (regular, distance learning, etc ) '
        ],
        'student_discount_report': [
            'Can I get a report on the discounts applied to students ',
            'Where can I view the details of student discount transactions ',
            'How do I generate a report showing the discounts applied to students ',
            'Can I track the total discounts given to students over the term ',
            'Where can I see a summary of the discounts applied for each student ',
            'How do I find out the discount percentage granted to students ',
            'Can I generate a discount report for students receiving financial aid ',
            'Where can I view the discount report for students with merit-based discounts ',
            'How do I check the total amount of discount granted to students ',
            'Can I get a breakdown of discounts given to students by category ',
            'Where can I see the discounts given to students based on their academic performance ',
            'How do I track discounts applied to students based on their socioeconomic status ',
            'Can I view the student discount report for different scholarship categories ',
            'Where can I find a detailed report of discounts for students in different programs ',
            'How do I track discounts given to students for early payments ',
            'Can I generate a discount report for students receiving special grants ',
            'Where can I view the discount data for students based on their enrollment year ',
            'How do I check if a student received a discount for extracurricular activities ',
            'Can I track the discounts granted to students for voluntary donations ',
            'Where can I find a report for discounts applied to students in the last month ',
            'How do I get a summary of discounts granted to international students ',
            'Can I see the discount report for students who were eligible for fee waivers ',
            'Where can I find the report on the fee discounts for students with disabilities ',
            'How do I generate a discount report for students who opted for specific services ',
            'Can I track the discounts applied to students for group registrations ',
            'Where can I see the discount history for a specific student ',
            'How do I check if a student received a sibling discount ',
            'Can I get a report on discounts given to students for attending extra workshops ',
            'Where can I track discounts applied based on student loyalty or history ',
            'How do I check the discounts granted for students who paid full fees upfront ',
            'Can I generate a report for discounts offered to students from a specific region ',
            'Where can I see the discounts given to students for academic excellence ',
            'How do I track the discounts offered to students with special circumstances ',
            'Can I get a breakdown of discounts given by faculty or department ',
            'Where can I find a report on discounts given to students during a particular event ',
            'How do I generate a report to compare discounts granted for different courses ',
            'Can I track the discounts provided to students who participated in specific programs ',
            'Where can I view a detailed report of the discount policies applied for each student ',
            'How do I get the student discount report for students who were exempted from certain fees ',
            'Can I generate a report of the total discount amount given for the year ',
            'Where can I track the discount history for students with installment payment plans ',
            'How do I check for students who received discounts for joining before a certain date ',
            'Can I track discounts applied for students based on their family income ',
            'Where can I find the breakdown of discounts based on course type ',
            'How do I track the discounts given to students for attending late registration ',
            'Can I get a report on the total discount percentage applied across all students ',
            'Where can I find the student discount details for a specific period ',
            'How do I check the report for discounts given during the last academic year ',
            'Can I track the discount report for students in a particular department ',
            'Where can I see the discounts for students who received fee reductions due to special policies ',
            'How do I generate a report on the discounts applied to students with hardship cases ',
            'Can I find a summary of discounts given to students for participating in special school activities '
        ],
        'fee_import': [
            'How do I import fee data into the system ',
            'Can I upload fee details for multiple students at once ',
            'Where can I find the option to import fee records ',
            'How do I import fee payment details for students ',
            'Can I upload fee transaction data from an Excel file ',
            'What is the format required to import fee data ',
            'How do I import fee structures for all students ',
            'Can I import previous fee records for all students ',
            'Is there an import feature for bulk fee entries ',
            'Where can I upload fee payments made by students ',
            'Can I import fee details for students from a CSV file ',
            'How do I import the fee details for students from last semester ',
            'What steps are needed to import fee payment data ',
            'Can I use a template to import fee transactions ',
            'Where can I find the file format for importing fee records ',
            'How do I import discounts applied to student fees ',
            'Can I import fee receipts for a specific period ',
            'How do I ensure that fee data is correctly imported ',
            'What data fields are required when importing fee details ',
            'Can I import fee details from a third-party system ',
            'How do I import the fee payment status for students ',
            'Is there a way to import fee data without manual entry ',
            'Where can I find the import option for fee records ',
            'How do I import fee receipts for students with scholarships ',
            'Can I update fee data through the import feature ',
            'What is the process for importing fee details in bulk ',
            'Can I import student fee details based on their enrollment date ',
            'How do I handle errors when importing fee data ',
            'Can I import fee payments made through online transactions ',
            'How can I import student fee records for a specific academic year ',
            'Can I import fee data for multiple courses at once ',
            'How do I import fee installments for students ',
            'What should I do if the imported fee data doesn’t match the records ',
            'Can I import student discounts along with fee data ',
            'How do I import the payment status for multiple students ',
            'Can I import data for students who made partial fee payments ',
            'Is there a way to check imported fee records for errors ',
            'How do I confirm that the imported fee data is correct ',
            'Can I import fee refund details for students ',
            'What do I do if I face issues while importing fee data ',
            'Can I schedule fee data imports automatically ',
            'How do I import fee structures for different programs ',
            'Can I import fee data from a legacy system ',
            'What formats can I use to import fee information ',
            'How do I manage fee imports if the data is from different years ',
            'Can I import fee payment receipts along with fee transactions ',
            'Where can I find the guidelines for importing fee data ',
            'Can I import fee details for students who were late with payments ',
            'How do I import and update student fee information regularly ',
            'What tools can I use to automate the import of fee data ',
            'How do I check the status of an imported fee file ',
            'Is it possible to import fee data for international students '
        ]
    },

        'general_accounting': {
        'ledger': [
            'What is the ledger in accounting ',
            'How do I view the ledger in the system ',
            'Where can I find the general ledger report ',
            'How can I add a new entry to the ledger ',
            'How do I filter transactions in the ledger ',
            'Can I generate a ledger report for a specific date range ',
            'How can I view the ledger for a particular account ',
            'What are the different types of ledgers ',
            'How do I close a ledger for the month ',
            'Can I export the ledger to Excel ',
            'How do I view a specific ledger account ',
            'Is it possible to create a custom ledger format ',
            'Can I get a ledger summary for all accounts ',
            'How do I adjust a ledger entry ',
            'What is the purpose of a ledger in accounting ',
            'How do I check the balance in my ledger ',
            'Can I reconcile accounts using the ledger ',
            'How do I view journal entries in the ledger ',
            'Can I track debits and credits in the ledger ',
            'Is there a way to automate ledger creation ',
            'Can I create a ledger entry for a specific transaction ',
            'How do I view the detailed ledger entries for a year ',
            'Can I search for specific transactions in the ledger ',
            'How can I categorize entries in the ledger ',
            'Is there an audit trail available for the ledger ',
            'How do I print the ledger report ',
            'Can I track the status of ledger transactions ',
            'How do I correct an incorrect ledger entry ',
            'What are the types of entries in the general ledger ',
            'Can I set up automatic journal entries in the ledger ',
            'Can I integrate the ledger with other accounting systems ',
            'How do I check for discrepancies in the ledger ',
            'Can I apply filters to the ledger report ',
            'What is the chart of accounts in the ledger ',
            'How do I assign accounts to different ledger entries ',
            'Can I track all transactions made in the ledger ',
            'How do I view the ledger for specific transactions ',
            'How can I delete an incorrect ledger entry ',
            'How do I add tax-related entries to the ledger ',
            'Can I generate ledger reports for different fiscal periods ',
            'What happens when I close a ledger account ',
            'Can I view the income statement from the ledger ',
            'How do I check the audit trail for ledger entries ',
            'Can I create recurring ledger entries ',
            'How do I analyze the ledger for accounting errors ',
            'Can I view the ledger in different currencies ',
            'How do I reconcile transactions in the ledger '
        ],
        'voucher_entry': [
            'What is a voucher entry in accounting ',
            'How do I create a voucher entry ',
            'Where can I find the voucher entry option ',
            'How do I record a payment using voucher entry ',
            'Can I use voucher entries for both debit and credit ',
            'How do I view all voucher entries made ',
            'What are the different types of voucher entries ',
            'How do I modify an existing voucher entry ',
            'Can I generate a voucher entry report ',
            'How do I enter a journal voucher ',
            'What are the fields required for voucher entry ',
            'How do I add a reference number to a voucher entry ',
            'Can I apply tax in a voucher entry ',
            'How do I delete an incorrect voucher entry ',
            'Can I track the status of a voucher entry ',
            'How do I categorize voucher entries ',
            'Can I import voucher entries into the system ',
            'How do I verify the accuracy of a voucher entry ',
            'Can I record vouchers for multiple transactions at once ',
            'What is the difference between a voucher entry and a journal entry ',
            'Can I create a recurring voucher entry ',
            'How do I view the audit trail for voucher entries ',
            'How do I add a memo to a voucher entry ',
            'Can I use voucher entries for cash and bank transactions ',
            'How do I attach documents to a voucher entry ',
            'Can I view pending voucher entries ',
            'How do I generate a voucher entry statement ',
            'How do I check the date for a voucher entry ',
            'Can I track voucher entries for a specific account ',
            'What is the purpose of a voucher entry in accounting ',
            'How do I apply discounts in a voucher entry ',
            'Can I link a voucher entry to a specific invoice ',
            'How do I check for errors in a voucher entry ',
            'How do I view a voucher entry’s history ',
            'Can I print a voucher entry report ',
            'How do I manage voucher entries for different departments ',
            'Can I integrate voucher entries with other systems ',
            'What should I do if I make a mistake in a voucher entry ',
            'Can I record payments for multiple accounts in one voucher ',
            'How do I check for pending payments in voucher entries ',
            'What are the accounting standards for voucher entries ',
            'Can I categorize voucher entries by type ',
            'How do I track the approval status of a voucher entry ',
            'Can I use vouchers for both payments and receipts ',
            'How do I confirm the accuracy of a voucher entry ',
            'Can I reconcile voucher entries with bank statements ',
            'How do I view the overall voucher entry summary ',
            'What is the purpose of a voucher entry report '
        ],
        'contra_entry': [
            'What is a contra entry in accounting ',
            'How do I create a contra entry ',
            'Where can I enter a contra entry in the system ',
            'Can I use contra entries for cash and bank transactions ',
            'How do I apply contra entries in a transaction ',
            'What are the different types of contra entries ',
            'Can I edit an existing contra entry ',
            'How do I delete a contra entry ',
            'How do I reconcile contra entries ',
            'Can I generate a contra entry report ',
            'How do I view all contra entries ',
            'What is the purpose of a contra entry in accounting ',
            'Can I record multiple contra entries at once ',
            'How do I check for errors in contra entries ',
            'How do I apply contra entries to bank accounts ',
            'Can I attach documents to a contra entry ',
            'Can I integrate contra entries with other systems ',
            'What is the role of contra entries in balance sheet ',
            'How do I check the status of contra entries ',
            'Can I print a contra entry report ',
            'How do I track contra entries for specific accounts ',
            'How do I categorize contra entries in the system ',
            'Can I create recurring contra entries ',
            'How do I view the history of contra entries ',
            'How do I track payments and receipts with contra entries ',
            'How do I verify a contra entry before posting it ',
            'Can I use contra entries for both payments and receipts ',
            'How do I delete incorrect contra entries ',
            'What happens when I reconcile contra entries ',
            'Can I use contra entries for inter-account transfers ',
            'Can I record contra entries for specific departments ',
            'How do I check the audit trail for contra entries ',
            'Can I use contra entries for adjusting balances ',
            'How do I manage contra entries in the system ',
            'How do I ensure the accuracy of contra entries ',
            'Can I apply contra entries for foreign exchange transactions ',
            'What should I do if a contra entry is incorrect ',
            'Can I add contra entries to financial reports ',
            'How do I track the approval status of a contra entry ',
            'How do I handle contra entries in the trial balance ',
            'How do I reconcile contra entries with bank statements ',
            'Can I use contra entries to offset transactions ',
            'How do I create an automatic contra entry ',
            'What is the general ledger’s role in contra entries ',
            'How do I check pending contra entries ',
            'How do I track pending contra entry approvals '
        ],
        'day_book_report': [
            'What is a day book report in accounting ',
            'How do I generate a day book report ',
            'Can I filter day book entries by date range ',
            'How do I view all day book transactions ',
            'How do I add new entries to the day book ',
            'Can I export the day book report to Excel ',
            'What is included in the day book report ',
            'How do I check pending transactions in the day book ',
            'Can I view day book reports for specific accounts ',
            'How do I print a day book report ',
            'Can I view day book entries for a specific period ',
            'What are the components of a day book ',
            'How do I categorize day book entries ',
            'Can I generate the day book report by account type ',
            'How do I filter the day book by transaction type ',
            'Can I reconcile day book entries with the bank statement ',
            'How do I track day book transactions for specific departments ',
            'How do I check for errors in the day book ',
            'How do I view the total balance from the day book ',
            'How do I modify an existing day book entry ',
            'Can I track the status of day book entries ',
            'What is the purpose of the day book report ',
            'How do I view summary reports from the day book ',
            'Can I create day book entries for different currencies ',
            'How do I categorize entries in the day book ',
            'What do I do if there is an error in the day book ',
            'Can I use the day book for both payments and receipts ',
            'How do I reconcile transactions in the day book ',
            'How can I get a detailed report for day book entries ',
            'How do I check the history of day book transactions ',
            'Can I adjust day book entries for previous months ',
            'How do I add notes to day book transactions ',
            'Can I filter day book entries by specific accounts ',
            'How do I track unpaid transactions in the day book ',
            'Can I reconcile the day book with the cash book ',
            'How do I ensure the accuracy of the day book ',
            'Can I create recurring day book entries ',
            'What are the steps to reconcile the day book with bank entries ',
            'How do I track the approval of day book entries ',
            'Can I integrate day book entries with other systems ',
            'What steps are needed to print a day book report ',
            'How do I categorize entries in the day book ',
            'Can I view day book transactions for a specific account ',
            'What is the process to create day book reports automatically '
        ],
        'balance_sheet_ledger': [
            'How do I generate a balance sheet ledger ',
            'Can I view the balance sheet ledger for a specific period ',
            'How do I add entries to the balance sheet ledger ',
            'What does the balance sheet ledger show ',
            'How do I reconcile the balance sheet ledger ',
            'Can I export the balance sheet ledger to Excel ',
            'What are the main components of a balance sheet ledger ',
            'How do I track assets in the balance sheet ledger ',
            'How do I track liabilities in the balance sheet ledger ',
            'How do I track equity in the balance sheet ledger ',
            'Can I categorize balance sheet entries ',
            'What is the purpose of a balance sheet ledger ',
            'How do I check discrepancies in the balance sheet ledger ',
            'Can I print the balance sheet ledger report ',
            'How do I filter balance sheet entries by account type ',
            'Can I reconcile the balance sheet with the general ledger ',
            'How do I update entries in the balance sheet ledger ',
            'What is the format for the balance sheet ledger ',
            'Can I track balance sheet entries for multiple companies ',
            'How do I calculate the final balance from the balance sheet ',
            'Can I generate the balance sheet ledger for different fiscal periods ',
            'How do I categorize assets and liabilities in the balance sheet ',
            'Can I view the balance sheet ledger for multiple branches ',
            'How do I verify the accuracy of the balance sheet ledger ',
            'Can I adjust previous balance sheet entries ',
            'How do I integrate balance sheet ledger data with other reports ',
            'How do I track balance sheet entries with external systems ',
            'What steps are needed to reconcile the balance sheet with bank records ',
            'Can I generate automated balance sheet reports ',
            'How do I add new balance sheet categories ',
            'How do I create a balance sheet ledger summary ',
            'Can I track changes in the balance sheet over time ',
            'How do I reconcile balance sheet entries with income statements ',
            'How do I verify the accuracy of the balance sheet ledger ',
            'Can I create reports from balance sheet entries automatically ',
            'How do I track assets and liabilities with the balance sheet ledger ',
            'Can I filter balance sheet ledger data by transaction type ',
            'Can I add notes or comments to balance sheet entries ',
            'What is the importance of the balance sheet ledger ',
            'How do I ensure the correctness of balance sheet entries ',
            'Can I import balance sheet entries from other systems ',
            'How do I handle discrepancies in the balance sheet ledger ',
            'What are the steps for closing the balance sheet ledger ',
            'Can I categorize entries in the balance sheet ledger by asset types '
        ],
        'receipts_and_payments': [
            'What is a receipt and payment report ',
            'How do I generate a receipts and payments report ',
            'How can I view receipts and payments for a specific date range ',
            'Can I filter receipts and payments by account ',
            'How do I categorize receipts and payments ',
            'Can I export receipts and payments to Excel ',
            'How do I track payments received ',
            'How do I view outstanding receipts and payments ',
            'Can I check the status of specific receipts and payments ',
            'How do I reconcile receipts and payments with bank statements ',
            'Can I create a detailed receipts and payments report ',
            'How do I track receipts and payments by department ',
            'Can I generate a report for both receipts and payments together ',
            'How do I handle refunds in receipts and payments ',
            'Can I automate receipts and payments entries ',
            'What should I do if there’s an error in a receipts or payments entry ',
            'Can I view the receipts and payments summary for the month ',
            'How do I reconcile discrepancies in receipts and payments ',
            'Can I print the receipts and payments report ',
            'How do I ensure accuracy in receipts and payments ',
            'Can I view receipts and payments for multiple accounts ',
            'What information is included in the receipts and payments report ',
            'Can I create recurring receipts or payments ',
            'How do I apply tax to receipts and payments ',
            'Can I filter receipts and payments by payment method ',
            'How do I add new receipts or payments ',
            'Can I track the history of receipts and payments ',
            'Can I link receipts and payments to invoices ',
            'How do I view receipts and payments for specific customers ',
            'How do I categorize payments as operational or non-operational ',
            'How do I track overdue receipts ',
            'Can I integrate receipts and payments data with other accounting systems ',
            'How do I manage refunds in receipts and payments ',
            'Can I categorize payments as operational expenses ',
            'What’s the purpose of receipts and payments reports ',
            'How do I track receipts and payments in different currencies ',
            'Can I automatically calculate outstanding balances in receipts and payments ',
            'Can I analyze receipts and payments trends ',
            'How do I handle partial payments in receipts and payments ',
            'Can I create a summary for receipts and payments ',
            'How do I manage receipts and payments for different fiscal periods ',
            'How do I ensure the completeness of receipts and payments entries ',
            'Can I integrate receipts and payments with bank reconciliation ',
            'How do I verify the accuracy of receipts and payments data ',
            'Can I manage receipts and payments based on payment due dates ',
            'Can I filter receipts and payments by customer or vendor ',
            'What steps do I follow to reconcile receipts and payments ',
            'How do I add notes to receipts and payments entries '
        ],
        'profit_and_loss': [
            'What is a profit and loss statement ',
            'How do I generate a profit and loss report ',
            'Can I view the profit and loss statement for a specific period ',
            'How do I track revenue and expenses in the profit and loss statement ',
            'How do I categorize income and expenses in the profit and loss statement ',
            'Can I export the profit and loss statement to Excel ',
            'How do I reconcile the profit and loss statement with the balance sheet ',
            'Can I generate a summary for the profit and loss report ',
            'How do I analyze profit and loss for different departments ',
            'How do I track gross profit and net profit ',
            'What information is included in the profit and loss statement ',
            'How do I view the profit and loss for multiple periods ',
            'Can I add custom categories to the profit and loss statement ',
            'Can I check the detailed breakdown of profits and losses ',
            'How do I categorize operational and non-operational income ',
            'Can I filter profit and loss data by account type ',
            'Can I track profit and loss by project or department ',
            'How do I handle unusual items in the profit and loss statement ',
            'Can I view profits and losses for different branches ',
            'How do I ensure accuracy in the profit and loss statement ',
            'How do I adjust profit and loss entries ',
            'Can I print the profit and loss statement ',
            'How do I track profit and loss for specific transactions ',
            'Can I create recurring profit and loss reports ',
            'Can I add notes to the profit and loss report ',
            'How do I reconcile profit and loss with receipts and payments ',
            'Can I compare profit and loss data across different fiscal periods ',
            'How do I categorize revenue and expense streams in the profit and loss statement ',
            'Can I create a forecast for the profit and loss statement ',
            'What’s the purpose of the profit and loss statement ',
            'How do I track cost of goods sold in the profit and loss statement ',
            'Can I apply tax adjustments in the profit and loss report ',
            'How do I ensure consistency in profit and loss reporting ',
            'Can I generate a profit and loss summary for the year ',
            'How do I view profit and loss data for different currencies ',
            'Can I integrate profit and loss with other financial statements ',
            'How do I manage and adjust profit and loss figures ',
            'Can I compare actual vs projected profit and loss ',
            'How do I calculate operating profit ',
            'Can I track expenses and income by category in the profit and loss statement ',
            'Can I reconcile discrepancies in the profit and loss statement ',
            'How do I track net income in the profit and loss statement ',
            'How do I use profit and loss reports for financial analysis ',
            'Can I generate monthly profit and loss reports automatically ',
            'Can I analyze the profitability of my business using profit and loss data ',
            'How do I determine financial health using the profit and loss statement '
        ],
        'income_expense_analysis': [
            'What is income and expense analysis ',
            'How do I perform an income and expense analysis ',
            'How can I track income and expenses over time ',
            'Can I create an income and expense report ',
            'What are the key components of income and expense analysis ',
            'How do I categorize income and expenses in the analysis ',
            'How do I track trends in income and expenses ',
            'Can I filter income and expense data by category ',
            'Can I generate a visual report for income and expense analysis ',
            'How do I analyze income and expenses for different departments ',
            'Can I view a detailed breakdown of income and expenses ',
            'Can I export the income and expense analysis to Excel ',
            'How do I analyze income and expenses by account type ',
            'What’s the purpose of performing an income and expense analysis ',
            'How do I track income and expenses by department ',
            'Can I perform income and expense analysis for multiple years ',
            'How do I reconcile income and expense data with financial statements ',
            'How do I track net income in the analysis ',
            'How do I ensure accuracy in income and expense analysis ',
            'Can I apply filters to the income and expense analysis ',
            'How do I handle one-time expenses in the income and expense analysis ',
            'Can I view income and expense data for different fiscal periods ',
            'How do I track income and expenses by source ',
            'Can I create recurring income and expense reports ',
            'How do I identify cost-saving opportunities in the analysis ',
            'Can I compare projected vs actual income and expenses ',
            'Can I track income and expenses for different projects ',
            'How do I integrate income and expense analysis with other financial data ',
            'Can I track expenses by type (fixed vs variable) ',
            'How do I assess the profitability of a business using income and expense analysis ',
            'Can I analyze both operational and non-operational income and expenses ',
            'How do I add adjustments to the income and expense analysis ',
            'Can I categorize income as operational or non-operational ',
            'How do I view income and expense analysis for different currencies ',
            'Can I generate a summary report for income and expenses ',
            'How do I manage and adjust income and expense entries ',
            'How do I track changes in income and expense over time ',
            'Can I categorize income and expenses by account ',
            'How do I view income and expenses by category ',
            'What are the steps for performing a complete income and expense analysis ',
            'Can I track income and expense performance for different regions ',
            'How do I create a detailed breakdown of income vs expenses ',
            'Can I analyze cash flow from the income and expense analysis ',
            'How do I manage income and expense records over time ',
            'Can I automate income and expense reports ',
            'How do I identify areas for improvement in income and expense management '
        ]
    },

        'academic': {
        'time_table': [
            'What is my timetable for today ',
            'Can I view my class schedule ',
            'How can I check my upcoming classes ',
            'Can I get the timetable for next week ',
            'What subjects do I have today ',
            'How do I view the timetable for a specific subject ',
            'Can I print my timetable ',
            'How do I change my class schedule ',
            'Can I see the timetable for a different teacher ',
            'How do I view my timetable for a particular semester ',
            'Can I export my timetable to Excel ',
            'How do I view the timetable for a specific day ',
            'Can I get the timetable for the entire term ',
            'How can I track my class schedule for the week ',
            'What time does my class start today ',
            'What classes do I have after lunch ',
            'Can I get reminders for my next class ',
            'How do I get notified of class changes ',
            'Can I view the timetable for all my subjects ',
            'How do I see class timings for a specific subject ',
            'Can I check my timetable on mobile ',
            'How do I update my timetable if a class is rescheduled ',
            'Can I see the timetable for other departments ',
            'How do I track the end time for my classes ',
            'What’s the best way to keep track of my timetable ',
            'Can I integrate my timetable with my calendar ',
            'How can I share my timetable with others ',
            'How do I see holidays in my timetable ',
            'Can I set class reminders in my timetable ',
            'How do I make changes to my timetable ',
            'Can I check the timetable of a specific teacher ',
            'How do I add a new class to my timetable ',
            'How do I manage overlapping classes ',
            'Can I view my class timings for the next few weeks ',
            'Can I filter the timetable by class type ',
            'How do I add or remove a subject from my timetable ',
            'How can I get the timetable for an exam period ',
            'Can I see the entire year’s timetable ',
            'How do I find the timetable for a particular course ',
            'Can I view my timetable by room number ',
            'How do I check the timetable of my classmates ',
            'How do I track missed classes on the timetable ',
            'Can I view the timetable for a specific class group ',
            'Can I adjust my timetable based on personal preferences ',
            'How do I see class schedules for elective courses ',
            'How do I check if a class is canceled ',
            'How do I set up automatic timetable reminders '
        ],
        'room_master': [
    		'What is room master ',
    		'Add a new classroom in room master',
    		'Edit room master details',
    		'Delete a room from room master',
    		'List all rooms in the master',
    		'Configure room capacities',
    		'Assign room types in master',
    		'How to manage room master ',
    		'Show me available rooms',
    		'How to update a classroom ',
    		'Classroom master entry',
    		'Room setup guide',
    		'Create room master record',
    		'Room data input form',
    		'Manage room information',
    		'Bulk upload room list',
    		'Import room master from Excel',
    		'Add lab rooms in master',
    		'Set floor-wise room list',
    		'Can I assign room numbers ',
    		'What’s in room master ',
    		'Define exam halls in room master',
    		'Add special rooms to master',
    		'Can I disable a room ',
    		'Configure room availability',
    		'Mark room as active or inactive',
    		'Add room with capacity of 50',
    		'List rooms on 1st floor',
    		'Room list format for master',
    		'Download room master list',
    		'Edit existing room entry',
    		'Change room code',
    		'Link rooms to time table',
    		'Room master validation',
    		'Are duplicate rooms allowed ',
    		'What fields are in room master ',
    		'Set room preferences',
    		'Remove old classroom',
    		'Configure smart class rooms',
    		'How to sort rooms ',
    		'Room label setup',
    		'Create temporary room',
    		'Room status report',
    		'Classroom occupancy master',
    		'Room allocation for timetable',
    		'Master list of all rooms',
    		'View or edit room info',
    		'Room registration page',
    		'New room entry form'
    	],
    	'time_slot_master': [
    		'What is time slot master ',
    		'Create a time slot',
    		'Edit existing time slot',
    		'Remove a time slot',
    		'View time slot details',
    		'Manage time slots',
    		'Define time intervals for classes',
    		'Add morning time slots',
    		'Setup afternoon time slots',
    		'What time slot format is used ',
    		'Set 45-minute periods',
    		'Configure break times',
    		'Adjust class period length',
    		'Add lab time slot',
    		'Change time slot duration',
    		'Time slot master guide',
    		'Link slots to timetable',
    		'Assign slots to weekdays',
    		'Can I overlap time slots ',
    		'Enable Saturday slots',
    		'Slot entry form',
    		'Master record for periods',
    		'Bulk upload slot timings',
    		'View complete slot list',
    		'Time slot validation rules',
    		'Morning session slots',
    		'Evening period settings',
    		'Time slot for assembly',
    		'What’s included in slot master ',
    		'Add new slot from 9 to 10',
    		'Class period master settings',
    		'Weekly slot setup',
    		'Set special slots for events',
    		'Edit or delete time periods',
    		'How to block time slots ',
    		'Assign class duration',
    		'Create flexible time slots',
    		'Custom period timing',
    		'Break slot entry',
    		'Lunch slot master',
    		'Holiday slot exclusion',
    		'Slot priority order',
    		'Period sequencing in slot master',
    		'How many slots allowed ',
    		'Time slot import template',
    		'Assign teacher to time slot',
    		'Generate time slot report',
    		'Slot master backup file',
    		'Default time slot settings'
    	],
    	'time_table_master': [
    		'What is time table master ',
    		'Create a new time table',
    		'Assign teachers to time slots',
    		'Generate timetable automatically',
    		'Edit existing timetable',
    		'Delete a time table entry',
    		'View timetable by class',
    		'Time table setup guide',
    		'Print class wise time table',
    		'Upload timetable data',
    		'Assign room to periods',
    		'Time table master for teachers',
    		'Period-wise subject entry',
    		'Create timetable for week',
    		'Define subject periods',
    		'Download timetable in PDF',
    		'Configure class periods',
    		'How to copy last year’s timetable ',
    		'Batch-wise timetable setup',
    		'Timetable entry form',
    		'How to lock the timetable ',
    		'Export timetable for mobile app',
    		'Add break periods',
    		'Customize timetable layout',
    		'Assign labs in timetable',
    		'Validate overlapping slots',
    		'View teacher availability',
    		'Check timetable conflicts',
    		'View student timetable',
    		'Teacher wise period distribution',
    		'Replace a teacher in timetable',
    		'Time table dashboard',
    		'Generate section wise timetable',
    		'Preview class time table',
    		'Set max periods per day',
    		'Weekday selection for timetable',
    		'How to add games period ',
    		'Manual vs auto generation',
    		'Merge sections in timetable',
    		'Subject-wise period limit',
    		'What is bell schedule ',
    		'Time table alerts',
    		'Email time table to parents',
    		'Assign fixed rooms',
    		'Time table template settings',
    		'How to publish timetable ',
    		'Copy timetable to next semester',
    		'Can I block slots ',
    		'Get all teacher timetables'
    	],
        'my_syllabus': [
            'How can I view my syllabus ',
            'Can I see the syllabus for my current semester ',
            'How do I get the syllabus for a specific subject ',
            'Where can I find the syllabus for my courses ',
            'Can I download my syllabus ',
            'How do I check for syllabus updates ',
            'Can I print my syllabus ',
            'How do I view the syllabus for upcoming subjects ',
            'Can I access my syllabus from my phone ',
            'How do I know if the syllabus has changed ',
            'Can I share my syllabus with others ',
            'How do I add new subjects to my syllabus ',
            'Can I search for a particular topic in my syllabus ',
            'How do I filter my syllabus by subject ',
            'Can I track which parts of the syllabus I have covered ',
            'How do I update my syllabus ',
            'Can I integrate my syllabus with my study planner ',
            'Can I view the syllabus for the next semester ',
            'How do I get the syllabus for elective courses ',
            'Can I see the syllabus for all subjects in one view ',
            'How do I check if my syllabus is available online ',
            'Can I view the syllabus for each unit separately ',
            'How do I see the topics for each class in my syllabus ',
            'Can I filter my syllabus by lecture or practical sessions ',
            'How can I track the progress of my syllabus ',
            'Can I print a specific unit of my syllabus ',
            'Can I request a syllabus update from the teacher ',
            'How do I set syllabus reminders for upcoming exams ',
            'How do I view the syllabus for a different subject ',
            'How do I access a syllabus for online courses ',
            'Can I track important dates from my syllabus ',
            'How do I prepare for exams based on my syllabus ',
            'How do I check the resources mentioned in my syllabus ',
            'Can I see the syllabus in different formats ',
            'How do I add extra materials to my syllabus ',
            'Can I set up automatic syllabus notifications ',
            'Can I access old syllabi from previous years ',
            'How do I view the syllabus for special courses ',
            'Can I use my syllabus to plan my studies ',
            'How do I view the recommended books in my syllabus ',
            'How do I add or remove topics from my syllabus ',
            'Can I view detailed notes for each section of my syllabus ',
            'How do I track the completion of my syllabus ',
            'How do I manage syllabus changes during the semester ',
            'Can I access my syllabus through the student portal '
        ],
        'online_class_manager': [
            'How do I manage my online classes ',
            'Can I schedule an online class ',
            'How do I join an online class ',
            'Can I view the list of upcoming online classes ',
            'How do I invite students to my online class ',
            'How do I check the attendance of students in an online class ',
            'Can I create new sessions for my online class ',
            'How do I share materials during an online class ',
            'Can I schedule recurring online classes ',
            'How do I manage break times in online classes ',
            'Can I track my teaching hours for online classes ',
            'How do I see the history of my past online classes ',
            'Can I download the recordings of online classes ',
            'How do I change the time or date of an online class ',
            'Can I monitor student participation in an online class ',
            'How do I conduct exams during online classes ',
            'Can I set reminders for upcoming online classes ',
            'How do I check the technical setup for my online class ',
            'Can I integrate a calendar for online class scheduling ',
            'How do I handle technical issues during an online class ',
            'Can I make changes to my class settings during the session ',
            'How do I share presentations during my online class ',
            'Can I assign tasks to students during an online class ',
            'How do I collect assignments in an online class ',
            'Can I set up breakout rooms in an online class ',
            'How do I moderate discussions during an online class ',
            'Can I send messages to students during the online class ',
            'How do I share study materials in an online class ',
            'How do I get feedback from students in an online class ',
            'How do I manage the schedule for multiple online classes ',
            'Can I use a whiteboard in my online class ',
            'How do I adjust the audio and video settings in an online class ',
            'Can I record the online class for later review ',
            'How do I monitor student progress in online classes ',
            'How do I prepare for an online class session ',
            'Can I manage attendance automatically for online classes ',
            'How do I manage assignments for online classes ',
            'How do I track homework submissions in online classes ',
            'Can I add co-teachers to my online class ',
            'Can I schedule online office hours for students ',
            'How do I communicate with students before and after an online class ',
            'How do I ensure internet stability for online classes ',
            'Can I integrate online tests and quizzes with my online class ',
            'How do I handle technical issues during live sessions ',
            'How do I maintain student engagement in online classes ',
            'Can I create and manage quizzes during an online class ',
            'How do I assess student participation in online classes ',
            'Can I share class notes during an online class ',
            'Can I assign group activities in an online class '
        ],
        'syllabus_master': [
            'How do I create a new syllabus for a subject ',
            'Can I update an existing syllabus ',
            'Where can I view the master syllabus for all subjects ',
            'How do I access the syllabus master list ',
            'Can I delete a syllabus from the master list ',
            'How do I add new topics to my syllabus master ',
            'Can I import syllabi from external sources ',
            'How can I search for a particular syllabus in the master list ',
            'How do I categorize syllabi in the syllabus master ',
            'Can I view the syllabus for a specific subject from the master ',
            'How do I assign a syllabus to a specific course ',
            'Can I export the syllabus master to a file ',
            'How do I check the syllabus revision history ',
            'Can I preview the syllabus before finalizing ',
            'How do I make sure the syllabus is aligned with the curriculum ',
            'Can I attach study materials to a syllabus in the master list ',
            'How do I manage syllabus versions ',
            'Can I track changes made to the syllabus master ',
            'How do I share the syllabus master with colleagues ',
            'Can I sync the syllabus master with my teaching platform ',
            'How do I set permissions for editing the syllabus master ',
            'Can I assign different syllabus versions to different batches ',
            'How do I ensure my syllabus is up-to-date ',
            'Can I get notifications about syllabus updates ',
            'How do I view syllabus changes made by others ',
            'Can I lock the syllabus to prevent further changes ',
            'How do I import a syllabus from a template ',
            'Can I view syllabus comments from other faculty members ',
            'How do I approve changes to the syllabus master ',
            'Can I get reports on syllabus usage across courses ',
            'How do I track the implementation of the syllabus ',
            'Can I get an overview of all subjects in the syllabus master ',
            'How do I delete a syllabus that is no longer in use ',
            'Can I copy a syllabus to create a new one ',
            'How do I group syllabi by course or department ',
            'Can I view syllabi from previous years ',
            'How do I add prerequisites to a syllabus in the master list ',
            'Can I link a syllabus to related course materials ',
            'How do I integrate the syllabus master with my teaching tools ',
            'Can I set deadlines for syllabus updates ',
            'How do I receive feedback on my syllabus ',
            'Can I schedule regular updates for my syllabus ',
            'How do I manage syllabus approvals and reviews ',
            'Can I assign a syllabus to multiple instructors ',
            'How do I monitor the completion of syllabus topics ',
            'Can I track syllabus completion by students ',
            'How do I generate reports based on the syllabus master ',
            'Can I link the syllabus master to a learning management system (LMS) '
        ],
        'teacher_subjects': [
            'Which subjects are assigned to me ',
            'How do I check the subjects I am teaching ',
            'Can I add a new subject to my teaching schedule ',
            'How do I modify my assigned subjects ',
            'Can I remove a subject from my teaching list ',
            'How do I check the subject details for my classes ',
            'Where can I view the list of all subjects ',
            'How do I assign a teacher to a subject ',
            'Can I view the subjects of other teachers ',
            'How do I change the subject I am teaching ',
            'How do I view the syllabus for my subjects ',
            'Can I get the timetable for the subjects I am teaching ',
            'How do I check the attendance for the subjects I teach ',
            'How do I get access to the study material for my subjects ',
            'Can I upload resources for my subjects ',
            'How do I view the number of students in my subjects ',
            'Can I assign homework to students for my subjects ',
            'How do I see upcoming assignments for my subjects ',
            'Can I assign grades for my subjects ',
            'How do I share teaching notes for my subjects ',
            'Can I create an exam schedule for my subjects ',
            'How do I link my subjects to the correct grade levels ',
            'Can I modify the teaching method for my subjects ',
            'How do I update the subject information ',
            'Can I add or update course materials for my subjects ',
            'How do I view the subject performance reports ',
            'Can I track the performance of students in my subjects ',
            'How do I set the teaching schedule for my subjects ',
            'Can I check if a subject is mandatory for my class ',
            'How do I get updates about changes in my subjects ',
            'Can I share my subject-specific resources with other teachers ',
            'How do I view the attendance of students in my subjects ',
            'Can I give feedback on the subjects I am teaching ',
            'How do I access the latest updates on the subjects I teach ',
            'Can I add additional notes or remarks for my subjects ',
            'How do I manage multiple classes for the same subject ',
            'Can I customize my subject schedule ',
            'How do I check for subject conflicts in my timetable ',
            'Can I set reminders for subject-related tasks ',
            'How do I assign teaching assistants to my subjects ',
            'Can I filter subjects by department ',
            'How do I assign multiple teachers to the same subject ',
            'Can I get a report on student performance for my subject ',
            'How do I manage subject prerequisites ',
            'Can I view or edit my subject schedule for the semester ',
            'How do I manage teaching resources for my subjects ',
            'Can I check the workload distribution for my subjects ',
            'How do I coordinate with other teachers on the same subject ',
            'Can I create a subject-specific student group ',
            'How do I get subject-specific announcements ',
            'Can I customize grading criteria for my subjects ',
            'How do I check the subject materials shared by other teachers ',
            'Can I add new topics to my subjects during the semester '
        ],
        'teaching_notes': [
            'Where can I find my teaching notes ',
            'How do I create new teaching notes ',
            'Can I update my existing teaching notes ',
            'Can I upload my teaching notes to the system ',
            'How do I share my teaching notes with other teachers ',
            'Can I categorize my teaching notes by subject ',
            'How do I access the teaching notes of other teachers ',
            'Can I attach resources to my teaching notes ',
            'How do I search for specific teaching notes ',
            'Can I print my teaching notes ',
            'How do I delete old teaching notes ',
            'Can I organize my teaching notes by lesson plans ',
            'How do I add references to my teaching notes ',
            'Can I make my teaching notes accessible to students ',
            'How do I get notified about updates to teaching notes ',
            'Can I view teaching notes shared by other departments ',
            'How do I align my teaching notes with the syllabus ',
            'Can I create teaching notes for group discussions ',
            'How do I get feedback on my teaching notes ',
            'Can I use multimedia (images/videos) in my teaching notes ',
            'How do I track changes in my teaching notes ',
            'Can I make teaching notes public for the whole school ',
            'How do I set permissions for editing teaching notes ',
            'Can I access previous versions of my teaching notes ',
            'How do I organize teaching notes by date ',
            'Can I link my teaching notes to my class timetable ',
            'How do I format my teaching notes ',
            'Can I share teaching notes with students outside school ',
            'How do I view a preview of my teaching notes ',
            'Can I collaborate with other teachers on teaching notes ',
            'How do I make my teaching notes interactive ',
            'Can I add teaching notes to a course curriculum ',
            'How do I include student feedback in my teaching notes ',
            'Can I share teaching notes via email ',
            'How do I categorize teaching notes for specific grades ',
            'Can I integrate my teaching notes with my classroom tools ',
            'How do I set reminders for updating my teaching notes ',
            'Can I check the status of my teaching notes ',
            'How do I track the use of my teaching notes by other teachers ',
            'Can I see the effectiveness of my teaching notes through feedback ',
            'How do I secure my teaching notes from unauthorized access ',
            'Can I sync my teaching notes with my teaching schedule ',
            'How do I make my teaching notes available offline ',
            'Can I review teaching notes from previous semesters ',
            'How do I check the most recent updates in my teaching notes ',
            'Can I create a syllabus using my teaching notes ',
            'How do I submit teaching notes for review ',
            'Can I organize my teaching notes by unit topics ',
            'How do I add teaching goals to my notes '
        ],
        'home_work_managers': [
            'How do I assign homework to my students ',
            'Where can I see all the homework I’ve assigned ',
            'How do I track the progress of homework submissions ',
            'Can I check if my students have submitted their homework ',
            'How do I set deadlines for homework submissions ',
            'Can I view the homework history for my students ',
            'How do I give feedback on homework ',
            'Can I grade homework online ',
            'How do I upload homework assignments ',
            'How do I check which students haven’t submitted homework ',
            'Can I organize homework by subjects ',
            'How do I assign homework in bulk ',
            'Can I set reminders for homework due dates ',
            'How do I attach resources to my homework assignments ',
            'Can I allow students to revise and resubmit homework ',
            'How do I create different types of homework (e g , essays, quizzes) ',
            'Can I allow students to collaborate on homework ',
            'How do I monitor homework submission rates ',
            'Can I set up homework categories (e g , homework, project, assignment) ',
            'How do I create a homework schedule ',
            'Can I view a report on homework completion ',
            'How do I send reminders to students about homework deadlines ',
            'Can I make certain homework assignments optional ',
            'How do I view students’ grades on their homework ',
            'Can I allow multiple submissions for a homework assignment ',
            'How do I send feedback to students on their homework ',
            'Can I see who has completed and who has not completed their homework ',
            'How do I delete or modify a homework assignment ',
            'Can I create group homework assignments ',
            'How do I track late homework submissions ',
            'Can I mark homework as graded ',
            'How do I link homework to lesson objectives ',
            'Can I upload solutions to homework after submission ',
            'How do I track homework progress across multiple classes ',
            'Can I share homework assignments with other teachers ',
            'How do I communicate homework expectations to students ',
            'Can I include multimedia in my homework assignments ',
            'How do I schedule recurring homework assignments ',
            'Can I view feedback from parents on homework ',
            'How do I share answers to homework questions with students ',
            'Can I assign homework with different difficulty levels ',
            'How do I share sample answers with students ',
            'Can I see the time spent by students on homework ',
            'How do I categorize homework by subject ',
            'Can I automatically grade homework assignments ',
            'How do I use homework for assessment purposes ',
            'Can I link homework assignments to the syllabus ',
            'How do I provide extra credit opportunities through homework '
        ],
        'my_online_class': [
            'How do I schedule an online class ',
            'Where can I see my upcoming online classes ',
            'How do I set up a meeting link for my online class ',
            'Can I record my online classes ',
            'How do I invite students to my online class ',
            'Can I check student attendance in my online class ',
            'How do I share course materials in my online class ',
            'Can I use video and audio in my online classes ',
            'How do I assign roles to students in my online class ',
            'Can I conduct quizzes in my online class ',
            'How do I manage online class participation ',
            'Can I set up group activities in my online class ',
            'How do I share a presentation in my online class ',
            'Can I view the attendance record for past online classes ',
            'How do I send reminders for my online classes ',
            'Can I organize the seating arrangement for my online class ',
            'How do I get feedback from students after the class ',
            'Can I host interactive polls in my online class ',
            'How do I mute and unmute participants during the class ',
            'Can I share my screen with the students ',
            'How do I create assignments for my online class ',
            'Can I integrate external tools into my online class ',
            'How do I track student performance in online classes ',
            'Can I add a virtual whiteboard to my online class ',
            'How do I manage break times during the class ',
            'Can I provide one-on-one assistance to students during the class ',
            'How do I upload recordings of my online classes for later viewing ',
            'Can I adjust the class time for different time zones ',
            'How do I send announcements to my online class ',
            'Can I set up discussion boards for my online class ',
            'How do I manage students joining the class late ',
            'Can I assign tasks during the online class ',
            'How do I manage student participation in group activities ',
            'Can I conduct live Q&A sessions in my online class ',
            'How do I share documents during my online class ',
            'Can I set specific permissions for students in my online class ',
            'How do I track the time spent by students in my online class ',
            'Can I share videos with my online class ',
            'How do I ensure that students are attentive during the class ',
            'Can I disable the camera or microphone of students ',
            'How do I upload supporting documents for the online class ',
            'Can I assign teaching assistants to help with online classes ',
            'How do I end my online class session ',
            'Can I organize my online classes by subject ',
            'How do I track who has completed online assignments ',
            'Can I provide certificates for online class completion ',
            'How do I integrate my calendar with my online classes ',
            'Can I use breakout rooms in my online class '
        ],
        'my_time_table': [
            'How do I view my class timetable ',
            'Can I add a new class to my timetable ',
            'How do I modify my existing timetable ',
            'Can I sync my timetable with my phone calendar ',
            'How do I check for timetable conflicts ',
            'Can I share my timetable with students ',
            'How do I remove a class from my timetable ',
            'Can I set reminders for my classes in my timetable ',
            'How do I change the time for a class in my timetable ',
            'Can I view the timetable for other teachers ',
            'How do I set my availability in the timetable ',
            'Can I print my class timetable ',
            'How do I categorize classes by subject in my timetable ',
            'Can I adjust my timetable based on holidays ',
            'How do I check my free periods in the timetable ',
            'Can I check my online class schedule in my timetable ',
            'How do I see upcoming class schedules ',
            'Can I sync my timetable with my teaching calendar ',
            'How do I set up recurring classes in my timetable ',
            'Can I create a timetable for a specific group of students ',
            'How do I update my timetable in case of an emergency ',
            'Can I check for classroom availability in my timetable ',
            'How do I share my timetable with my department ',
            'Can I track changes to my timetable ',
            'How do I assign classes to my colleagues through the timetable ',
            'Can I add breaks to my timetable ',
            'How do I customize the timetable for different terms ',
            'Can I view my previous timetables ',
            'How do I notify students about changes to the timetable ',
            'Can I set up reminders for upcoming classes ',
            'How do I add special events or exams to my timetable ',
            'Can I update the timetable based on student requests ',
            'How do I request time off from my timetable ',
            'Can I see the timetable for my department ',
            'How do I check which classroom I’m assigned to ',
            'Can I make my timetable visible to parents ',
            'How do I check the number of students in each class on the timetable ',
            'Can I set the duration for my classes in the timetable ',
            'How do I adjust my timetable to accommodate different time zones ',
            'Can I check if a class is scheduled during a holiday ',
            'How do I integrate my timetable with the school calendar ',
            'Can I generate a timetable report for my department ',
            'How do I manage multiple sections in my timetable ',
            'Can I set specific locations for my classes in the timetable '
        ],
        'my_teaching_notes': [
            'How can I access my teaching notes ',
            'Where are my latest teaching notes ',
            'Can I search through my teaching notes ',
            'How do I organize my teaching notes ',
            'Can I edit my existing teaching notes ',
            'Can I add more content to my teaching notes ',
            'How do I download my teaching notes ',
            'How do I share my teaching notes with others ',
            'Can I print my teaching notes ',
            'Can I categorize my teaching notes by subject ',
            'How do I add images or files to my teaching notes ',
            'Can I sync my teaching notes across devices ',
            'How do I organize teaching notes by lesson ',
            'Can I collaborate on teaching notes with other teachers ',
            'How do I ensure that my teaching notes are up to date ',
            'Can I integrate teaching notes with my lesson plan ',
            'How do I add student feedback to my teaching notes ',
            'How do I add my teaching notes to the course curriculum ',
            'Can I make my teaching notes available offline ',
            'How do I view my teaching notes from previous terms ',
            'Can I link my teaching notes to the assignments ',
            'How do I create templates for my teaching notes ',
            'Can I search for keywords within my teaching notes ',
            'How do I organize my teaching notes for each subject ',
            'Can I add references and citations in my teaching notes ',
            'How do I make my teaching notes interactive ',
            'Can I track changes made to my teaching notes ',
            'How do I lock certain sections of my teaching notes ',
            'Can I attach documents to my teaching notes ',
            'How do I view revisions of my teaching notes ',
            'Can I access my teaching notes from other devices ',
            'How do I make my teaching notes public or private ',
            'Can I share teaching notes with my students ',
            'How do I get notified when my teaching notes are updated ',
            'Can I archive my teaching notes for future use ',
            'How do I delete old or outdated teaching notes ',
            'Can I link teaching notes with upcoming exams or tests ',
            'How do I set reminders for updating my teaching notes ',
            'Can I embed videos in my teaching notes ',
            'How do I review teaching notes shared by others ',
            'Can I create collaborative groups to work on teaching notes ',
            'How do I check for plagiarism in my teaching notes ',
            'Can I attach links to external resources in my teaching notes ',
            'How do I make a copy of my teaching notes ',
            'Can I track which students have accessed my teaching notes ',
            'How do I get feedback on my teaching notes ',
            'Can I integrate my teaching notes with the digital library ',
            'How do I incorporate feedback from peers into my teaching notes '
        ],
    
        'mark_attendance': [
            'How do I mark attendance for my class ',
            'Can I mark attendance for my online class ',
            'How do I mark attendance for a specific student ',
            'Can I mark attendance for a group of students ',
            'How do I check which students were absent ',
            'Can I edit attendance once it’s marked ',
            'How do I view the attendance history for my class ',
            'Can I set attendance reminders for my classes ',
            'How do I view the attendance status of all students ',
            'Can I export attendance data ',
            'How do I track attendance for multiple sections ',
            'Can I mark attendance in bulk ',
            'How do I track which students have irregular attendance ',
            'Can I send notifications to students about their attendance ',
            'How do I mark attendance for a student who is late ',
            'Can I make a note on a student’s attendance ',
            'How do I check attendance for previous classes ',
            'Can I set up automated attendance tracking ',
            'How do I mark attendance for special events ',
            'Can I share attendance reports with parents ',
            'How do I integrate my attendance system with my timetable ',
            'Can I see attendance trends over time ',
            'How do I remove a student from the attendance record ',
            'Can I set attendance rules for my classes ',
            'How do I mark partial attendance for students ',
            'Can I allow students to mark their own attendance ',
            'How do I handle attendance for students with excused absences ',
            'Can I monitor real-time attendance during my class ',
            'How do I get a summary of my class’s attendance ',
            'Can I create custom attendance categories (e g , Present, Late) ',
            'How do I compare attendance across multiple courses ',
            'Can I track student participation as part of attendance ',
            'How do I adjust attendance based on school holidays ',
            'Can I mark attendance for a student who joined late ',
            'How do I notify students when their attendance is marked ',
            'Can I generate reports based on attendance data ',
            'How do I check if there are attendance discrepancies ',
            'Can I review attendance for different subjects or periods ',
            'How do I adjust attendance for missed classes ',
            'Can I automate attendance for recurring classes ',
            'How do I check the attendance for specific periods ',
            'Can I add notes for students with low attendance ',
            'How do I notify parents about their child’s attendance ',
            'Can I integrate attendance data with my gradebook ',
            'How do I manage attendance for extracurricular activities ',
            'Can I track student attendance across multiple semesters '
        ],
        
        'mark_bulk_attendance': [
            'How do I mark bulk attendance for my class ',
            'Can I mark attendance for multiple classes at once ',
            'How do I mark attendance for an entire group of students ',
            'Can I upload an attendance list for bulk marking ',
            'How do I mark attendance for multiple subjects ',
            'Can I use a CSV file to mark bulk attendance ',
            'How do I check bulk attendance for past sessions ',
            'Can I edit bulk attendance after submission ',
            'How do I track bulk attendance for a term ',
            'Can I add specific remarks for bulk attendance ',
            'How do I mark bulk attendance during school events ',
            'Can I review bulk attendance for each subject ',
            'How do I send attendance notifications after bulk marking ',
            'Can I generate reports from bulk attendance records ',
            'How do I view bulk attendance by section or group ',
            'Can I set a rule for bulk attendance (e g , all students marked present) ',
            'How do I remove or update entries in bulk attendance ',
            'Can I generate a summary report from bulk attendance ',
            'How do I synchronize bulk attendance with the timetable ',
            'Can I track individual attendance in bulk marking ',
            'How do I mark attendance for large groups of students ',
            'Can I assign different statuses to students in bulk attendance ',
            'How do I get feedback on bulk attendance submission ',
            'Can I monitor bulk attendance trends ',
            'How do I view bulk attendance data from previous months ',
            'Can I export bulk attendance records to Excel ',
            'How do I share bulk attendance reports with parents ',
            'Can I automate attendance for recurring events ',
            'How do I validate bulk attendance data before finalizing ',
            'Can I bulk mark attendance for special programs ',
            'How do I make bulk attendance adjustments for missed days ',
            'Can I set a deadline for submitting bulk attendance records ',
            'How do I track bulk attendance for extracurricular activities ',
            'Can I create attendance categories for bulk marking ',
            'How do I ensure bulk attendance data is accurate ',
            'Can I add or remove students from bulk attendance records ',
            'How do I track bulk attendance for school-wide events ',
            'Can I track attendance across multiple school terms ',
            'How do I add late arrivals in bulk attendance marking ',
            'Can I set permissions for who can submit bulk attendance ',
            'How do I handle bulk attendance for absent students ',
            'Can I integrate bulk attendance with the school’s database ',
            'How do I manage bulk attendance for a specific department ',
            'Can I view the attendance status of each student after bulk marking '
        ],
        'my_attendance': [
            'How do I view my attendance record ',
            'Can I track my attendance for all subjects ',
            'Where can I see my attendance for this semester ',
            'Can I see my attendance for a specific date ',
            'How do I view my overall attendance for the course ',
            'Can I get an attendance summary ',
            'How do I know if I’ve exceeded the allowed absences ',
            'Can I check my attendance for specific classes ',
            'Where can I see a breakdown of my attendance status ',
            'How can I track my attendance for online classes ',
            'How do I view my attendance in the last month ',
            'Can I see how many classes I’ve missed ',
            'How do I view attendance for each subject ',
            'Can I see the attendance percentage ',
            'How do I track attendance for activities ',
            'Can I view my attendance for co-curricular activities ',
            'How do I get notified about my attendance status ',
            'Can I track my attendance using a mobile app ',
            'How do I check if I’m meeting the attendance requirements ',
            'Can I review attendance history ',
            'How do I request a correction for my attendance ',
            'Can I download my attendance records ',
            'How do I share my attendance details with my parents ',
            'Can I view attendance trends over time ',
            'How do I mark an absence or late arrival ',
            'Can I receive alerts when my attendance is low ',
            'How do I see my attendance status for extracurricular activities ',
            'How do I request leave and see its effect on my attendance ',
            'How do I calculate my attendance percentage ',
            'Can I access my attendance from previous semesters ',
            'Can I view my attendance across multiple subjects ',
            'How do I know if I have excused absences ',
            'Can I see which classes I attended today ',
            'How do I track my attendance in real-time ',
            'Can I see attendance details in a graph or chart ',
            'How do I mark attendance for field trips or special events ',
            'Can I view detailed attendance information in a report ',
            'How do I compare my attendance to my classmates ',
            'How do I check if there are any attendance discrepancies ',
            'Can I request a report of my attendance for a specific period ',
            'How do I see a list of excused absences ',
            'Can I add notes or comments about my attendance ',
            'How do I track my attendance for multiple terms ',
            'Can I add special remarks for missed classes ',
            'How do I track late arrivals in my attendance record ',
            'Can I track attendance for practical classes separately ',
            'How do I monitor my participation in class activities ',
            'Can I see my attendance with a breakdown by session ',
            'How do I manage my attendance if I’m sick or on leave ',
            'Can I view the attendance status for all of my courses '
        ],
    
        'my_notes': [
            'How can I view my notes ',
            'Can I search for specific notes in my collection ',
            'How do I organize my notes for each subject ',
            'Can I add images or files to my notes ',
            'How do I categorize my notes by topic ',
            'Can I download my notes for offline access ',
            'How do I share my notes with others ',
            'Can I edit my notes after they are created ',
            'How do I track changes made to my notes ',
            'Can I sync my notes across devices ',
            'How do I add new notes to my collection ',
            'Can I create notes for specific lessons ',
            'How do I organize my notes by subject or class ',
            'Can I integrate my notes with my teaching materials ',
            'How do I categorize my notes by priority ',
            'Can I highlight or mark important sections in my notes ',
            'How do I make my notes available to other students ',
            'Can I collaborate with others on my notes ',
            'How do I add comments or annotations to my notes ',
            'Can I link my notes with assignments or exams ',
            'How do I store references or links in my notes ',
            'Can I print my notes for future use ',
            'How do I view my notes from different courses together ',
            'Can I track revisions made to my notes ',
            'How do I archive my old notes for future reference ',
            'Can I protect my notes with a password ',
            'How do I get notifications when my notes are updated ',
            'Can I share my notes with parents or guardians ',
            'How do I incorporate feedback into my notes ',
            'Can I create templates for my notes ',
            'How do I track the creation date of my notes ',
            'Can I see a history of the changes made to my notes ',
            'How do I find specific content within my notes ',
            'Can I add links to external resources in my notes ',
            'How do I add multimedia to my notes ',
            'Can I make my notes accessible offline ',
            'How do I see a list of notes related to a subject ',
            'Can I set reminders to review my notes ',
            'How do I get suggestions for new notes ',
            'Can I attach my notes to specific dates or events ',
            'How do I track my progress with the notes I’ve written ',
            'Can I integrate my notes with my assignments ',
            'How do I make my notes available to my students ',
            'Can I check my notes for spelling or grammar errors ',
            'How do I keep my notes organized during the semester ',
            'Can I mark certain parts of my notes as completed ',
            'How do I use my notes to prepare for exams ',
            'Can I sync my notes across all my devices ',
            'How do I make my notes more interactive ',
            'Can I create summaries or outlines in my notes ',
            'How do I categorize my notes based on difficulty or relevance '
        ],
    
        'my_homework': [
            'How do I view my homework assignments ',
            'Can I track due dates for my homework ',
            'How do I submit my homework online ',
            'Can I add notes to my homework submissions ',
            'How do I know if my homework is graded ',
            'Can I see feedback on my homework ',
            'How do I organize my homework by subject ',
            'Can I get reminders about upcoming homework deadlines ',
            'How do I track my homework progress ',
            'Can I mark homework as complete ',
            'How do I see my homework grades ',
            'Can I collaborate with classmates on homework assignments ',
            'How do I download homework assignments for offline work ',
            'Can I attach files to my homework submissions ',
            'How do I check my homework for plagiarism ',
            'Can I upload my homework from a mobile app ',
            'How do I get notified when new homework is assigned ',
            'Can I see a list of all my past homework assignments ',
            'How do I get updates on my homework status ',
            'Can I request an extension for my homework deadline ',
            'How do I check my homework progress over time ',
            'Can I share my homework with my teacher ',
            'How do I submit group homework assignments ',
            'Can I attach research materials to my homework ',
            'How do I track the time I spend on homework ',
            'Can I see homework from previous terms ',
            'How do I revise my homework if I made mistakes ',
            'Can I get help on my homework from my teacher ',
            'How do I organize homework based on priority ',
            'Can I submit my homework in multiple formats ',
            'How do I view homework related to specific lessons ',
            'Can I view homework for specific dates or weeks ',
            'How do I get reminders for homework reviews ',
            'Can I add comments or questions to my homework ',
            'How do I check my homework submission history ',
            'Can I view homework assignments by subject ',
            'How do I prepare for exams using my homework assignments ',
            'Can I mark certain homework tasks as urgent ',
            'How do I keep track of all my homework across courses ',
            'Can I sync my homework assignments across devices ',
            'How do I ensure that my homework submission is correct ',
            'Can I print my homework assignments ',
            'How do I see homework assignments by due date ',
            'Can I get notifications for homework grading ',
            'How do I manage homework for multiple subjects ',
            'Can I track homework status for group projects ',
            'How do I ensure that my homework is submitted on time ',
            'Can I see my progress in completing all homework tasks ',
            'How do I submit homework in different languages ',
            'Can I integrate homework assignments with my schedule '
        ],
        'co_curricular_activity': [
            'What are the upcoming co-curricular activities ',
            'How can I register for co-curricular activities ',
            'What are the available clubs and societies ',
            'How do I join a new co-curricular activity ',
            'Can I participate in multiple co-curricular activities ',
            'How do I view my participation in co-curricular activities ',
            'What are the benefits of participating in co-curricular activities ',
            'How do I track my achievements in co-curricular activities ',
            'Can I create my own co-curricular activity ',
            'How do I get notified about co-curricular activity events ',
            'Can I receive reminders for co-curricular activities ',
            'Where can I see the schedule for co-curricular activities ',
            'How can I contribute to organizing a co-curricular event ',
            'Can I participate in co-curricular activities after school hours ',
            'How do I check my attendance for co-curricular activities ',
            'Can I track my hours of participation in co-curricular activities ',
            'How do I submit my achievements for co-curricular activities ',
            'Can I get a certificate for participating in co-curricular activities ',
            'How do I get credit for my co-curricular activity participation ',
            'Where can I see a list of co-curricular activities I’ve joined ',
            'How do I manage my schedule for co-curricular activities ',
            'How do I know if I meet the requirements to join a co-curricular activity ',
            'Can I view co-curricular activities for a specific term ',
            'How do I register for co-curricular activities for the next term ',
            'How can I add a new activity to my co-curricular schedule ',
            'Can I participate in co-curricular activities online ',
            'How do I connect with other students in my co-curricular activities ',
            'Can I join co-curricular activities related to my major ',
            'How do I get feedback on my performance in co-curricular activities ',
            'How do I see upcoming events in my co-curricular club ',
            'Can I volunteer for co-curricular activities ',
            'How do I know if a co-curricular activity is mandatory ',
            'Can I withdraw from a co-curricular activity ',
            'How do I get involved in leadership roles in co-curricular activities ',
            'Can I propose a new activity for my school’s co-curricular program ',
            'How do I track my involvement in sports activities ',
            'Can I receive updates about my co-curricular group ',
            'How do I see achievements and awards in co-curricular activities ',
            'How do I communicate with coordinators for co-curricular activities ',
            'Can I collaborate with other students on co-curricular projects ',
            'How do I submit a project for a co-curricular activity ',
            'Can I showcase my skills through co-curricular activities ',
            'How do I see the benefits of participating in specific co-curricular activities ',
            'Can I integrate my co-curricular activities into my academic portfolio ',
            'How do I balance co-curricular activities with academics ',
            'Can I join co-curricular activities as a volunteer ',
            'How do I create a project related to co-curricular activities ',
            'Can I get an endorsement for my involvement in co-curricular activities ',
            'How do I register for special co-curricular workshops ',
            'Can I check if I’m eligible for specific co-curricular programs ',
            'How do I track my performance in a sports event as part of co-curricular activities '
        ]
    },

        'examination': {
        'examination_master': [
            "What is the examination schedule for this year ",
            "Can I get details about the upcoming exams ",
            "What are the exam rules ",
            "How can I prepare for my exams ",
            "What is the exam format for this term ",
            "Where can I find the exam syllabus ",
            "How do I access my exam timetable ",
            "When is the next exam scheduled ",
            "Can I reschedule my exam ",
            "What happens if I miss an exam ",
            "Can I apply for a re-examination ",
            "Where can I view the exam regulations ",
            "Are there any changes in the exam schedule ",
            "What is the exam duration ",
            "How do I get my exam hall ticket ",
            "Can I get a seat change for my exam ",
            "How do I appeal my exam results ",
            "Is there any specific attire for the exam ",
            "Can I bring any materials into the exam hall ",
            "What items are prohibited during the exam ",
            "Where is the exam center located ",
            "How do I check if I’m eligible to sit for the exam ",
            "What do I need to bring to the exam hall ",
            "Can I request an extra time for the exam ",
            "Are the exams conducted online or offline ",
            "How can I get access to previous exam papers ",
            "Where can I check my exam eligibility ",
            "Can I change my exam subjects ",
            "How do I know if I have any backlogs in my exams ",
            "What happens if I fail an exam ",
            "Can I get extra time for my exam if I have a disability ",
            "Where can I find the exam guidelines ",
            "How do I check if I’ve registered for all required exams ",
            "Can I take a break during the exam ",
            "What happens if I forget to bring my exam ID ",
            "Can I take the exam from home ",
            "Is there a revaluation process for exams ",
            "How do I check my exam results ",
            "Can I get a provisional exam certificate ",
            "What if I have a conflict between two exams ",
            "How do I apply for an exam deferral ",
            "Where can I see the exam result release date ",
            "Can I get exam preparation tips ",
            "What happens if I miss my exam ",
            "Can I request a change in my exam subject or schedule ",
            "Where can I find the exam notification ",
            "How do I appeal if my exam results are wrong "
        ],
        "grade_master": [
            "What is grade master in examinations ",
            "How to add a new grade ",
            "Edit existing grade master entry",
            "Delete a grade from master",
            "List all grade levels",
            "What are grade thresholds ",
            "Set grade boundaries",
            "Can I customize grades ",
            "Upload grades in bulk",
            "Download grade master format",
            "Assign grade to marks range",
            "Is GPA part of grade master ",
            "Define grade rules",
            "How to map grades to assessments ",
            "Generate grade list",
            "Preview all grades",
            "Sort grade master entries",
            "Create grade A to E",
            "Grade percentage range setup",
            "Grade with remarks entry",
            "Are grades editable ",
            "Check grade configuration",
            "Subject-wise grade allocation",
            "Class-wise grade template",
            "Set pass/fail threshold",
            "Can I have custom grade labels ",
            "Upload grades from Excel",
            "Validate grade inputs",
            "How are grades calculated ",
            "Create CGPA based grades",
            "How to define credit-based grades ",
            "Import last year’s grades",
            "Restrict grade editing",
            "View archived grades",
            "Manage multiple grade sets",
            "Sync grades with mark entry",
            "Allow decimal grade range",
            "Use same grade for all subjects",
            "Apply grade schemes",
            "Grade point average settings",
            "Include remarks with grade",
            "Check student grades in master",
            "Apply grades automatically",
            "Map grade master to students",
            "Link grade with reports",
            "What is the grading system ",
            "Can I use letter grades ",
            "Customize result with grades",
            "Show grades on report card",
            "Reset grade master settings"
        ],
        "assessment_master": [
            "What is assessment master ",
            "How to define assessments ",
            "Add a new assessment entry",
            "Edit an assessment",
            "Delete assessment from list",
            "View all assessments",
            "Assign weightage to assessments",
            "Map assessments to terms",
            "Class-wise assessment setup",
            "Assessment master structure",
            "How to import assessments ",
            "Assessment grading rules",
            "Assessment type definition",
            "Group assessments by subject",
            "Generate assessment scheme",
            "Assessment format download",
            "How to customize assessments ",
            "Assessment code configuration",
            "Weightage entry for assessments",
            "Assign assessment to subjects",
            "Assessment remarks entry",
            "Enable/disable assessments",
            "Apply default assessments",
            "Assessment template upload",
            "What fields are in assessment master ",
            "Set maximum marks for assessments",
            "Assessment mapping to grade",
            "Convert marks to grade in assessment",
            "Assessment type filtering",
            "Create internal assessments",
            "Mark entry based on assessment",
            "Assign assessment to student group",
            "Bulk assessment entry",
            "Assessment master validation",
            "Assessment categories",
            "Term wise assessment setup",
            "Create co-scholastic assessments",
            "Assessment result generation",
            "How to define sub-assessments ",
            "Assessment configuration for online exam",
            "Assessment rules setup",
            "Mark limits in assessment",
            "Assessment with negative marking",
            "Assessment visibility in portal",
            "Assessment history for student",
            "Copy assessment master to next year",
            "Assessment evaluation method",
            "Auto calculate grade in assessments",
            "Delete inactive assessments",
            "Schedule based on assessment master"
        ],
        "skill_groups": [
            "What are skill groups ",
            "Create a new skill group",
            "Edit skill group details",
            "Delete a skill group",
            "List all skill groups",
            "Define skill categories",
            "Assign skills to group",
            "Skill group setup guide",
            "Skill group for co-scholastic",
            "How many skills per group ",
            "View student-wise skill groups",
            "Skill group entry form",
            "Can I customize skill groups ",
            "Import skill groups from file",
            "Skill group template download",
            "Skill evaluation setup",
            "Skill mapping with class",
            "How to evaluate skill group ",
            "Group-wise skill report",
            "Add soft skills in group",
            "Skill group type definition",
            "Assign skill groups to grades",
            "Use skill group in exams",
            "Skill group progress tracking",
            "Skill group observations entry",
            "Bulk skill group upload",
            "Skill group marking system",
            "Skill group data export",
            "Skill groups visibility to parents",
            "Skill groups in report card",
            "Skill master linking with groups",
            "Assign skill group to teacher",
            "Skill grading inside group",
            "Skill descriptors in group",
            "Skill rubric definition",
            "Skill group validation rules",
            "Skill domain entry",
            "Student-wise skill evaluation",
            "Track skill improvement",
            "Create observation checklist",
            "View archived skill groups",
            "Customize group names",
            "Skill group chart report",
            "Apply skill groups in portal",
            "Reassign skills between groups",
            "Skill group with sub-skills",
            "Grade-wise skill group settings",
            "Skill observation frequency",
            "Delete unused skill groups"
        ],
        "skill_master": [
            "What is skill master ",
            "Add a new skill entry",
            "Edit existing skill details",
            "Remove a skill from master",
            "View list of all skills",
            "Assign skills to group",
            "Skill rubric setup",
            "How to mark skills ",
            "Skill scoring range",
            "Custom skill entry",
            "Define soft skill master",
            "Hard skill entry options",
            "Skill remarks entry",
            "Link skill with student profile",
            "Skill-wise report generation",
            "Track student skill growth",
            "Import skills to system",
            "Skill master export",
            "What skills are available ",
            "Skill levels definition",
            "Skill category linking",
            "Create core skills",
            "Manage student skillset",
            "Assign skill teacher wise",
            "Skill ID generation",
            "Bulk upload skills",
            "Skill evaluation matrix",
            "Class-wise skill master",
            "Skill grading scale",
            "Define performance indicators",
            "Create skill test format",
            "Skill rubric download",
            "Skill remarks settings",
            "Skill point table",
            "Use skill master in exams",
            "Skill validation logic",
            "Reorder skill entries",
            "Observation note for skill",
            "Skill status toggle",
            "Assign priority for skills",
            "Create student skill bank",
            "Skill importance weightage",
            "Skill descriptor input",
            "Teacher assessment for skills",
            "Skill calendar setup",
            "Skill review process",
            "Student feedback on skill",
            "Evaluation history of skills",
            "Skill archive list",
            "Reset skill master"
        ],
        "exam_description_type": [
            "What is exam description type ",
            "Add new exam type",
            "Edit existing exam description",
            "Delete exam description type",
            "Define exam types",
            "List all exam description types",
            "Customize exam categories",
            "How to manage exam description ",
            "Exam type master entry",
            "Exam duration and weightage",
            "Assign description to term",
            "Exam remarks linking",
            "Exam type for internal assessments",
            "External exam description setup",
            "Exam pattern details",
            "Configure theory/practical types",
            "Attach type to subject",
            "Exam scheme creation",
            "Exam rules per description",
            "Subject wise exam type linking",
            "Student-wise exam type report",
            "Exam schedule based on type",
            "Term-wise description types",
            "Upload exam description",
            "Download exam format",
            "Exam type remarks setup",
            "Set description display name",
            "Enable/disable exam types",
            "Mapping of exam types",
            "Auto link exam with description",
            "Exam paper description setup",
            "Generate report per description",
            "Assign teacher for exam type",
            "Sync with mark entry",
            "Validate exam type selection",
            "Exam category sorting",
            "Show only active exam types",
            "Exam type filtering in UI",
            "Apply description in mobile app",
            "Exam description backup",
            "Archive old exam types",
            "Assign default description",
            "Define exam code & name",
            "Custom logic for exam types",
            "Exam evaluation by type",
            "Print report by exam type",
            "Link description to grade system",
            "Can I copy description types ",
            "Delete unused exam types",
            "Multiple descriptions per exam"
        ],
        "subject_group": [
            "What is a subject group ",
            "Create a new subject group",
            "Edit subject group settings",
            "Delete a subject group",
            "Assign subjects to a group",
            "View list of subject groups",
            "Group elective subjects",
            "Mandatory vs optional group",
            "Configure group subjects",
            "Subject group master setup",
            "Class-wise subject groups",
            "Subject group for report cards",
            "Define subject streams",
            "Batch-wise subject group",
            "Bulk upload subject groups",
            "Subject group data export",
            "Assign teacher to group",
            "Manage subject combinations",
            "Mark entry based on groups",
            "Streamwise subject grouping",
            "Subject groups for term exam",
            "Grade linking with group",
            "Subject group logic setup",
            "Can I assign one subject to multiple groups ",
            "Subject grouping for electives",
            "Group entry validation",
            "Subject group weightage",
            "Time table generation by group",
            "Group-wise syllabus mapping",
            "Add lab subjects in group",
            "Group-wise grading policy",
            "Student subject allocation",
            "How to track group performance ",
            "Create subject group code",
            "Subject group with skill mapping",
            "Set default subject group",
            "Download subject group template",
            "Subject groups for entrance prep",
            "Optional subject group creation",
            "Validate group selection",
            "Change subject group mid-year",
            "Assign group based on class",
            "Apply group to section",
            "Add co-scholastic in group",
            "Subject group visibility in portal",
            "Teacher wise group summary",
            "Group creation rules",
            "Remove inactive groups",
            "Subject group mapping history"
        ],
        'exam_creation': [
            "How do I create an exam for the semester ",
            "What is the process of creating an exam ",
            "Can I add different sections to the exam ",
            "How do I set the exam time limit ",
            "Can I select the exam date ",
            "How do I assign marks for each question in the exam ",
            "Is it possible to create an online exam ",
            "How do I add multiple subjects in the exam ",
            "Can I include practicals in the exam creation ",
            "How do I create an exam with multiple choice questions ",
            "Can I set negative marking for the exam ",
            "How do I assign different exam supervisors ",
            "Can I schedule multiple exams at the same time ",
            "How do I create an exam for specific groups of students ",
            "Can I preview the exam before finalizing it ",
            "How do I upload exam questions from a file ",
            "Can I set different difficulty levels for questions in the exam ",
            "How do I add time-bound questions to the exam ",
            "Can I create an exam with essay-type questions ",
            "How do I create a custom exam for a particular course ",
            "Can I set automatic grading for the exam ",
            "How do I ensure that no student repeats a question in the exam ",
            "Can I limit the number of attempts for each student ",
            "How do I make the exam available to selected students only ",
            "Can I schedule the exam to be available on specific dates ",
            "How do I set up a password-protected exam ",
            "Can I assign individual exam weights to each section ",
            "How do I add images or videos to exam questions ",
            "Can I create a timed exam with automatic submission ",
            "Can I set up an exam with randomized questions for each student ",
            "How do I choose whether the exam is open-book or closed-book ",
            "Can I schedule the exam for multiple batches of students ",
            "How do I set the passing criteria for the exam ",
            "Can I set up auto-evaluation for objective type questions ",
            "How do I notify students about the exam schedule ",
            "Can I create an exam with practical assessments ",
            "How do I define the exam duration based on subjects ",
            "Can I set different rules for the exam based on subject ",
            "How do I assign different types of questions in one exam ",
            "Can I allow students to save and continue their exam later ",
            "Can I include a break time in the exam schedule ",
            "How do I test the exam before making it live for students ",
            "Can I change the exam settings after creating it ",
            "How do I add a comment section for students in the exam ",
            "Can I schedule multiple exam shifts for different timings "
        ],
        'criteria_based': [
            "How do I create an exam based on specific criteria ",
            "Can I set eligibility criteria for an exam ",
            "What types of criteria can be applied for exams ",
            "How do I select students based on certain criteria for an exam ",
            "Can I create a custom criteria for exam eligibility ",
            "How do I restrict exam participation based on attendance ",
            "How can I include grade-based criteria in the exam ",
            "Can I define prerequisites for students before they take the exam ",
            "How do I create a criteria-based exam schedule ",
            "Can I select students with a certain minimum grade for the exam ",
            "How do I specify which students are eligible for the exam ",
            "Can I set up an automatic criteria check for exam eligibility ",
            "How do I manage criteria for different courses in the exam ",
            "Can I create a custom criteria for each subject’s exam ",
            "How do I ensure that students meet the criteria before the exam ",
            "Can I filter students based on specific criteria for exam seating ",
            "How do I create separate criteria for practical and theory exams ",
            "Can I apply a cutoff percentage as a criterion for the exam ",
            "Can I set different criteria for re-exams ",
            "How do I check if a student meets the exam eligibility criteria ",
            "Can I include non-academic criteria like extracurriculars for exam eligibility ",
            "How do I handle students who don’t meet the criteria for the exam ",
            "Can I include attendance in the criteria for taking an exam ",
            "How do I inform students about the criteria for exam participation ",
            "Can I create a customized criteria for international students taking the exam ",
            "How do I modify the criteria once the exam is scheduled ",
            "Can I automatically update criteria for each exam session ",
            "How do I check the validity of the criteria set for the exam ",
            "Can I create a waitlist based on criteria for an exam ",
            "How do I set different criteria for regular and distance learning students ",
            "Can I allow exceptions to the criteria under specific circumstances ",
            "Can I change the criteria based on student feedback ",
            "How do I define the criteria for optional exams ",
            "Can I apply specific age-based criteria for the exam ",
            "How do I include multiple criteria for exam entry ",
            "Can I combine academic performance and extracurriculars for exam eligibility ",
            "Can I restrict access to an exam based on past performance ",
            "How do I include criteria like student behavior or discipline in the exam schedule ",
            "Can I set time-bound criteria for students before they can sit for an exam ",
            "How do I set criteria for students opting for special accommodations during the exam ",
            "Can I define eligibility criteria based on a student’s major or department ",
            "How do I inform students when they do not meet the criteria for the exam "
        ],
        "criteria_master": [
            "What is criteria master ",
            "How to add a new criteria ",
            "Edit criteria master details",
            "Delete a criteria from the master",
            "List all available exam criteria",
            "Define grading criteria",
            "Set up criteria for subjects",
            "Criteria master format download",
            "Import exam criteria",
            "Custom exam criteria entry",
            "Create assessment-based criteria",
            "Configure pass/fail criteria",
            "Set marks range in criteria",
            "Criteria for internal exams",
            "Criteria for practical exams",
            "Term-wise criteria setup",
            "Add weightage to criteria",
            "Criteria for different classes",
            "Link criteria to assessments",
            "What are default exam criteria ",
            "How to enable/disable criteria ",
            "Criteria label customization",
            "Is there a template for criteria ",
            "Create custom logic in criteria",
            "Criteria used in reports ",
            "Link criteria with exam types",
            "Can I assign criteria per grade ",
            "Criteria ID generation",
            "Bulk upload of criteria",
            "How to validate exam criteria ",
            "Create multi-subject criteria",
            "Group-wise criteria setup",
            "Duplicate criteria option",
            "Criteria summary report",
            "Marks distribution in criteria",
            "Criteria master for terms",
            "Delete unused criteria",
            "Archive old criteria",
            "Criteria-based result settings",
            "Customize marks evaluation",
            "How many criteria allowed ",
            "Import from previous year",
            "Exam rules using criteria",
            "Attach remarks to criteria",
            "Subject-wise criteria linking",
            "Marks range per criteria level",
            "Allow decimal marks in criteria",
            "Auto calculate based on criteria",
            "Criteria name and code entry",
            "Review criteria settings"
        ],
        "criteria_association": [
            "What is criteria association ",
            "Associate criteria to subjects",
            "Link criteria to class and term",
            "View existing criteria associations",
            "Add new criteria association",
            "Edit a criteria link",
            "Remove an associated criteria",
            "Criteria association template",
            "Bulk upload criteria associations",
            "Criteria association validation",
            "Assign criteria by grade level",
            "How to set criteria mappings ",
            "Criteria per student group",
            "Subject to criteria linkage",
            "Is criteria reusable ",
            "Copy criteria associations",
            "Use same criteria across terms",
            "Default criteria associations",
            "Can we change associated criteria ",
            "Associate skill criteria",
            "Mark entry based on association",
            "Track associated criteria changes",
            "Show associated criteria in portal",
            "Preview criteria associations",
            "Assign multiple criteria per subject",
            "Remove old associations",
            "Reassign criteria by exam type",
            "Link practical and theory criteria",
            "Validate linked criteria",
            "Sync with exam master",
            "Set priority for criteria",
            "Criteria subject mapping list",
            "Update existing associations",
            "Criteria associations in reports",
            "Auto assign criteria",
            "Check criteria association conflicts",
            "Class section-wise criteria",
            "List of unlinked subjects",
            "Re-link deleted associations",
            "Can I override associations ",
            "Batch-wise criteria association",
            "Assign by assessment group",
            "Use criteria groups for subjects",
            "Criteria in co-scholastic areas",
            "Upload association format",
            "Import criteria associations from Excel",
            "Delete invalid associations",
            "Filter criteria by subject group",
            "Assign exam-wise criteria sets"
        ],
        "criteria_marks_entry": [
            "What is criteria marks entry ",
            "How to enter marks for criteria ",
            "Edit marks under specific criteria",
            "Delete criteria marks entry",
            "Enter marks based on grade criteria",
            "Bulk marks entry for criteria",
            "Class-wise criteria marks entry",
            "Section-wise entry for criteria",
            "Criteria marks format download",
            "Enable criteria marks input",
            "Mark entry for internal exams",
            "Enter practical marks by criteria",
            "Criteria wise mark entry per student",
            "Sync criteria marks with report card",
            "Import marks entry from sheet",
            "Subject-wise criteria marks input",
            "Update entered marks",
            "Criteria marks audit report",
            "Are marks auto-calculated ",
            "Allow decimal in marks entry",
            "Criteria marks by assessment",
            "Marks validation rules",
            "Criteria based result calculation",
            "Can we review marks before saving ",
            "Assign evaluator for criteria entry",
            "Preview all entered marks",
            "Marks entry locked after date ",
            "Exam wise criteria marks input",
            "Reset marks entry form",
            "Custom remarks for marks entry",
            "Skill-based criteria marks entry",
            "Mark upload in bulk for criteria",
            "Auto-fill zero for missing entries",
            "Student feedback for criteria marks",
            "Who can enter marks for criteria ",
            "Reopen marks entry window",
            "Report generation from marks entry",
            "Are marks saved instantly ",
            "Track marks changes by user",
            "Criteria-based error detection",
            "Enter criteria score per session",
            "Recalculate marks with criteria",
            "Notify teachers to enter marks",
            "Time limit for marks entry",
            "Marks approval workflow",
            "Enable/disallow re-entry",
            "Criteria marks entry for co-scholastic",
            "Is marks entry mobile-friendly ",
            "Can I export marks entry ",
            "Batch processing for marks input"
        ],
        'exam_mark_entry': [
            "How do I enter exam marks for students ",
            "Can I update exam marks after they have been entered ",
            "What is the format for entering exam marks ",
            "Can I upload exam marks in bulk ",
            "How do I enter marks for multiple subjects ",
            "Can I modify individual marks once entered ",
            "How do I enter marks for practical exams ",
            "Can I enter marks for an individual student ",
            "How do I check if marks are entered correctly ",
            "What happens if I enter wrong marks by mistake ",
            "Can I validate marks before submitting them ",
            "How do I update the marks for absent students ",
            "Can I assign marks based on assignments and projects ",
            "How do I enter marks for internal assessments ",
            "Can I add remarks while entering marks ",
            "How do I adjust marks if there was an error in the exam ",
            "Can I set a grading scale for marks entry ",
            "How do I calculate total marks after entering individual scores ",
            "Can I enter marks for an exam that has multiple sections ",
            "How do I handle partial marks entry for subjective questions ",
            "Can I apply a standardization formula when entering marks ",
            "How do I enter bonus marks for students ",
            "Can I enter marks for re-exams ",
            "How do I ensure there are no discrepancies in the marks entry ",
            "Can I track changes made to exam marks ",
            "How do I enter marks for a group of students at once ",
            "Can I review the entered marks before final submission ",
            "How do I enter marks for an online exam ",
            "Can I enter marks for different exam formats like MCQ or essays ",
            "How do I enter marks for a student who was late to the exam ",
            "Can I enter marks for a student who was absent for a particular exam ",
            "How do I enter marks for students with special accommodations ",
            "Can I enter grades instead of raw marks ",
            "How do I link the entered marks to the respective subjects ",
            "Can I save the entered marks before submitting ",
            "How do I ensure that I’ve entered all marks correctly ",
            "Can I export the marks after entry ",
            "How do I enter marks for students who took the exam at different times ",
            "Can I assign marks based on peer assessments ",
            "How do I enter marks for a mixed exam format with objective and subjective questions ",
            "Can I review and adjust marks after the final submission ",
            "How do I track the overall performance of students based on marks ",
            "Can I send out notifications after marks are entered ",
            "How do I handle marks for multiple exam sessions ",
            "Can I set individual weightage for each question while entering marks "
        ],
        'exam_remark_entry': [
            "How do I enter remarks for an exam ",
            "Can I add remarks for a student after the exam is graded ",
            "What is the process for adding remarks to exam results ",
            "Can I add positive and negative remarks for students ",
            "How do I add remarks for absent students ",
            "Can I add remarks for students who have special needs ",
            "Can I add detailed feedback in the exam remarks ",
            "How do I add remarks for both practical and theory exams ",
            "Can I modify remarks after they have been submitted ",
            "How do I add comments for specific questions on the exam ",
            "Can I include encouragement in the remarks for students ",
            "Can I set standard remark templates for the exams ",
            "How do I add remarks for students with low performance ",
            "Can I categorize remarks based on exam sections ",
            "Can I add remarks in different languages ",
            "How do I add remarks for students who failed the exam ",
            "Can I include suggestions for improvement in the remarks ",
            "How do I ensure the remarks align with the grading ",
            "Can I add remarks for re-exam results ",
            "How do I make sure that the remarks are constructive ",
            "Can I add remarks about student behavior during the exam ",
            "How do I enter exam-related feedback for teachers ",
            "Can I add comments for students based on their exam attendance ",
            "Can I remove remarks once they are entered ",
            "How do I ensure that the remarks are confidential ",
            "Can I track changes made to the remarks after submission ",
            "How do I communicate remarks to students after the exam ",
            "Can I add remarks for students who missed part of the exam ",
            "Can I add remarks in the exam report card ",
            "How do I assign remarks for students based on their class participation ",
            "Can I include remarks for students who attempted the exam poorly ",
            "How do I ensure that all remarks are standardized across exams ",
            "Can I give personalized remarks for individual students ",
            "How do I add remarks for students in the case of cheating or misconduct ",
            "Can I delete remarks if they were entered in error ",
            "How do I add remarks for a student after re-evaluating their exam ",
            "Can I set up automatic generation of remarks based on marks ",
            "How do I ensure the remarks are consistent with the grading scheme ",
            "Can I review and edit remarks before finalizing them ",
            "Can I include remarks for the entire exam as feedback ",
            "How do I ensure that remarks are clear and precise ",
            "Can I send the exam remarks directly to the students ",
            "How do I add constructive feedback to help students improve ",
            "Can I categorize remarks based on performance categories ",
            "How do I enter remarks for group-based exams or projects ",
            "Can I add exam-related remarks for students with attendance issues "
        ],
        'exam_grade_entry': [
            "How do I enter grades for the exam ",
            "Can I assign grades based on the marks obtained ",
            "How do I define the grade scale for the exam ",
            "Can I enter grades for both theory and practical exams ",
            "How do I ensure the grades are accurate and reflect the marks ",
            "Can I modify the grades once they are entered ",
            "How do I categorize students based on their grades ",
            "Can I apply a grading curve when entering grades ",
            "How do I convert marks into grades ",
            "Can I set the grade boundaries for the exam ",
            "How do I assign grades for re-exam students ",
            "Can I assign different grading systems for different subjects ",
            "How do I handle students who failed the exam in grade entry ",
            "Can I set grades based on both class participation and exam performance ",
            "How do I enter grades for students with special accommodations ",
            "Can I enter grades for different exam formats like MCQs and essays ",
            "How do I apply grades for students who took the exam online ",
            "Can I update grades after reviewing the student’s performance ",
            "How do I check the grade distribution once entered ",
            "Can I use a percentage-based grading system for the exam ",
            "How do I handle grade entry for students who missed the exam ",
            "Can I enter grades for a student who took the exam at a different time ",
            "How do I assign grades for students based on their overall performance ",
            "Can I set different grading scales for different batches of students ",
            "How do I enter grades for a mixed exam with multiple sections ",
            "Can I adjust the grades based on student feedback ",
            "How do I enter grades for group exams or projects ",
            "Can I enter grades for students based on their contributions to group work ",
            "How do I handle grade changes after re-evaluation ",
            "Can I assign provisional grades for students pending exam results ",
            "How do I ensure consistency in grade assignment ",
            "Can I assign grades based on a rubric for subjective questions ",
            "How do I convert letter grades into GPA ",
            "Can I review and edit grades before final submission ",
            "How do I enter grades for students who submitted late ",
            "Can I generate a grade report for students after entering the grades ",
            "How do I handle discrepancies in grade entry ",
            "Can I notify students once their grades are entered ",
            "How do I ensure that grades are confidential ",
            "Can I enter grades for students who have special circumstances ",
            "How do I verify that the grades entered are correct ",
            "Can I assign grades based on a weighted grading system ",
            "How do I apply a grading scale based on subject difficulty ",
            "Can I calculate the overall grade from the individual scores ",
            "How do I assign grades based on attendance and participation ",
            "Can I assign different weights to the exam and assignments in grade entry ",
            "How do I handle grades for students who didn’t attempt all questions "
        ],
        'exam_report_card': [
            "How do I generate the exam report card for students ",
            "Can I view the report card before finalizing it ",
            "How do I customize the format of the exam report card ",
            "Can I include remarks in the exam report card ",
            "How do I include student’s exam scores in the report card ",
            "Can I download the exam report card for all students ",
            "How do I generate the report card for a specific student ",
            "Can I print the exam report card for distribution ",
            "How do I include both grades and marks in the report card ",
            "Can I generate a report card for a student who missed the exam ",
            "How do I add attendance details to the report card ",
            "Can I add the student’s overall performance in the report card ",
            "How do I include student behavior and remarks in the report card ",
            "Can I generate the report card for a specific subject ",
            "How do I include extra-curricular activity details in the report card ",
            "Can I automatically update the report card after grade entry ",
            "How do I handle discrepancies in the exam report card ",
            "Can I include exam attendance in the report card ",
            "How do I ensure that the exam report card is accurate ",
            "Can I generate a report card for both theory and practical exams ",
            "How do I include comments from the examiners in the report card ",
            "Can I send the exam report card directly to the student ",
            "How do I ensure that all students’ report cards are ready in time ",
            "Can I include the student’s academic ranking in the report card ",
            "How do I add subjects to the report card if they were missing ",
            "Can I generate the report card in different languages ",
            "How do I add additional sections to the exam report card ",
            "Can I include co-curricular activity grades in the report card ",
            "How do I print the report card for multiple students at once ",
            "Can I track changes made to the exam report card ",
            "How do I include the result of the student’s previous exams ",
            "Can I add the date of the exam in the report card ",
            "How do I review the report card before sharing it with parents ",
            "Can I export the exam report card to a PDF or Excel file ",
            "How do I generate the report card for a student who had special accommodations ",
            "Can I send the exam report card via email to the student ",
            "How do I handle confidential information in the report card ",
            "Can I add comments about the student’s exam performance ",
            "How do I ensure the formatting of the report card is consistent ",
            "Can I provide a detailed breakdown of the student’s exam scores in the report card ",
            "How do I customize the layout of the report card ",
            "Can I set up automatic generation of report cards for students ",
            "How do I handle re-exam results in the report card ",
            "Can I include a personal message to the student in the report card ",
            "How do I handle exam-related remarks in the report card ",
            "Can I generate a report card for a student in the middle of the year ",
            "How do I ensure the accuracy of the data in the report card "
        ],
        'import_exam_marks': [
            "How do I import exam marks for students ",
            "Can I import exam marks from an Excel file ",
            "How do I ensure the exam marks are correctly imported ",
            "Can I import marks for multiple subjects at once ",
            "What file formats can I use to import exam marks ",
            "Can I validate the imported exam marks before saving ",
            "How do I import marks for practical exams ",
            "Can I import exam marks for a specific batch of students ",
            "How do I handle missing data during exam marks import ",
            "Can I use a template for importing exam marks ",
            "How do I ensure the import process doesn’t overwrite existing data ",
            "Can I import marks for a specific group of students ",
            "How do I check the imported marks for accuracy ",
            "Can I import marks for students who were absent ",
            "How do I import marks for students from different classes ",
            "Can I import marks for re-exams separately ",
            "How do I ensure there are no duplicates when importing marks ",
            "Can I import marks for both online and offline exams ",
            "How do I check for errors during the import process ",
            "Can I import marks for multiple exam sessions ",
            "How do I handle discrepancies between the imported data and the system data ",
            "Can I preview the imported exam marks before saving them ",
            "How do I track the imported exam marks history ",
            "Can I map subjects to marks while importing ",
            "How do I import marks for students who took different versions of the exam ",
            "Can I use a CSV file to import exam marks ",
            "How do I import exam marks for a specific subject group ",
            "Can I filter the imported marks by date ",
            "How do I ensure that the import doesn’t affect other records ",
            "Can I import marks for students who took exams at different times ",
            "How do I import exam marks for a student with special needs ",
            "Can I import marks for a mixed exam format with objective and subjective questions ",
            "How do I validate the data before importing exam marks ",
            "Can I import grades instead of raw marks ",
            "How do I handle partial marks import for subjective questions ",
            "Can I schedule the exam marks import process ",
            "How do I update existing marks using the import feature ",
            "Can I export the imported marks for further review ",
            "How do I handle missing students during marks import ",
            "Can I import marks and grades together ",
            "How do I import exam attendance along with marks ",
            "Can I import multiple exam marks for a single student ",
            "How do I avoid common errors while importing marks ",
            "Can I import exam marks from an external system ",
            "How do I resolve conflicts during the marks import process ",
            "Can I import marks for students in different exam versions or question papers ",
            "How do I deal with incorrect formatting during marks import "
        ],
        'exam_attendance': [
            "How do I mark attendance for the exam ",
            "Can I mark exam attendance for multiple students at once ",
            "How do I handle students who arrived late for the exam ",
            "Can I update exam attendance after the exam has started ",
            "How do I mark students who were absent for the exam ",
            "Can I track students' attendance across multiple exam sessions ",
            "How do I manage attendance for online exams ",
            "Can I generate an attendance list for the exam ",
            "How do I mark partial attendance for a student who attended only part of the exam ",
            "Can I add notes for absent students ",
            "How do I ensure that attendance is marked for every student ",
            "Can I mark attendance for a student who missed the exam due to illness ",
            "How do I track attendance for practical exams ",
            "Can I mark attendance based on student ID cards ",
            "How do I handle students who arrive late but still take the exam ",
            "Can I mark attendance for a student who was excused from the exam ",
            "How do I manage exam attendance for students with special needs ",
            "Can I print an attendance sheet for the exam ",
            "How do I track changes made to the exam attendance ",
            "Can I send out reminders to students who missed the exam ",
            "How do I validate that all students have been marked for attendance ",
            "Can I track attendance for online or remote exams ",
            "How do I update the attendance list if there were errors in the original marking ",
            "Can I export the exam attendance to a CSV file ",
            "How do I mark attendance for students who missed the exam for personal reasons ",
            "Can I add attendance details to the student’s report card ",
            "How do I deal with students who were excused from the exam ",
            "Can I mark attendance for students who had a special exam schedule ",
            "How do I ensure that there is no duplicate attendance entry ",
            "Can I track the attendance status for each exam session ",
            "How do I handle attendance for multiple exam sessions on the same day ",
            "Can I mark attendance for re-exam students separately ",
            "How do I ensure the attendance data is accurate ",
            "Can I adjust attendance if a student was marked incorrectly ",
            "How do I handle attendance for group exams ",
            "Can I mark attendance for different exam formats ",
            "How do I generate a report of student attendance during the exam ",
            "Can I track the reasons for a student’s absence during the exam ",
            "How do I mark attendance for students who take the exam at a different time ",
            "Can I review the attendance list before finalizing it ",
            "How do I prevent errors while marking exam attendance "
        ],
            
        'exam_consolidated_marks': [
            "How do I view the consolidated marks for all students ",
            "Can I generate a consolidated marks report for the whole class ",
            "How do I include marks from different exams into a single report ",
            "Can I create a consolidated marks report based on subjects ",
            "How do I check if the consolidated marks calculation is correct ",
            "Can I customize the layout of the consolidated marks report ",
            "How do I download the consolidated marks for a specific batch ",
            "Can I generate consolidated marks for different sections separately ",
            "How do I handle missing exam marks in the consolidated report ",
            "Can I add extra columns in the consolidated marks report ",
            "How do I print the consolidated marks report ",
            "Can I export the consolidated marks report to Excel or PDF ",
            "How do I calculate average marks in the consolidated report ",
            "Can I include grades along with marks in the consolidated report ",
            "How do I view subject-wise consolidated marks ",
            "Can I view consolidated marks based on student groups ",
            "How do I include internal and external marks together in consolidation ",
            "Can I compare two exam consolidated results side-by-side ",
            "How do I ensure all subjects are reflected in the consolidated marks ",
            "Can I include exam attendance in the consolidated report ",
            "How do I update consolidated marks if an individual mark entry is changed ",
            "Can I view consolidated marks for optional subjects separately ",
            "How do I deal with students having incomplete exam data ",
            "Can I filter consolidated marks by performance level ",
            "How do I generate a class average from the consolidated report ",
            "Can I include a graphical analysis in the consolidated report ",
            "How do I merge different exam results into one consolidated sheet ",
            "Can I weight exams differently in the consolidated marks ",
            "How do I sort students based on consolidated marks ",
            "Can I rank students using consolidated marks ",
            "How do I verify the consolidated marks before finalizing ",
            "Can I schedule auto-generation of the consolidated marks report ",
            "How do I share the consolidated marks with parents ",
            "Can I consolidate marks for only selected subjects ",
            "How do I include extracurricular activities in consolidated marks ",
            "Can I view the pass/fail status in the consolidated report ",
            "How do I recalculate consolidated marks after a re-exam ",
            "Can I set a minimum passing mark in the consolidated report ",
            "How do I secure the consolidated marks report from editing ",
            "Can I mark students for special awards based on consolidated marks ",
            "How do I customize grade scales in the consolidated marks report ",
            "Can I generate department-wise consolidated marks ",
            "How do I generate term-wise consolidated marks separately ",
            "Can I hide certain exam results while consolidating ",
            "How do I lock the final consolidated marks after approval ",
            "Can I send consolidated marks reports via email ",
            "How do I create a backup of consolidated marks data ",
            "Can I adjust marks manually before consolidating ",
            "How do I set rules for automatic consolidation of marks ",
            "Can I access previous years’ consolidated marks reports "
        ],
        'exam_promotion_report': [
            "How do I generate the exam promotion report ",
            "Can I view students eligible for promotion based on exam marks ",
            "How do I set passing criteria for promotions ",
            "Can I customize the promotion report layout ",
            "How do I promote students automatically based on exam performance ",
            "Can I manually promote a student even if they fail ",
            "How do I handle detained students in the promotion report ",
            "Can I filter students who failed a subject but still got promoted ",
            "How do I print the promotion report ",
            "Can I export the promotion report to Excel or PDF ",
            "How do I verify the accuracy of the promotion list ",
            "Can I set different promotion rules for different classes ",
            "How do I promote students subject-wise ",
            "Can I view past years' promotion records ",
            "How do I re-promote a student who was detained previously ",
            "Can I generate a summary of promoted and detained students ",
            "How do I include attendance in the promotion decision ",
            "Can I generate a promotion report section-wise ",
            "How do I promote students who have supplementary exams ",
            "Can I customize grade-based promotion rules ",
            "How do I rollback promotions if there was an error ",
            "Can I promote only selected students manually ",
            "How do I promote students across departments ",
            "Can I view promotion eligibility before finalizing ",
            "How do I auto-calculate CGPA for promotion criteria ",
            "Can I add remarks in the promotion report ",
            "How do I schedule generation of promotion reports ",
            "Can I track promoted students' new class or section ",
            "How do I view students who did not qualify for promotion ",
            "Can I promote students with special considerations ",
            "How do I highlight toppers in the promotion report ",
            "Can I apply grace marks during promotion evaluation ",
            "How do I merge multiple exam results before promotion ",
            "Can I block promotions for fee defaulters ",
            "How do I finalize the promotion list before academic year-end ",
            "Can I share promotion results with parents through email ",
            "How do I display pass/fail status along with promotion ",
            "Can I create custom rules for border case students ",
            "How do I promote students considering co-curricular achievements ",
            "Can I lock the final promotion list after approval ",
            "How do I auto-generate roll numbers after promotion ",
            "Can I add notes for each student during promotion ",
            "How do I show previous marks with the promotion report ",
            "Can I access detailed reports for promoted vs detained students ",
            "How do I backup promotion report data ",
            "Can I perform mass promotions for entire grades ",
            "How do I set a minimum attendance requirement for promotion ",
            "Can I filter promoted students by percentage range ",
            "How do I generate promotion certificates automatically ",
            "Can I adjust marks manually before deciding promotions "
        ]
    },

        'portfolio': {
        'master': [
            "How do I set up the portfolio master ",
            "Can I edit portfolio categories in the master ",
            "How to add a new portfolio type ",
            "Where do I manage the portfolio master list ",
            "Can I delete a portfolio master entry ",
            "How do I view all portfolio masters ",
            "Can I reorder portfolio master items ",
            "How to assign a portfolio to a student in master ",
            "Is there an option to bulk upload portfolio masters ",
            "Can I duplicate an existing portfolio master ",
            "How do I set rules for portfolio submissions in master ",
            "Can I link portfolio master to specific classes ",
            "How do I archive a portfolio master entry ",
            "Can I search within the portfolio master list ",
            "How to categorize portfolios in the master settings ",
            "Can I assign weights to portfolios in master ",
            "Is it possible to set deadlines in portfolio master ",
            "How to create a template for new portfolio entries ",
            "Can I hide certain portfolios from students ",
            "How to manage grading options in portfolio master ",
            "Can I assign mentors through the portfolio master ",
            "How to set different portfolio rules for each grade ",
            "Can portfolio master be integrated with academic records ",
            "Can I mark a portfolio as compulsory from the master ",
            "How to restrict editing in portfolio master once published ",
            "How to preview a portfolio master item ",
            "Can portfolio masters be class-specific ",
            "How to filter active vs inactive portfolio masters ",
            "How to define the assessment method in portfolio master ",
            "Can I import portfolios from previous academic years ",
            "How do I update an existing portfolio master ",
            "How to delete multiple portfolio masters at once ",
            "Can I set notifications for portfolio master updates ",
            "How to duplicate settings from one master to another ",
            "How to restrict portfolios to specific grades ",
            "Can I transfer portfolios to a new academic session ",
            "How do I check usage statistics for a portfolio master ",
            "How to batch edit portfolio masters ",
            "Is there an approval process for portfolio masters ",
            "Can I view the history of changes in portfolio master ",
            "How to temporarily disable a portfolio master ",
            "How can students see the portfolio master list ",
            "How to add instructions in the portfolio master ",
            "How to set a default portfolio master for new students ",
            "Can I assign deadlines per portfolio master item ",
            "How to manage master portfolios for special needs students ",
            "How to automatically archive old portfolios ",
            "Can I link master portfolios with class performance ",
            "How to allow or restrict edits after portfolio creation "
        ],
        'entry_page': [
            "How do I create a portfolio entry ",
            "Can students submit their portfolios on the entry page ",
            "How do I edit an existing portfolio entry ",
            "Where do I view submitted portfolio entries ",
            "Can I upload multiple files in a portfolio entry ",
            "How do I approve or reject a portfolio submission ",
            "Is there a draft mode on the portfolio entry page ",
            "How to link a portfolio entry to a specific event ",
            "Can I comment on portfolio entries ",
            "How to mark a portfolio entry as complete ",
            "Can students edit their entries after submission ",
            "How to attach documents or media to an entry ",
            "Is there a word limit for portfolio submissions ",
            "How to sort entries based on submission date ",
            "Can I assign grades directly from the entry page ",
            "How do I track revisions in a portfolio entry ",
            "Can I lock a portfolio entry after approval ",
            "How to duplicate a previous portfolio entry ",
            "Is there a notification when a portfolio is submitted ",
            "How to reject a portfolio with feedback ",
            "Can parents view portfolio entries ",
            "How do I filter portfolio entries by category ",
            "How to mass approve multiple entries ",
            "Can I allow peer review for portfolio entries ",
            "How to integrate portfolio entry with classwork ",
            "Is there an option to save as draft before final submission ",
            "How do I bulk upload entries for students ",
            "Can portfolio entries be private between teacher and student ",
            "How do I export portfolio entries ",
            "Can portfolio entries be showcased publicly ",
            "Can I set a submission deadline for entries ",
            "How to send reminders for pending portfolio submissions ",
            "Can students submit video portfolios ",
            "How to categorize portfolio entries ",
            "Can I assign multiple students to a single portfolio entry ",
            "How to monitor portfolio engagement ",
            "Can I view comments history on portfolio entries ",
            "How to track incomplete portfolio entries ",
            "Can entries be sorted by grades received ",
            "How to highlight exceptional portfolio entries ",
            "How to link a portfolio to project-based learning ",
            "Can I track time spent on each portfolio entry ",
            "How to restrict portfolio visibility based on roles ",
            "Can portfolios be tagged with skills or subjects ",
            "How to enable plagiarism check on portfolio submissions ",
            "Can I create a collaborative portfolio entry ",
            "How to add mentors' feedback to entries ",
            "How to auto-save progress on portfolio entries ",
            "How to move entries between categories if needed "
        ],
        'report': [
            "How do I generate a portfolio report ",
            "Can I filter portfolio reports by student ",
            "How to view portfolios grade-wise in reports ",
            "Can I print portfolio reports ",
            "How to export portfolio reports to Excel ",
            "How do I analyze portfolio submissions ",
            "Can I view pending vs approved portfolios in reports ",
            "How to group reports by class or subject ",
            "Is there a summary report for portfolios ",
            "Can I track portfolio submission timelines ",
            "How to show portfolio grades in reports ",
            "Can I create customized portfolio reports ",
            "How to view portfolio participation statistics ",
            "Can I add teacher feedback in portfolio reports ",
            "How to track late submissions in portfolio report ",
            "Is there a graphical view of portfolio performance ",
            "How to download portfolio report cards ",
            "Can I filter reports by portfolio type ",
            "How to view portfolio activities over the semester ",
            "Can parents access portfolio reports ",
            "How do I show student reflections in reports ",
            "Can I generate comparative portfolio reports ",
            "How to track top portfolio submissions ",
            "Can I see rejected portfolios in reports ",
            "How to show the number of entries per student ",
            "Can I highlight exceptional portfolios in reports ",
            "How to display portfolio marks distribution ",
            "Can I track improvement across multiple portfolios ",
            "How to show comments in portfolio report card ",
            "Can portfolio reports be automatically emailed ",
            "Can I generate year-wise portfolio reports ",
            "How to identify top-performing students via portfolios ",
            "Can reports be scheduled to run automatically ",
            "How to check portfolio trends over time ",
            "Can I compare portfolio submissions between classes ",
            "How to view detailed student-level portfolio performance ",
            "Can I show portfolio improvement graphs ",
            "Can I merge multiple portfolios into one report ",
            "Can I check pending feedback in portfolio reports ",
            "How to analyze portfolio by submission type ",
            "How to create a heatmap of portfolio submissions ",
            "Can I track how often students update portfolios ",
            "How to tag portfolios in reports ",
            "Can I share portfolio reports with students ",
            "How to include awards or recognitions in portfolio reports ",
            "Can I exclude incomplete portfolios from reports ",
            "How to download section-wise portfolio reports ",
            "How to integrate portfolio reports with academic progress reports ",
            "How to view audit trails in portfolio reporting "
        ]
    },

        'transport': {
        'transport_master': [
            "How do I access the transport master ",
            "What details are stored in transport master ",
            "How can I edit transport master entries ",
            "Can I add a new route in transport master ",
            "What is the purpose of transport master ",
            "How to view all transport master entries ",
            "Can I delete a record from transport master ",
            "How do I manage vehicles in transport master ",
            "Where can I find bus routes in transport master ",
            "How to update transport details ",
            "Can I export transport master data ",
            "How do I link vehicles to routes in transport master ",
            "Is there a way to search in transport master ",
            "Can I filter buses in transport master ",
            "How to add driver information in transport master ",
            "How do I deactivate a transport entry ",
            "What is the process to update route details ",
            "Can I duplicate a transport entry ",
            "How do I add multiple transport entries at once ",
            "Can I see inactive transport records ",
            "How to assign boarding points to a route ",
            "How to allocate students via transport master ",
            "Can I map transport routes to vendors ",
            "Where can I find the bus driver information ",
            "How to check which buses are active ",
            "Can I print transport master data ",
            "How to check vehicle capacity in transport master ",
            "Can I generate a transport report from transport master ",
            "What are the fields available in transport master ",
            "How can I bulk update transport master ",
            "Where do I set timings for buses ",
            "How do I view a specific transport route ",
            "Is transport master connected to student allocation ",
            "How to set boarding sequence through transport master ",
            "Can I track bus maintenance via transport master ",
            "How do I import transport data ",
            "Can I export student transport list ",
            "How do I map students to transport master ",
            "Is there an approval process in transport master ",
            "Can I check assigned drivers in transport master ",
            "How to manage special buses in transport master ",
            "How to check route changes in transport master ",
            "Can I update bus fee details in transport master ",
            "How to view a driver's assigned route ",
            "How to filter routes based on distance ",
            "Can I set emergency contacts in transport master ",
            "How can I mark a bus out of service ",
            "How to add a substitute driver for a bus ",
            "Can I set maximum capacity per bus ",
            "How to map GPS devices in transport master "
        ],
        "route_master": [
			"Add a new transport route ",
			"Show me all existing transport routes ",
			"Can I update a transport route ",
			"How to delete a transport route ",
			"List all active transport routes ",
			"Create a new bus route ",
			"Modify route for school bus 4 ",
			"Search for route by name ",
			"Get all student routes ",
			"Can I copy route data ",
			"Assign a driver to a route ",
			"Update pickup and drop times ",
			"Rename an existing route ",
			"Deactivate an old transport route ",
			"How to merge two routes ",
			"Print transport route details ",
			"View route details for today ",
			"Filter routes by area ",
			"Assign stops to a route ",
			"Export route master to Excel ",
			"Sync route master with bus data ",
			"See full route path map ",
			"Import route data ",
			"Fix errors in route mapping ",
			"Does this route have enough capacity ",
			"Validate transport route ",
			"Route data not saving properly ",
			"Route master isn't opening ",
			"Can a student change routes ",
			"Set default route per class ",
			"Allow temporary route change ",
			"Route creation guide please ",
			"How do I handle overlapping routes ",
			"Who manages route assignments ",
			"Set distance for route ",
			"Route ID is missing ",
			"Print route assignment sheet ",
			"Track students per route ",
			"Auto-assign route based on address ",
			"Disable transport route for holidays ",
			"Route master validation errors ",
			"Add remarks to a route ",
			"Set route start and end time ",
			"Configure weekend route ",
			"Display only active routes ",
			"Is this route assigned to a bus ",
			"Which students are on this route ",
			"Show routes with issues ",
			"Generate transport route QR code ",
			"Route name already exists, help "
		],
		"stage_master": [
			"Add a new stage to a route ",
			"Edit stage details ",
			"Delete a transport stage ",
			"List all transport stages ",
			"Show stage list for Route A ",
			"How do I reorder the stages ",
			"Assign bus stop to a stage ",
			"Set distance for a stage ",
			"Update pickup time for stage ",
			"Can I copy stage from one route to another ",
			"Export stage data ",
			"Stage master entry guide ",
			"Assign stage to multiple buses ",
			"Print stage list ",
			"Stage not appearing in drop-down ",
			"Rename a transport stage ",
			"Track student count per stage ",
			"Fix wrong stage mapping ",
			"Search stage by location ",
			"Add GPS location to stage ",
			"Can a student change stage ",
			"Assign fees based on stage ",
			"Is stage linked to cost fee ",
			"Filter stages by zone ",
			"What are the stages for Route 3 ",
			"Set default stage for new admission ",
			"Stage form not saving ",
			"Stage master loading slowly ",
			"Bulk import stages ",
			"Create new stage template ",
			"Restrict access to stage master ",
			"Stage not showing in report ",
			"Mark stage as inactive ",
			"Duplicate stage name issue ",
			"View route from stage details ",
			"Assign route from stage screen ",
			"Show live data of stage usage ",
			"Is there a max stage limit ",
			"Who changed the stage order ",
			"Track attendance by stage ",
			"Disable pickup at this stage ",
			"Stage and boarding mismatch ",
			"Sort stage list alphabetically ",
			"Find stage closest to area ",
			"Are stage names unique ",
			"View all stage logs ",
			"Backup stage data ",
			"Link student with specific stage ",
			"How many buses use this stage "
		],
		"boarding_master": [
			"Create a new boarding point ",
			"Assign students to a boarding point ",
			"Edit boarding point details ",
			"Delete a boarding point ",
			"List all boarding points ",
			"Search boarding by name ",
			"Set boarding time ",
			"Link boarding to stage ",
			"Map boarding with bus ",
			"Update boarding location ",
			"Show boarding list per route ",
			"Which students use this boarding ",
			"Add remarks to boarding master ",
			"Fix wrong boarding allocation ",
			"Can a student have multiple boardings ",
			"Print boarding sheet ",
			"How do I change student boarding ",
			"Can I temporarily disable a boarding ",
			"Mark boarding as inactive ",
			"View boarding change history ",
			"Upload photo of boarding point ",
			"Track late boardings ",
			"Backup boarding data ",
			"Get boarding capacity ",
			"Auto-assign boarding on admission ",
			"Does boarding affect cost ",
			"How many students at this boarding ",
			"Is this boarding assigned to a stage ",
			"Sort boarding by zone ",
			"Assign fee group based on boarding ",
			"Show boarding on map ",
			"Generate QR for boarding point ",
			"Send alert for boarding delay ",
			"Create boarding schedule ",
			"Sync boarding with stage master ",
			"Lock boarding assignment ",
			"Edit boarding timing ",
			"Allow parent-requested boarding change ",
			"Set rules for boarding changes ",
			"Boarding master not opening ",
			"Show all active boardings ",
			"Find nearest boarding point ",
			"Create temporary boarding ",
			"Import boarding data ",
			"Export boarding data ",
			"Boarding report per class ",
			"Assign helper to boarding point ",
			"Link RFID to boarding "
		],
		"mapping_master": [
			"Create a new mapping ",
			"Map students to their routes ",
			"Mapping not working, fix it ",
			"Edit an existing mapping ",
			"Delete incorrect mapping ",
			"Bulk map students to transport ",
			"View mapping by class ",
			"Search student transport mapping ",
			"Can I remap a student ",
			"Assign mapping during admission ",
			"Mapping master user guide ",
			"Update mapping with new routes ",
			"Show mapping status ",
			"Export all mappings ",
			"Print mapping summary ",
			"Import mapping file ",
			"Lock mapping after finalizing ",
			"How to verify transport mapping ",
			"See live changes in mapping ",
			"Mapping auto-save not working ",
			"Duplicate mapping entries error ",
			"Student not found in mapping list ",
			"Undo last mapping action ",
			"Send mapping summary to parent ",
			"Route and stage mapping mismatch ",
			"Mapping not appearing in report ",
			"Map based on address zone ",
			"Mapping master is blank ",
			"Generate QR based on mapping ",
			"Create default mapping structure ",
			"Which teacher manages mappings ",
			"Mapping not saving correctly ",
			"Assign fee group during mapping ",
			"Route update not reflecting in mapping ",
			"Auto-suggest mapping options ",
			"Can mapping be time-based ",
			"Do mappings affect attendance ",
			"Set priority mapping rules ",
			"Sort mappings by grade ",
			"Mapping data backup option ",
			"Can mapping be done per term ",
			"Audit log of mapping changes ",
			"Restrict mapping to admin only ",
			"Student already mapped error ",
			"Map only active students ",
			"Which route is this student mapped to ",
			"Allow temporary mapping override ",
			"Get unmapped students report ",
			"Mapping by stage or boarding "
		],
		"cost_fee_master": [
			"Set transport cost per route ",
			"Edit cost for a specific stage ",
			"View all transport cost entries ",
			"How to calculate transport cost ",
			"Link cost to fee master ",
			"Add cost for boarding point ",
			"Assign fee amount based on distance ",
			"Export transport cost report ",
			"Can I adjust cost fee mid-year ",
			"Assign transport fee manually ",
			"How is cost calculated for students ",
			"Apply transport discount ",
			"Create fee slab for transport ",
			"Transport cost master error fix ",
			"Cost not reflecting in receipt ",
			"Configure cost rules by grade ",
			"Print cost sheet ",
			"Enable auto cost update ",
			"Check total transport revenue ",
			"Fee structure for transport ",
			"Assign fee category based on route ",
			"Are transport costs prorated ",
			"Apply transport fee only on weekdays ",
			"Cost fee master not loading ",
			"Set transport fee exemption ",
			"Route update not affecting cost ",
			"Generate cost breakdown ",
			"Student fee not showing transport cost ",
			"Bulk update cost entries ",
			"Import cost data ",
			"Cost based on attendance days ",
			"Monthly transport fee calculation ",
			"Annual cost setup for transport ",
			"Define peak season transport charges ",
			"Apply fuel surcharge in transport fee ",
			"Enable tax on transport cost ",
			"Can cost fee be refunded ",
			"Late payment charge on transport cost ",
			"Assign different costs to siblings ",
			"See old cost history ",
			"Change transport fee mid-session ",
			"Recalculate fees after route change ",
			"Who configures transport cost ",
			"Fee mismatch for transport ",
			"Multiple cost slabs for one route ",
			"Auto-adjust transport fee on leave ",
			"Assign transport fee during promotion ",
			"Does transport fee vary by class ",
			"Show cost details to parents "
		],
        'vendor_master': [
            "How do I access vendor master ",
            "What details are stored in vendor master ",
            "How can I add a new vendor ",
            "Can I edit vendor information ",
            "How do I delete a vendor record ",
            "What is the purpose of vendor master ",
            "Can I upload vendor documents ",
            "How do I view all vendors ",
            "Where can I find vendor contact details ",
            "Can I search vendors by name ",
            "How do I update a vendor's address ",
            "How to link vendors with transport routes ",
            "Is it possible to deactivate a vendor ",
            "How can I add multiple vendors at once ",
            "Can I export vendor master data ",
            "How do I assign buses to vendors ",
            "Can I see active and inactive vendors separately ",
            "How to set vendor payment terms ",
            "How to add vendor license information ",
            "Can I track vendor performance ",
            "How do I change a vendor's service area ",
            "Is there a vendor rating option ",
            "Can I bulk update vendor records ",
            "How do I add emergency contact details for vendors ",
            "Can vendors be assigned to multiple routes ",
            "How to check vendor contract expiry dates ",
            "Can I upload vendor agreements ",
            "How to manage vendor renewals ",
            "Is it possible to blacklist a vendor ",
            "How do I view vendor transaction history ",
            "Where can I store vendor insurance details ",
            "How to send notifications to vendors ",
            "How do I verify a vendor’s compliance documents ",
            "Can I print vendor master data ",
            "Is vendor approval required before assignment ",
            "Can I archive old vendor records ",
            "How to assign backup vendors for a route ",
            "How can I manage vendor penalties ",
            "Can I set multiple contact numbers for vendors ",
            "How to update vendor GST information ",
            "Can vendors be associated with specific shifts ",
            "How to view vendors linked to a vehicle ",
            "How to generate a report of all vendors ",
            "How to check overdue payments to vendors ",
            "Can vendors be evaluated on performance ",
            "How to manage vendor complaints ",
            "Is vendor reassignment possible ",
            "How to maintain alternate vendor lists ",
            "Can I view vendor service history ",
            "How to link vendor services to cost centers "
        ],
        'vehicle_master': [
            "How do I add a new vehicle ",
            "Where can I view all vehicle details ",
            "How to update vehicle information ",
            "Can I delete a vehicle record ",
            "What data is stored in vehicle master ",
            "How to upload vehicle registration documents ",
            "Can I assign drivers to vehicles ",
            "How to track vehicle insurance expiry ",
            "How do I record vehicle maintenance ",
            "Can I search vehicles by number ",
            "How to assign vehicles to routes ",
            "Can I deactivate a vehicle temporarily ",
            "How do I update vehicle capacity ",
            "Is there a way to bulk add vehicles ",
            "How to export vehicle master data ",
            "How to check vehicle compliance status ",
            "Where to find vehicle fitness certificate ",
            "How to add vehicle pollution certificate ",
            "Can I manage vehicle ownership details ",
            "How do I set vehicle tracking devices ",
            "Is it possible to map vehicle routes ",
            "How to view active and inactive vehicles ",
            "Can I link vehicle to a vendor ",
            "How to store vehicle purchase date ",
            "How to record vehicle insurance provider ",
            "Where can I upload road tax documents ",
            "How do I manage vehicle loan details ",
            "Can I view a vehicle service history ",
            "How to check vehicle warranty status ",
            "Can I maintain vehicle fuel records ",
            "How to assign multiple drivers to one vehicle ",
            "Is vehicle maintenance schedule available ",
            "How to mark a vehicle under repair ",
            "Can I keep emergency tools list per vehicle ",
            "How to set vehicle inspection reminders ",
            "Can I track fuel efficiency of vehicles ",
            "Is it possible to attach documents to vehicles ",
            "How to manage vehicle-related fines ",
            "Can vehicles be reassigned between routes ",
            "How to verify vehicle ownership certificates ",
            "How do I view vehicles due for servicing ",
            "How to manage vehicle spare parts inventory ",
            "Is there a report for vehicle wise expenses ",
            "How to view expired insurance vehicles ",
            "Can I monitor vehicle running status ",
            "How do I check seating capacity per vehicle ",
            "Is there a provision to tag vehicle types ",
            "Can I assign specific shifts to vehicles ",
            "How to record vehicle accidents ",
            "Can I view fuel expense per vehicle "
        ],
        'boarding_sequence': [
            "How do I set the boarding sequence ",
            "Where can I edit boarding points ",
            "Can I rearrange the boarding order ",
            "How to assign students to boarding points ",
            "Is it possible to add new stops ",
            "How do I delete a boarding point ",
            "Can I update the pickup and drop sequence ",
            "Where to view all boarding sequences ",
            "How to link boarding sequence to vehicle routes ",
            "How can I change the order of boarding ",
            "Is there a way to bulk upload boarding points ",
            "Can I print the boarding sequence list ",
            "How to manage boarding sequence timings ",
            "Is it possible to group nearby stops ",
            "Can students have different boarding points for pickup and drop ",
            "How do I track stop-wise student allocation ",
            "Can I set boarding points by class or section ",
            "How to view students per boarding point ",
            "Can boarding sequences vary by day ",
            "How do I activate or deactivate stops ",
            "Is it possible to color code boarding points ",
            "Where can I assign default boarding points ",
            "Can I manage boarding points for special routes ",
            "How to adjust the sequence during exams ",
            "Can I have multiple boarding sequences for the same vehicle ",
            "How to prioritize main stops in sequence ",
            "Where can I configure distance between stops ",
            "Can I mark temporary stops for events ",
            "Is there a report for boarding point details ",
            "How to manage pickup points for staff ",
            "Can I update boarding sequence for each session ",
            "How do I set sequence based on timing ",
            "Can the boarding points auto-update ",
            "How to handle emergency changes in stops ",
            "Is student feedback considered for boarding updates ",
            "How often should I review boarding sequences ",
            "Can I create boarding groups for trips ",
            "Is GPS tracking linked to boarding sequence ",
            "How can I tag students who change stops frequently ",
            "Can I assign temporary boarding points ",
            "How do I audit changes in boarding sequences ",
            "Is boarding sequence approval needed ",
            "How to integrate boarding with attendance ",
            "How to manage boarding points for late comers ",
            "Can I check boarding density at each stop ",
            "How to synchronize boarding with vehicle schedules ",
            "Is there an option to archive old sequences ",
            "Can I send notifications for updated boarding points ",
            "How do I set maximum students per boarding stop ",
            "Can I export the boarding sequence data "
        ],
        'student_allocation': [
            "How do I allocate students to buses ",
            "Can I assign a student to a transport route ",
            "Where to manage student transport allocation ",
            "How to bulk allocate students to transport ",
            "Can a student change their allocated bus ",
            "How to view students assigned to a vehicle ",
            "Is there a report for student bus allocation ",
            "How do I allocate based on boarding points ",
            "Can I filter students without bus allocation ",
            "How to reassign students to a new vehicle ",
            "Is it possible to allocate different pickup and drop routes ",
            "How to update allocation if the student shifts residence ",
            "Can students opt out of transport after allocation ",
            "How to set default buses for students ",
            "Can I allocate students class-wise to buses ",
            "Is there a limit on students per bus ",
            "How to view vacant seats in a vehicle before allocation ",
            "Can allocation change during mid-term ",
            "Where can parents see student transport allocation ",
            "How to assign special transport for events ",
            "Can I allocate staff with students in the same vehicle ",
            "How to check previous year’s student allocations ",
            "Is auto-allocation possible based on address ",
            "How to import student allocation data ",
            "Can I generate allocation slips for students ",
            "How to prioritize allocation for distant students ",
            "Is there a waiting list for transport allocation ",
            "Can I manually adjust seat numbers for students ",
            "Where to edit incorrect student allocations ",
            "Can students have different morning and evening routes ",
            "How to mark students as transport users or non-users ",
            "Can transport fee depend on student allocation ",
            "How to suspend transport allocation temporarily ",
            "Can students opt for transport mid-session ",
            "How to allocate siblings to the same vehicle ",
            "Can I segregate boys and girls allocation ",
            "Where can I mass update student allocations ",
            "How to set transport zones for allocation ",
            "Can I allocate students based on distance slabs ",
            "How to deal with over-allocated buses ",
            "Is there an alert for allocation conflicts ",
            "Can I allocate students directly from their profile ",
            "How to track changes in student allocation ",
            "Can boarding points be changed during allocation ",
            "Is dynamic allocation available for different days ",
            "How to handle disabled students' transport allocation ",
            "Can I add comments during student allocation ",
            "How to cancel allocation if student leaves school ",
            "Can student allocation impact attendance module ",
            "Is it possible to allocate transport based on hostel location "
        ],
        'attendance': [
            "How do I mark attendance for transport users ",
            "Can I take attendance for students using transport ",
            "Where can I view transport attendance for the day ",
            "Can transport attendance be marked based on routes ",
            "How to mark attendance for students who missed the bus ",
            "Is there a report for transport attendance ",
            "How to track student attendance using transport ",
            "Can I mark attendance for both morning and evening routes ",
            "Can I automate transport attendance based on vehicle tracking ",
            "How to mark a student's absence in transport ",
            "Can transport attendance be integrated with the main attendance system ",
            "How to mark attendance for students on field trips ",
            "Is there a system to confirm if a student boarded the bus ",
            "Can I mark transport attendance before or after school hours ",
            "How to adjust attendance if a student missed the bus ",
            "Where to find the summary of transport attendance ",
            "Can I mark attendance for students based on route stops ",
            "How to update attendance if a student is dropped off early ",
            "Can I generate transport attendance reports for specific dates ",
            "How do I know if a student boarded the bus today ",
            "Is there an option to mark attendance for absent students manually ",
            "How to view transport attendance for the entire month ",
            "How to verify transport attendance with GPS tracking ",
            "Can I mark attendance for bus assistants too ",
            "How to handle students who switch buses for the day ",
            "Can attendance be marked for students on sick leave ",
            "How to track students who are late for the bus ",
            "How do I import attendance data for transport ",
            "How to set rules for marking transport attendance ",
            "Can I set up a system to notify if a student misses transport ",
            "How do I check if a student used transport after school ",
            "How do I mark attendance for students in different routes ",
            "Can I set an alert if a student is marked absent in transport ",
            "Is there a way to track transport attendance in real-time ",
            "Can parents view their child's transport attendance ",
            "How to update attendance records in case of transport delays ",
            "Can attendance be auto-marked if a student is on the bus ",
            "How do I mark attendance for students taking different transport routes ",
            "How to mark a student's attendance if they traveled by car today ",
            "Can I view the attendance for each vehicle separately ",
            "How to view reports for attendance of different buses ",
            "Can I download transport attendance reports in Excel format ",
            "How to manage transport attendance for holidays or non-school days ",
            "Can I add notes for transport attendance for specific days ",
            "Can I generate reports for bus attendance based on routes ",
            "How do I handle attendance if the bus has an unexpected breakdown ",
            "How to link transport attendance with academic attendance "
        ],
        'vehicle_maintenance': [
            "How do I log vehicle maintenance for a bus ",
            "Can I schedule maintenance for a specific vehicle ",
            "How to check the maintenance status of a vehicle ",
            "Can I set reminders for vehicle servicing ",
            "What are the maintenance intervals for a school bus ",
            "How do I request vehicle maintenance ",
            "Where can I find the maintenance history of a vehicle ",
            "Can I update vehicle maintenance records ",
            "How do I track if a bus needs repair ",
            "Can I schedule regular inspections for school vehicles ",
            "How to add a maintenance request for a specific vehicle ",
            "How do I report an issue with a school bus ",
            "Can I assign a mechanic to handle vehicle maintenance ",
            "Where do I find vehicle maintenance reports ",
            "How do I know when a vehicle is due for servicing ",
            "Is there a system to track vehicle repairs ",
            "Can I set an alert when vehicle maintenance is due ",
            "How to track parts replaced during vehicle maintenance ",
            "How to manage vehicle service history ",
            "Can I track fuel consumption and maintenance together ",
            "How do I update the vehicle maintenance schedule ",
            "Can I view upcoming maintenance tasks for school vehicles ",
            "How to manage unexpected repairs for vehicles ",
            "How do I schedule emergency maintenance for a vehicle ",
            "Can I view the list of vehicles needing maintenance ",
            "How do I log the maintenance work completed on a bus ",
            "What information is required for vehicle maintenance ",
            "Can I view the cost of repairs for each vehicle ",
            "How to keep track of spare parts used in vehicle maintenance ",
            "Can I generate maintenance reports for vehicles ",
            "How do I track maintenance costs for school vehicles ",
            "Can I attach photos of vehicle damage for maintenance records ",
            "How do I approve maintenance requests for vehicles ",
            "Can I view maintenance tasks by vehicle type ",
            "How do I assign a priority level to vehicle repairs ",
            "How do I know if a vehicle is ready for use after maintenance ",
            "How to handle maintenance scheduling conflicts ",
            "Can I access maintenance logs for older vehicles ",
            "How to check if a vehicle's warranty covers repairs ",
            "Can I send reminders to drivers for vehicle check-ups ",
            "How do I ensure the safety of vehicles after maintenance ",
            "Can I create recurring maintenance tasks for vehicles ",
            "How to maintain the maintenance records of vehicles for audits ",
            "How to log vehicle accidents that require maintenance ",
            "Can I track the frequency of vehicle breakdowns ",
            "How do I monitor the performance of vehicles post-repair ",
            "Can I schedule maintenance for a vehicle during off-peak hours ",
            "Can I integrate vehicle maintenance with fuel usage tracking "
        ],
        'student_move_next_year': [
            "How do I move a student to the next year ",
            "Can I transfer students to the next academic year ",
            "What is the process for promoting students to the next grade ",
            "How to move students who have passed to the next year ",
            "Can I manually promote a student to the next year ",
            "What happens to the student's records when they move to the next year ",
            "Is there an option to automatically promote students ",
            "Can I bulk move students to the next year ",
            "How do I check if a student is eligible for promotion ",
            "Can I change the student's grade level for the next year ",
            "What information is required to move a student to the next year ",
            "How do I handle students who repeat the same grade ",
            "Is there a report for students promoted to the next year ",
            "How can I track students who have been moved to the next year ",
            "How do I manage the transition for students moving to the next year ",
            "Can I confirm the student’s promotion status before moving them ",
            "Can I undo the action if a student is mistakenly moved to the next year ",
            "How do I update the student’s class and section for the next year ",
            "Can I use a list to promote multiple students at once ",
            "Is there a way to check the promotion history for a student ",
            "Can I set criteria for student promotions to the next year ",
            "How do I check if the student has met all the promotion requirements ",
            "Can I add new subjects for the student when they move to the next year ",
            "How do I ensure the student’s performance is updated for the next year ",
            "What happens to the student’s fees when they move to the next year ",
            "Can I transfer students from one class to another when they are promoted ",
            "Can I edit student records before moving them to the next year ",
            "How do I move a student to a different stream in the next year ",
            "How do I handle students who are transferring from another school to the next year ",
            "Can I assign new teachers to students when they move to the next year ",
            "How do I move students who are being transferred to a different campus ",
            "Can I transfer a student to a new school year if they are not passing ",
            "Can I send notifications to parents when their child is moved to the next year ",
            "How do I archive student records after they move to the next year ",
            "What happens to the student’s extracurricular activities when they are promoted ",
            "Can I track students’ performance through a dashboard after moving to the next year ",
            "How do I update student addresses when they move to the next year ",
            "Can I keep track of the student’s medical history when they are moved to the next year ",
            "How can I integrate the promotion process with the school’s other systems ",
            "What is the deadline for moving students to the next academic year ",
            "Can I bulk upload student promotions for the next year ",
            "How do I handle student’s elective subject choices when moving to the next year ",
            "Can I check the prerequisites for moving students to the next year ",
            "What steps should I follow to promote students to the next grade ",
            "Can I assign students to their respective classes after moving to the next year ",
            "How do I notify the class teacher about the promotion ",
            "Can I generate a list of students who need to be moved to the next year ",
            "How can I view students that have been moved to the next year in a report "
        ],
        'reports': [
            "How can I generate a transport report ",
            "Can I get a summary of the attendance report for transport ",
            "What is included in the vehicle route summary report ",
            "How do I check the vehicle information in the report ",
            "Can I get a transporter report for the last month ",
            "How can I track vehicle due payments through a report ",
            "Where can I find the route-wise student report ",
            "Can I generate an MIS report for transport ",
            "How do I check the driver's list from the transport report ",
            "Can I view the class-wise bus student report ",
            "Where can I find the vehicle capacity report ",
            "How do I access the vehicle expense report ",
            "Is there a way to export the transport report to Excel ",
            "Can I filter reports based on specific vehicle routes ",
            "How do I analyze transport expenses from the reports ",
            "Can I view the attendance data of all students in the transport report ",
            "Is there a report that shows the transport schedule for the next month ",
            "Can I get a detailed report on transport expenses ",
            "How do I track the maintenance history of vehicles in the report ",
            "Can I generate a report of all the students allocated to vehicles ",
            "Is it possible to get a historical vehicle route summary ",
            "Can I generate a report to show vehicle maintenance costs ",
            "How can I filter the attendance report by date ",
            "Can I track the fuel consumption in the vehicle report ",
            "Is there a way to view reports based on the driver's performance ",
            "How do I view the class-wise transport allocation for students ",
            "Can I print the transport reports for physical records ",
            "How do I analyze transport performance across routes ",
            "Can I view a report of all students who used transport last week ",
            "Where can I find the transport reports for the past year ",
            "How can I get a report showing the monthly collection summary ",
            "Can I access a report for transport vendor payments ",
            "How can I get a breakdown of transport costs per route ",
            "Can I get a report that includes all vehicles' maintenance records ",
            "Is there a way to visualize transport data in a graph ",
            "Can I see a list of vehicles due for servicing in the report ",
            "How do I generate a report for driver performance statistics ",
            "Can I create a custom report for transport expenses ",
            "How do I get the report for students' transport attendance for a specific month ",
            "Is there a report that lists all vehicles' current status ",
            "Can I filter the vehicle information summary report by vehicle type ",
            "How can I analyze the usage of each transport route in a report ",
            "Can I get an overview of all the transport routes in a report ",
            "Where can I view detailed transport reports for different bus routes ",
            "Can I generate a report showing the total distance covered by each vehicle ",
            "Can I generate a driver attendance report along with the transport report ",
            "How can I view transport reports by specific time slots ",
            "Can I create a detailed report on transport vehicle insurance "
        ],
        'attendance_report': [
            "How can I generate a transport attendance report ",
            "Can I get a report of student attendance for transport ",
            "What is included in the transport attendance report ",
            "Can I see the attendance details for each student in the report ",
            "How do I filter attendance data in the transport report ",
            "Can I get a summary of student attendance for the last month ",
            "How can I track daily student attendance in transport ",
            "Is it possible to view the attendance percentage in transport reports ",
            "Can I generate a report of students who were absent from the transport ",
            "How do I generate a report showing transport attendance trends ",
            "Can I generate a detailed transport attendance report by route ",
            "Can I download the attendance report in Excel format ",
            "Can I filter the attendance report based on date ",
            "How do I export the transport attendance report ",
            "How can I check the number of students absent in transport ",
            "Can I generate the attendance report for specific transport routes ",
            "Is there a way to visualize transport attendance in a graph ",
            "Can I generate a monthly transport attendance report ",
            "How do I get the attendance data of students for a specific month ",
            "Can I track transport attendance for individual vehicles ",
            "How do I find the attendance report for students using transport on holidays ",
            "Can I get a detailed report on students who were absent repeatedly ",
            "How do I see the attendance status of students on different routes ",
            "Can I track the attendance status of a student over time ",
            "Is there a way to include the student names in the transport attendance report ",
            "Can I generate a report that shows absenteeism trends over months ",
            "Can I filter the attendance report by bus route ",
            "How do I view the total number of absentees for each vehicle ",
            "Can I create a custom transport attendance report for specific dates ",
            "How can I check if a student attended the transport service regularly ",
            "Can I view the attendance report for students during holidays ",
            "Can I generate an absentee report for students using transport ",
            "How do I check which bus routes had the highest attendance ",
            "Can I compare attendance reports for different months ",
            "Can I include transport schedules in the attendance report ",
            "How do I view the attendance data for the entire transport system ",
            "Can I track the attendance of students by vehicle number ",
            "Is it possible to add remarks to the transport attendance report ",
            "How do I track attendance for specific student categories ",
            "Can I filter the attendance report by vehicle type ",
            "How do I view transport attendance on a weekly basis ",
            "Can I generate a report for the attendance of students for the current year ",
            "Can I compare transport attendance across different routes "
        ],
        'vehicle_route_summary': [
            "How can I generate a summary of vehicle routes ",
            "Can I get a summary of the routes taken by the vehicles ",
            "What is included in the vehicle route summary report ",
            "How do I filter the vehicle routes in the summary ",
            "Can I see the number of students assigned to each vehicle route ",
            "How do I check the vehicle usage across different routes ",
            "Is there a way to compare routes based on vehicle capacity ",
            "Can I see the details of each vehicle's assigned route ",
            "How do I track the vehicle routes for the current month ",
            "Can I view the list of vehicles on specific routes ",
            "Can I get a summary report of routes with the highest student count ",
            "How do I filter the vehicle routes by bus type ",
            "Can I generate a report showing the distance covered by each vehicle ",
            "How can I check the assigned routes for each vehicle ",
            "Can I track vehicle route changes in the report ",
            "Is there a way to visualize vehicle routes on a map in the summary ",
            "How do I get the vehicle route summary for the previous week ",
            "Can I generate a report for the routes taken by vehicles on a specific day ",
            "How can I check which vehicle is assigned to which route ",
            "Can I get a detailed breakdown of vehicle routes by school district ",
            "Can I see the vehicle route summary for the last semester ",
            "Can I generate a summary of vehicle routes based on student demographics ",
            "How do I find the vehicle route that services the highest number of students ",
            "Can I include the driver's name in the vehicle route summary ",
            "Can I check if there are any delays in the vehicle routes ",
            "How do I check the total mileage of all vehicle routes ",
            "Can I generate a report that shows vehicle routes and their maintenance status ",
            "How do I get the average time spent by vehicles on each route ",
            "Can I compare vehicle route summaries across different time periods ",
            "Is there a way to track vehicle routes with the least student occupancy ",
            "How can I view the route summary for the upcoming week ",
            "Can I filter the route summary based on specific vehicles ",
            "Can I view a graphical representation of the vehicle routes ",
            "How do I track vehicle routes for specific regions or areas ",
            "Can I get a report showing the most frequently used vehicle routes ",
            "How do I generate a report showing the distance covered by vehicles per route ",
            "Can I see the vehicle route summary for a particular time of day ",
            "How do I track vehicle performance on different routes ",
            "Can I generate a report showing the average time for each vehicle on each route ",
            "Can I filter the vehicle route summary based on the school year ",
            "How do I view the route assignment history for vehicles ",
            "Can I generate a report for vehicle routes with the highest fuel consumption ",
            "Can I track the status of vehicles on different routes in the summary ",
            "How do I generate a report that compares vehicle routes for the current month ",
            "Can I include vehicle maintenance records in the route summary ",
            "Can I track the number of students who use a particular vehicle route ",
            "How do I generate a summary that shows vehicle routes by day of the week ",
            "Can I compare vehicle routes by cost and distance "
        ],
        'vehicle_information_summary': [
            "How can I generate a summary of vehicle information ",
            "Can I get a report on the vehicle details ",
            "What information does the vehicle information summary include ",
            "How do I filter the vehicle information by vehicle type ",
            "Can I see a list of all vehicles with their details ",
            "How do I get information on the vehicle's registration status ",
            "Can I get the summary of vehicle details including make and model ",
            "How do I track the maintenance history of all vehicles ",
            "Can I see a summary of vehicle insurance details ",
            "How do I get a list of vehicles with their current operational status ",
            "Can I check the seating capacity of all vehicles in the summary ",
            "How can I track the fuel efficiency of each vehicle in the report ",
            "Can I get the total number of vehicles by type in the summary ",
            "How do I get details about each vehicle's driver assignment ",
            "Can I view the vehicle's maintenance due dates in the summary ",
            "How do I track the warranty details for each vehicle ",
            "Can I get a report showing the vehicle's last service date ",
            "How do I check the vehicle's condition and current status ",
            "Can I filter the summary by vehicles that require maintenance ",
            "How do I get a summary of vehicles with expired insurance ",
            "Can I view the list of vehicles in use and their assigned routes ",
            "How can I track the fleet's total mileage in the report ",
            "Can I generate a report showing vehicles due for inspection ",
            "How do I view the detailed specifications of each vehicle ",
            "Can I get a summary of vehicle registration numbers ",
            "How do I check the fuel consumption for each vehicle ",
            "Can I see the total number of vehicles assigned to specific routes ",
            "How do I generate a report showing the vehicle's service cost ",
            "Can I get the details of vehicles that have been sold or retired ",
            "How can I track the age of each vehicle in the fleet ",
            "Can I generate a report for vehicles that are currently not in service ",
            "How do I get a summary of vehicles based on their registration type ",
            "Can I see the list of vehicles and their last inspection date ",
            "How do I track the current status of vehicles in the fleet ",
            "Can I generate a report showing the fuel type for each vehicle ",
            "How can I view the operational hours for each vehicle in the fleet ",
            "Can I get a summary of vehicles that require insurance renewal ",
            "How do I filter vehicles that need to be replaced soon ",
            "Can I track the tire condition for each vehicle ",
            "Can I get the history of accidents or repairs for each vehicle ",
            "How do I track vehicle expenses for each vehicle in the fleet ",
            "Can I view the maximum load capacity for each vehicle ",
            "How do I get a summary of vehicles by their registration year ",
            "Can I get the list of vehicles by their usage type ",
            "How can I see a report on the maintenance costs per vehicle ",
            "Can I generate a report that shows the lifespan of each vehicle ",
            "How do I track the number of breakdowns for each vehicle ",
            "Can I generate a report showing the vehicle ownership details ",
            "How do I view the vehicle's fuel capacity in the summary ",
            "Can I track the vehicle usage hours for each vehicle "
        ],
        'transporter_report': [
            "Can I generate a transporter report for all vehicles ",
            "How do I get a detailed report for the transporters ",
            "What details are included in the transporter report ",
            "Can I see a report that lists all transporters in the system ",
            "How do I filter the transporter report by vehicle type ",
            "Can I generate a report showing transporters' contact details ",
            "How do I get a list of transporters by region ",
            "Can I see the transporter report based on vehicle usage ",
            "Can I track the transporter's payment details in the report ",
            "How do I get a transporter report showing assigned routes ",
            "Can I generate a report showing the transporter's performance ",
            "How do I view the transporter's fleet size in the report ",
            "Can I track transporters' delivery times in the report ",
            "How do I filter the transporter report by active transporters ",
            "Can I see a list of transporters and their vehicle assignments ",
            "How do I generate a report on transporters with pending payments ",
            "Can I track transporters' compliance status in the report ",
            "How do I see the transporter rating and feedback in the report ",
            "Can I generate a report of all transporters with expired contracts ",
            "How do I view the transporter service history in the report ",
            "Can I track the revenue generated by each transporter in the report ",
            "How do I get a list of transporters with overdue deliveries ",
            "Can I generate a report showing transporters' fleet maintenance history ",
            "How do I track transporters' insurance status in the report ",
            "Can I view the transporter's performance based on delivery speed ",
            "How do I generate a transporter report based on service ratings ",
            "Can I see the total distance covered by each transporter in the report ",
            "How do I view transporters' contract expiration dates in the report ",
            "Can I generate a report showing the transporter's payment history ",
            "How do I track transporters' fleet utilization in the report ",
            "Can I see a transporter report for a specific time period ",
            "How do I track the transporter's service frequency in the report ",
            "Can I filter the transporter report by service type ",
            "How do I generate a report showing transporters' insurance costs ",
            "Can I track transporters' fuel consumption in the report ",
            "How do I view transporters' vehicle maintenance costs in the report ",
            "Can I generate a report showing transporters' total earnings ",
            "How do I get a report on transporters by their payment method ",
            "Can I track transporters' service violations in the report ",
            "How do I filter the transporter report by active contracts ",
            "Can I view a list of transporters based on vehicle availability ",
            "How do I get a report showing transporters' cost per trip ",
            "Can I track transporters' delivery deadlines in the report ",
            "How do I generate a report showing transporters' operational hours ",
            "Can I filter the transporter report by service reliability ",
            "How do I track transporters' vehicle types in the report ",
            "Can I generate a report showing the transporter's area of operation ",
            "How do I view transporters' legal compliance status in the report ",
            "Can I track transporters' driver performance in the report "
        ],
        'vehicle_due_report': [
            "Can I generate a report of all vehicles with dues ",
            "How do I view the list of vehicles with pending dues ",
            "What details are included in the vehicle due report ",
            "Can I see a report of vehicles with overdue payments ",
            "How do I filter the vehicle due report by due amount ",
            "Can I track the payment due dates in the vehicle due report ",
            "How do I generate a report for vehicles with outstanding payments ",
            "Can I see a list of vehicles and their payment status ",
            "How do I track the due amount for each vehicle ",
            "Can I generate a report showing vehicles with late payments ",
            "How do I filter the report by vehicles with high due amounts ",
            "Can I view a report showing the vehicle dues for a specific time period ",
            "How do I see which vehicles have the highest pending dues ",
            "Can I track vehicles' payment due dates in the report ",
            "How do I generate a vehicle due report for the last month ",
            "Can I generate a report showing due amounts by vehicle type ",
            "How do I see which vehicles have overdue dues ",
            "Can I track the due payments for each vehicle in the report ",
            "How do I get a report on vehicles with unpaid dues ",
            "Can I filter the vehicle due report by due payment status ",
            "How do I see the total dues for all vehicles in the system ",
            "Can I generate a report showing overdue dues by vehicle owner ",
            "How do I view a report of vehicles with unresolved dues ",
            "Can I generate a report showing vehicles with overdue maintenance payments ",
            "How do I track the payment due dates for vehicle maintenance ",
            "Can I see the outstanding dues for vehicles with expired insurance ",
            "How do I track overdue insurance payments for vehicles ",
            "Can I view vehicles with unpaid road tax in the report ",
            "How do I generate a vehicle due report based on overdue road tax ",
            "Can I track the overdue maintenance costs for each vehicle ",
            "How do I view pending vehicle registration payments in the report ",
            "Can I generate a report showing vehicles with unpaid registration fees ",
            "How do I track vehicle insurance payment dues ",
            "Can I see a list of vehicles with pending fines in the report ",
            "How do I track overdue parking fines in the vehicle due report ",
            "Can I view vehicles with outstanding toll fees in the report ",
            "How do I track the vehicle payment history in the due report ",
            "Can I generate a report of vehicles with overdue toll payments ",
            "How do I filter the vehicle due report by payment method ",
            "Can I generate a report showing overdue vehicle dues based on service ",
            "How do I get a report showing which vehicles are overdue for maintenance ",
            "Can I track the due payments for vehicle fuel charges ",
            "How do I generate a report for vehicles with outstanding fuel payments ",
            "Can I see a vehicle due report based on overdue taxes ",
            "How do I track overdue tax payments for each vehicle ",
            "Can I generate a report showing overdue dues for vehicle service contracts ",
            "How do I see the vehicles with delayed service payment in the report ",
            "Can I track vehicles' due payments by owner in the report ",
            "How do I filter the report by vehicles with high overdue payments ",
            "Can I generate a report for vehicles that have dues from the past 6 months ",
            "How do I track the total overdue dues across all vehicles in the system "
        ],
        'routewise_student_report': [
            "Can I generate a route-wise student report ",
            "How do I see the student details for each route ",
            "Can I get a list of students grouped by their transport route ",
            "How do I filter the route-wise student report by route ",
            "Can I view the student distribution across different bus routes ",
            "How do I get a report showing students assigned to specific routes ",
            "Can I generate a report showing the students per route for today ",
            "How do I see the student count for each transport route ",
            "Can I get a route-wise student count report ",
            "How do I see a detailed list of students for each vehicle route ",
            "Can I filter the report by students traveling on specific bus routes ",
            "How do I get a report showing all students assigned to route 5 ",
            "Can I view the list of students for multiple routes at once ",
            "How do I see which students are assigned to bus route 7 ",
            "Can I track the number of students per route for the entire month ",
            "How do I generate a report showing students on route 3 ",
            "Can I get a route-wise report showing student names ",
            "How do I see the bus route assignments for students ",
            "Can I generate a route-wise report including student pickup locations ",
            "How do I see the assigned bus route for each student ",
            "Can I get a route-wise student attendance report ",
            "How do I track student attendance by bus route ",
            "Can I get a route-wise report with students’ contact details ",
            "How do I generate a report showing students’ route assignments by grade ",
            "Can I filter the report to show students by route and bus stop ",
            "How do I view the student distribution by route for the academic year ",
            "Can I generate a route-wise report including student fees ",
            "How do I see the fee status of students by their route ",
            "Can I track the number of students per route with pending fees ",
            "How do I get a route-wise report including the student’s pickup time ",
            "Can I generate a route-wise student report based on school branch ",
            "How do I track the routes assigned to students for different grades ",
            "Can I view the student assignments for routes on a specific date ",
            "How do I see the historical route assignments for students ",
            "Can I get a report that shows all students assigned to route 10 ",
            "How do I generate a report by route and stop time for each student ",
            "Can I see the number of students assigned to each route for the semester ",
            "How do I generate a route-wise report showing students with specific needs ",
            "Can I track students with disabilities in the route-wise report ",
            "How do I see which students have opted for a specific route ",
            "Can I generate a report with detailed route-wise student data ",
            "How do I view the student count for each route in a specific month ",
            "Can I filter the route-wise report to show only active students ",
            "How do I generate a report showing the students who travel on route 4 ",
            "Can I see the contact information for students in the route-wise report ",
            "How do I track the student location by route in the report ",
            "Can I get a detailed breakdown of students assigned to specific routes ",
            "How do I see a route-wise report of students who need special transport arrangements ",
            "Can I track the transportation status of students by route in the report ",
            "How do I see the list of students who switched routes recently ",
            "Can I generate a report showing students who missed their assigned transport route ",
            "How do I view the transport details for students traveling on specific routes "
        ],
        'mis_report': [
            "Can I generate an MIS report for transport ",
            "How do I create a report for transport management system ",
            "Can I see a detailed MIS report for the transport department ",
            "How do I generate an MIS report for vehicle operations ",
            "Can I get an MIS report that includes transport usage statistics ",
            "How do I view the overall transport system performance in the MIS report ",
            "Can I track the number of students assigned to each route in the MIS report ",
            "How do I generate an MIS report for vehicle maintenance status ",
            "Can I view the transport-related expenses in an MIS report ",
            "How do I track transport operational costs in the MIS report ",
            "Can I generate an MIS report showing student transport attendance ",
            "How do I get an overview of all transport activities in an MIS report ",
            "Can I include student transportation fees in the MIS report ",
            "How do I generate an MIS report with vehicle route details ",
            "Can I filter the MIS report by specific vehicle types ",
            "How do I view a detailed report of transport vendor performance in the MIS report ",
            "Can I generate an MIS report showing the status of vehicle repairs ",
            "How do I view the utilization rate of each transport vehicle in the MIS report ",
            "Can I track vehicle route efficiency in the MIS report ",
            "How do I view the fuel consumption details for each vehicle in the MIS report ",
            "Can I see the maintenance cost of each vehicle in the MIS report ",
            "How do I generate an MIS report showing the transport department’s budget ",
            "Can I track transportation expenses per route in the MIS report ",
            "How do I view the student count per bus in the MIS report ",
            "Can I see which buses have the highest student occupancy in the MIS report ",
            "How do I generate an MIS report that shows the number of students per route ",
            "Can I track the condition of each transport vehicle in the MIS report ",
            "How do I generate an MIS report for transport infrastructure ",
            "Can I see vehicle route-wise occupancy in the MIS report ",
            "How do I track transport vendor payments in the MIS report ",
            "Can I get a summary of all transportation services in the MIS report ",
            "How do I view a monthly summary of transport operations in the MIS report ",
            "Can I generate an MIS report showing the status of pending transport issues ",
            "How do I track vehicle maintenance history in the MIS report ",
            "Can I include transport-related complaints in the MIS report ",
            "How do I view the total revenue from transportation services in the MIS report ",
            "Can I track transport service quality metrics in the MIS report ",
            "How do I view the performance of each transport vehicle in the MIS report ",
            "Can I see a detailed breakdown of transport route expenses in the MIS report ",
            "How do I generate an MIS report to analyze transport service efficiency ",
            "Can I see the total number of students assigned to transport routes in the MIS report ",
            "How do I view the transport system’s budget allocation in the MIS report ",
            "Can I track transportation incidents or accidents in the MIS report ",
            "How do I generate an MIS report showing the number of transport vehicles in service ",
            "Can I see the average occupancy of buses in the MIS report ",
            "How do I track the on-time performance of transport vehicles in the MIS report ",
            "Can I generate an MIS report to analyze transport vendor performance ",
            "How do I track transport driver performance in the MIS report ",
            "Can I see an overview of all transport-related incidents in the MIS report "
        ],
        'driverlist_report': [
            "Can I see a list of all transport drivers in the report ",
            "How do I generate a report of all transport drivers ",
            "Can I view the list of active drivers in the transport department ",
            "How do I see the driver assignments for each vehicle ",
            "Can I get a detailed report on each transport driver's performance ",
            "How do I generate a report that includes driver names and vehicle assignments ",
            "Can I track which drivers are assigned to which routes in the driver report ",
            "How do I generate a driver performance report ",
            "Can I see a report showing driver attendance for each day ",
            "How do I generate a driver report that includes their working hours ",
            "Can I get a summary of all driver shifts in the report ",
            "How do I view the list of drivers and their assigned vehicles ",
            "Can I include driver license numbers in the driver list report ",
            "How do I track the total number of hours worked by each driver ",
            "Can I view a report of all drivers with expired licenses ",
            "How do I generate a report showing the number of accidents involving each driver ",
            "Can I track driver complaints in the driver list report ",
            "How do I generate a report to see the driving record of each driver ",
            "Can I view the driver performance rating in the report ",
            "How do I track the number of days each driver has been on leave ",
            "Can I see a report that lists the drivers with the most accidents ",
            "How do I view driver disciplinary actions in the report ",
            "Can I track the driver salary details in the driver report ",
            "How do I view the total driving hours for each driver in a report ",
            "Can I generate a report that lists all driver promotions ",
            "How do I track driver training sessions and certifications in the report ",
            "Can I get a report on drivers who have recently completed training ",
            "How do I view the assignment history of each driver ",
            "Can I track the vehicles each driver has been assigned to in the report ",
            "How do I generate a report to see which drivers have the most on-time performance ",
            "Can I track driver performance based on vehicle condition in the report ",
            "How do I view a summary of driver work schedules in the report ",
            "Can I see the total number of days each driver worked in a report ",
            "How do I generate a report showing drivers who have been on leave ",
            "Can I track the reasons for driver absenteeism in the report ",
            "How do I generate a report to see which drivers have the most complaints ",
            "Can I view a report showing the number of routes assigned to each driver ",
            "How do I track driver safety compliance in the report ",
            "Can I see a report that lists the vehicles each driver is qualified to drive ",
            "How do I track driver certifications in the report ",
            "Can I generate a driver turnover report ",
            "How do I track the training hours completed by each driver in the report ",
            "Can I view a report showing the total distance driven by each driver ",
            "How do I generate a report to see drivers' performance ratings over time ",
            "Can I see which drivers have had the most safety incidents in the report ",
            "How do I generate a report that includes driver feedback surveys ",
            "Can I track the completion of safety checks by each driver in the report ",
            "How do I generate a report showing the total number of trips each driver completed "
        ],
        'classwise_bus_students_report': [
            "Can I generate a report showing bus students by class ",
            "How do I view a list of students assigned to each bus based on their class ",
            "Can I see the number of students using the bus for each class ",
            "How do I view the student count per bus for each class ",
            "Can I get a report on bus routes with student details per class ",
            "How do I generate a report that shows bus assignments for each class ",
            "Can I track which students from each class are using the school bus ",
            "How do I view the student list per class that uses the bus service ",
            "Can I generate a report for students using the bus for specific classes ",
            "How do I track the number of students who are availing bus transport for each class ",
            "Can I get a report showing which students are assigned to each bus by class ",
            "How do I view the bus routes for students of each class ",
            "Can I generate a report to see the bus service usage by class ",
            "How do I track student transportation usage based on their class ",
            "Can I view a report showing bus assignments for all classes ",
            "How do I generate a report that shows the number of students in each bus by class ",
            "Can I see which classes have the most students using the bus ",
            "How do I track which students from each class use the morning or afternoon bus ",
            "Can I view the details of bus usage per class in a report ",
            "How do I see a breakdown of bus routes by class ",
            "Can I generate a report showing the bus route for each class ",
            "How do I track student attendance on the bus for each class ",
            "Can I get a report showing the buses assigned to different classes ",
            "How do I view the details of students using the bus from specific classes ",
            "Can I generate a report showing which bus routes are most popular with certain classes ",
            "How do I track the student count per bus for morning and evening shifts ",
            "Can I generate a report showing students assigned to buses across different classes ",
            "How do I view a breakdown of class-wise student assignments for each bus ",
            "Can I track how many students from each class are using the bus service ",
            "How do I generate a report to see the list of students on each bus by class ",
            "Can I get a summary of all bus routes per class ",
            "How do I view which bus routes are associated with each class ",
            "Can I generate a report showing student names assigned to each bus for each class ",
            "How do I track the capacity utilization of each bus by class ",
            "Can I generate a class-wise bus transport summary report ",
            "How do I view the details of bus assignments based on student class ",
            "Can I track bus transport expenses per class in a report ",
            "How do I generate a report that shows the bus service usage for various classes ",
            "Can I view a report showing the number of students traveling per bus per class ",
            "How do I see the transportation preferences of students by class ",
            "Can I generate a report showing bus transportation for specific grades or sections ",
            "How do I view which students are assigned to each bus for different grade levels ",
            "Can I track the attendance of students using the bus by class ",
            "How do I generate a report showing the bus routes for different academic years ",
            "Can I get a report showing the bus routes for specific student grades or classes ",
            "How do I generate a report that includes the class details for bus users ",
            "Can I track the usage of bus services per class in the report ",
            "How do I get the list of students assigned to different buses per class ",
            "Can I generate a report to compare bus service usage across classes "
        ],
        'vehicle_capacity_report': [
            "Can I get a report showing the capacity of each vehicle ",
            "How do I view the capacity of buses and vehicles used for transport ",
            "Can I track the seating capacity of each vehicle in a report ",
            "How do I check the capacity utilization of each bus ",
            "Can I get a report of vehicle capacities across different routes ",
            "How do I view the total seating capacity of all vehicles ",
            "Can I generate a report showing the occupancy rate of buses by class ",
            "How do I get the list of vehicles with their seating capacity ",
            "Can I track the vehicle capacity per route in a report ",
            "How do I view a breakdown of bus seating capacity for different routes ",
            "Can I generate a report showing the maximum and current capacity of vehicles ",
            "How do I check vehicle capacity against the number of students assigned ",
            "Can I see a report that compares vehicle capacity to actual usage ",
            "How do I track vehicle capacity per bus route ",
            "Can I generate a vehicle capacity report by transportation department ",
            "How do I get a report showing the utilization of vehicle seats ",
            "Can I track how many seats are occupied in each vehicle ",
            "How do I view the remaining capacity in each vehicle ",
            "Can I generate a report that shows vehicle capacity versus student count ",
            "How do I see which vehicles are under or over-utilized ",
            "Can I generate a report showing the vehicle capacity based on each route ",
            "How do I check if any vehicle is exceeding its seating capacity ",
            "Can I get a report that shows the seating capacity of different vehicles ",
            "How do I generate a report that shows the total capacity of vehicles used for transport ",
            "Can I view the occupancy percentage of each vehicle in a report ",
            "How do I generate a report showing the seating capacity of school buses ",
            "Can I track how many seats are available in each vehicle ",
            "How do I view the capacity utilization of different types of vehicles ",
            "Can I get a summary of vehicle seating capacities for the transport department ",
            "How do I view the capacity report for each vehicle on different routes ",
            "Can I track if any vehicles are exceeding their capacity limit ",
            "How do I check the total number of occupied seats in the fleet ",
            "Can I generate a report comparing vehicle capacity with the student load ",
            "How do I track vehicle capacity for different vehicle types ",
            "Can I see which routes have the highest vehicle capacity utilization ",
            "How do I view the vehicle's capacity in relation to student assignments ",
            "Can I generate a vehicle capacity summary for all vehicles in service ",
            "How do I check the number of unoccupied seats in the transport fleet ",
            "Can I view the total capacity of buses for specific routes ",
            "How do I track the seat availability in each vehicle for the upcoming routes ",
            "Can I get a report showing the average capacity utilization for all buses ",
            "How do I track the vehicle capacity and compare it with student requirements ",
            "Can I generate a vehicle capacity report to evaluate the fleet's performance ",
            "How do I check the seating capacity of vehicles based on the number of students per route ",
            "Can I generate a report showing which vehicles are closest to reaching full capacity ",
            "How do I see the capacity details for the most utilized vehicles ",
            "Can I track which buses have the lowest occupancy in a report ",
            "How do I view the total student load versus vehicle capacity in a report ",
            "Can I get a vehicle capacity report for the school’s transport service "
        ],
        'vehicle_expense_report': [
            "Can I get a report showing the expenses of each vehicle ",
            "How do I track the maintenance expenses of all vehicles ",
            "Can I see the fuel consumption and expenses for each vehicle ",
            "How do I generate a report for vehicle expenses over a period ",
            "Can I view the total cost of vehicle maintenance ",
            "How do I track the operational expenses of each vehicle ",
            "Can I get a breakdown of the expenses for each transport vehicle ",
            "How do I generate a report showing the cost of vehicle repairs ",
            "Can I track fuel costs and maintenance expenses for school buses ",
            "How do I see the total expenses for vehicle operation in a month ",
            "Can I view a detailed expense report for each vehicle ",
            "How do I track insurance costs for each vehicle ",
            "Can I generate a report for the total expenses of the transport department ",
            "How do I view the expense history of each vehicle ",
            "Can I get a report showing how much was spent on vehicle repairs ",
            "How do I track the expenses of different types of vehicles ",
            "Can I generate a detailed vehicle expense report by category ",
            "How do I track tire replacement expenses for vehicles ",
            "Can I see a report of the total fuel expenses for all vehicles ",
            "How do I check the cost of servicing vehicles over the past year ",
            "Can I view the cost of vehicle maintenance for each route ",
            "How do I generate an expense report showing fuel, maintenance, and repairs ",
            "Can I get a report for the cost of vehicle registration ",
            "How do I track the fuel expenses for each vehicle by route ",
            "Can I generate a report that compares vehicle expenses across different routes ",
            "How do I track the total maintenance cost for all school buses ",
            "Can I view the overall expenses of the vehicle fleet ",
            "How do I track and manage vehicle repair expenses ",
            "Can I generate a report showing the expenses for each transport vehicle type ",
            "How do I see how much has been spent on vehicle insurance ",
            "Can I view a report showing the total cost of vehicle fuel for the month ",
            "How do I generate a report for vehicle maintenance and repair costs over the past year ",
            "Can I track the overall transport expenses in a detailed report ",
            "How do I view the total operating cost for all vehicles in the fleet ",
            "Can I generate a report showing the expenses for vehicles with high maintenance ",
            "How do I track expenses by fuel type for each vehicle ",
            "Can I view the total cost for repairs and maintenance of each vehicle ",
            "How do I generate a report showing vehicle repair costs by month ",
            "Can I track the total expenses for transportation vehicles by year ",
            "How do I view the vehicle expenses for specific types of maintenance ",
            "Can I get a detailed report on fuel, repair, and insurance expenses for each vehicle ",
            "How do I see the vehicle expenses compared with the income from student transport fees ",
            "Can I track the cost of operating specific vehicles like school buses ",
            "How do I generate a vehicle expense report by vendor for parts and services ",
            "Can I view a report showing how much is spent per vehicle in terms of fuel ",
            "How do I generate a report showing the cost efficiency of each vehicle ",
            "Can I track the total vehicle operating expenses and compare them with the budget ",
            "How do I generate a report showing the impact of vehicle repairs on transport costs ",
            "Can I view a report comparing fuel costs of different vehicles ",
            "How do I track the overall transport costs including maintenance and repairs ",
            "Can I get a vehicle expense summary for the fleet for the year "
        ]
    },

        'communication' : {
        'send_sms': [
            "How do I send an SMS to a student ",
            "Can I send an SMS to multiple students ",
            "How can I send a reminder SMS to parents ",
            "Can I send SMS notifications for upcoming events ",
            "How do I send an SMS to a specific group ",
            "Can I send an SMS to all the students of a particular class ",
            "How can I send SMS alerts to staff members ",
            "Can I schedule an SMS to be sent later ",
            "How do I send an SMS with attachments ",
            "Can I send a bulk SMS to all users ",
            "How do I send a personal SMS to a student ",
            "Can I send an SMS with event details ",
            "How do I send SMS reminders for upcoming exams ",
            "Can I send SMS notifications for fee due dates ",
            "How can I send an SMS for attendance reminders ",
            "How do I send an SMS to students with low attendance ",
            "Can I send SMS notifications for holiday updates ",
            "How do I send an SMS for emergency alerts ",
            "Can I send an SMS confirming student registration ",
            "How do I send SMS updates about school events ",
            "Can I send SMS about transportation details ",
            "How do I send personalized SMS for fee payments ",
            "Can I send an SMS for results announcement ",
            "How do I send an SMS to notify about class cancellations ",
            "Can I send SMS alerts about important school updates ",
            "How do I send SMS about upcoming school trips ",
            "Can I send reminder SMS to teachers about meetings ",
            "How do I send SMS to inform about teacher absences ",
            "Can I send a confirmation SMS for exam registrations ",
            "How do I send an SMS to parents about their child's performance ",
            "Can I send SMS notifications about school holidays ",
            "How do I send an SMS for attendance confirmation ",
            "Can I send bulk SMS to notify about school closures ",
            "How do I send an SMS alert for urgent announcements ",
            "Can I send birthday wish SMS to students ",
            "How do I send SMS to notify about parent-teacher meetings ",
            "Can I send SMS with exam result details ",
            "How do I send an SMS alert about transport changes ",
            "Can I send a birthday SMS to teachers ",
            "How do I send an SMS for fee receipt confirmations ",
            "Can I send SMS reminders for school events ",
            "How do I send SMS notifications for new assignments ",
            "Can I send an SMS for school activity notifications ",
            "How do I send SMS alerts for exam schedule changes ",
            "Can I send an SMS for school registration details ",
            "How do I send SMS for fee dues ",
            "Can I send SMS for parent-teacher conference updates ",
            "How do I send a farewell SMS to students "
        ],
        'sms_send_report': [
            "Can I view the report of all sent SMS ",
            "How do I check the SMS sending history ",
            "How do I see a report of SMS sent to parents ",
            "Can I get a detailed report of all SMS sent to students ",
            "How do I generate an SMS send report by date ",
            "Can I track which SMS were sent to specific groups ",
            "How can I get a report on all SMS sent by teachers ",
            "How do I generate a report of failed SMS deliveries ",
            "Can I track the delivery status of each SMS sent ",
            "How do I view a report for SMS sent to staff members ",
            "Can I get a report on the number of SMS sent daily ",
            "How do I view the delivery rate of sent SMS ",
            "Can I see a report of SMS sent for event reminders ",
            "How do I get a report on SMS sent for attendance notifications ",
            "Can I track the SMS sent for fee reminders ",
            "How do I generate a report of SMS sent for exam results ",
            "Can I check the status of SMS deliveries for student notifications ",
            "How do I generate a report for SMS sent to parents regarding school events ",
            "Can I track the SMS sent for emergency alerts ",
            "How do I generate a detailed SMS report for all class updates ",
            "Can I check the SMS delivery failures ",
            "How do I get a report of SMS sent for school closures ",
            "Can I track SMS delivery for notices about transport ",
            "How do I check SMS sent for parent-teacher meeting reminders ",
            "Can I generate a report for birthday wish SMS sent ",
            "How do I check the SMS delivery for exam schedule notifications ",
            "Can I get a report for SMS sent about school trips ",
            "How do I track SMS sent for holiday notifications ",
            "Can I check a report for SMS sent for school registrations ",
            "How do I get the history of all bulk SMS sent ",
            "Can I track SMS sent for school activity updates ",
            "How do I generate a report for SMS sent for class cancellations ",
            "Can I view the SMS reports for student transport updates ",
            "How do I track SMS sent for student performance updates ",
            "Can I see a report of SMS reminders sent for homework ",
            "How do I get a summary of all SMS sent to teachers ",
            "Can I track SMS sent for assignment notifications ",
            "How do I view SMS delivery rates for urgent announcements ",
            "Can I generate a report of all SMS for attendance reminders ",
            "How do I track SMS sent to notify about fee payments ",
            "Can I check the status of all birthday wish SMS sent ",
            "How do I generate a report for SMS sent for school event notifications ",
            "Can I track the SMS sent for registration confirmations ",
            "How do I get a report on SMS sent to confirm exam registrations ",
            "Can I generate a detailed report for SMS sent to different student groups ",
            "How do I track all SMS notifications sent to parents ",
            "Can I see the delivery status for SMS about school activities ",
            "How do I get a report for SMS sent to teachers regarding meetings "
        ],
        'send_email': [
            "How do I send an email to a student ",
            "Can I send an email to multiple students at once ",
            "How can I send an email with the school event details ",
            "Can I send a bulk email to all the staff members ",
            "How do I send an email for fee reminders ",
            "Can I send an email to parents about upcoming exams ",
            "How do I send an email to notify about school holidays ",
            "Can I send an email with attachments for students ",
            "How can I send an email about the school timetable ",
            "Can I send an email to inform about transportation updates ",
            "How do I send a confirmation email for student registration ",
            "Can I send an email for school closure announcements ",
            "How do I send an email about school trips to parents ",
            "Can I send an email for urgent school updates ",
            "How do I send personalized emails for students' performance ",
            "Can I send an email for results announcements ",
            "How do I send an email for upcoming events at school ",
            "Can I send an email to parents about parent-teacher meetings ",
            "How do I send an email to notify about class cancellations ",
            "Can I send an email for assignments updates ",
            "How do I send an email with school activity details ",
            "Can I send an email with exam schedule updates ",
            "How do I send a thank you email to teachers ",
            "Can I send an email to notify about staff meetings ",
            "How do I send an email for birthday wishes to students ",
            "Can I send an email to parents about their child’s attendance ",
            "How do I send an email for urgent alerts ",
            "Can I send an email to confirm exam registrations ",
            "How do I send an email with fee payment details ",
            "Can I send an email for transport service alerts ",
            "How do I send an email to announce the results ",
            "Can I send a reminder email to parents about upcoming events ",
            "How do I send an email with the latest school news ",
            "Can I send an email to announce parent-teacher conferences ",
            "How do I send an email for exam result notification ",
            "Can I send an email about a student’s registration status ",
            "How do I send an email to inform about school clubs ",
            "Can I send an email to notify about special events ",
            "How do I send an email with attendance reminder ",
            "Can I send an email to students about class notes ",
            "How do I send an email to inform about school exams ",
            "Can I send an email about assignments due dates ",
            "How do I send an email for upcoming holiday notifications ",
            "Can I send an email for new school policy updates ",
            "How do I send an email to parents about school activities ",
            "Can I send an email for emergency school updates ",
            "How do I send an email to inform about late arrivals ",
            "Can I send an email for student behavior reports ",
            "How do I send an email for school registration confirmations "
        ],
        'email_send_report': [
            "Can I view the report of the emails I've sent ",
            "How do I check the status of emails I've sent ",
            "Where can I find the report for sent emails ",
            "Is there a way to track the emails I've already sent ",
            "How do I see a summary of the emails sent to parents ",
            "Can I get a report of emails sent to all students ",
            "How do I view the list of emails sent to teachers ",
            "Where can I find the sent email log for the last week ",
            "Can I check if any of my sent emails were undelivered ",
            "How do I get a report of all emails sent to staff members ",
            "Can I export a report of sent emails to a CSV file ",
            "How do I filter sent email reports by date ",
            "Is there a way to see how many emails were opened ",
            "Can I see which students received the email about the exam ",
            "How do I find emails that failed to be sent ",
            "Where can I find a report of failed emails ",
            "Can I check which emails had attachments ",
            "How do I view the sent email history for a specific student ",
            "Can I get a daily report of emails sent to parents ",
            "How do I see the status of emails sent about school events ",
            "Can I see the number of emails sent per month ",
            "How do I check the success rate of sent emails ",
            "Can I view a report for emails sent to a specific class ",
            "How do I get a report of all email communications with teachers ",
            "Can I get a report on email responses ",
            "How do I check if my email was delivered to all parents ",
            "Can I get a report on the email delivery status ",
            "How do I see the email delivery rates ",
            "Can I filter the sent email report by email subject ",
            "How do I check the list of attachments sent in emails ",
            "Can I get a report for emails sent regarding school fees ",
            "How do I find out if any email attachments failed to send ",
            "Can I see if an email with a timetable was received by all students ",
            "How do I get a report on email open rates ",
            "Can I see which emails were delivered and which were bounced ",
            "How do I get a report of emails sent to the entire school staff ",
            "Can I see a report of emails sent for school trips ",
            "How do I get a summary report of emails sent in the last month ",
            "Can I view the details of a specific sent email ",
            "How do I check the status of a particular email sent to parents ",
            "Can I see if any sent email had any errors ",
            "How do I get a report of emails sent for parent-teacher meetings ",
            "Can I track if a sent email has been marked as read ",
            "How do I get a report on email failures ",
            "Can I get a detailed report on the emails sent to students about assignments ",
            "How do I know if my email about school holidays was delivered ",
            "Can I filter the report to show emails sent to a specific department ",
            "How do I get a detailed sent email log by date range ",
            "Can I get a list of all emails sent to students about their results "
        ],
        'notification_setting': [
            "How do I manage my notification settings ",
            "Can I change the frequency of notifications ",
            "Where can I adjust my notification preferences ",
            "How do I turn off email notifications ",
            "Can I set notifications for specific events ",
            "How do I set up notifications for upcoming exams ",
            "Can I get notified about fee due dates ",
            "How can I enable notifications for class updates ",
            "Can I receive notifications for new homework assignments ",
            "How do I customize my notification preferences ",
            "Can I turn off notifications for certain subjects ",
            "How can I receive notifications for student absences ",
            "How do I set up notifications for student results ",
            "Can I receive notifications for parent-teacher meetings ",
            "How do I change the sound for notifications ",
            "Can I set quiet hours for notifications ",
            "How do I adjust notification settings for reports ",
            "Can I receive notifications for new student admissions ",
            "How do I set notifications for school events ",
            "Can I get reminders for upcoming holidays ",
            "How do I enable notifications for attendance marking ",
            "Can I turn off notifications for school news ",
            "How do I enable notifications for school fees due ",
            "How do I set up notifications for assignment deadlines ",
            "Can I receive notifications for new homework uploaded ",
            "How do I customize the types of notifications I get ",
            "Can I turn off notifications for email reports ",
            "How can I receive notifications about co-curricular activities ",
            "How do I enable notifications for school meetings ",
            "Can I set up notifications for leave approvals ",
            "How do I get a notification when my child's exam results are out ",
            "How do I set notifications for my child's attendance ",
            "Can I turn off notifications for system updates ",
            "How do I set custom notification preferences for my account ",
            "Can I get notifications for fee payments ",
            "How do I enable SMS notifications for school announcements ",
            "Can I receive notifications for school transport changes ",
            "How do I get notifications about changes in the timetable ",
            "How do I set up notifications for exam scheduling ",
            "Can I get notifications for new teaching materials ",
            "How do I receive notifications when new grades are posted ",
            "How do I set notifications for class attendance summaries ",
            "Can I get notified when new messages are posted in the forum ",
            "How do I enable notifications for school events like sports day ",
            "Can I choose the type of notifications I want to receive ",
            "How do I disable push notifications on my mobile ",
            "Can I set different notification preferences for students and parents ",
            "How do I receive a notification when a student is marked absent ",
            "Can I receive daily updates on assignments and homework "
        ],
        'wishes_details': [
            "Can I send wishes to students ",
            "How do I send birthday wishes to students ",
            "Can I send anniversary wishes to teachers ",
            "How can I send personalized wishes to staff members ",
            "Is it possible to send festive wishes to the entire school ",
            "Can I set up automatic birthday wishes for students ",
            "How do I configure wishes for teachers' special days ",
            "Can I customize the message for birthday wishes ",
            "How do I send congratulations for academic achievements ",
            "Can I send wishes on national holidays ",
            "How do I send end-of-term wishes to students ",
            "Can I create a special message for exam results ",
            "How can I send congratulations for school events ",
            "Can I schedule wishes for a specific date ",
            "Can I send New Year wishes to parents and staff ",
            "How do I send personalized holiday wishes ",
            "Can I set up automated wishes for school staff ",
            "How can I send end-of-year wishes to students ",
            "Can I send congratulatory messages for sports achievements ",
            "How do I customize wishes for a student’s academic milestones ",
            "Can I send farewell wishes to students leaving the school ",
            "How do I send wishes for festivals like Diwali and Christmas ",
            "Can I send wishes for the first day of school ",
            "How can I send appreciation messages for teachers ",
            "How do I send a thank you message to staff after an event ",
            "Can I send wishes for teacher appreciation week ",
            "How do I set up wishes for teachers’ day ",
            "Can I send wishes for Parent-Teacher meeting days ",
            "How do I send congratulatory wishes for student scholarships ",
            "How can I create a template for sending wishes ",
            "Can I schedule multiple wishes for different occasions ",
            "Can I send birthday wishes via email to parents ",
            "How do I send a graduation wish to students ",
            "Can I send wishes for achievements in extracurricular activities ",
            "How can I send wishes for a student's first job ",
            "Can I send personalized wishes to alumni ",
            "How do I send festive greetings for school holidays ",
            "Can I send congratulations for winning competitions ",
            "How do I send messages of encouragement before exams ",
            "Can I automate wishes based on student data like age or grade ",
            "How do I send general well wishes at the beginning of the school year ",
            "Can I send special messages for sports day ",
            "How do I send congratulations for completing a course ",
            "Can I send wishes for the school foundation day ",
            "How can I create and send special messages for students’ parents ",
            "How do I send a thank you message for teachers on appreciation day ",
            "Can I send special wishes for staff anniversaries ",
            "How do I send well wishes to new students "
        ],
        'home_work': [
            "Can I assign homework to students ",
            "How do I upload homework assignments for students ",
            "Can I set a deadline for homework submission ",
            "How can I track homework submissions ",
            "Can I send reminders to students for homework ",
            "Can I assign homework to a specific class ",
            "How do I view the status of homework submissions ",
            "Can I assign homework with attachments ",
            "Can I set up automated reminders for homework ",
            "How do I check homework assignments submitted by students ",
            "Can I send a message to students about their homework ",
            "How do I mark homework as completed ",
            "Can I allow students to submit homework via email ",
            "How do I upload multiple homework assignments at once ",
            "Can I edit homework assignments after posting them ",
            "How do I attach a document to a homework assignment ",
            "Can I send homework assignments in different formats (PDF, Word, etc ) ",
            "Can I schedule homework assignments for future dates ",
            "How do I check if a student has completed their homework ",
            "Can I assign group homework tasks ",
            "How do I categorize homework assignments by subject ",
            "Can I assign homework for multiple subjects at once ",
            "How do I assign individual homework to students ",
            "Can I set multiple deadlines for different parts of a homework task ",
            "How do I send feedback on homework submissions ",
            "Can I track late homework submissions ",
            "Can I assign bonus points for early homework submissions ",
            "Can I share homework solutions with students ",
            "How do I inform parents about homework assignments ",
            "Can I mark homework as pending ",
            "Can I view homework assignment history ",
            "How do I remind students to complete their homework ",
            "Can I assign homework based on student performance ",
            "How do I notify students of upcoming homework deadlines ",
            "Can I assign holiday homework ",
            "Can I assign project work as homework ",
            "How do I upload homework for online classes ",
            "Can I assign creative homework tasks ",
            "Can I send homework assignments as part of a weekly schedule ",
            "How do I create a list of required resources for homework ",
            "Can I send homework assignments through the school's communication app ",
            "How do I share detailed homework guidelines with students ",
            "Can I assign homework to students based on their grade level ",
            "How do I send homework updates to parents via SMS or email ",
            "Can I assign peer-reviewed homework tasks ",
            "Can I share homework deadlines on the school portal ",
            "How do I send homework reminders through push notifications ",
            "Can I receive notifications when students submit homework ",
            "How do I create homework assignments with multiple parts "
        ],
        'app_user_report': [
            "How do I generate a report for app users ",
            "Can I view a list of all users in the app ",
            "How do I track user activity in the app ",
            "Can I generate reports for user sign-ups ",
            "How do I filter users by their activity in the app ",
            "Can I get a report of active users in the app ",
            "How do I find the most frequent users of the app ",
            "Can I view a report on users' last login times ",
            "How can I track new user registrations in the app ",
            "Can I generate reports for users' app usage patterns ",
            "How do I export user reports from the app ",
            "Can I see which users have logged in within the last 24 hours ",
            "How do I generate reports for user demographics ",
            "Can I track the number of users who have completed a specific action in the app ",
            "How do I generate reports based on user locations ",
            "Can I get a report on users who have used the app most frequently ",
            "How do I generate reports for users who haven't logged in recently ",
            "Can I filter users by the date they registered in the app ",
            "How do I track which features users engage with most ",
            "Can I get a report for users who have left feedback or reviews ",
            "How do I monitor the user retention rate over time ",
            "Can I see reports for users who have shared content from the app ",
            "How do I track user behavior based on device type ",
            "Can I see how many users have downloaded the app ",
            "How do I generate a report on app crashes or issues reported by users ",
            "Can I analyze user activity based on app versions ",
            "How do I track app usage trends over a specific time period ",
            "Can I get a report on users who have made in-app purchases ",
            "How do I track how many users have completed a specific tutorial or guide ",
            "Can I filter users by their subscription or payment status ",
            "How do I track users' engagement with push notifications ",
            "Can I see a report of users who have unsubscribed from push notifications ",
            "How do I analyze user feedback data collected through the app ",
            "Can I track user activity based on geographic region ",
            "How do I get insights into user retention and engagement metrics ",
            "Can I see user activity reports for a specific date range ",
            "How do I track user interactions with app notifications ",
            "Can I get a report for users who have participated in surveys or polls ",
            "How do I find users who have opted for premium features ",
            "Can I track users' frequency of using specific features in the app ",
            "How do I generate reports on users who have encountered issues with the app ",
            "Can I get insights into the age distribution of app users ",
            "How do I track user interactions with different content in the app ",
            "Can I generate reports on users who have completed app updates ",
            "How do I track users based on the time of day they use the app ",
            "Can I track the average session duration of app users ",
            "How do I analyze user behavior based on the user's app session frequency ",
            "Can I generate a report on users' app ratings or feedback scores ",
            "How do I monitor app usage trends to identify potential issues "
        ]
    },

        'library' : {
        'library_masters': [
            "How do I manage library masters ",
            "Can I add a new library master ",
            "How do I update library master records ",
            "What information is required for creating a library master ",
            "Can I view the list of library masters ",
            "How can I delete a library master entry ",
            "Is it possible to edit the details of a library master ",
            "Can I filter library masters by category ",
            "How do I assign categories to library masters ",
            "Can I search for a specific library master by name ",
            "How do I categorize library masters into sections ",
            "Can I upload a list of library masters in bulk ",
            "How do I manage library master details for book entries ",
            "Can I associate authors with library masters ",
            "How can I update the status of a library master ",
            "Is it possible to import library masters from an external file ",
            "How do I set access levels for library masters ",
            "Can I sort library masters by creation date ",
            "How can I track changes to library masters ",
            "Can I create sub-categories for library masters ",
            "How do I assign library masters to specific sections of the library ",
            "Can I delete outdated library master records ",
            "How do I manage the availability of library masters ",
            "Can I assign tags to library masters ",
            "How do I review the history of a library master ",
            "Can I see which library masters have been borrowed recently ",
            "How do I link library masters to specific locations ",
            "Can I print a list of all library masters ",
            "How do I manage library master information for new acquisitions ",
            "Can I customize the fields for library master records ",
            "How do I ensure library masters are properly cataloged ",
            "Can I integrate library masters with other library modules ",
            "How do I track inventory levels for library masters ",
            "Can I set up alerts for overdue library masters ",
            "How do I manage library masters for multiple branches ",
            "Can I assign barcodes to library masters ",
            "How do I review detailed records of library masters ",
            "Can I add notes to library master records ",
            "How do I manage the availability status of library masters ",
            "Can I create reports for library master categories ",
            "How do I verify the accuracy of library master details ",
            "Can I restrict access to certain library masters ",
            "How do I link library masters to specific student groups ",
            "Can I track the usage of library masters ",
            "How do I integrate library masters with the book issue system ",
            "Can I audit library master entries for accuracy ",
            "How do I delete duplicate library master records "
        ],
        'dashboard': [
            "How do I access the library dashboard ",
            "What information is displayed on the library dashboard ",
            "Can I customize the layout of the library dashboard ",
            "How do I get real-time statistics on library usage ",
            "Can I view a summary of book issues and returns on the dashboard ",
            "How do I track the status of books in the library on the dashboard ",
            "Can I see overdue book details on the dashboard ",
            "How do I monitor book requisition requests through the dashboard ",
            "Can I get alerts for books that need to be returned on the dashboard ",
            "How do I filter library data by date range on the dashboard ",
            "Can I view the most popular books on the dashboard ",
            "How do I check the number of library attendees on the dashboard ",
            "Can I track the daily activity of the library on the dashboard ",
            "How do I display the current number of available books on the dashboard ",
            "Can I manage book reservations directly from the dashboard ",
            "How do I generate quick reports from the dashboard ",
            "Can I track the total number of active users on the dashboard ",
            "How do I view library membership statistics on the dashboard ",
            "Can I check the number of books issued today on the dashboard ",
            "How do I access library transaction summaries from the dashboard ",
            "Can I set alerts for library attendance on the dashboard ",
            "How do I track the library's book stock from the dashboard ",
            "Can I see trends in library usage on the dashboard ",
            "How do I add new modules to the dashboard ",
            "Can I track student activity in the library on the dashboard ",
            "How do I filter dashboard data by user type ",
            "Can I view library reports directly on the dashboard ",
            "How do I export data from the dashboard ",
            "Can I integrate the dashboard with other library systems ",
            "How do I display upcoming book releases on the dashboard ",
            "Can I monitor library attendance patterns on the dashboard ",
            "How do I keep track of overdue books and fines on the dashboard ",
            "Can I see the library's financial status on the dashboard ",
            "How do I manage library staff activity through the dashboard ",
            "Can I set up personalized views for different users on the dashboard ",
            "How do I display book issue and return data on the dashboard ",
            "Can I view detailed reports on book history from the dashboard ",
            "How do I get notifications for system updates on the dashboard ",
            "Can I track the number of books borrowed by category on the dashboard ",
            "How do I manage library settings through the dashboard ",
            "Can I configure the dashboard to show real-time library data ",
            "How do I check library attendance trends over time on the dashboard ",
            "Can I get alerts for new book arrivals on the dashboard ",
            "How do I view detailed usage patterns for each library section ",
            "Can I track the status of book requisitions from the dashboard ",
            "How do I access library circulation data from the dashboard ",
            "Can I add new features to the library dashboard "
        ],
        'book_master': [
            "How do I add a new book to the master list ",
            "Can I update the details of an existing book in the master list ",
            "How do I delete a book from the book master ",
            "Can I import a list of books into the book master ",
            "How do I categorize books in the master list ",
            "Can I link authors to books in the master list ",
            "How do I assign publishers to books in the book master ",
            "Can I track the availability of books in the master list ",
            "How do I manage book editions in the master list ",
            "Can I add multiple copies of a book to the master list ",
            "How do I search for a specific book in the master list ",
            "Can I update book information such as title, author, and category ",
            "How do I assign ISBN numbers to books in the master list ",
            "Can I set a price for each book in the master list ",
            "How do I track the publication date of each book ",
            "Can I filter books by category in the book master ",
            "How do I assign book copies to library sections ",
            "Can I review detailed records for each book in the master list ",
            "How do I update the status of a book (e g , available, checked out) ",
            "Can I assign a barcode to each book in the master list ",
            "How do I add notes to book records in the master list ",
            "Can I set up alerts for low stock of books in the master list ",
            "How do I assign library users to borrow books from the master list ",
            "Can I generate reports on book usage from the master list ",
            "How do I track overdue books from the master list ",
            "Can I integrate the book master with the issue/return system ",
            "How do I manage book requisitions from the master list ",
            "Can I organize books by genre in the master list ",
            "How do I review the inventory of all books in the master list ",
            "Can I track how many copies of a book are available in the library ",
            "How do I manage book condition (e g , new, damaged) in the master list ",
            "Can I update book pricing based on edition or condition ",
            "How do I add book reviews or ratings in the master list ",
            "Can I assign book locations to specific library sections ",
            "How do I ensure that book records are properly maintained ",
            "Can I merge duplicate book entries in the master list ",
            "How do I review transaction history for each book ",
            "Can I categorize books by language or publication region ",
            "How do I track books by their circulation frequency ",
            "Can I archive old book records in the master list ",
            "How do I update the cataloging system for new acquisitions ",
            "Can I track which books are in high demand ",
            "How do I manage donated books in the master list ",
            "Can I update the book master with external data sources ",
            "How do I create and update book categories for easier searching ",
            "Can I generate detailed book history reports from the master list "
        ],
        'book_issue_return': [
            "How do I issue a book to a student ",
            "What is the process for returning a book ",
            "Can I extend the due date for a book issue ",
            "How do I track overdue books ",
            "Can I issue multiple books at once ",
            "How do I manage the return of books ",
            "Can I see the list of issued books ",
            "How do I check if a book is available for issue ",
            "Can I reserve a book for later issue ",
            "How do I issue books for a specific category ",
            "Can I print a receipt for a book issue ",
            "How do I manage book issue for different students ",
            "Can I track how long a book has been issued ",
            "How do I handle damaged or lost books during return ",
            "Can I issue a book to a student from a specific section ",
            "How do I update the status of a book after return ",
            "Can I allow students to issue books online ",
            "How do I handle the issue of reference books ",
            "Can I issue books without assigning a return date ",
            "How do I set up reminders for overdue books ",
            "Can I issue books based on availability ",
            "How do I issue books to multiple students at once ",
            "Can I manage book returns for different departments ",
            "How do I calculate late fees for overdue books ",
            "Can I issue books based on student type (e g , staff, student) ",
            "How do I check the return status of a specific book ",
            "Can I issue books in bulk for an event or class ",
            "How do I view the book return history ",
            "Can I issue books to guest users ",
            "How do I restrict the issue of certain books to users ",
            "Can I update the due date for a returned book ",
            "How do I record the condition of books during return ",
            "Can I track if a student has returned all issued books ",
            "How do I manage book issue for special users (e g , teachers) ",
            "Can I create reports for book issues and returns ",
            "How do I check which books are most frequently issued ",
            "Can I assign specific due dates for each issued book ",
            "How do I manage fines for overdue books ",
            "Can I apply a grace period for book returns ",
            "How do I issue books based on user requests ",
            "Can I restrict the number of books a student can issue ",
            "How do I see the status of a particular book issue ",
            "Can I extend the borrowing period for certain books ",
            "How do I track the availability of returned books ",
            "Can I issue books to staff members only ",
            "How do I generate reports on book issues and returns ",
            "Can I issue books with special conditions (e g , damaged) ",
            "How do I cancel a book issue request ",
            "Can I issue books based on student category (e g , grade level) ",
            "How do I handle bulk book issues for library events ",
            "Can I add notes to book issue records "
        ],
        'library_attendance': [
            "How do I mark attendance for library users ",
            "Can I track who is using the library at any given time ",
            "How do I set up library attendance for different sections ",
            "Can I generate a report for daily library attendance ",
            "How do I manage student attendance in the library ",
            "Can I mark attendance for online library sessions ",
            "How do I track staff attendance in the library ",
            "Can I set up notifications for missed attendance ",
            "How do I handle absent students in the library ",
            "Can I mark attendance for library events ",
            "How do I view attendance history for library users ",
            "Can I filter attendance data by date ",
            "How do I track library attendance based on department ",
            "Can I generate reports on attendance trends ",
            "How do I handle irregular attendance in the library ",
            "Can I mark attendance for group sessions in the library ",
            "How do I manage attendance for faculty using the library ",
            "Can I track attendance by student type (e g , teacher, student) ",
            "How do I mark attendance for staff training sessions in the library ",
            "Can I record attendance for virtual library sessions ",
            "How do I ensure accurate attendance tracking in the library ",
            "Can I create attendance patterns for different library users ",
            "How do I set attendance rules for library access ",
            "Can I monitor library attendance by time of day ",
            "How do I manage attendance for students attending library events ",
            "Can I track library attendance on weekends or holidays ",
            "How do I generate monthly attendance reports for the library ",
            "Can I add comments to student library attendance records ",
            "How do I track who has accessed the library's digital resources ",
            "Can I mark attendance for book clubs or study groups ",
            "How do I handle attendance for library workshops ",
            "Can I generate an overall attendance report for the library ",
            "How do I track the number of hours spent by students in the library ",
            "Can I set up automated attendance reminders for users ",
            "How do I assign attendance for library classes or orientations ",
            "Can I integrate library attendance with other systems ",
            "How do I manage late arrivals and early departures in the library ",
            "Can I review and edit attendance records for the library ",
            "How do I track student attendance across multiple library visits ",
            "Can I export library attendance data to Excel or CSV ",
            "How do I track library attendance for special events ",
            "Can I mark group attendance for students using the library together ",
            "How do I ensure accurate reporting of library attendance data ",
            "Can I use a barcode scanner for marking attendance in the library ",
            "How do I review attendance trends over time for the library ",
            "Can I set up attendance rules based on student behavior ",
            "How do I manage student late fees based on attendance ",
            "Can I monitor library usage based on attendance data "
        ],
        'book_requisitions': [
            "How do I request a new book for the library ",
            "Can I see the list of pending book requisitions ",
            "How do I approve a book requisition ",
            "How do I track the status of a book requisition ",
            "Can I cancel a book requisition ",
            "How do I add requisition details for a new book ",
            "Can I assign requisitions to specific departments ",
            "How do I manage requisitions for multiple copies of a book ",
            "Can I request books for a specific subject area ",
            "How do I process requisitions for journals and magazines ",
            "Can I create a requisition for rare or special collection books ",
            "How do I see the requisition history for a specific department ",
            "Can I approve multiple requisitions at once ",
            "How do I request books for upcoming events or classes ",
            "Can I track book requisitions for different sections ",
            "How do I filter requisitions by status (approved, pending) ",
            "Can I notify users when their book requisition is approved ",
            "How do I track the delivery status of requested books ",
            "Can I request books based on user demand ",
            "How do I handle requisitions for books that are out of stock ",
            "Can I prioritize requisitions for high-demand books ",
            "How do I create a requisition report for books ",
            "Can I request e-books for the library collection ",
            "How do I manage requisitions for textbooks ",
            "Can I set a budget for book requisitions ",
            "How do I receive books after requisition approval ",
            "Can I track the budget spent on book requisitions ",
            "How do I manage requisitions for reference materials ",
            "Can I set up notifications for requisition deadlines ",
            "How do I handle book requisitions from staff members ",
            "Can I create requisition requests for library expansions ",
            "How do I manage requisitions from different departments ",
            "Can I approve requisitions for multiple libraries in the system ",
            "How do I track books requested for library events ",
            "Can I request books from specific publishers ",
            "How do I integrate requisition data with other library systems ",
            "Can I request books for students with special needs ",
            "How do I manage requisitions for library renovations ",
            "Can I restrict requisitions based on department needs ",
            "How do I process requisitions for books already available ",
            "Can I review and edit existing requisitions ",
            "How do I manage requisitions for digital resources ",
            "Can I print requisition forms for record-keeping ",
            "How do I approve requisitions based on priority ",
            "Can I set requisition limits for each department ",
            "How do I reject a book requisition ",
            "Can I monitor the delivery timeline of requisitioned books ",
            "How do I handle international requisitions ",
            "Can I track requisitions from multiple suppliers ",
            "How do I ensure accurate record-keeping of requisition data "
        ],
        'book_history': [
            "How do I view the borrowing history of a book ",
            "Can I see the issue and return history of a specific book ",
            "How do I find the book history for a student ",
            "Can I check if a book was ever issued ",
            "How do I see the status of a particular book in the library ",
            "Can I check the historical demand for a specific book ",
            "How do I track if a book has been damaged in the past ",
            "Can I generate a report of all borrowed books ",
            "How do I view the past availability of a book ",
            "Can I check when a book was last issued ",
            "How do I view who borrowed a particular book ",
            "Can I view the history of all books issued by a student ",
            "How do I find out which books are overdue based on history ",
            "Can I check the book history by department ",
            "How do I manage book history for rare books ",
            "Can I view the history of book returns ",
            "How do I track the status of a book from its history ",
            "Can I see the history of books issued to staff ",
            "How do I track if a book was part of a book fair or event ",
            "Can I check the historical issue patterns of popular books ",
            "How do I see the list of most frequently borrowed books ",
            "Can I generate reports based on the book issue history ",
            "How do I view the lending history of a book ",
            "Can I track books that have been requested multiple times ",
            "How do I review the return history of a book ",
            "Can I track the condition of a book over time ",
            "How do I monitor a book’s history of damages or fines ",
            "Can I search book history based on the issue date ",
            "How do I see if a book has been withdrawn in the past ",
            "Can I view the detailed borrowing history for a student ",
            "How do I find the most borrowed books by year ",
            "Can I track book transfers between libraries ",
            "How do I track historical borrowing patterns of departments ",
            "Can I check if a book has been discontinued in the past ",
            "How do I access the complete history of a book’s circulation ",
            "Can I generate historical reports on book usage ",
            "How do I compare the borrowing patterns of multiple books ",
            "Can I check the history of a specific author’s books ",
            "How do I filter book history by user type (e g , student, staff) ",
            "Can I check the frequency of issues for a particular genre ",
            "How do I view the book’s past requisitions ",
            "Can I track the historical usage of library digital resources ",
            "How do I monitor the frequency of returns for a particular book ",
            "Can I generate reports for books with long borrowing history ",
            "How do I check the historical library attendance linked to a book ",
            "Can I track if a book was part of any special library programs ",
            "How do I view the historical collection of a specific book "
        ],
        'book_search': [
            "How do I search for a book in the library ",
            "Can I search for a book by author ",
            "How do I search for a book by title ",
            "Can I search for books by genre ",
            "How do I find books by ISBN ",
            "Can I search for books by publisher ",
            "How do I filter my book search by availability ",
            "Can I search for books by publication date ",
            "How do I search for books by language ",
            "Can I search for books by subject ",
            "How do I find books by year of release ",
            "Can I check if a book is available at my local library branch ",
            "How do I search for a specific edition of a book ",
            "Can I search for books based on the book series ",
            "How do I use the book search feature on the library portal ",
            "Can I search for e-books in the library catalog ",
            "How do I search for reference materials in the library ",
            "Can I search for rare books in the library catalog ",
            "How do I search for books that are out of stock ",
            "Can I search for books using keywords or phrases ",
            "How do I search for books by format (hardcover, paperback) ",
            "Can I search for books that are currently checked out ",
            "How do I search for books with high ratings ",
            "Can I search for books by reading level ",
            "How do I search for books that are suitable for children ",
            "Can I search for books by author’s nationality ",
            "How do I find new arrivals in the library ",
            "Can I search for textbooks available for my course ",
            "How do I search for books by the number of pages ",
            "Can I search for books by their availability for reservation ",
            "How do I search for books based on their condition ",
            "Can I search for books in my preferred language ",
            "How do I search for books from specific award-winning authors ",
            "Can I search for books related to a specific event ",
            "How do I search for books with a specific cover design ",
            "Can I filter my search results by price ",
            "How do I search for books based on review scores ",
            "Can I search for audiobooks in the library catalog ",
            "How do I search for book summaries in the catalog ",
            "Can I search for books based on their availability in multiple locations ",
            "How do I search for books by a specific translator ",
            "Can I search for books with supplementary study materials ",
            "How do I search for books on a particular academic topic ",
            "Can I search for books by their shelf number ",
            "How do I search for books by an author’s pen name ",
            "Can I search for books based on customer reviews ",
            "How do I search for books from a particular country ",
            "Can I search for out-of-print books ",
            "How do I search for books by a particular publisher’s catalog ",
            "Can I search for books in multiple formats (e-book, audiobook) "
        ],
        'library_reports': [
            "How do I generate a library report ",
            "Can I generate a report for borrowed books ",
            "How do I generate a report for overdue books ",
            "Can I generate a report based on book categories ",
            "How do I create a report on the number of new books added ",
            "Can I generate a report on library usage ",
            "How do I view a report for library staff attendance ",
            "Can I generate a report for books that are rarely borrowed ",
            "How do I generate a report for fines collected in the library ",
            "Can I generate a monthly or yearly library usage report ",
            "How do I generate a report for student borrowings ",
            "Can I generate a report for damaged books ",
            "How do I generate a report for book requisitions ",
            "Can I create a report to track missing books ",
            "How do I generate a report on the book circulation status ",
            "Can I generate a library report for a specific branch ",
            "How do I generate a report for book requests ",
            "Can I generate a report on library book conditions ",
            "How do I create a report for returned books ",
            "Can I generate a report based on book genres ",
            "How do I generate a report of unreturned library items ",
            "Can I create a report for student library fines ",
            "How do I generate a library stock report ",
            "Can I generate a library usage trend report ",
            "How do I view a detailed library book transaction report ",
            "Can I create a report of library memberships ",
            "How do I generate a report for books with high demand ",
            "Can I create a report on book reserves and holds ",
            "How do I generate a report of library inventory ",
            "Can I generate a report for books borrowed by faculty ",
            "How do I generate a library resource allocation report ",
            "Can I create a report to compare library borrowings by month ",
            "How do I generate a report on library events and activities ",
            "Can I generate a book return report ",
            "How do I track and generate reports for library donations ",
            "Can I generate a report for books based on availability ",
            "How do I create a report on library membership renewals ",
            "Can I generate a report for inter-library loans ",
            "How do I create a report for late fees and fines ",
            "Can I generate a report for book purchase history ",
            "How do I view reports for lost or stolen library items ",
            "Can I generate a report of library systems access ",
            "How do I generate a report on the most popular books ",
            "Can I track library usage patterns using a report ",
            "How do I create a report on e-book borrowings ",
            "Can I generate a report for book inventories across different branches "
        ],
        'library_books_import': [
            "How do I import books into the library system ",
            "Can I import a list of books from a CSV file ",
            "How do I upload new book records to the library system ",
            "Can I bulk import book details into the catalog ",
            "How do I import e-books into the library catalog ",
            "Can I import books from a different library system ",
            "How do I validate data when importing books ",
            "Can I import books based on ISBN ",
            "How do I import book titles and authors from an external source ",
            "Can I bulk import book stock details ",
            "How do I check for duplicates when importing books ",
            "Can I import books with their descriptions and metadata ",
            "How do I import books in different formats (PDF, EPUB) ",
            "Can I import book images along with the details ",
            "How do I import new books while ensuring correct categorization ",
            "Can I import library books with different languages ",
            "How do I import publisher information with book records ",
            "Can I import multiple copies of the same book ",
            "How do I handle import errors for books ",
            "Can I import books using API integrations ",
            "How do I import book borrowing limits with each book ",
            "Can I import books with their purchase price ",
            "How do I manage metadata while importing books ",
            "Can I import digital books from various sources ",
            "How do I import books in bulk without conflicts ",
            "Can I schedule book imports for regular updates ",
            "How do I track imported books in the library system ",
            "Can I import books in batches to avoid system overload ",
            "How do I import new book editions into the system ",
            "Can I import books and map them to library departments ",
            "How do I manage book imports for new library branches ",
            "Can I update existing book information through imports ",
            "How do I delete or overwrite incorrect book imports ",
            "Can I import books based on author’s collection ",
            "How do I import e-books and associate them with specific categories ",
            "Can I use third-party libraries to automate book imports ",
            "How do I import book pricing information from suppliers ",
            "Can I import book transaction data from an external file ",
            "How do I handle missing data during book imports ",
            "Can I track the status of each imported book record ",
            "How do I test the book import functionality before going live ",
            "Can I import historical book data into the library system ",
            "How do I ensure imported books comply with library standards ",
            "Can I batch update book details during import ",
            "How do I add tags and keywords when importing books ",
            "Can I automate book imports from publishers’ databases ",
            "How do I import books for special collections or archives "
        ]
    },

        'payroll' : {
        'payroll_masters': [
            "What is payroll masters ",
            "How do I access payroll masters ",
            "Can I edit payroll masters ",
            "What information is included in payroll masters ",
            "How do I add new employees to payroll masters ",
            "Can I delete records in payroll masters ",
            "How do I manage payroll masters for different departments ",
            "Can payroll masters be integrated with other modules ",
            "How do I update payroll masters after employee changes ",
            "Can I search for an employee in payroll masters ",
            "How do I handle salary adjustments in payroll masters ",
            "Can I filter payroll masters by job role ",
            "How do I view employee payment details in payroll masters ",
            "What fields are required in payroll masters ",
            "How do I add deductions to payroll masters ",
            "Can I export payroll masters data ",
            "How do I handle salary increments in payroll masters ",
            "Can I assign payroll masters data to a specific department ",
            "How do I update payroll masters for multiple employees ",
            "Can payroll masters track employee benefits ",
            "How do I assign bonuses to employees in payroll masters ",
            "Can I manage overtime details in payroll masters ",
            "How do I handle tax details in payroll masters ",
            "Can I import employee data to payroll masters ",
            "How do I set up payroll masters for contractors ",
            "Can payroll masters handle different pay frequencies ",
            "How do I link payroll masters to attendance data ",
            "How do I maintain accuracy in payroll masters ",
            "Can payroll masters handle union dues ",
            "How do I add benefits like health insurance to payroll masters ",
            "Can payroll masters accommodate bonuses and commissions ",
            "How do I manage paid time off in payroll masters ",
            "How do I ensure compliance with labor laws in payroll masters ",
            "Can I generate reports from payroll masters ",
            "How do I manage payroll masters for seasonal employees ",
            "Can I track payroll deductions in payroll masters ",
            "How do I manage retirement contributions in payroll masters ",
            "Can I merge payroll masters data from different sources ",
            "How do I ensure data privacy in payroll masters ",
            "Can payroll masters handle multiple job titles for an employee ",
            "How do I update payroll masters for employee promotions ",
            "Can I assign payroll masters based on work location ",
            "How do I correct errors in payroll masters ",
            "Can I link payroll masters with HRMS modules "
        ],
        "financial_year": [
            "Show me the current financial year ",
            "Which financial year is active ",
            "How do I set the financial year ",
            "Change the financial year settings ",
            "List all financial years ",
            "Add a new financial year ",
            "Delete the old financial year ",
            "Update the current financial year ",
            "View financial year details ",
            "How many financial years are added ",
            "Switch to the next financial year ",
            "Set start and end date of financial year ",
            "Is the financial year closed ",
            "Open previous financial year ",
            "Activate a financial year ",
            "Which financial year is used in payroll ",
            "Check financial year for salary processing ",
            "Get financial year list ",
            "Edit a financial year entry ",
            "Remove an unwanted financial year ",
            "Show me financial year calendar ",
            "Financial year in which month starts ",
            "Does the financial year follow April-March ",
            "Configure the academic financial year ",
            "Assign financial year to employees ",
            "Where to manage financial years ",
            "Financial year setup location ",
            "How to archive financial years ",
            "Which year is being used ",
            "Can I rename financial year ",
            "Copy data to new financial year ",
            "Enable financial year for reports ",
            "How to migrate data across financial years ",
            "What is this year’s financial period ",
            "Get payroll year status ",
            "Is this year closed for payroll ",
            "Save changes to financial year ",
            "Create a dummy financial year ",
            "How to check current year status ",
            "Move data to another financial year ",
            "Compare two financial years ",
            "How many records in this financial year ",
            "Can I lock financial year changes ",
            "Enable reminders for year end closing ",
            "What’s the financial year ID ",
            "View complete year timeline ",
            "Show year-based payroll summaries ",
            "Export year information ",
            "List all historical financial years ",
            "How to validate financial year periods "
        ],
        "parameter_master": [
            "Open the parameter master ",
            "Show all payroll parameters ",
            "How do I configure salary parameters ",
            "Where to update the parameter master ",
            "List of parameters used in salary ",
            "Add a new payroll parameter ",
            "Edit parameter details ",
            "Delete a salary parameter ",
            "Is there a parameter for overtime ",
            "Check attendance-related parameters ",
            "Can I change the parameter label ",
            "View active salary parameters ",
            "Which parameters are enabled ",
            "Filter parameter master list ",
            "What are the default parameters ",
            "Parameter used for deductions ",
            "Show parameter values for bonuses ",
            "Map parameter to employee group ",
            "Can I hide unused parameters ",
            "View complete list of payroll parameters ",
            "Parameter list for allowances ",
            "Set rules in parameter master ",
            "Add performance bonus parameter ",
            "Edit parameter calculation formula ",
            "Where are fixed parameters defined ",
            "Enable grade-based parameter ",
            "Parameter master report view ",
            "Download all payroll parameters ",
            "Can I link parameter with shift ",
            "Show parameter history logs ",
            "Get parameter configuration rules ",
            "Define custom parameters ",
            "Enable formula-based parameter ",
            "What is parameter master for ",
            "List system-defined payroll parameters ",
            "Payroll parameter for late penalty ",
            "Update tax parameters ",
            "Where to manage hourly wage parameter ",
            "Review parameter changes log ",
            "Assign parameters to department ",
            "How to reset parameters ",
            "Clone parameter settings ",
            "Employee type-based parameters ",
            "View summary of parameter master ",
            "Do we use parameter for ESIC ",
            "Can parameters be conditional ",
            "Setup incentives in parameter master ",
            "Is parameter master accessible to HR ",
            "Which parameters are mandatory ",
            "Import parameters from Excel "
        ],
        "professional_tax": [
            "Show professional tax settings ",
            "How to configure professional tax ",
            "What is the professional tax slab ",
            "Update professional tax rules ",
            "Add a new professional tax rule ",
            "Delete a professional tax record ",
            "Edit the professional tax slab ",
            "Apply professional tax to employees ",
            "Professional tax by state ",
            "Show professional tax report ",
            "Professional tax calculation method ",
            "Which employees are exempted ",
            "How is professional tax calculated ",
            "Slab-wise professional tax view ",
            "Can I change the professional tax percentage ",
            "Monthly deduction for professional tax ",
            "Professional tax limit for this month ",
            "Tax deduction proof required ",
            "Configure tax for different grades ",
            "State-wise professional tax details ",
            "Enable professional tax compliance ",
            "Where to set up professional tax ",
            "Is professional tax auto deducted ",
            "Professional tax applicability ",
            "Who manages professional tax settings ",
            "Generate professional tax statement ",
            "Link tax with payroll ",
            "Get professional tax summary ",
            "How to remove professional tax from payroll ",
            "Display historical tax deductions ",
            "Setup multi-state professional tax ",
            "Verify tax rule updates ",
            "Tax deduction threshold ",
            "Bulk assign tax to employees ",
            "Can interns be taxed ",
            "Show pending tax deductions ",
            "How to disable professional tax ",
            "Export tax slab as PDF ",
            "Professional tax comparison chart ",
            "Change tax slab date range ",
            "Create professional tax templates ",
            "Professional tax vs income tax ",
            "Professional tax rules for part-time staff ",
            "Professional tax applicability date ",
            "What is the tax exemption limit ",
            "Auto-calculate professional tax ",
            "Professional tax for current month ",
            "View state tax notification ",
            "Show deducted tax summary ",
            "Can I generate tax receipts "
        ],
        "pf_master": [
            "Show me the PF master settings ",
            "How do I configure PF for employees ",
            "Update PF rates in master ",
            "Add a new PF entry ",
            "Edit PF master record ",
            "Delete a PF rule ",
            "What is the current PF contribution rate ",
            "Is employer contribution in PF ",
            "How is PF calculated ",
            "View list of PF rules ",
            "Which employees have PF enabled ",
            "Can I disable PF for some employees ",
            "Set up PF for new staff ",
            "Apply PF to all employees ",
            "Monthly PF deduction settings ",
            "Yearly PF contribution details ",
            "Generate PF report ",
            "PF master download option ",
            "Show employer vs employee PF contribution ",
            "Enable automatic PF calculation ",
            "PF percentage setting location ",
            "Add voluntary PF contribution ",
            "PF rule for contractual staff ",
            "Does PF vary by department ",
            "Where to enter UAN in PF master ",
            "Which PF slab is currently active ",
            "Reset PF configuration ",
            "Import PF data from file ",
            "What is the minimum wage for PF ",
            "Are interns included in PF ",
            "Link PF rule with salary structure ",
            "PF rules for part-time employees ",
            "Get historical PF contribution logs ",
            "PF deduction threshold value ",
            "View PF contribution timeline ",
            "Where is PF master in the payroll module ",
            "Edit PF contribution formula ",
            "Define fixed PF amount ",
            "Is there a cap on PF contribution ",
            "How to exempt an employee from PF ",
            "PF contribution changes for this year ",
            "Assign PF rule by grade ",
            "Create new PF templates ",
            "Bulk update PF master ",
            "Generate PF ledger ",
            "PF master configuration summary ",
            "Show active PF policies ",
            "Can I duplicate an old PF record ",
            "What does PF master manage ",
            "Save PF setup changes "
        ],
        "esic_master": [
            "Open the ESIC master settings ",
            "How do I configure ESIC in payroll ",
            "Add new ESIC record ",
            "Edit an ESIC policy ",
            "Delete ESIC rule from system ",
            "Current ESIC contribution rate ",
            "Who is eligible for ESIC ",
            "ESIC limit for employees ",
            "Update ESIC contribution values ",
            "Monthly ESIC deduction settings ",
            "Enable ESIC for certain employees ",
            "How to remove ESIC from employee ",
            "What is the salary limit for ESIC ",
            "Assign ESIC rules in payroll ",
            "Generate ESIC summary report ",
            "Can I apply different ESIC slabs ",
            "Where to manage ESIC data ",
            "Employer vs employee ESIC split ",
            "How is ESIC calculated ",
            "ESIC master configuration guide ",
            "Is ESIC compulsory ",
            "Setup ESIC for newly joined staff ",
            "Where to enter ESIC number ",
            "Get ESIC deduction history ",
            "Link ESIC with department rules ",
            "Create ESIC template ",
            "Can I disable ESIC for one month ",
            "Get monthly ESIC ledger ",
            "Who qualifies for ESIC benefit ",
            "What is ESIC contribution formula ",
            "ESIC exemption rules ",
            "ESIC for part-time workers ",
            "Salary range for ESIC eligibility ",
            "Bulk update ESIC info ",
            "Is ESIC auto calculated ",
            "Check ESIC applicability ",
            "Enable ESIC for specific roles ",
            "Where to add ESIC policy document ",
            "What is covered under ESIC ",
            "Can I view ESIC per employee ",
            "How to verify ESIC details ",
            "ESIC deduction cap value ",
            "How to print ESIC summary ",
            "Can interns be under ESIC ",
            "Show all ESIC rules ",
            "Edit ESIC based on new government rules ",
            "Deactivate old ESIC rule ",
            "Import ESIC data from Excel ",
            "ESIC policy management page ",
            "Save changes in ESIC master "
        ],
        "exclude_parameters": [
            "How do I exclude parameters from payroll ",
            "Show excluded parameters ",
            "Add parameter to exclusion list ",
            "Remove parameter from exclusions ",
            "What is the exclude parameter section for ",
            "Can I temporarily exclude a parameter ",
            "List of all excluded salary parameters ",
            "Why exclude a parameter ",
            "Which parameters are currently excluded ",
            "Exclude bonus from current month ",
            "Exclude overtime payment ",
            "How to set conditional exclusions ",
            "Exclude parameters for interns ",
            "Bulk exclude salary components ",
            "Edit exclusion rules ",
            "View reason for parameter exclusion ",
            "How to override exclusion settings ",
            "Exclude specific allowance ",
            "Exclude parameter for selected department ",
            "Setup monthly exclusions ",
            "Where to find exclude parameter master ",
            "Can I exclude parameter by employee group ",
            "Exclude tax from net salary ",
            "Show summary of excluded items ",
            "Exclude all deductions this month ",
            "Why are some parameters not processed ",
            "Temporarily skip a parameter in salary ",
            "What happens when a parameter is excluded ",
            "Re-include a previously excluded parameter ",
            "View excluded parameter history ",
            "Auto exclude inactive parameters ",
            "Parameter exclusion policy setup ",
            "Download excluded parameter list ",
            "Who authorized parameter exclusion ",
            "Show excluded parameters per employee ",
            "Define new exclusion rules ",
            "Save exclusion settings ",
            "Reset all excluded parameters ",
            "Can I exclude multiple parameters ",
            "Schedule parameter exclusion ",
            "Enable alerts for exclusions ",
            "Is exclusion permanent ",
            "What are system default exclusions ",
            "Exclude a parameter from reports ",
            "Mark allowance as excluded ",
            "Exclude specific grade parameters ",
            "Compare excluded vs included parameters ",
            "How to lock excluded parameters ",
            "Exclude with justification note ",
            "Recalculate salary after exclusion "
        ],
        'salary_configuration': [
            "How do I configure salary details ",
            "Can I change the salary structure ",
            "How do I set up salary bands ",
            "Can I configure salary increments ",
            "How do I define bonus structures in salary configuration ",
            "How do I set up deductions in salary configuration ",
            "Can I configure employee-specific salary rules ",
            "How do I adjust for tax deductions in salary configuration ",
            "Can I configure overtime payment in salary configuration ",
            "How do I link salary configuration to payroll masters ",
            "Can I set salary configurations for different departments ",
            "How do I include allowances in salary configuration ",
            "Can I customize salary configuration based on job roles ",
            "How do I configure salary payment frequency ",
            "Can I configure salary payment methods ",
            "How do I manage salary increases in salary configuration ",
            "Can I configure special bonuses in salary configuration ",
            "How do I configure medical benefits in salary configuration ",
            "Can I configure retirement benefits in salary configuration ",
            "How do I configure pay scales in salary configuration ",
            "Can I configure salary structure for different grades ",
            "How do I apply salary deductions for loans ",
            "Can I configure salary based on work location ",
            "How do I include performance-based incentives in salary configuration ",
            "Can I set up salary configuration for contract employees ",
            "How do I include tax exemptions in salary configuration ",
            "Can I configure salary rules for full-time vs part-time employees ",
            "How do I configure salary based on experience level ",
            "Can I set salary for apprentices and interns ",
            "How do I configure salary payment based on attendance ",
            "Can I configure salary adjustments for leave ",
            "How do I configure salary payment for remote workers ",
            "Can I configure salary increases automatically ",
            "How do I link salary configuration to employee performance ",
            "Can I set salary increments for promotions ",
            "How do I configure salary rules for temporary employees ",
            "Can I configure different salary bands for different skill levels ",
            "How do I set up salary configuration for seasonal workers ",
            "Can I configure salary payment based on sales targets ",
            "How do I configure tax calculations for salary ",
            "Can I link salary configuration to employee benefits ",
            "How do I manage salary configuration for employees on probation ",
            "Can I configure different salary for different shift types ",
            "How do I adjust salary for changes in government regulations ",
            "Can I apply salary changes retroactively ",
            "How do I track salary configuration changes over time "
        ],
        'process_attendance': [
            "How do I process attendance for employees ",
            "Can I process attendance for a specific date ",
            "How do I mark attendance in the payroll system ",
            "Can I process attendance based on time-in and time-out ",
            "How do I handle partial attendance in payroll ",
            "Can I process attendance for overtime work ",
            "How do I mark leave in attendance processing ",
            "Can I mark half-days in attendance processing ",
            "How do I correct attendance errors after processing ",
            "Can I process attendance for remote employees ",
            "How do I handle employee absences in attendance processing ",
            "Can I process attendance for different shifts ",
            "How do I manage holidays in attendance processing ",
            "Can I process attendance manually for employees ",
            "How do I integrate attendance data with payroll ",
            "Can I generate attendance reports after processing ",
            "How do I manage attendance for contract workers ",
            "Can I handle attendance for employees working from home ",
            "How do I adjust attendance processing for public holidays ",
            "Can I process attendance for employees on leave ",
            "How do I process attendance for part-time employees ",
            "Can I configure attendance rules for different teams ",
            "How do I handle attendance discrepancies after processing ",
            "Can I update attendance records after processing ",
            "How do I process attendance data in bulk ",
            "Can I process attendance based on biometric data ",
            "How do I handle attendance for multiple work locations ",
            "Can I mark attendance for employees using mobile apps ",
            "How do I process attendance for shift-based workers ",
            "Can I import attendance data from external systems ",
            "How do I manage attendance for salaried employees ",
            "Can I adjust attendance for unexpected leave ",
            "How do I process attendance for employees on business trips ",
            "Can I process attendance for employees in multiple departments ",
            "How do I process attendance for trainees or interns ",
            "Can I check attendance records after processing ",
            "How do I process attendance for employees on sabbatical ",
            "Can I exclude certain hours from attendance processing ",
            "How do I process attendance in case of system errors ",
            "Can I calculate wages based on processed attendance ",
            "How do I handle late arrivals in attendance processing ",
            "Can I generate alerts for incorrect attendance records ",
            "How do I process attendance for a specific department ",
            "Can I process attendance based on employee ID ",
            "How do I process attendance data for a specific period "
        ],
        'keyboard_parameters': [
            "How do I set keyboard parameters in payroll ",
            "Can I customize keyboard parameters for payroll processing ",
            "How do I change the keyboard shortcuts in payroll settings ",
            "Can I adjust keyboard parameters for specific payroll functions ",
            "How do I configure keyboard parameters for attendance entry ",
            "Can I disable certain keyboard shortcuts in payroll ",
            "How do I reset the keyboard parameters to default settings ",
            "Can I modify the keyboard layout for payroll ",
            "How do I set keyboard parameters for generating salary slips ",
            "Can I assign specific keys for salary processing tasks ",
            "How do I configure keyboard shortcuts for payroll reports ",
            "Can I adjust keyboard parameters for different users ",
            "How do I change the default keyboard commands in payroll ",
            "Can I set keyboard parameters for salary release ",
            "How do I modify keyboard settings for payroll masters ",
            "Can I assign special functions to keyboard keys in payroll ",
            "How do I configure keyboard shortcuts for employee data entry ",
            "Can I create custom keyboard shortcuts for payroll processing ",
            "How do I use keyboard parameters for data entry in payroll ",
            "Can I enable or disable keyboard shortcuts for specific actions ",
            "How do I assign functions like save, submit to specific keys ",
            "Can I assign new functions to keyboard shortcuts in payroll ",
            "How do I reset keyboard parameters if they are not working ",
            "Can I use keyboard shortcuts for processing salary calculations ",
            "How do I modify the key bindings for specific payroll tasks ",
            "Can I use special keys for payroll data verification ",
            "How do I configure keyboard parameters for report generation ",
            "Can I assign keyboard shortcuts for specific payroll modules ",
            "How do I set up user-specific keyboard preferences in payroll ",
            "Can I configure keyboard shortcuts for payroll approval ",
            "How do I enable hotkeys for payroll processes ",
            "Can I change the function of specific keys in payroll ",
            "How do I assign a key for processing salary adjustments ",
            "Can I enable numeric keyboard parameters for data entry ",
            "How do I configure key press durations in payroll ",
            "Can I restrict certain keyboard inputs in payroll processing ",
            "How do I link keyboard parameters to payroll settings ",
            "Can I integrate custom keyboard parameters into payroll ",
            "How do I save and apply keyboard parameters in payroll "
        ],
        'process_salary': [
            "How do I process salaries in the payroll system ",
            "Can I process salary for multiple employees at once ",
            "How do I calculate salary for an employee ",
            "Can I process salary adjustments after calculation ",
            "How do I include bonuses in salary processing ",
            "Can I deduct taxes while processing salary ",
            "How do I handle overtime in salary processing ",
            "Can I process salary for part-time employees ",
            "How do I process salary for employees on leave ",
            "Can I generate salary slips after processing ",
            "How do I ensure correct salary calculation for all employees ",
            "Can I process salary based on attendance data ",
            "How do I process salary for contract workers ",
            "Can I generate salary reports after processing ",
            "How do I process salary for employees working different shifts ",
            "Can I process salary for remote employees ",
            "How do I ensure compliance with tax regulations during salary processing ",
            "Can I adjust salary based on performance during processing ",
            "How do I process salary for employees in different departments ",
            "Can I exclude certain employees from salary processing ",
            "How do I handle salary adjustments for new hires ",
            "Can I verify salary details before processing ",
            "How do I process salary using automated systems ",
            "Can I reprocess salary after errors are found ",
            "How do I process salary payments based on employee grade ",
            "Can I set salary processing for different frequencies ",
            "How do I ensure salary payments are accurate ",
            "Can I process salary using predefined configurations ",
            "How do I check for discrepancies in salary processing ",
            "Can I generate a report of processed salaries ",
            "How do I process salary for seasonal employees ",
            "Can I automate salary processing for future months ",
            "How do I calculate deductions and contributions during salary processing ",
            "Can I modify salary figures after processing ",
            "How do I ensure timely salary payments during processing ",
            "Can I link salary processing to employee benefits ",
            "How do I process salary for employees on special leave ",
            "Can I process salary for employees in multiple locations ",
            "How do I calculate and deduct loan payments during salary processing ",
            "Can I process salary for trainees and apprentices "
        ],
        'release_salary': [
            "How do I release salary for employees ",
            "Can I release salary for all employees at once ",
            "How do I ensure salary release is accurate ",
            "Can I delay salary release for certain employees ",
            "How do I release salary based on performance ",
            "Can I release salary early for some employees ",
            "How do I set up salary release dates ",
            "Can I release salary via direct bank transfer ",
            "How do I confirm salary release for employees ",
            "Can I print salary slips before releasing salary ",
            "How do I generate salary release reports ",
            "Can I schedule salary release for the future ",
            "How do I ensure compliance with salary release regulations ",
            "Can I release salary in different currencies ",
            "How do I release salary after deductions ",
            "Can I release salary only after approval ",
            "How do I track salary release for different departments ",
            "Can I release salary via mobile payment systems ",
            "How do I prevent errors during salary release ",
            "Can I release salary for employees on leave ",
            "How do I release salary to contractors ",
            "Can I configure salary release for part-time employees ",
            "How do I adjust salary release for employees on probation ",
            "Can I release salary for seasonal workers ",
            "How do I release salary after a salary increase ",
            "Can I generate an alert for upcoming salary release dates ",
            "How do I ensure employees receive salary on time ",
            "Can I verify salary release status ",
            "How do I release salary for employees working multiple shifts ",
            "Can I link salary release to attendance records ",
            "How do I notify employees about salary release ",
            "Can I release salary based on work location ",
            "How do I handle errors during salary release ",
            "Can I release salary after correcting discrepancies ",
            "How do I release salary for employees with loans ",
            "Can I release salary payments in installments ",
            "How do I ensure salary release meets legal requirements ",
            "Can I cancel salary release after it’s been initiated ",
            "How do I release salary for contract workers "
        ],
        'payroll_reports': [
            "How do I generate payroll reports ",
            "Can I generate a detailed payroll report ",
            "How do I filter payroll reports by department ",
            "Can I export payroll reports to Excel ",
            "How do I generate tax reports from payroll ",
            "Can I generate salary summaries in payroll reports ",
            "How do I view employee earnings in payroll reports ",
            "Can I filter payroll reports by date range ",
            "How do I generate bonus reports in payroll ",
            "Can I track deductions in payroll reports ",
            "How do I generate payroll reports for part-time employees ",
            "Can I include overtime details in payroll reports ",
            "How do I generate payroll reports for contractors ",
            "Can I customize payroll report formats ",
            "How do I generate a payroll report for a specific employee ",
            "Can I generate payroll reports by job title ",
            "How do I generate payroll reports for the entire company ",
            "Can I generate payroll reports for specific payroll cycles ",
            "How do I include tax deductions in payroll reports ",
            "Can I generate reports for employee benefits ",
            "How do I track salary adjustments in payroll reports ",
            "Can I generate payroll reports for a single department ",
            "How do I manage payroll reports for multiple years ",
            "Can I analyze payroll data from reports ",
            "How do I handle errors in payroll reports ",
            "Can I generate payroll reports for seasonal workers ",
            "How do I generate payroll reports for employees on leave ",
            "Can I analyze payroll trends from reports ",
            "How do I generate attendance-based payroll reports ",
            "Can I generate reports on employee deductions ",
            "How do I include loan repayments in payroll reports ",
            "Can I generate payroll reports for multiple locations ",
            "How do I generate payroll reports based on salary band ",
            "Can I generate payroll reports for different job roles ",
            "How do I generate summary reports for payroll ",
            "Can I compare payroll reports across different departments ",
            "How do I track employee leave in payroll reports ",
            "Can I generate payroll reports for union employees ",
            "How do I generate salary adjustment reports ",
            "Can I generate payroll tax reports for auditing ",
            "How do I include deductions for benefits in payroll reports ",
            "Can I generate payroll reports for different pay frequencies ",
            "How do I manage the generation of payroll reports "
        ],
        "consolidated_sheet": [
            "Show me the consolidated payroll sheet ",
            "Download the salary consolidated report ",
            "How can I view the overall salary summary ",
            "Where is the payroll consolidated sheet ",
            "Give me a summary of all employee salaries ",
            "Generate consolidated salary statement ",
            "View monthly consolidated salary sheet ",
            "Export consolidated payroll data ",
            "Where can I find the full salary sheet ",
            "Print consolidated payroll sheet ",
            "Is there a summary of all payroll ",
            "Payroll consolidated report download ",
            "How do I get a complete salary overview ",
            "Full payroll report in one place ",
            "I need a payroll sheet for all staff ",
            "Company-wide salary consolidated sheet ",
            "Summary of gross and net salaries ",
            "All-in-one salary sheet needed ",
            "Get a full month salary summary ",
            "Can I see all salaries together ",
            "Generate all department salary sheet ",
            "Monthly salary consolidated view ",
            "Create payroll consolidated sheet ",
            "Give me payroll report by department ",
            "Where is the final payroll sheet ",
            "Consolidated sheet with all deductions ",
            "View complete payroll deductions ",
            "Single sheet with all employee payments ",
            "Can I filter payroll by department ",
            "Give me summary of this month's salaries ",
            "I need a salary breakdown report ",
            "Payroll monthly overview report ",
            "Summarized salary report request ",
            "Where to find consolidated payroll records ",
            "Department-wise payroll summary ",
            "Export payroll summary to Excel ",
            "Consolidated payroll PDF report ",
            "Full payroll summary dashboard ",
            "List all salaries in one sheet ",
            "Salary register report ",
            "Consolidated net pay report ",
            "Gross vs net pay summary ",
            "Total salary paid report ",
            "Print payroll summary chart ",
            "Complete payroll audit sheet ",
            "Payroll overview for all staff ",
            "Merged payroll record report ",
            "All-inclusive salary sheet ",
            "Payroll overall summary file ",
            "Salary breakdown by group "
        ],
        "salary_slip": [
            "Show me my salary slip ",
            "Download employee salary slip ",
            "How do I get my pay slip ",
            "Where can I view the salary slip ",
            "Generate monthly salary slip ",
            "Print last month's salary slip ",
            "Salary slip for March please ",
            "Get salary slip in PDF format ",
            "I need a soft copy of my salary slip ",
            "Email my salary slip ",
            "Salary slip for all employees ",
            "Download salary slips in bulk ",
            "Generate pay slip for selected employee ",
            "View employee pay details ",
            "Check net pay in salary slip ",
            "Show me salary breakup ",
            "How do I create a pay slip ",
            "Customize salary slip template ",
            "Show deductions in my salary slip ",
            "Allowances section of salary slip ",
            "Gross vs net pay in slip ",
            "Can I regenerate an old salary slip ",
            "Print salary slip with signature ",
            "Export salary slip to Excel ",
            "Get detailed salary slip report ",
            "How many days counted in this pay slip ",
            "What is HRA in my salary slip ",
            "Basic salary in slip preview ",
            "Show yearly salary slips ",
            "Do I have bonuses in my pay slip ",
            "Generate salary slip by department ",
            "Group-wise salary slip report ",
            "Get auto-generated salary slip ",
            "Salary slip approval process ",
            "HR view of employee salary slips ",
            "Download pay slip for specific date ",
            "How is PF shown in the salary slip ",
            "TDS value in my pay slip ",
            "Include leave days in salary slip ",
            "Salary slip with UAN details ",
            "Is salary slip sent via email ",
            "Salary slip format settings ",
            "Pay slip with all components ",
            "Upload salary slip to employee portal ",
            "How to distribute salary slips ",
            "Employee pay slip record ",
            "One-click salary slip download ",
            "Get historical salary slips ",
            "View salary slip by ID ",
            "Salary slip with bank details "
        ],
        "consolidated_salary_yearly": [
            "Give me the yearly consolidated salary report ",
            "Download annual salary summary ",
            "Show me year-wise salary data ",
            "Yearly payroll summary please ",
            "All employees' yearly pay report ",
            "How much salary paid this year ",
            "Get employee annual salary total ",
            "Total yearly payout report ",
            "Generate full-year salary report ",
            "Annual salary summary by employee ",
            "Year-end payroll summary ",
            "Consolidated yearly pay statement ",
            "Salary paid to each staff this year ",
            "Download salary report by year ",
            "I want total yearly salary expenses ",
            "Year-wise salary breakdown ",
            "Monthly salary vs yearly report ",
            "Department-wise annual salary report ",
            "Yearly salary sheet PDF ",
            "Print annual payroll data ",
            "Annual payroll overview ",
            "Total gross and net pay yearly ",
            "Employee annual pay history ",
            "How much has HR spent this year ",
            "Year-end salary balance sheet ",
            "Create consolidated salary for year ",
            "Total earnings per employee annually ",
            "Download all employee annual salaries ",
            "Track annual salary by designation ",
            "Export yearly salary data ",
            "Generate report for last fiscal year ",
            "How many increments this year ",
            "Salary growth report by year ",
            "Annual gross pay report ",
            "Payroll data for financial year ",
            "Total amount paid this year ",
            "Print yearly pay chart ",
            "Company salary payout for the year ",
            "Year-wise bonus report ",
            "Employee-wise annual pay record ",
            "Download complete yearly report ",
            "Compare yearly payroll with last year ",
            "Salary distribution throughout the year ",
            "Average yearly salary ",
            "Max and min salary of the year ",
            "Full year payroll audit report ",
            "Taxable income for the year ",
            "List all yearly pay slips ",
            "Cumulative salary report ",
            "Annual consolidated salary dashboard "
        ],
        'employee_salary_year': [
        	"Show me the employee salary report for the year",
        	"I want to see the yearly salary of an employee",
        	"How can I check the employee's salary year-wise ",
        	"Give me the annual salary report of staff",
        	"Can I get the salary report of this year ",
        	"Please provide employee salary data for the full year",
        	"Fetch the year-end salary summary of employees",
        	"Where is the employee's salary info for the entire year ",
        	"Get me the total salary paid this year to employees",
        	"Need the employee salary report of 2024",
        	"Generate the salary details of employees for last year",
        	"Download the yearly salary report",
        	"I need last year's employee salary summary",
        	"Give me a breakdown of annual salaries",
        	"Show the total salary given to staff this year",
        	"Employee annual payroll report, please",
        	"Summarize employee salaries for the year",
        	"Where can I get the yearly salary info ",
        	"View staff annual salary",
        	"Display full-year salary details",
        	"I want to see salary of employee for full year",
        	"List the yearly salary figures",
        	"Give complete year salary information",
        	"Get employee yearly payslip report",
        	"How much salary did employees get this year ",
        	"Employee pay for the whole year",
        	"Get full salary details for 2023",
        	"Summarize salary for the fiscal year",
        	"Generate yearly pay summary",
        	"Annual salary overview",
        	"Can you pull yearly salary data ",
        	"I need yearly payroll sheet",
        	"Show me yearly earnings of employees",
        	"Monthly to yearly salary transition report",
        	"Pull employee salary data for the whole session",
        	"Staff salary for academic year",
        	"Fetch yearly salary file",
        	"Download full employee salary report",
        	"Show cumulative salary for staff this year",
        	"Give me employee total salary this year",
        	"Report on yearly salary",
        	"Show consolidated salary of employee",
        	"Employee’s yearly take-home pay",
        	"Total employee salary paid annually",
        	"Where is the full year salary info ",
        	"Generate salary sheet for financial year",
        	"Get me a copy of the yearly employee salary",
        	"How to view employee salary report annually ",
        	"Year-wise salary details please"
        ],
        'pf_report': [
        	"Show me the PF report",
        	"I need to see the Provident Fund details",
        	"Where can I find the PF data ",
        	"Download employee PF report",
        	"Give PF contribution report",
        	"How much PF was deducted ",
        	"Provide the staff PF summary",
        	"Generate PF details of employees",
        	"Show Provident Fund deduction report",
        	"I want to view employee PF records",
        	"Get me the monthly PF contribution report",
        	"PF breakdown by employee",
        	"Year-wise PF data",
        	"View staff PF deductions",
        	"List PF amount employee-wise",
        	"Show PF deduction history",
        	"PF contribution overview",
        	"Where is the PF report for 2024 ",
        	"Total PF submitted this year",
        	"Employee PF amount details",
        	"PF deduction list",
        	"Summarize employee Provident Fund",
        	"PF savings report",
        	"Download PF transaction data",
        	"Give last month's PF report",
        	"PF details by department",
        	"Check employee Provident Fund entries",
        	"Generate PF data by date",
        	"PF summary for last quarter",
        	"Show total PF contributions this month",
        	"PF amount generated per employee",
        	"Employee-wise PF sheet",
        	"PF deduction table",
        	"Report on employee PF",
        	"Export PF contribution file",
        	"PF compliance summary",
        	"Verify Provident Fund deductions",
        	"Get employee PF status",
        	"Monthly Provident Fund report",
        	"Show PF and employer contribution",
        	"Detailed Provident Fund report",
        	"Show me staff-wise PF",
        	"Total organization PF deductions",
        	"PF ledger report",
        	"Employee Provident Fund sheet",
        	"Where’s the PF summary ",
        	"PF transactions for the month",
        	"Get Provident Fund details for HR",
        	"Generate company PF report"
        ],
        'ptax_report': [
        	"Show PTAX report",
        	"Provide Professional Tax report",
        	"I want to see PTAX data",
        	"Download Professional Tax records",
        	"Where can I view PTAX info ",
        	"Professional Tax deduction summary",
        	"Generate PTAX details",
        	"Give me the staff PTAX report",
        	"Show monthly PTAX deduction",
        	"PTAX contribution breakdown",
        	"Get PTAX data for employees",
        	"Year-wise Professional Tax info",
        	"PTAX deduction report",
        	"View Professional Tax history",
        	"PTAX report for 2023",
        	"Department-wise PTAX details",
        	"Export PTAX transaction report",
        	"List employee PTAX payments",
        	"PTAX salary deduction list",
        	"Report on employee tax deduction",
        	"Monthly PTAX ledger",
        	"Professional Tax analysis",
        	"PTAX compliance sheet",
        	"PTAX breakdown by date",
        	"Total PTAX for organization",
        	"Employee-wise PTAX summary",
        	"Fetch PTAX contributions",
        	"PTAX ledger download",
        	"Detailed PTAX file",
        	"Give PTAX report for HR",
        	"PTAX deduction by staff",
        	"Professional Tax remittance list",
        	"PTAX contribution of all employees",
        	"Where is PTAX report ",
        	"Company-wide PTAX record",
        	"Download PTAX file",
        	"Monthly Professional Tax details",
        	"Get staff-wise PTAX sheet",
        	"PTAX paid per employee",
        	"Summarize PTAX data",
        	"Show Professional Tax for last month",
        	"PTAX deducted by finance",
        	"Professional Tax amount overview",
        	"PTAX compliance data",
        	"PTAX due list",
        	"Download organization PTAX summary",
        	"List PTAX by employee ID",
        	"Professional Tax collected in 2023",
        	"Give me PTAX records for audit"
        ],
        'bank_statement': [
        	"Show employee bank statement",
        	"Download payroll bank file",
        	"Where is the salary bank statement ",
        	"Get bank salary transfer details",
        	"Employee bank transaction list",
        	"View staff salary credited to bank",
        	"Generate bank transfer report",
        	"Bank statement for employee salaries",
        	"Payroll bank summary needed",
        	"Show bank credited salaries",
        	"Staff bank transfer sheet",
        	"Monthly bank credit report",
        	"Download employee bank slip",
        	"View salary bank records",
        	"Report of salary transferred to bank",
        	"Bank-wise salary credit list",
        	"Statement showing bank payments",
        	"Give me salary bank transaction report",
        	"HR bank salary report",
        	"List of bank credited payrolls",
        	"Bank deposit report of employee salary",
        	"Summary of bank salary transactions",
        	"Total salary paid via bank",
        	"View bank ledger of salary",
        	"Generate salary bank log",
        	"Get salary bank transaction list",
        	"Employee salary deposit report",
        	"Show bank transfer payroll",
        	"Download bank deposit sheet",
        	"List employee payments to bank",
        	"Bank payroll export file",
        	"Monthly salary bank log",
        	"Statement of payroll via bank",
        	"Get bank transaction details for HR",
        	"Staff payment report to banks",
        	"Display payroll bank report",
        	"Get bank transaction summary",
        	"Check salary credited through bank",
        	"Employee bank ledger for salary",
        	"Bank record of staff salaries",
        	"Show list of salaries paid via bank",
        	"Retrieve bank statements for payroll",
        	"Bank-wise payroll transfer list",
        	"Give HR the payroll bank record",
        	"Show employee-wise bank salary log",
        	"Report of salary bank statements",
        	"Export bank payroll statement",
        	"Payroll salary deposit record",
        	"Print employee bank payroll statement"
        ],
        'comparative_salary': [
        	"Show comparative salary report",
        	"Compare salaries of employees",
        	"Display salary difference chart",
        	"Get salary comparison data",
        	"Provide comparative salary sheet",
        	"Employee salary comparison report",
        	"Generate staff salary comparison",
        	"Compare current vs previous salary",
        	"View salary hike trends",
        	"Employee increment comparison",
        	"Compare salaries by department",
        	"Compare salaries by year",
        	"Show increase in employee salaries",
        	"List salary variations",
        	"Salary increment report needed",
        	"Employee-wise salary comparison",
        	"Download comparative salary file",
        	"Generate pay hike chart",
        	"Monthly vs yearly salary comparison",
        	"Yearly employee salary hike report",
        	"Department salary comparison",
        	"Cross-employee salary comparison",
        	"Employee growth chart in salary",
        	"Compare last year vs this year salaries",
        	"Salary analysis report",
        	"Show improvement in employee pay",
        	"View salary raise across departments",
        	"Comparison sheet for HR",
        	"Payroll growth report",
        	"Give raise percentage data",
        	"Trend of salary over years",
        	"Monthly comparative payroll report",
        	"Year-end salary comparison",
        	"Get staff-wise comparative salaries",
        	"Report comparing payroll by time",
        	"Compare staff salaries per quarter",
        	"Salary hike visualization",
        	"Show old vs new salary",
        	"Compare all employees’ salaries",
        	"Get growth trend in salary",
        	"Analyze staff salary progress",
        	"Employee raise tracking report",
        	"Employee salary over time chart",
        	"Compare staff packages",
        	"Salary upgrade record",
        	"Download comparative report",
        	"Show department-level salary hike",
        	"HR needs comparative payroll report",
        	"Salary increase by designation",
        	"Comparative pay report requested"
        ],
        'esic_report': [
        	"Show ESIC report",
        	"Download ESIC contribution file",
        	"Give employee ESIC summary",
        	"Where is the ESIC report ",
        	"Employee-wise ESIC deduction",
        	"Monthly ESIC report needed",
        	"Generate ESIC data",
        	"List ESIC contributions",
        	"ESIC summary for HR",
        	"Detailed ESIC deduction log",
        	"Provide ESIC employee sheet",
        	"Generate ESIC history",
        	"Show ESIC deduction for last month",
        	"List ESIC employee details",
        	"View employee insurance contributions",
        	"Employee ESIC transactions",
        	"ESIC monthly log",
        	"Give organization ESIC report",
        	"List ESIC by department",
        	"Total ESIC deducted this year",
        	"Show latest ESIC report",
        	"Give HR ESIC sheet",
        	"ESIC contributions this quarter",
        	"Where’s staff ESIC info ",
        	"List ESIC deductions per staff",
        	"ESIC deduction analysis",
        	"Provide full ESIC ledger",
        	"Show ESIC data per employee",
        	"ESIC report download required",
        	"ESIC payroll breakdown",
        	"Insurance contribution report",
        	"View ESIC per month",
        	"Show ESIC ledger",
        	"Generate ESIC statements",
        	"Download ESIC log",
        	"Employee-wise ESIC deductions summary",
        	"Display ESIC tracking",
        	"Provide HR with ESIC sheet",
        	"Company-wide ESIC record",
        	"Where to check ESIC details ",
        	"Check staff ESIC records",
        	"ESIC payment proof list",
        	"List of ESIC payees",
        	"Insurance compliance report",
        	"Display staff health insurance deductions",
        	"Print ESIC ledger",
        	"HR ESIC monthly summary",
        	"Give payroll team the ESIC summary",
        	"Provide insurance report for audit"
        ],
        'tds_report': [
        	"Show TDS report",
        	"I need TDS details",
        	"Download employee TDS file",
        	"Where is the tax deduction summary ",
        	"Generate staff TDS data",
        	"List TDS deductions",
        	"Yearly TDS report required",
        	"Employee-wise TDS info",
        	"Provide tax deduction file",
        	"Show monthly TDS deductions",
        	"List of staff with TDS cut",
        	"Get TDS contribution details",
        	"Tax deducted from employee salaries",
        	"TDS summary for the year",
        	"Get employee tax ledger",
        	"Display employee TDS details",
        	"Generate tax report",
        	"Employee income tax report",
        	"Where’s the TDS data ",
        	"HR needs employee tax summary",
        	"Employee-wise TDS chart",
        	"Get TDS deductions this quarter",
        	"Tax ledger of staff",
        	"Monthly tax report",
        	"Provide tax deducted list",
        	"Get TDS slip",
        	"Show tax payment by employee",
        	"List employee TDS contribution",
        	"TDS sheet for payroll",
        	"Download TDS sheet",
        	"TDS file for audit",
        	"Export income tax deductions",
        	"Salary TDS breakdown",
        	"Tax credit report",
        	"TDS deduction overview",
        	"Print employee tax report",
        	"TDS history of staff",
        	"Company TDS ledger",
        	"Summarize employee TDS",
        	"TDS per month",
        	"Employee-wise tax sheet",
        	"TDS amount deducted per person",
        	"Get tax report by date",
        	"Download monthly TDS ledger",
        	"Show full-year TDS details",
        	"View tax deducted till date",
        	"Total tax collected from staff",
        	"HR TDS reporting",
        	"Send TDS data to finance",
        	"Employee income tax deduction log"
        ]
    },

        'content' : {
        "document_category": [
          "Show me the list of document categories",
          "What document categories are available ",
          "Can I view all document categories ",
          "List document categories for me",
          "Give me the document categories",
          "Display all document category types",
          "Which document categories are in use ",
          "How many document categories exist ",
          "Provide document category details",
          "Tell me about the document categories",
          "Where can I find document categories ",
          "Show document category records",
          "Access the document category list",
          "Document categories available now ",
          "Retrieve document category list",
          "Can I get the document categories ",
          "Which categories are used for documents ",
          "Available document classification types ",
          "Give list of document category names",
          "Get me document categories",
          "Tell me all the categories for documents",
          "Search for document categories",
          "Document category master list",
          "Fetch document category options",
          "View document categorization types",
          "Types of categories for documents ",
          "Category names for stored documents ",
          "Need document categories",
          "I want to check document categories",
          "Can you show document categories ",
          "What's in the document category list ",
          "Let me see all document categories",
          "Details about document categories please",
          "Access document category section",
          "Open the document category module",
          "Document category configuration ",
          "Manage document categories",
          "Edit document categories",
          "Update document category list",
          "Download document categories",
          "View current document categories",
          "Document category master data",
          "Help me find document categories",
          "List all types of document categories",
          "What are the document groups ",
          "Check document classification",
          "Are there document categories in the system ",
          "Document category list view please",
          "Document category filter options",
          "Open document category page"
        ],
        "album_master": [
            "Show me the album master",
            "Open the album master section",
            "I want to see the album master",
            "List all albums in the album master",
            "Access album master records",
            "Display all albums from the master",
            "Get details of the album master",
            "Retrieve album master data",
            "Can you show the album master list ",
            "What albums are in the album master ",
            "Browse album master entries",
            "Which albums are listed in album master ",
            "Show album categories in album master",
            "Album master list please",
            "View the album master data",
            "Tell me the album master records",
            "Fetch album master entries",
            "Look up album master info",
            "Album master overview",
            "Access the album master module",
            "Where can I find the album master ",
            "How many albums are in the album master ",
            "Is there an album master section ",
            "Open the album master page",
            "What is inside album master ",
            "Show album master dashboard",
            "Explore album master options",
            "Display album master settings",
            "Give me the album master details",
            "Album master configuration view",
            "Show all photo albums in album master",
            "Album master table view",
            "Update album master list",
            "Edit entries in album master",
            "Create a new entry in album master",
            "Manage albums through album master",
            "Album master data table",
            "Album categories in master section",
            "Albums available in the master module",
            "Album master details required",
            "Print album master list",
            "Check all entries in album master",
            "Show latest albums in album master",
            "Which albums are configured in master ",
            "What does the album master contain ",
            "Search album master records",
            "Find album master content",
            "Album master item list",
            "Upload to album master",
            "Album master report view"
        ],
        "announcement_type": [
            "Show me the announcement types",
            "List all available announcement types",
            "What are the different announcement types ",
            "Display the announcement type list",
            "Give me all the announcement types",
            "I want to view announcement types",
            "Fetch announcement type options",
            "Which announcement types are there ",
            "Open the announcement type section",
            "View announcement types in the system",
            "What types of announcements can we make ",
            "Access announcement type settings",
            "Check all announcement types",
            "How many announcement types exist ",
            "Browse announcement type master",
            "Display all types of announcements",
            "Can you show the announcement categories ",
            "Categories under announcement types ",
            "Provide announcement type records",
            "Retrieve announcement type entries",
            "Where can I manage announcement types ",
            "Announcement types available right now ",
            "Edit the list of announcement types",
            "Add new announcement type",
            "Announcement type configuration panel",
            "Open announcement type module",
            "What announcement types are defined ",
            "Show me custom announcement types",
            "Announcement type options view",
            "Download announcement types",
            "Upload announcement types",
            "Announcement type master list please",
            "I need all the announcement types",
            "Check available types of announcements",
            "Review announcement type data",
            "Manage announcement types",
            "Announcement type section overview",
            "Create a new announcement type",
            "Display list of defined announcement types",
            "What are the existing announcement types ",
            "Show all system announcement types",
            "Give announcement type data",
            "Announcement type records please",
            "Get announcement type table",
            "Open the announcement type configuration",
            "Any updates on announcement types ",
            "What kind of announcement types are there ",
            "List out announcement types available",
            "Can I view the announcement type options ",
            "Find announcement type list"
        ],
        "calender_category": [
            "Show me the calendar categories",
            "List all calendar categories",
            "What calendar categories are available ",
            "Display the calendar category list",
            "Give me all the calendar categories",
            "I want to see calendar categories",
            "Fetch calendar category options",
            "Which calendar categories are defined ",
            "Open the calendar category section",
            "View calendar category master",
            "What are the types of calendar categories ",
            "Access the calendar category records",
            "Check available calendar categories",
            "How many calendar categories are there ",
            "Browse calendar category settings",
            "Display all types of calendar categories",
            "Can you show the calendar category list ",
            "Categories listed under calendar module ",
            "Provide calendar category data",
            "Retrieve calendar category entries",
            "Where can I manage calendar categories ",
            "Calendar categories currently used ",
            "Edit the calendar category list",
            "Add a new calendar category",
            "Calendar category configuration panel",
            "Open calendar category module",
            "What calendar category types exist ",
            "Show me custom calendar categories",
            "Calendar category options view",
            "Download calendar categories",
            "Upload calendar categories",
            "Calendar category master list please",
            "I need the calendar categories",
            "Check all available calendar types",
            "Review calendar category records",
            "Manage calendar categories",
            "Calendar category section overview",
            "Create a new calendar category",
            "Display calendar category definitions",
            "What are the current calendar categories ",
            "Show all system calendar categories",
            "Give calendar category information",
            "Calendar category records please",
            "Get the calendar category table",
            "Open the calendar category settings",
            "Any updates on calendar categories ",
            "List all available types of calendar categories",
            "Can I view the calendar category data ",
            "Find calendar category master",
            "Calendar category list needed"
        ],
        "document_master": [
            "Show me the document master",
            "List all entries in document master",
            "What records are in the document master ",
            "Display the document master list",
            "Give me all document master records",
            "I want to access document master",
            "Fetch data from document master",
            "Which documents are in the master ",
            "Open the document master section",
            "View document master module",
            "What is stored in the document master ",
            "Access the document master details",
            "Check document master contents",
            "How many entries are in document master ",
            "Browse the document master records",
            "Display all types in document master",
            "Can you show the document master data ",
            "Categories under document master ",
            "Provide document master information",
            "Retrieve document master entries",
            "Where can I manage document master ",
            "Document master entries currently present ",
            "Edit the document master list",
            "Add a new entry to document master",
            "Document master configuration panel",
            "Open the document master module",
            "What types are defined in document master ",
            "Show me custom document master records",
            "Document master data view",
            "Download document master records",
            "Upload documents to master",
            "Document master list needed",
            "I need document master data",
            "Check all available document types",
            "Review the document master list",
            "Manage document master entries",
            "Document master dashboard overview",
            "Create a new document record in master",
            "Display master document types",
            "What are the current entries in document master ",
            "Show all stored documents in master",
            "Give document master report",
            "Document master records please",
            "Get document master table",
            "Open document master configuration",
            "Any new records in document master ",
            "List all document types in master",
            "Can I view document master data ",
            "Search document master entries",
            "Document master list please"
        ],
        "photo_gallery": [
            "Show me the photo gallery",
            "Open the photo gallery section",
            "I want to view the photo gallery",
            "Display the school photo gallery",
            "Access the photo gallery now",
            "List all images in the photo gallery",
            "Fetch photo gallery albums",
            "Can you show the latest photos ",
            "Where is the photo gallery ",
            "Browse the photo gallery",
            "What photos are in the gallery ",
            "Show all gallery photos",
            "Photo gallery records please",
            "Get images from the photo gallery",
            "View events in the photo gallery",
            "Look at photo gallery entries",
            "View pictures in photo gallery",
            "Photo gallery album list",
            "Display recent uploads in gallery",
            "Access school photo gallery",
            "What albums are in the photo gallery ",
            "Photo gallery data please",
            "Photo gallery overview",
            "Open school gallery section",
            "Show categories in photo gallery",
            "Download images from photo gallery",
            "Upload to photo gallery",
            "Edit entries in the photo gallery",
            "Add new photos to gallery",
            "List categories in photo gallery",
            "Photo gallery display view",
            "School event images gallery",
            "Show past event photos",
            "Display photo gallery dashboard",
            "Which photos are uploaded recently ",
            "Search for photos in the gallery",
            "Find images in the photo gallery",
            "Print images from gallery",
            "Show all photos under annual day",
            "Photo gallery sorted by event",
            "Display photo albums",
            "School trip photos in gallery",
            "Event-based photo gallery",
            "Recent activities photo gallery",
            "Update entries in photo gallery",
            "Can I see the student activity photos ",
            "How to access photo gallery ",
            "I want gallery images",
            "Display image list in gallery",
            "Gallery photo section"
        ],
        "dairy_or_notice": [
            "Show me the diary or notice",
            "Open the school notice board",
            "Display today's diary entries",
            "Access the student diary section",
            "View all notices and updates",
            "List the latest school notices",
            "Check diary or notice updates",
            "What is in today’s notice ",
            "Display recent diary notes",
            "Get all diary and notices",
            "Fetch today's notice for students",
            "Where can I find school notices ",
            "What are the latest diary updates ",
            "Show the diary for this week",
            "Notices for this month please",
            "Open diary and announcements",
            "Access today's diary messages",
            "List all diary and notice entries",
            "School diary records please",
            "Fetch the parent notice updates",
            "I want to view the diary section",
            "What notices were sent today ",
            "Download the latest school notice",
            "Upload a new diary note",
            "Send today’s notice to students",
            "Who received the diary notice today ",
            "Check past diary entries",
            "Open teacher diary and notices",
            "Review the diary communication",
            "List circulars or notices",
            "Display noticeboard messages",
            "Check diary status for students",
            "Update school diary entry",
            "Share today’s school diary",
            "Parent notices from diary please",
            "Daily diary updates section",
            "School notice master view",
            "Where is today’s notice content ",
            "Add new school diary information",
            "Send notices through diary",
            "I want to add a diary note",
            "View all communication from diary",
            "School notice board open",
            "Diary entries for class 10",
            "Show diary records for today",
            "Notice and diary dashboard",
            "Print today’s diary message",
            "Diary message log view",
            "Announcement in today’s diary ",
            "Diary notes for last week",
            "Class-wise diary or notice"
        ],
        "news_and_events": [
            "Show me the news and events",
            "List all upcoming events",
            "What news and events are there ",
            "Display the latest news and events",
            "Can you show upcoming school events ",
            "Get all news and event updates",
            "Fetch recent school events",
            "Where can I find the latest news ",
            "What events are scheduled for this month ",
            "Open news and events section",
            "List all the school events",
            "Show me the event calendar",
            "Display all event details",
            "Check the latest news for students",
            "What’s happening in the school ",
            "News and events updates please",
            "What are the upcoming events ",
            "Show school events and activities",
            "Browse the latest school news",
            "Open events and news feed",
            "I want to see the event schedule",
            "Show me the news section",
            "View school events this week",
            "Check out the latest announcements",
            "Get the news and events dashboard",
            "List news updates for the school",
            "Show me the school calendar",
            "List upcoming events and news",
            "Update the news and events page",
            "Add a new event to the calendar",
            "Which events are happening next ",
            "Give me the news articles",
            "View event details in the system",
            "Download the latest event updates",
            "Show the event master list",
            "I want news and events from last week",
            "Manage news and events entries",
            "Publish an event in the school news",
            "Display upcoming school events",
            "How can I add an event to news ",
            "What are the major school events ",
            "School event and news notification",
            "Can I view event categories ",
            "Where can I find past news articles ",
            "Check events scheduled for this term",
            "Create a new event in the system",
            "Search for news and event topics",
            "Event categories available in system",
            "News and event record search",
            "Show me news updates for the school year",
            "Latest event details and descriptions"
        ],
        "calender_event": [
            "Show me the calendar events",
            "List all calendar events",
            "What events are on the calendar ",
            "Display the upcoming calendar events",
            "Can you show me the events on the calendar ",
            "Fetch the scheduled calendar events",
            "Where can I see the calendar events ",
            "What events are happening this month ",
            "View the calendar event list",
            "What’s on the calendar this week ",
            "Open the calendar event section",
            "Browse all calendar events",
            "Check out the calendar event schedule",
            "List the events happening in school",
            "Display all school calendar events",
            "Check calendar events for the next week",
            "Event details for this month",
            "List of all upcoming calendar events",
            "Show me the school events calendar",
            "How many events are on the calendar ",
            "Can I add a new event to the calendar ",
            "View the current month’s events",
            "Display all events scheduled for today",
            "Get the full calendar event list",
            "Open the event calendar module",
            "Show events for this semester",
            "Update the calendar with new events",
            "Create a new calendar event",
            "Manage the school calendar events",
            "What’s the next event on the calendar ",
            "Show past calendar events",
            "Check the school’s calendar schedule",
            "Browse calendar event options",
            "Calendar event details page",
            "Show me school events today",
            "Download calendar events list",
            "Upload a new event to the calendar",
            "Check for event conflicts on the calendar",
            "Create a calendar event for the next month",
            "Can you show the event for today ",
            "Show me the events for this year",
            "Event calendar overview",
            "List upcoming calendar events",
            "View event details for this term",
            "Display events on the school calendar",
            "Can I modify an existing calendar event ",
            "Where can I check event details ",
            "Show all events scheduled this week",
            "View all calendar events for next semester",
            "Add an event to the calendar",
            "What events are already scheduled "
        ],
        "message_of_day": [
            "Show me the message of the day",
            "What is today's message ",
            "Display the message for today",
            "Can you show me the message of the day ",
            "Fetch today's message for me",
            "Give me the message of the day",
            "What's the message for today ",
            "I want to see the message of the day",
            "Show the motivational message of the day",
            "What is the announcement for today ",
            "Open the message of the day",
            "Display the school’s daily message",
            "Get today’s school message",
            "Check the message for today",
            "Show the daily quote or message",
            "What is today's thought ",
            "Message of the day please",
            "Give me the quote for today",
            "What's the message from the principal ",
            "Show me today's announcement",
            "Display the day’s inspirational message",
            "Can I view today’s message ",
            "Read out today’s message",
            "Can you tell me the message of today ",
            "I want to know today’s message",
            "What’s the school message today ",
            "What is the message from the teacher today ",
            "Today’s motivational message please",
            "Fetch the daily school message",
            "Show today’s notice or message",
            "Can you display the message for today ",
            "What's the theme for today ",
            "Give me the day's important message",
            "I need today’s message from the school",
            "What message is being sent today ",
            "Open today’s inspirational message",
            "Message of the day content please",
            "Fetch the daily announcement",
            "Show the educational message of the day",
            "Display today's important message",
            "Send me today’s motivational note",
            "Show the daily words of wisdom",
            "Message of the day from school",
            "What’s the daily quote in school ",
            "What's the school’s message of the day ",
            "Can I see today’s motivational message ",
            "I want the school’s quote for today",
            "Read today’s message aloud",
            "Message for today from the principal"
        ]
    },

        "food_menu": {
        "menu_slot": [
            "Show me the menu slot",
            "What are the available menu slots ",
            "Display the available time slots for food",
            "What are the available meal slots ",
            "Can you show the menu time slots ",
            "List the available slots for food",
            "Get the menu slots for today",
            "What time is food being served ",
            "When are the food slots available ",
            "List all the menu slots",
            "Show the available time for meals",
            "Display all available meal timings",
            "What are the different slots for meals ",
            "Open menu slots for today",
            "Can I see the meal slot times ",
            "Show me the slots for breakfast",
            "Display lunch and dinner slots",
            "What time is lunch served ",
            "Get meal slots for this week",
            "List the available food slots",
            "What are the time slots for meals ",
            "Show the food slots for the day",
            "Meal timings for today",
            "When does the next meal slot open ",
            "Display meal slots for the week",
            "Check the menu time slot for dinner",
            "Get available meal timings",
            "What’s the lunch time slot ",
            "Meal slot details for today",
            "Are there any slots for snacks ",
            "Show me the available dinner slots",
            "What are the food slots today ",
            "What time is breakfast served ",
            "How long are the meal slots open ",
            "Meal slot list for the day",
            "What are the breakfast slot hours ",
            "Get the next available meal slot",
            "Show all time slots for meals",
            "Can I book a menu slot ",
            "What’s the next meal slot ",
            "Meal slots for next week",
            "List all available time slots for food",
            "What are the available meal hours ",
            "Check meal availability for today",
            "When is the next available food slot "
        ],
        "my_menu": [
            "Show me my menu",
            "What’s on my personal menu ",
            "Display my customized menu",
            "Can you show my food menu ",
            "Open my meal plan",
            "Give me my food preferences",
            "What food items are on my menu ",
            "Can I see my meal selections ",
            "My personal menu for today",
            "What’s available for me today ",
            "Show my selected food items",
            "Display my favorite meals",
            "Get my food options for today",
            "What’s on my menu this week ",
            "List all the items on my menu",
            "Can I change my personal menu ",
            "Show my breakfast menu",
            "What are my meal choices today ",
            "Get my lunch menu",
            "Show me my favorite dishes",
            "What meals are on my personalized menu ",
            "My menu for this week",
            "Display my personalized food list",
            "Show me my regular menu items",
            "What’s my dinner menu ",
            "Can I add meals to my personal menu ",
            "Customize my menu for today",
            "Show me my special food selections",
            "What food do I prefer today ",
            "List my menu items",
            "What’s on my menu for breakfast ",
            "What meals can I select today ",
            "Open my food preferences menu",
            "View my favorite meal options",
            "Can I change my meal choices ",
            "My meal selection for today",
            "Can I edit my menu ",
            "My personalized food menu",
            "What are my special food items ",
            "Show me my menu for this month",
            "Display my selected meals",
            "Open my customized food list",
            "Can I remove items from my menu ",
            "Show me my meal history",
            "Get my menu selection for this year",
            "View my regular food items",
            "What’s on my menu for lunch today "
        ],
        "menu_item_detail": [
            "Show me the details of this menu item",
            "Can you provide details of this dish ",
            "Display details for this menu item",
            "Give me more info on this dish",
            "What is the description of this item ",
            "Tell me about this food item",
            "What’s included in this menu item ",
            "Show the nutritional details of this dish",
            "Give me the ingredient list for this menu item",
            "What’s the price of this food item ",
            "How many calories are in this item ",
            "Can you describe this menu item ",
            "Show the ingredients of this food item",
            "What’s in this menu item ",
            "What are the key ingredients of this dish ",
            "Display the recipe for this menu item",
            "Can I get more details on this food ",
            "Tell me the serving size of this dish",
            "How much does this food item cost ",
            "What’s the portion size for this dish ",
            "What’s the cooking method for this item ",
            "Give me the nutritional facts for this dish",
            "What’s the calorie count of this item ",
            "Show the allergen information for this dish",
            "Can I get the calorie breakdown ",
            "What’s the preparation time for this food item ",
            "How long does it take to make this dish ",
            "What’s the protein content of this item ",
            "Can I see a detailed breakdown of the dish ",
            "What are the main ingredients in this food ",
            "Display all details for this menu item",
            "Tell me the cooking method for this dish",
            "What’s the serving suggestion for this food ",
            "Give me the detailed description of this dish",
            "What are the side ingredients for this item ",
            "Show the price of this menu item",
            "Give me a full overview of this food",
            "What’s the source of this food item ",
            "What’s the origin of this dish ",
            "Give me the full ingredients list",
            "Show the detailed ingredients for this food",
            "Tell me about the preparation of this dish",
            "What’s the weight of this food item ",
            "Display the item details including calories",
            "What are the dietary restrictions for this dish "
        ],
        "ingredient_master": [
            "Show me the ingredient master list",
            "What ingredients are available in the master list ",
            "Display the ingredient master",
            "Can you show all the ingredients ",
            "Get the full list of ingredients",
            "List all available ingredients",
            "What ingredients are in the system ",
            "Show the master list of ingredients",
            "Display ingredients for the menu",
            "What’s in the ingredient database ",
            "Get ingredient details from the master list",
            "What ingredients are used in the dishes ",
            "Show me the full list of ingredients available",
            "Display all ingredients used in the food",
            "Where can I find the ingredient master list ",
            "List all ingredients in the food menu",
            "Can I add a new ingredient to the master list ",
            "What’s in the ingredient list ",
            "Show me the ingredients used today",
            "Ingredient master overview",
            "Get the details of ingredients",
            "What ingredients are in the menu ",
            "List of all food ingredients",
            "Display ingredient information",
            "Can I search ingredients in the master list ",
            "Show ingredients for today’s meals",
            "Add new ingredients to the master list",
            "Edit ingredient details in the list",
            "Check ingredient availability",
            "Can I view the ingredient categories ",
            "View ingredient categories",
            "What’s the origin of this ingredient ",
            "What are the sources of ingredients ",
            "What ingredients are used for desserts ",
            "Display ingredients used in lunch menu",
            "Show all spice ingredients",
            "Check ingredients for vegetarian dishes",
            "Can I delete an ingredient from the list ",
            "What’s the allergen info for this ingredient ",
            "Get all available ingredient categories",
            "List ingredients for breakfast menu",
            "What ingredients are used in drinks ",
            "What are the prices of ingredients ",
            "Show ingredients for special dishes",
            "Ingredient list by category",
            "Ingredient selection for school meals",
            "Display ingredient types available"
        ]
    },

        'stock_and_inventory' : {
        'stock_inventory_masters': [
            "How do I manage stock inventory masters ",
            "Can I add new stock items to the inventory ",
            "How do I update stock inventory details ",
            "Can I delete stock items from the inventory ",
            "How do I categorize stock items in the inventory ",
            "Can I track stock levels in inventory ",
            "How do I set minimum and maximum stock levels ",
            "Can I import stock inventory data from a file ",
            "How do I assign stock codes to items ",
            "Can I manage suppliers for stock items ",
            "How do I view details of all stock inventory items ",
            "Can I create a new stock category ",
            "How do I manage stock prices in the inventory ",
            "Can I update the stock quantity for multiple items at once ",
            "How do I track the purchase history of stock items ",
            "Can I assign a barcode to each stock item ",
            "How do I link stock items to specific vendors ",
            "Can I generate a report of all inventory items ",
            "How do I manage stock items in different locations ",
            "Can I set reorder points for stock items ",
            "How do I track expiry dates for perishable items ",
            "Can I update inventory based on stock movements ",
            "How do I assign tags to stock inventory items ",
            "Can I manage multiple warehouses in inventory ",
            "How do I view stock availability for different locations ",
            "Can I integrate the stock inventory system with accounting ",
            "How do I track the cost price of stock items ",
            "Can I import inventory details from external systems ",
            "How do I search for specific stock items in the inventory ",
            "Can I restrict access to certain stock items in the inventory "
        ],
        'stock_module': [
            "How do I manage stock in the stock module ",
            "Can I add stock entries for products ",
            "How do I adjust stock levels in the stock module ",
            "Can I generate stock reports in the stock module ",
            "How do I transfer stock between locations ",
            "Can I track stock movement in the stock module ",
            "How do I perform stock audits in the module ",
            "Can I set stock alerts for low stock levels ",
            "How do I update stock quantities after a sale ",
            "Can I include stock adjustments in the stock module ",
            "How do I integrate stock purchases into the module ",
            "Can I manage stock returns in the module ",
            "How do I set stock reordering rules in the module ",
            "Can I track stock receipts in the stock module ",
            "How do I track stock issues in the module ",
            "Can I filter stock data by product category ",
            "How do I handle stock discrepancies in the module ",
            "Can I generate stock valuation reports ",
            "How do I track stock aging in the module ",
            "Can I manage stock for multiple warehouses in the module ",
            "How do I track stock sales in the module ",
            "Can I perform stock reconciliation in the module ",
            "How do I add bulk stock entries in the module ",
            "Can I set up automatic stock reordering in the module ",
            "How do I manage stock levels for seasonal items ",
            "Can I track stock losses and damages in the module ",
            "How do I perform stock movement tracking across different locations ",
            "Can I perform stock entry validations in the module "
        ],
        'stock_report': [
            "How do I generate stock reports ",
            "Can I generate a stock report for a specific product ",
            "How do I filter stock reports by category ",
            "Can I export stock reports to Excel or PDF ",
            "How do I generate a stock status report ",
            "Can I track stock movement in reports ",
            "How do I generate a stock valuation report ",
            "Can I analyze stock levels across multiple locations ",
            "How do I include stock aging details in reports ",
            "Can I track stock purchases and sales in reports ",
            "How do I generate stock balance reports ",
            "Can I filter stock reports by date range ",
            "How do I generate reports for stock returns ",
            "Can I include stock discrepancies in the report ",
            "How do I generate reports for stock wastage ",
            "Can I generate a stock report by product code ",
            "How do I generate a stock report for a specific warehouse ",
            "Can I analyze stock turnover in reports ",
            "How do I generate reports for low stock items ",
            "Can I track stock audits in reports ",
            "How do I generate a report on stock movements between locations ",
            "Can I track stock adjustments in reports ",
            "How do I generate reports on stock sales performance ",
            "Can I generate reports on stock in transit ",
            "How do I include stock expiry details in the report ",
            "Can I generate a report on stock aging for perishable goods ",
            "How do I generate reports on stock ordering and reordering ",
            "Can I compare stock reports from different periods ",
            "How do I generate a summary stock report "
        ]
    },

        'health_card_management': {
        'major_criteria': [
            "What are the major health criteria for students ",
            "Can you explain the major criteria for the health card ",
            "Which health parameters are considered major ",
            "How do you define major health issues ",
            "What major conditions are recorded in the health card ",
            "Is a major illness recorded differently ",
            "How are major health problems categorized ",
            "What are examples of major health concerns ",
            "Can you list the major health checks required ",
            "How often are major health criteria updated ",
            "Is vaccination part of major criteria ",
            "Are chronic illnesses included in major health criteria ",
            "How do you evaluate major health issues ",
            "What medical tests fall under major criteria ",
            "Can a student be exempt from major health criteria ",
            "Is allergy considered a major criterion ",
            "Does surgery history fall under major criteria ",
            "Are hospitalization records considered major ",
            "How are emergencies classified in health cards ",
            "Are mental health issues major criteria ",
            "Is diabetes part of major health recording ",
            "Does the health card show family medical history ",
            "Can parents update major health records ",
            "Do major criteria differ by grade level ",
            "Is asthma a major health criterion ",
            "What is the impact of major criteria on school activities ",
            "Are major injuries tracked separately ",
            "Can major criteria affect student participation ",
            "What happens if a major health issue is found ",
            "Who reviews major health entries ",
            "Are vaccines a major health concern ",
            "How detailed are major health entries ",
            "Is physical disability a major criterion ",
            "Does a major surgery need special reporting ",
            "Can students appeal major health entries ",
            "What is the screening process for major criteria ",
            "Do you need parental consent for major criteria ",
            "Can teachers see major health issues ",
            "Who can access major health information ",
            "Is mental health a separate major criterion ",
            "How secure is major health data ",
            "Can outdated major health issues be removed ",
            "Are parents informed about major health updates ",
            "Is there a checklist for major criteria ",
            "Can school nurses edit major health criteria ",
            "What are the common major health issues found ",
            "Does the school assist students with major health issues ",
            "Are special accommodations provided for major health cases ",
            "Can a health card be updated after initial submission ",
            "What if a major health issue is misreported "
        ],
        'minor_criteria': [
            "What are minor health criteria ",
            "Can you explain minor health conditions recorded ",
            "Which issues are considered minor ",
            "How often are minor health records updated ",
            "Are seasonal allergies considered minor ",
            "Is common cold recorded under minor criteria ",
            "Are minor injuries noted in the health card ",
            "What is classified as a minor health issue ",
            "Do minor illnesses need doctor certification ",
            "How are minor health issues reported ",
            "Are minor infections tracked ",
            "What defines a minor versus major health issue ",
            "Can parents report minor illnesses ",
            "Are cuts and bruises part of minor criteria ",
            "How long are minor health issues stored ",
            "Can teachers update minor health records ",
            "What happens if a minor issue becomes major ",
            "Are dental issues minor or major ",
            "Is fever categorized as minor ",
            "How frequently are minor criteria reviewed ",
            "Can a student attend school with minor health issues ",
            "Are minor surgeries listed separately ",
            "Can minor issues be updated in real-time ",
            "Is a doctor’s note required for minor issues ",
            "Can parents view minor health updates ",
            "Are food allergies considered minor ",
            "Does vaccination reaction count as minor ",
            "Can students self-report minor health concerns ",
            "Is fatigue classified under minor criteria ",
            "Are minor health issues ignored for activities ",
            "Can minor records affect sports participation ",
            "Who approves minor health entries ",
            "Can students hide minor health information ",
            "Is stomach ache a minor health criterion ",
            "Is eye infection a minor record ",
            "Can minor health data be exported ",
            "Are minor health concerns noted by school nurses ",
            "Can minor criteria be customized ",
            "Are seasonal flu cases recorded ",
            "What happens after recording a minor health event ",
            "Is there a minor criteria checklist ",
            "Can minor injuries be updated daily ",
            "Are minor issues discussed with parents ",
            "Is counseling available for minor health matters ",
            "Do minor health issues affect student grades ",
            "Is minor illness attendance tracked ",
            "Are minor allergy details mandatory ",
            "Can multiple minor issues be recorded at once ",
            "Are vaccination side-effects minor criteria ",
            "Can old minor records be archived "
        ],
        'health_card_entry': [
            "How do I make a new health card entry ",
            "What details are required for health card entry ",
            "Can health card entries be updated ",
            "Who can create a health card entry ",
            "Where do I input health data for students ",
            "Is parental approval needed for health card entry ",
            "Can students view their health card entries ",
            "What documents are needed for health card entry ",
            "How often should health entries be updated ",
            "Is health card entry mandatory ",
            "Can teachers make health entries ",
            "Is there a deadline for health card entries ",
            "Are vaccination records part of health card entries ",
            "How secure is the health card entry system ",
            "Can errors in health card entries be corrected ",
            "How long does a health card entry process take ",
            "Is medical history part of the health entry ",
            "Can health card entry be done online ",
            "Who verifies health card entries ",
            "What happens if health data is missing ",
            "Can a health entry be deleted ",
            "Are allergies entered into health cards ",
            "Can minor illnesses be entered ",
            "What is the format for entering health records ",
            "Can previous health entries be updated ",
            "Is blood group information mandatory ",
            "Can fitness certificates be attached ",
            "Can health records be printed ",
            "Are biometric details part of health cards ",
            "Is a doctor’s verification needed for entries ",
            "How to report incorrect health card data ",
            "Are hospital visit details entered ",
            "Is emergency contact information added ",
            "Is past surgery information mandatory ",
            "Can parents update health card entries ",
            "How are health card entries protected ",
            "Can vaccination dates be recorded ",
            "Are dental checkup records needed ",
            "Is vision test data entered ",
            "Can fitness activities be recorded ",
            "Can a health card include psychological assessments ",
            "Are injuries during school recorded ",
            "Is TB screening data required ",
            "Are hearing tests mandatory ",
            "Can you track previous health entries ",
            "Are hospital discharge summaries attached ",
            "How many times can entries be edited ",
            "Is health insurance information recorded ",
            "What to do if health entry fails ",
            "Is there a mobile app for health entries "
        ],
        'health_report': [
            "How can I generate a health report ",
            "Where can I view the health report ",
            "Who can access student health reports ",
            "Can parents download the health report ",
            "Is there a format for health reports ",
            "Are health reports updated automatically ",
            "How often are health reports generated ",
            "Can health reports be customized ",
            "Can teachers view health reports ",
            "How detailed are the health reports ",
            "Are vaccination details shown in reports ",
            "Do health reports show past illnesses ",
            "Can medical fitness be shown in the report ",
            "Are allergies included in the health report ",
            "Can health trends be analyzed ",
            "Is psychological health included ",
            "Can you email health reports ",
            "Can health reports be printed ",
            "Is there a health report for sports clearance ",
            "Can parents request health report corrections ",
            "How secure are health reports ",
            "Is doctor verification shown on the report ",
            "Are all checkups listed in the health report ",
            "Can you filter the health report by date ",
            "Can hospitalization records be viewed ",
            "Does the health report show vaccination expiry ",
            "Can diet recommendations be included ",
            "Can we get a PDF copy of the health report ",
            "Is fitness level part of the report ",
            "Can vision and hearing tests be shown ",
            "Can sports injuries be recorded in the report ",
            "Is the health report shared with sports coaches ",
            "Does the health report show emergency contacts ",
            "Can chronic diseases be highlighted ",
            "Are minor illnesses shown in the report ",
            "How do you edit mistakes in a health report ",
            "Can you merge multiple health reports ",
            "Are medication details included ",
            "Can lab results be attached to the report ",
            "Can we request a health report yearly ",
            "Is there an option for urgent health reports ",
            "Can mental health evaluations be included ",
            "Are BMI and weight statistics shown ",
            "Does the health report indicate doctor's advice ",
            "Can a health report be archived ",
            "Are previous years' reports available ",
            "Is there a health report summary section ",
            "Is blood group mentioned in health reports ",
            "Can you check vaccination compliance from the report ",
            "Is the health report available in the parent portal "
        ]
    },
        'nep_hpc_hub': {
        'dashboard': [
          "Open the NEP dashboard",
          "Show me the HPC Hub dashboard",
          "Where can I find the NEP dashboard ",
          "View NEP-HPC dashboard",
          "Access the holistic progress dashboard",
          "Go to NEP HPC dashboard",
          "Display NEP dashboard stats",
          "Show NEP dashboard summary",
          "Dashboard overview of NEP",
          "HPC dashboard insights",
          "Show me the overview of NEP HPC",
          "Pull up NEP dashboard",
          "Launch NEP dashboard view",
          "What’s on the NEP dashboard ",
          "Give me the NEP HPC dashboard",
          "Access holistic progress card dashboard",
          "NEP dashboard data please",
          "I want to see the NEP dashboard",
          "Dashboard of NEP hub please",
          "Where is the HPC dashboard ",
          "Open holistic progress card dashboard",
          "Display NEP-HPC Hub dashboard",
          "Give me dashboard of NEP-HPC",
          "Show stats on the NEP dashboard",
          "Access national education policy dashboard",
          "HPC insights on dashboard",
          "Start NEP dashboard view",
          "Show me foundation stage stats on dashboard",
          "See preparatory details on dashboard",
          "Dashboard for middle stage NEP",
          "Display summary in HPC hub dashboard",
          "NEP report summary on dashboard",
          "Dashboard data of HPC activities",
          "Let’s open NEP hub dashboard",
          "Dashboard details for NEP stages",
          "I need to access the NEP dashboard",
          "Can I see the NEP HPC dashboard ",
          "Where’s the dashboard for NEP ",
          "Give NEP learning stage dashboard",
          "Show education policy dashboard",
          "Load NEP stage overview dashboard",
          "HPC card dashboard summary",
          "Launch NEP hub main dashboard",
          "Summary of NEP dashboard please",
          "Show me NEP-HPC progress report on dashboard",
          "Can I get dashboard for holistic progress ",
          "Get me holistic card dashboard",
          "Overview dashboard of NEP foundation/preparatory/middle",
          "Summarize NEP on dashboard",
          "Open NEP & HPC hub stats"
        ],
        'foundation_stage': [
          "Access the foundation stage",
          "Show me foundation stage details",
          "Open foundation stage module",
          "Give me info on foundation stage",
          "Start foundation stage page",
          "What is included in the foundation stage ",
          "View NEP foundation stage",
          "Go to foundation stage dashboard",
          "Foundation stage data please",
          "I want foundation stage insights",
          "Show learning progress in foundation stage",
          "Details about the foundation stage",
          "Foundation phase information",
          "Show curriculum for foundation stage",
          "Open the NEP foundation stage",
          "Explore foundation stage module",
          "Which activities are under foundation stage ",
          "Get foundation stage details",
          "What’s taught in foundation stage ",
          "Overview of foundation stage",
          "Breakdown of foundation stage components",
          "Subjects in foundation stage",
          "Syllabus for foundation stage",
          "Foundation stage education levels",
          "Check foundation stage inputs",
          "See foundation stage progress",
          "How is the foundation stage managed ",
          "Foundation stage assessment methods",
          "Get learning plan for foundation stage",
          "Foundation curriculum info",
          "Early education under foundation stage",
          "Stages included in foundation module",
          "What covers foundation stage ",
          "Access foundation NEP level",
          "Data on foundation level",
          "Age group for foundation stage",
          "Duration of foundation stage",
          "Who manages foundation stage ",
          "Key skills in foundation stage",
          "Foundation stage student data",
          "Foundation stage report",
          "Details of students in foundation stage",
          "Foundation stage schedule",
          "How many classes in foundation stage ",
          "Open early years learning (foundation)",
          "Start NEP foundation module",
          "Give me full list of foundation stage items",
          "View holistic progress in foundation stage",
          "Foundation level results",
          "View foundation stage activity list"
        ],
        'foundation_stage_activity': [
          "Show foundation stage activities",
          "List all activities in foundation stage",
          "Give me foundation activities",
          "Which activities belong to foundation stage ",
          "Open activity list for foundation stage",
          "Foundation learning activities",
          "What are kids doing in foundation stage ",
          "Educational activities in foundation phase",
          "View learning tasks in foundation stage",
          "List NEP foundation stage tasks",
          "Interactive activities for foundation stage",
          "Hands-on tasks for foundation level",
          "Early learning activities NEP",
          "Foundation activity dashboard",
          "Get NEP activity log for foundation",
          "Details of foundation activities",
          "What’s happening in foundation stage ",
          "Show activity records foundation stage",
          "Curriculum activities for foundation",
          "Stage-based learning activities",
          "Foundation class activities",
          "Get play-based activities in foundation",
          "Creative tasks in foundation module",
          "Open student activities for foundation",
          "Activity tracking for foundation stage",
          "Games and play in foundation",
          "Foundation activities planned",
          "Daily activities foundation phase",
          "How many tasks in foundation stage ",
          "Learning through play - foundation",
          "Get group activities foundation stage",
          "Log of activities - foundation NEP",
          "Hands-on learning for foundation stage",
          "Get student activity progress",
          "Activities planned for NEP foundation",
          "Explore learning goals of foundation",
          "Practical learning in foundation level",
          "Activity-based education foundation",
          "Open activity board of foundation",
          "Access foundation stage timeline",
          "Foundation stage daily plan",
          "Open weekly activities foundation phase",
          "Monitor foundation activities",
          "Holistic activities - foundation",
          "Get summary of activities foundation",
          "Access class-wise foundation activities",
          "Review foundation stage activities",
          "Which events happened in foundation ",
          "Pull activity data - foundation stage",
          "Fetch learning records - foundation"
        ],
        "preparatory_stage": [
            "Access the preparatory stage module",
            "Open preparatory stage details",
            "Show preparatory stage overview",
            "Give me information on preparatory stage",
            "Preparatory stage content please",
            "What’s in the preparatory stage ",
            "Subjects under preparatory stage",
            "Syllabus of preparatory stage",
            "View NEP preparatory stage section",
            "Open preparatory learning phase",
            "Which classes are in preparatory stage ",
            "Details about preparatory education",
            "Preparatory level learning data",
            "I want to see the preparatory stage module",
            "How does preparatory stage work ",
            "Access syllabus for preparatory students",
            "List classes under preparatory level",
            "Open NEP preparatory module",
            "Preparatory subjects and topics",
            "Preparatory curriculum overview",
            "Check preparatory syllabus",
            "Show NEP preparatory phase details",
            "Who manages the preparatory stage ",
            "Preparatory level curriculum details",
            "What age group is preparatory stage ",
            "Preparatory level lessons",
            "Give preparatory academic structure",
            "Preparatory stage subject list",
            "Preparatory phase education info",
            "Check student records in preparatory stage",
            "Preparatory learning plan",
            "Overview of preparatory syllabus",
            "Preparatory topics summary",
            "Preparatory schedule information",
            "What’s included in the preparatory level ",
            "Preparatory level report",
            "Stages in preparatory education",
            "Overview of preparatory stage syllabus",
            "Preparatory phase student data",
            "Open overview of preparatory classes",
            "Summary of preparatory teaching",
            "Class structure in preparatory stage",
            "Load preparatory stage information",
            "Launch preparatory academic page",
            "Review NEP preparatory stage module",
            "Preparatory structure and goals",
            "How long is the preparatory stage ",
            "Preparatory stage purpose",
            "Get overview for preparatory learners",
            "Preparatory learning outcomes"
          ],
          "preparatory_stage_team": [
            "Show preparatory stage team",
            "Who are in the preparatory team ",
            "List all preparatory team members",
            "Details of the preparatory team",
            "Open team assigned for preparatory stage",
            "Check faculty for preparatory stage",
            "Staff list for preparatory level",
            "Teachers in preparatory group",
            "Who manages the preparatory stage ",
            "Preparatory stage staff details",
            "Open list of preparatory stage mentors",
            "Team behind preparatory phase",
            "Preparatory teachers and faculty",
            "Faculty details for preparatory stage",
            "Team structure for preparatory stage",
            "Management of preparatory group",
            "Preparatory support staff info",
            "Head of preparatory stage",
            "Staff assignments in preparatory level",
            "Who is in charge of preparatory stage ",
            "NEP preparatory team members",
            "Assigned faculty for preparatory stage",
            "Access preparatory team list",
            "See list of preparatory stage instructors",
            "Preparatory department details",
            "Teaching and non-teaching staff preparatory",
            "Preparatory stage coordinator info",
            "Advisors of preparatory phase",
            "Teachers handling preparatory stage",
            "Get preparatory leadership team",
            "Prep stage staff structure",
            "Instructors assigned to preparatory phase",
            "Preparatory team members with designation",
            "Preparatory mentors overview",
            "Preparatory guide list",
            "Open list of teaching staff preparatory stage",
            "Team planning preparatory syllabus",
            "Administration of preparatory stage",
            "Personnel involved in preparatory education",
            "Teaching staff for NEP preparatory stage",
            "Support team of preparatory level",
            "Class in-charges of preparatory classes",
            "Teachers assigned to NEP preparatory",
            "Open info for preparatory team",
            "Review preparatory stage staff",
            "Faculty roster for preparatory students",
            "Preparatory level teacher list",
            "Preparatory department members",
            "Preparatory coordinators and teachers",
            "Show department team for preparatory"
          ],
          "preparatory_stage_activity": [
            "Show activities for preparatory stage",
            "List all preparatory stage tasks",
            "Open activity list for preparatory level",
            "Give me activities under preparatory phase",
            "What activities are in preparatory stage ",
            "Educational activities for preparatory students",
            "Open NEP preparatory activity records",
            "Preparatory learning activities",
            "Play-based tasks for preparatory",
            "Skill-based activities for preparatory level",
            "Preparatory classroom activities",
            "Hands-on learning in preparatory stage",
            "Get preparatory project list",
            "Review academic activities for preparatory",
            "Open list of scheduled preparatory tasks",
            "Interactive tasks for preparatory students",
            "Get practical exercises preparatory phase",
            "What kind of activities are done in preparatory ",
            "Learning events for preparatory level",
            "Preparatory team-based activities",
            "Activity dashboard for preparatory stage",
            "Summarize learning tasks in preparatory",
            "Creative learning in preparatory module",
            "NEP activities under preparatory stage",
            "Skill development in preparatory stage",
            "Games and tasks in preparatory stage",
            "Activity-wise breakdown of preparatory stage",
            "Preparatory daily activity chart",
            "Weekly activity plan for preparatory",
            "Monthly activities under preparatory stage",
            "List of student engagements - preparatory",
            "Open classroom activity register",
            "Exploration-based activities for preparatory",
            "Preparatory learning task details",
            "Subject-wise activities in preparatory",
            "Time-table of preparatory activities",
            "Curriculum-based tasks for preparatory",
            "Get NEP preparatory activity calendar",
            "Activity progress report preparatory",
            "Preparatory event schedule",
            "Extracurricular activities in preparatory",
            "Activity list for student evaluation",
            "Preparatory assessment-related activities",
            "Fun learning in preparatory stage",
            "Preparatory learning modules and activities",
            "Activity tracking in preparatory phase",
            "Group tasks in preparatory",
            "Creative projects for preparatory learners",
            "Foundational learning tasks - preparatory",
            "Preparatory learning outcomes via activities"
          ],
          "middle_stage": [
            "Open middle stage module",
            "Show middle stage overview",
            "Access NEP middle stage",
            "Give details of middle stage",
            "View middle stage academics",
            "Subjects under middle stage",
            "Middle stage learning plan",
            "What is in the middle stage ",
            "Middle stage curriculum details",
            "Check middle stage syllabus",
            "Which grades are in middle stage ",
            "Middle stage overview please",
            "Launch middle stage information",
            "Curriculum in NEP middle stage",
            "Explore middle stage syllabus",
            "View classes in middle stage",
            "Open education structure of middle stage",
            "Middle stage module access",
            "Middle stage course plan",
            "List subjects in middle stage",
            "NEP middle stage topics",
            "What is taught in middle stage ",
            "Subjects and topics of middle stage",
            "Get syllabus for middle stage",
            "Check student data in middle stage",
            "Open NEP module for middle stage",
            "Summarize middle stage academics",
            "Middle stage learning objectives",
            "Syllabus topics for middle stage",
            "Open details of middle education",
            "Middle stage curriculum overview",
            "Classes included in middle stage",
            "Middle level NEP details",
            "Overview of middle classes",
            "Check education plan for middle students",
            "Middle stage learning structure",
            "Student list of middle stage",
            "NEP content under middle stage",
            "Middle education overview",
            "Course topics in middle stage",
            "Open progress data for middle stage",
            "Middle grade curriculum",
            "Information on middle stage students",
            "Student-wise middle stage report",
            "Academic flow in middle stage",
            "Explain middle stage NEP module",
            "Learning flow of middle level",
            "Middle stage planning report",
            "Education strategy of middle stage",
            "Middle level academic framework"
          ],
          "middle_stage_activity": [
            "List activities in middle stage",
            "Show middle stage activities",
            "Open NEP middle stage activity details",
            "Middle stage learning tasks",
            "Middle stage academic events",
            "Check middle stage educational activities",
            "What activities are done in middle stage ",
            "Middle stage class tasks",
            "Project-based activities in middle stage",
            "Middle stage subject-wise activities",
            "Hands-on activities in middle stage",
            "Interactive activities in middle stage",
            "Middle school learning tasks",
            "Get list of classroom activities for middle stage",
            "Middle stage fun learning activities",
            "Check activity plan for middle stage",
            "Group activities in middle stage",
            "Middle level student engagements",
            "Middle stage NEP task list",
            "Extracurriculars in middle stage",
            "Activity calendar for middle level",
            "Skill development tasks in middle stage",
            "Creative learning activities for middle stage",
            "Middle stage student involvement",
            "Middle stage daily learning events",
            "Weekly activity chart for middle stage",
            "Middle grade student projects",
            "Report on middle stage activities",
            "Open list of academic events in middle level",
            "Middle level NEP activities",
            "Team-based activities in middle education",
            "Middle stage monthly activities",
            "Middle stage subject-linked tasks",
            "Get scheduled activities for middle students",
            "Middle school exploration activities",
            "Middle stage assessment activities",
            "Access creative tasks for middle stage",
            "Student development activities in middle stage",
            "Middle stage guided learning tasks",
            "Middle stage syllabus-integrated activities",
            "Classroom engagements at middle stage",
            "Middle stage holistic activities",
            "Physical activities in middle level",
            "Language activities for middle students",
            "Science-based tasks in middle stage",
            "Middle education program tasks",
            "Academic competition list middle stage",
            "Open middle stage activity report",
            "Co-curricular activities for middle classes",
            "Check completed activities middle stage"
          ],
          "middle_stage_team": [
            "Show middle stage team",
            "List members of the middle stage team",
            "Open faculty list for middle stage",
            "Middle stage staff details",
            "Team handling middle stage",
            "View NEP middle stage instructors",
            "Faculty assigned to middle stage",
            "Teaching staff of middle stage",
            "Middle stage support team",
            "Instructors for middle level students",
            "Management of middle stage",
            "Who are in the middle stage team ",
            "Teaching and admin staff for middle stage",
            "Open team structure of middle stage",
            "Coordinator of middle stage",
            "Middle stage department list",
            "Middle stage team overview",
            "Faculty for NEP middle module",
            "Get list of middle stage faculty",
            "Who is leading middle stage ",
            "Department in charge of middle students",
            "Show teaching assignments for middle stage",
            "Head of middle stage academics",
            "Middle stage staff and coordinators",
            "Staff list for NEP middle stage",
            "Teachers involved in middle learning",
            "Middle stage subject handlers",
            "Middle stage academic team info",
            "Middle school instructors",
            "Review team of middle stage",
            "Open list of assigned mentors - middle stage",
            "Access middle stage management info",
            "Faculty assignment report for middle stage",
            "Middle stage educators",
            "Teacher list for middle NEP program",
            "See middle education support team",
            "Faculty and mentors - middle level",
            "Advisors for middle stage",
            "Academic support for middle students",
            "Open NEP middle stage personnel",
            "Middle stage team with designations",
            "Faculty responsibilities in middle stage",
            "Assigned teachers to middle classes",
            "Middle stage organizational structure",
            "Load middle team information",
            "Middle stage faculty planning",
            "Middle stage instructor group",
            "Review assigned mentors for middle level",
            "Staff overseeing middle stage students",
            "Educators for NEP middle module"
        ]
    },

        'portfolio' : {
        'master': [
    		"Open portfolio master",
    		"Show portfolio master settings",
    		"Access master section in portfolio",
    		"View master data of portfolio",
    		"Configure portfolio master",
    		"Portfolio master configuration",
    		"Portfolio master setup",
    		"Edit portfolio master",
    		"Portfolio master entry page",
    		"Add data to portfolio master",
    		"Portfolio master details",
    		"Portfolio module master view",
    		"List all items in portfolio master",
    		"Portfolio master access",
    		"Update portfolio master data",
    		"Portfolio master definitions",
    		"Portfolio master structure",
    		"Check portfolio master",
    		"Open master tab in portfolio",
    		"View all portfolio masters",
    		"Navigate to portfolio master page",
    		"Portfolio master options",
    		"Portfolio master records",
    		"Search portfolio master",
    		"Edit master entries in portfolio",
    		"Portfolio main master screen",
    		"Portfolio base master",
    		"Master module under portfolio",
    		"Portfolio configuration master",
    		"Portfolio default master settings",
    		"Portfolio master setup page",
    		"View structure of portfolio master",
    		"Portfolio base master page",
    		"Open configuration master in portfolio",
    		"Portfolio master section view",
    		"How to open portfolio master ",
    		"Load portfolio master records",
    		"Show current portfolio master data",
    		"Portfolio master interface",
    		"Master screen of portfolio",
    		"Launch portfolio master",
    		"Open and edit portfolio master",
    		"Create master record in portfolio",
    		"Master section inside portfolio module",
    		"Modify portfolio master entries",
    		"Master details of portfolio module",
    		"Portfolio master registry",
    		"Portfolio master information",
    		"Portfolio master dashboard"
    	],
    	"entry_page": [
    		"Open portfolio entry page",
    		"Show data entry in portfolio",
    		"Where is the portfolio entry page ",
    		"Portfolio module entry screen",
    		"View portfolio entry form",
    		"Access entry page in portfolio",
    		"Portfolio form entry section",
    		"Add portfolio data",
    		"Portfolio input page",
    		"Navigate to portfolio entry page",
    		"Portfolio entry tab",
    		"Submit data to portfolio entry",
    		"Portfolio module data entry",
    		"Create entry in portfolio",
    		"Entry page for student portfolio",
    		"Portfolio entry section view",
    		"Entry area of portfolio",
    		"Portfolio data submission screen",
    		"Enter student details in portfolio",
    		"Portfolio input form access",
    		"Insert new portfolio record",
    		"Portfolio information entry",
    		"Portfolio data entry panel",
    		"Enter academic records in portfolio",
    		"Student record entry in portfolio",
    		"Fill the portfolio entry form",
    		"View portfolio entry interface",
    		"Where to input portfolio data ",
    		"Open and use portfolio entry screen",
    		"Portfolio student details entry page",
    		"Log student achievements in portfolio",
    		"Portfolio input tab details",
    		"Create student portfolio entry",
    		"Launch entry module for portfolio",
    		"Portfolio section for new entry",
    		"Portfolio academic entry screen",
    		"Portfolio data update page",
    		"Student records input - portfolio",
    		"Portfolio entry form for teachers",
    		"Update student portfolio record",
    		"Input academic data in portfolio",
    		"Open profile entry in portfolio",
    		"Fill student portfolio info",
    		"Portfolio data upload page",
    		"Portfolio input submission form",
    		"Open data entry window for portfolio",
    		"Student performance entry - portfolio",
    		"Portfolio log entry tab",
    		"Start entering portfolio records",
    		"Portfolio entry sheet"
    	],
    	"report": [
    		"Show portfolio report",
    		"Generate student portfolio report",
    		"Access portfolio reports",
    		"Portfolio reporting page",
    		"Print portfolio report",
    		"Download portfolio report",
    		"Portfolio analysis report",
    		"Open portfolio report section",
    		"Portfolio data summary",
    		"View complete portfolio report",
    		"Report of portfolio entries",
    		"Where is the portfolio report ",
    		"Report for academic portfolio",
    		"Portfolio report generation",
    		"Portfolio records report",
    		"Portfolio report preview",
    		"Get full portfolio report",
    		"Student performance portfolio report",
    		"Portfolio data report view",
    		"Check reports in portfolio",
    		"Open report tab under portfolio",
    		"Portfolio reporting dashboard",
    		"Portfolio achievement report",
    		"Summary report for portfolio module",
    		"Assessment portfolio report",
    		"Report card from portfolio",
    		"PDF report of student portfolio",
    		"Show complete report from portfolio",
    		"Portfolio report insights",
    		"Open progress report - portfolio",
    		"Generate detailed portfolio report",
    		"Report for submitted portfolios",
    		"Portfolio academic tracking report",
    		"Report section for student profiles",
    		"Display portfolio report",
    		"Generate and print portfolio report",
    		"Search student portfolio reports",
    		"Monthly portfolio report view",
    		"Portfolio reporting tools",
    		"Overview report of portfolio entries",
    		"Open portfolio marks report",
    		"Export portfolio data as report",
    		"Portfolio documentation report",
    		"Summary of student portfolio",
    		"Individual portfolio report",
    		"Performance report from portfolio",
    		"Retrieve portfolio report data",
    		"Report tab in portfolio section",
    		"Academic report through portfolio"
    	]
    },
        'settings' : {
        "system_setting": [
            "Open system settings",
            "Show me the system settings page",
            "Where can I access the system configuration ",
            "Go to the system settings tab",
            "I want to change the system preferences",
            "Update the global system settings",
            "Navigate to system setup",
            "System setting panel access",
            "Open configuration options",
            "I need to modify system settings",
            "Adjust system behavior settings",
            "How do I change the system settings ",
            "Manage the current system setup",
            "Display system configuration",
            "System parameters control",
            "Enter the system setup section",
            "I want to review system options",
            "Reset system defaults",
            "Access system menu",
            "Launch system settings screen",
            "Find system settings now",
            "Enable or disable system options",
            "Show global system preferences",
            "Configure platform settings",
            "Update my system environment",
            "Where is system setup ",
            "I want to tweak the system configurations",
            "Open admin settings",
            "Enter the configuration area",
            "Customize system-level controls",
            "Change settings related to system core",
            "Switch to system control mode",
            "Change the setup options",
            "See all system preferences",
            "Bring up the system tab",
            "Reconfigure system settings",
            "Update operational settings",
            "Enable hidden system features",
            "View default system controls",
            "Admin system setting access",
            "System setup and configuration",
            "Control panel for the system",
            "Edit global settings",
            "Modify preferences in system area",
            "Enter system admin panel",
            "System level changes",
            "I want to configure the main system",
            "System backend settings",
            "Open advanced system controls",
            "Navigate to system preferences"
          ],
          "system_initialisation": [
            "Start system initialization",
            "Initialize system now",
            "Begin setting up the system",
            "Launch the system initializer",
            "How to initialize this system ",
            "Run system setup",
            "I want to configure and initialize",
            "Start platform initialization",
            "Trigger system boot process",
            "Where is the initialization menu ",
            "Open system startup configuration",
            "Reset and initialize system",
            "Access first time setup",
            "Run initial system routine",
            "Prepare the system for first use",
            "Initialise the environment",
            "Execute setup initializer",
            "Configure system for initial use",
            "Perform base system setup",
            "Open system startup tab",
            "Launch system init mode",
            "Start default configuration wizard",
            "Initialization page access",
            "How to reinitialize the system ",
            "System pre-launch settings",
            "System boot and setup",
            "Open system install/setup",
            "Run initial setup module",
            "Trigger environment initializer",
            "Create initial configurations",
            "Reconfigure system from scratch",
            "Access onboarding setup",
            "Get me started with initialization",
            "Kick off platform setup",
            "Begin fresh system install",
            "Restart setup mode",
            "Initialization control panel",
            "Enter first-time setup screen",
            "System start process guide",
            "Initial system config needed",
            "I want to start over with system setup",
            "Access configuration assistant",
            "Trigger platform boot sequence",
            "Activate initialization steps",
            "How to run the setup tool ",
            "System setup wizard start",
            "Open zero-state setup",
            "Platform boot configuration",
            "Launch base setup tools",
            "System default initialization"
          ],
          "global_column_settings": [
            "Access global column settings",
            "Change global columns",
            "Edit the column structure",
            "Modify global table columns",
            "Where are column settings ",
            "Set default column views",
            "Adjust global column widths",
            "Change visibility of columns",
            "I want to configure columns globally",
            "Update column layout",
            "Change table column settings",
            "Enable or disable columns",
            "Rearrange global columns",
            "Customize global fields",
            "Where do I set global columns ",
            "Reset column preferences",
            "Column settings per module",
            "Edit global column headers",
            "Manage global display columns",
            "Column configuration dashboard",
            "Column visibility control",
            "Set default table columns",
            "Administer global column layout",
            "Add or remove global columns",
            "Switch global column order",
            "Column mapping settings",
            "Setup default columns view",
            "Column configuration options",
            "Control which columns appear",
            "Define global table schema",
            "Column customization area",
            "Apply column changes globally",
            "Review system-wide column setup",
            "Launch column settings tab",
            "Table view configuration",
            "Global field layout settings",
            "Edit columns for all modules",
            "Column layout controller",
            "Control column fields globally",
            "Make changes to global fields",
            "Access the global columns tab",
            "Customize appearance of columns",
            "Where is column layout defined ",
            "Tweak table field settings",
            "Control default column widths",
            "Set field order globally",
            "System-wide column settings",
            "Platform-wide column manager",
            "Field preferences configuration",
            "Update data field arrangement"
          ],
          "app_permissions": [
            "Set application permissions",
            "Change who can access the app",
            "Manage app user roles",
            "Adjust application access rights",
            "Who has permission for this app ",
            "Show app permission list",
            "App access configuration",
            "Modify permission settings",
            "Control application access",
            "Set roles for the app",
            "Define app-level access",
            "Edit user access for app",
            "Change app usage rights",
            "Application permission dashboard",
            "Restrict certain users",
            "Set permissions for app modules",
            "Give permission to app tools",
            "Limit app features by role",
            "Where is permission setup ",
            "Application role assignment",
            "Assign app access rights",
            "Revoke user access to app",
            "Permit user to use this app",
            "Grant admin access",
            "Manage app privileges",
            "Change who can edit in app",
            "App user authorization",
            "App security settings",
            "Show who can use what",
            "Update access privileges",
            "Control module-level permissions",
            "Application access manager",
            "Check app permission levels",
            "Reconfigure access rights",
            "Configure who can log in",
            "Toggle user access to apps",
            "Create custom app roles",
            "Delete a permission",
            "App access audit settings",
            "Restrict access to some parts",
            "Enable feature access per role",
            "Permission mapping",
            "Setup app access roles",
            "App-wide access settings",
            "Control visibility of app items",
            "Role-based permission editor",
            "Open permissions editor",
            "Adjust admin privileges",
            "Show all role settings",
            "Limit view/edit rights"
          ],
          "user_roles": [
            "Add a new user role",
            "Define user roles",
            "Edit existing user role",
            "Create a role for teachers",
            "Delete a user role",
            "What are the current user roles ",
            "Show me all user roles",
            "Assign roles to a user",
            "Modify user permissions",
            "Change user role names",
            "Manage system user roles",
            "User role configuration page",
            "Update role permissions",
            "Enable a user role",
            "Disable specific roles",
            "How to create a user role ",
            "Remove a user from a role",
            "Link roles to modules",
            "Customize roles",
            "Where is user role management ",
            "Open user role dashboard",
            "Create access roles",
            "Define role-based permissions",
            "Assign multiple roles",
            "Role setup wizard",
            "Edit role description",
            "Limit roles to departments",
            "Assign role-based access",
            "Role rights editor",
            "Change default roles",
            "Configure user access roles",
            "What permissions does this role have ",
            "User category permissions",
            "Manage custom roles",
            "Role hierarchy configuration",
            "Revoke role privileges",
            "Create temporary roles",
            "Apply roles to groups",
            "Setup new role templates",
            "View assigned roles",
            "List of predefined roles",
            "How to manage roles ",
            "Enable or disable a role",
            "Set role visibility",
            "Open roles module",
            "Who has this role ",
            "System role configuration",
            "Create role with limited access",
            "I want to define a manager role",
            "Change roles assigned to a user"
        ],
        "user_master": [
            "Add a new user",
            "Show me the user list",
            "Edit user details",
            "Delete a user",
            "Create a user account",
            "Update user information",
            "View all users",
            "Open user master screen",
            "Access user profiles",
            "Find a specific user",
            "Search users by name",
            "Show active users",
            "Show inactive users",
            "Change user status",
            "Add multiple users",
            "Bulk upload users",
            "Reset user account",
            "View user roles",
            "Assign role to user",
            "Edit user role",
            "Deactivate user",
            "Reactivate user",
            "What are all user details ",
            "Update contact info of user",
            "Edit user login settings",
            "Configure user access",
            "Show all usernames",
            "Download user data",
            "Who created a user ",
            "Create system users",
            "Define staff accounts",
            "View teacher accounts",
            "Add admin user",
            "Update user photo",
            "Export all users",
            "List student users",
            "How to create a user ",
            "Set user permissions",
            "Open user registry",
            "Check user credentials",
            "Manage user identities",
            "Search user by ID",
            "Control user access",
            "Enable a user profile",
            "Disable user login",
            "Enter user setup area",
            "Display user data fields",
            "User master page access",
            "Modify existing user",
            "Create new user profile"
          ],
          "password_manager": [
            "Open password manager",
            "Change my password",
            "Reset user password",
            "Set default password",
            "Show password policy",
            "How to update password ",
            "Enforce password rules",
            "Manage password security",
            "Enable password expiration",
            "Force password change",
            "Update login password",
            "Change admin password",
            "Access password controls",
            "View stored passwords",
            "Secure password list",
            "Add new password rule",
            "Set password complexity",
            "What is my password status ",
            "Password reset options",
            "Configure password settings",
            "System password manager",
            "Re-enter password manager",
            "Check password age",
            "Lock account after failed logins",
            "Require special characters",
            "Create password policy",
            "Define user password format",
            "Disable weak passwords",
            "Open password panel",
            "Modify password strength",
            "Password rule editor",
            "How many characters allowed ",
            "Change password for role",
            "Link password to username",
            "Check recent password changes",
            "Find password reset logs",
            "Customize password limits",
            "Security via password control",
            "Enable password hints",
            "Change password expiry time",
            "Restore forgotten passwords",
            "Generate secure password",
            "Password manager section",
            "See password history",
            "How to set default user passwords ",
            "Who changed their password ",
            "Edit user password manually",
            "Restrict password reuse",
            "Set lockout attempts"
          ],
          "login_preview": [
            "Show login screen preview",
            "Preview the login page",
            "How does the login screen look ",
            "Open login UI preview",
            "Access login page template",
            "Check login layout",
            "Customize login view",
            "View sample login interface",
            "Show login display mode",
            "Edit login background",
            "What does login look like ",
            "Login appearance settings",
            "Design login preview page",
            "Change login screen elements",
            "Where is login preview ",
            "See login page before launch",
            "Login theme preview",
            "Set login screen image",
            "Preview login page settings",
            "Update login UI",
            "Simulate login view",
            "Enable login preview feature",
            "Show current login layout",
            "Edit welcome message",
            "Test login interface",
            "Demo login screen",
            "Configure login appearance",
            "Change login page design",
            "Adjust login page theme",
            "Open login customization",
            "Access login interface preview",
            "Set login logo",
            "Edit login instructions",
            "Preview login theme",
            "View changes to login screen",
            "User login preview",
            "Open login page demo",
            "Preview role-based login page",
            "See login for students",
            "Login preview for admin",
            "Design sample login view",
            "Show default login layout",
            "Check updated login screen",
            "Login customization section",
            "Design login welcome",
            "Preview institute login view",
            "How to preview login changes ",
            "Open login screen builder",
            "Test login visuals"
          ],
          "sibling_list": [
            "Show sibling list",
            "Which students are siblings ",
            "View siblings in the system",
            "Find related students",
            "List sibling relationships",
            "Open sibling master",
            "Siblings of this student ",
            "Are they siblings ",
            "Sibling report",
            "Show all sibling pairs",
            "Match siblings by family",
            "Display sibling data",
            "Link siblings together",
            "Update sibling list",
            "Enter sibling info",
            "How to find siblings ",
            "List students with same parents",
            "Check for sibling connection",
            "Add new sibling link",
            "Remove sibling entry",
            "Are these two siblings ",
            "Search for sibling name",
            "Sibling database access",
            "Generate sibling records",
            "How to tag siblings ",
            "Add sibling details",
            "Group siblings by family",
            "Which students share a guardian ",
            "Sibling entry form",
            "Sibling list for class",
            "Match sibling profiles",
            "Manage sibling data",
            "Review sibling connections",
            "Find siblings in same grade",
            "Identify siblings in records",
            "Who is this student's sibling ",
            "View family linked students",
            "Get sibling profile list",
            "Export sibling list",
            "Connect students as siblings",
            "Display student siblings",
            "Edit sibling group",
            "Family link between students",
            "Find sibling info quickly",
            "Sibling tag manager",
            "System sibling grouping",
            "Class-wise sibling list",
            "Guardian-wise sibling info",
            "Student sibling association"
          ],
          "institute_master": [
            "Show institute master data",
            "Add a new institute",
            "Edit institute details",
            "Remove an institute",
            "Update institute name",
            "View all institute entries",
            "Where is institute master ",
            "Change institute info",
            "Open institute record",
            "Display school details",
            "Modify campus name",
            "Institute master list",
            "Add multiple institutes",
            "Download institute master",
            "Set default institute",
            "Manage institute branches",
            "Assign institute codes",
            "Edit institute description",
            "Who added the institute ",
            "Enable or disable an institute",
            "Configure institute settings",
            "Check institute creation date",
            "Open school database",
            "Update official institute name",
            "Show registered institutes",
            "Search for an institute",
            "Change institute logo",
            "Upload institute details",
            "Find all school records",
            "Institute registration info",
            "Display full institute data",
            "Open institute profile",
            "Institute information manager",
            "Setup institute master",
            "Edit details of this institute",
            "School record update",
            "Create new institute entry",
            "Where is campus information ",
            "Modify head of institute",
            "Institute configuration settings",
            "School group editor",
            "Main campus information",
            "Institute record view",
            "View institute summary",
            "Edit institute level fields",
            "How to add a new campus ",
            "Institute overview screen",
            "Access main institute panel",
            "Activate new institute",
            "Deactivate old institute"
        ],
        "academic_year_or_batch": [
            "Show academic year list",
            "Add a new academic year",
            "Edit academic batch",
            "Remove academic year",
            "View current academic year",
            "What is the active batch ",
            "Change the academic year",
            "Update batch dates",
            "Set academic year duration",
            "Academic year master data",
            "Search academic batch",
            "Add batch for new session",
            "Open academic year master",
            "List of academic years",
            "Which year is active ",
            "Define academic year range",
            "Academic year details screen",
            "Configure academic batches",
            "Activate next academic year",
            "Deactivate old academic year",
            "Batch master entry",
            "Show all past batches",
            "Add session for new students",
            "Update existing academic year",
            "Edit batch name",
            "Download academic year data",
            "Show archived academic years",
            "Upload batch details",
            "Manage academic sessions",
            "View batch-wise reports",
            "List of academic sessions",
            "When does batch start ",
            "Get academic calendar",
            "Assign subjects to batch",
            "Change year-wise settings",
            "Setup academic batches",
            "Edit start and end date",
            "New academic session input",
            "Show batch master setup",
            "Link students to batch",
            "Academic year timeline",
            "How to define academic year ",
            "Assign exams to session",
            "Edit academic calendar year",
            "Academic session year data",
            "Preview academic session",
            "Configure academic terms",
            "View academic year settings",
            "Switch to new academic year"
          ],
          "educational_stages": [
            "List educational stages",
            "Add a new stage",
            "Edit educational stage",
            "Remove education stage",
            "Set education levels",
            "Show stage details",
            "Update educational hierarchy",
            "Define stage names",
            "School educational stages",
            "What are the learning stages ",
            "Setup academic levels",
            "Stages of education in system",
            "Display school levels",
            "Edit primary stage",
            "Add pre-primary stage",
            "Manage school stages",
            "Add class stages",
            "View stage mapping",
            "Educational stage chart",
            "Define stages for curriculum",
            "Set level structure",
            "Educational tiers setup",
            "Stage-wise configuration",
            "Assign classes to stage",
            "Which stage is this class ",
            "Search for education stage",
            "Set school stage policy",
            "View list of all stages",
            "Change class to stage mapping",
            "Setup KG stage",
            "Define high school stage",
            "Educational framework stages",
            "Stages of learning screen",
            "Update academic level info",
            "School education flow",
            "Stage-wise subjects mapping",
            "List stages by curriculum",
            "Stage name configuration",
            "Assign grade to stage",
            "Display educational tier list",
            "School level categorization",
            "Educational step details",
            "Create new learning stage",
            "How to setup school stages ",
            "Edit education level master",
            "Institution stage list",
            "Manage stages for grades",
            "Predefined academic stages",
            "Academic stage controller"
          ],
          "section_category": [
            "Show section categories",
            "Add a new section category",
            "Edit section category",
            "Delete a section category",
            "List all section types",
            "What is the section category ",
            "Create section grouping",
            "View section names",
            "Open section category master",
            "Section categories overview",
            "Define class section category",
            "Update section group info",
            "School section category setup",
            "Edit section category list",
            "Display section types",
            "Group sections into categories",
            "What are my section categories ",
            "Assign students by category",
            "Set up section category names",
            "Customize section group",
            "Show section type settings",
            "Configure section division",
            "Set labels for section groups",
            "Manage section types",
            "Section tag configuration",
            "Group category of sections",
            "Show student section category",
            "Classroom section categories",
            "Assign section to class",
            "Section categorization rules",
            "School-level section category",
            "Rename section group",
            "Edit section group master",
            "Section category format",
            "System section type list",
            "Find section categories",
            "Define special section types",
            "Categorize students by section",
            "Standard section groupings",
            "Update class section label",
            "Add multiple section categories",
            "Modify section type",
            "Export section group list",
            "Who belongs to which section ",
            "Add new type of section",
            "How to create section groups ",
            "Master list of section categories",
            "Manage section templates",
            "Access section category screen"
          ],
          "division_master": [
            "Show division list",
            "Add a new division",
            "Edit division name",
            "Remove a division",
            "List all class divisions",
            "Division master setup",
            "Configure school divisions",
            "Display division groups",
            "How many divisions are there ",
            "Open division manager",
            "Setup class-wise divisions",
            "Define divisions for grades",
            "Update division names",
            "Assign students to division",
            "Manage division codes",
            "Division details screen",
            "Search for division",
            "Check division availability",
            "Set up division structure",
            "Classroom division list",
            "What are the divisions for grade 1 ",
            "Change division for student",
            "Group students by division",
            "Division-wise student view",
            "Configure division fields",
            "Rename existing division",
            "Show division master screen",
            "Division-level setup",
            "Class division details",
            "Edit student division",
            "Which class belongs to this division ",
            "Add divisions to new class",
            "Division panel",
            "Access division setup",
            "Create division labels",
            "Delete classroom division",
            "Show all divisions",
            "Manage grade divisions",
            "Set default division",
            "List of school divisions",
            "Division section overview",
            "Update division rules",
            "Division name master",
            "Student division grouping",
            "Change student division data",
            "Export division master",
            "Define section-wise divisions",
            "Set division parameters",
            "Link students to division"
          ],
          "academic_session_date": [
            "Show academic session dates",
            "Add a new session date",
            "Edit session calendar",
            "Update academic sessions",
            "Delete session date",
            "View school session calendar",
            "Configure academic sessions",
            "Set session start and end date",
            "Academic session overview",
            "What is the current session date ",
            "Setup session period",
            "Define school academic sessions",
            "Academic term date setup",
            "Edit session timeline",
            "Show session duration",
            "Calendar view for sessions",
            "Input academic dates",
            "Configure term-wise session",
            "Change academic calendar",
            "Academic schedule editor",
            "Master of session dates",
            "Check active session period",
            "Add term-wise session",
            "Assign sessions to year",
            "List session entries",
            "Session-wise academic plan",
            "Show school year timeline",
            "Create new term calendar",
            "Academic cycle dates",
            "Set vacation dates in session",
            "Define school year start",
            "Semester calendar update",
            "What are this year’s sessions ",
            "Academic plan configuration",
            "Create new academic timeline",
            "Change school year calendar",
            "Open academic schedule",
            "System session calendar",
            "Show institute session plan",
            "Session-based teaching days",
            "Upload calendar dates",
            "Term-wise session details",
            "How to set academic dates ",
            "Update exam session dates",
            "Input calendar holidays",
            "Session duration entry",
            "Academic timeline manager",
            "Session milestone dates",
            "School year calendar manager",
            "Academic term setup"
        ],
        "academic_year_or_batch": [
            "Show academic year list",
            "Add a new academic year",
            "Edit academic batch",
            "Remove academic year",
            "View current academic year",
            "What is the active batch ",
            "Change the academic year",
            "Update batch dates",
            "Set academic year duration",
            "Academic year master data",
            "Search academic batch",
            "Add batch for new session",
            "Open academic year master",
            "List of academic years",
            "Which year is active ",
            "Define academic year range",
            "Academic year details screen",
            "Configure academic batches",
            "Activate next academic year",
            "Deactivate old academic year",
            "Batch master entry",
            "Show all past batches",
            "Add session for new students",
            "Update existing academic year",
            "Edit batch name",
            "Download academic year data",
            "Show archived academic years",
            "Upload batch details",
            "Manage academic sessions",
            "View batch-wise reports",
            "List of academic sessions",
            "When does batch start ",
            "Get academic calendar",
            "Assign subjects to batch",
            "Change year-wise settings",
            "Setup academic batches",
            "Edit start and end date",
            "New academic session input",
            "Show batch master setup",
            "Link students to batch",
            "Academic year timeline",
            "How to define academic year ",
            "Assign exams to session",
            "Edit academic calendar year",
            "Academic session year data",
            "Preview academic session",
            "Configure academic terms",
            "View academic year settings",
            "Switch to new academic year"
          ],
          "educational_stages": [
            "List educational stages",
            "Add a new stage",
            "Edit educational stage",
            "Remove education stage",
            "Set education levels",
            "Show stage details",
            "Update educational hierarchy",
            "Define stage names",
            "School educational stages",
            "What are the learning stages ",
            "Setup academic levels",
            "Stages of education in system",
            "Display school levels",
            "Edit primary stage",
            "Add pre-primary stage",
            "Manage school stages",
            "Add class stages",
            "View stage mapping",
            "Educational stage chart",
            "Define stages for curriculum",
            "Set level structure",
            "Educational tiers setup",
            "Stage-wise configuration",
            "Assign classes to stage",
            "Which stage is this class ",
            "Search for education stage",
            "Set school stage policy",
            "View list of all stages",
            "Change class to stage mapping",
            "Setup KG stage",
            "Define high school stage",
            "Educational framework stages",
            "Stages of learning screen",
            "Update academic level info",
            "School education flow",
            "Stage-wise subjects mapping",
            "List stages by curriculum",
            "Stage name configuration",
            "Assign grade to stage",
            "Display educational tier list",
            "School level categorization",
            "Educational step details",
            "Create new learning stage",
            "How to setup school stages ",
            "Edit education level master",
            "Institution stage list",
            "Manage stages for grades",
            "Predefined academic stages",
            "Academic stage controller"
          ],
          "section_category": [
            "Show section categories",
            "Add a new section category",
            "Edit section category",
            "Delete a section category",
            "List all section types",
            "What is the section category ",
            "Create section grouping",
            "View section names",
            "Open section category master",
            "Section categories overview",
            "Define class section category",
            "Update section group info",
            "School section category setup",
            "Edit section category list",
            "Display section types",
            "Group sections into categories",
            "What are my section categories ",
            "Assign students by category",
            "Set up section category names",
            "Customize section group",
            "Show section type settings",
            "Configure section division",
            "Set labels for section groups",
            "Manage section types",
            "Section tag configuration",
            "Group category of sections",
            "Show student section category",
            "Classroom section categories",
            "Assign section to class",
            "Section categorization rules",
            "School-level section category",
            "Rename section group",
            "Edit section group master",
            "Section category format",
            "System section type list",
            "Find section categories",
            "Define special section types",
            "Categorize students by section",
            "Standard section groupings",
            "Update class section label",
            "Add multiple section categories",
            "Modify section type",
            "Export section group list",
            "Who belongs to which section ",
            "Add new type of section",
            "How to create section groups ",
            "Master list of section categories",
            "Manage section templates",
            "Access section category screen"
          ],
          "division_master": [
            "Show division list",
            "Add a new division",
            "Edit division name",
            "Remove a division",
            "List all class divisions",
            "Division master setup",
            "Configure school divisions",
            "Display division groups",
            "How many divisions are there ",
            "Open division manager",
            "Setup class-wise divisions",
            "Define divisions for grades",
            "Update division names",
            "Assign students to division",
            "Manage division codes",
            "Division details screen",
            "Search for division",
            "Check division availability",
            "Set up division structure",
            "Classroom division list",
            "What are the divisions for grade 1 ",
            "Change division for student",
            "Group students by division",
            "Division-wise student view",
            "Configure division fields",
            "Rename existing division",
            "Show division master screen",
            "Division-level setup",
            "Class division details",
            "Edit student division",
            "Which class belongs to this division ",
            "Add divisions to new class",
            "Division panel",
            "Access division setup",
            "Create division labels",
            "Delete classroom division",
            "Show all divisions",
            "Manage grade divisions",
            "Set default division",
            "List of school divisions",
            "Division section overview",
            "Update division rules",
            "Division name master",
            "Student division grouping",
            "Change student division data",
            "Export division master",
            "Define section-wise divisions",
            "Set division parameters",
            "Link students to division"
          ],
          "academic_session_date": [
            "Show academic session dates",
            "Add a new session date",
            "Edit session calendar",
            "Update academic sessions",
            "Delete session date",
            "View school session calendar",
            "Configure academic sessions",
            "Set session start and end date",
            "Academic session overview",
            "What is the current session date ",
            "Setup session period",
            "Define school academic sessions",
            "Academic term date setup",
            "Edit session timeline",
            "Show session duration",
            "Calendar view for sessions",
            "Input academic dates",
            "Configure term-wise session",
            "Change academic calendar",
            "Academic schedule editor",
            "Master of session dates",
            "Check active session period",
            "Add term-wise session",
            "Assign sessions to year",
            "List session entries",
            "Session-wise academic plan",
            "Show school year timeline",
            "Create new term calendar",
            "Academic cycle dates",
            "Set vacation dates in session",
            "Define school year start",
            "Semester calendar update",
            "What are this year’s sessions ",
            "Academic plan configuration",
            "Create new academic timeline",
            "Change school year calendar",
            "Open academic schedule",
            "System session calendar",
            "Show institute session plan",
            "Session-based teaching days",
            "Upload calendar dates",
            "Term-wise session details",
            "How to set academic dates ",
            "Update exam session dates",
            "Input calendar holidays",
            "Session duration entry",
            "Academic timeline manager",
            "Session milestone dates",
            "School year calendar manager",
            "Academic term setup"
        ],
        "subject_type": [
            "Add a new subject type",
            "List all subject types",
            "Delete a subject type",
            "Edit subject type name",
            "Show core subject types",
            "Display elective subject types",
            "What are the types of subjects ",
            "Set up new subject category",
            "Update subject type details",
            "Define subject type labels",
            "Add compulsory subject",
            "Include optional subject type",
            "Set subject types for class",
            "Show categorized subjects",
            "List core and optional subjects",
            "Subject type classification",
            "Assign type to subject",
            "Subject category setup",
            "Add new academic subject type",
            "Custom subject type entry",
            "Edit subject classification",
            "Delete subject category",
            "Subject group configuration",
            "Mark subject as practical",
            "Theoretical vs practical subjects",
            "Set subject to elective",
            "Assign subjects to types",
            "Subject labeling screen",
            "Define core subject group",
            "Filter subject by type",
            "Subject type screen master",
            "Group subject as vocational",
            "Enable practical type subject",
            "Assign type to new subject",
            "Rename subject type",
            "Show subject type mapping",
            "How to categorize subjects ",
            "Setup subjects by type",
            "Create new subject group",
            "Which subjects are optional ",
            "Update list of subject types",
            "Subject structure classification",
            "Institution subject type config",
            "Setup co-curricular subjects",
            "Change type of subject",
            "View type-wise subject list",
            "Configure theory subject type",
            "Display practical subject type",
            "Subject management by type",
            "Master of subject types"
          ],
          "subject_master": [
            "Show all subjects",
            "Add a new subject",
            "Delete subject from list",
            "Edit subject details",
            "List active subjects",
            "Create new subject entry",
            "Assign subject to class",
            "Subject master overview",
            "Edit subject short name",
            "Define subject for syllabus",
            "Mark subject as compulsory",
            "Set subject as optional",
            "Subject list by class",
            "View subject mapping",
            "Update subject info",
            "Subject name correction",
            "Set subject display order",
            "Add subject description",
            "Group subject under stream",
            "Import subject list",
            "Export subject master",
            "Bulk add subjects",
            "Setup new academic subject",
            "Subjects assigned to grade",
            "Show deleted subjects",
            "Restore deleted subject",
            "Show subject entry screen",
            "Subject configuration setup",
            "Enable subject for exams",
            "Assign teacher to subject",
            "Subject icon or code setup",
            "Search subject master list",
            "Change subject label",
            "Setup subject alias",
            "How to edit subjects ",
            "Institution subject listing",
            "Edit subject preferences",
            "Add subject notes",
            "Attach documents to subject",
            "View all subject entries",
            "Delete unused subject",
            "Rearrange subject order",
            "Change class subject structure",
            "Subject master screen",
            "Enable subject for report card",
            "Setup co-curricular subject",
            "Map subject to stream",
            "Show all subject groups",
            "Edit subject configuration"
          ],
          "subject_master_upload": [
            "Upload subject master file",
            "Import subject data",
            "Show subject upload option",
            "How to upload subject master ",
            "Upload subjects via Excel",
            "Add subjects using CSV",
            "Batch upload subjects",
            "Bulk import subject list",
            "Upload new subject entries",
            "Upload academic subjects",
            "Import subject names",
            "Subject bulk entry screen",
            "File format for subject upload",
            "CSV upload for subjects",
            "Upload optional subjects",
            "Subject master import tool",
            "Attach subject file to upload",
            "Validate subject upload",
            "Subject master data upload",
            "Upload class-wise subjects",
            "How to use upload template ",
            "Download subject upload sheet",
            "Subject bulk upload master",
            "Show subject import interface",
            "Auto-map subjects on upload",
            "Subject uploading guide",
            "Can I upload multiple subjects ",
            "Subject upload error fix",
            "Check uploaded subjects",
            "Upload subjects with category",
            "Add elective subjects via upload",
            "Use template to upload subjects",
            "Edit uploaded subject file",
            "Re-upload subject master",
            "Delete uploaded subject",
            "Fix subject import issues",
            "Upload screen for subjects",
            "Where is subject upload option ",
            "Upload teacher-subject mapping",
            "Subject master upload settings",
            "Data import for subjects",
            "Upload syllabus-wise subjects",
            "Add subjects with stream upload",
            "Show upload result",
            "Batch-wise subject import",
            "Preview uploaded subjects",
            "Subject upload success list",
            "Subject upload with errors",
            "Subject import progress",
            "Show upload summary"
          ],
          "class_teacher_master": [
            "Show class teacher master list",
            "Add new class teacher",
            "Edit class teacher details",
            "Remove class teacher assignment",
            "Assign teacher to class",
            "Set class in-charge",
            "Who is the class teacher ",
            "Class teacher mapping screen",
            "List of all class teachers",
            "Update teacher-class link",
            "Class teacher configuration",
            "Reassign class teacher",
            "Delete class teacher entry",
            "Bulk assign class teachers",
            "View class teacher panel",
            "Teacher class mapping table",
            "Edit assigned class",
            "Download teacher assignment",
            "Upload class teacher mapping",
            "Assign teacher to division",
            "Mark class teacher for session",
            "Teacher-class edit screen",
            "Assign default class teacher",
            "Add multiple class teachers",
            "Search class teacher records",
            "Update class-wise teachers",
            "Set class teacher status",
            "Class teacher master data",
            "Replace existing class teacher",
            "Link staff with class",
            "Assign teacher role",
            "Change teacher assignment",
            "Show active class teachers",
            "Set class-wise in-charge",
            "Map teacher to class",
            "Class and teacher configuration",
            "Class teacher report",
            "Define teacher-class matrix",
            "Remove teacher-class link",
            "Assign backup teacher",
            "Enable teacher login",
            "Assign academic class teacher",
            "Teacher-in-charge assignment",
            "Get assigned class teacher",
            "Institution class teacher setup",
            "Master screen of class teachers",
            "Allotted teacher to class",
            "Show teaching staff assignment",
            "Teacher map preview"
          ],
          "calendar_day_setting": [
            "Set academic calendar days",
            "Add working day",
            "Mark holiday on calendar",
            "Setup school calendar",
            "Set working and non-working days",
            "Add half day to calendar",
            "View day-wise calendar settings",
            "Edit calendar day master",
            "Add term days",
            "Show weekly off settings",
            "Holiday list configuration",
            "How to set school calendar ",
            "Assign events to days",
            "Calendar day management",
            "Setup national holidays",
            "Set weekends",
            "Define school working days",
            "Add exam day to calendar",
            "Mark Sunday as holiday",
            "Change weekly off days",
            "School calendar configuration",
            "Create academic day list",
            "Edit holiday list",
            "Configure day types",
            "School term calendar setup",
            "Add vacation period",
            "Block calendar days",
            "Working day setup",
            "Non-working day update",
            "Calendar day rules",
            "Institution calendar settings",
            "Calendar day type master",
            "Upload calendar holidays",
            "Set alternate working days",
            "Holiday and working day mapping",
            "Assign day type to date",
            "Mark exam day on calendar",
            "Calendar management system",
            "Download calendar settings",
            "Add special event days",
            "Define seasonal breaks",
            "Mark Friday as holiday",
            "Set up school day planner",
            "Set weekly calendar schedule",
            "Mark classes off on festival",
            "Holiday master calendar",
            "Set term end date",
            "Map subject to calendar",
            "Set calendar for new session",
            "Preview school calendar"
        ],
        "login_duplicate": [
            "Prevent duplicate logins",
            "How to block multiple logins ",
            "Enable single user session",
            "Restrict login to one device",
            "Stop user from logging in twice",
            "Login duplicate settings",
            "Disable simultaneous sessions",
            "Manage concurrent login control",
            "Restrict login duplicates",
            "Single sign-on enforcement",
            "Avoid duplicate account login",
            "Limit login to one user",
            "Enforce one-session-per-user rule",
            "Logout other sessions",
            "Allow only one active login",
            "Detect login duplication",
            "Control multiple logins",
            "How to prevent same user login twice ",
            "Block multiple sessions for user",
            "Duplicate login detection system",
            "Disable multi-session logins",
            "Enable session restriction",
            "One login at a time policy",
            "Set login control policy",
            "Restrict parallel login attempts",
            "Logout duplicate login users",
            "Track concurrent logins",
            "Prevent login cloning",
            "System to stop duplicate user login",
            "Block second login attempt",
            "Handle duplicate user sessions",
            "Same user login restriction",
            "Duplicate session termination",
            "Control account simultaneous login",
            "Duplicate login attempt alert",
            "Check for active sessions",
            "Force logout on duplicate login",
            "System login restriction settings",
            "Detect and stop multiple logins",
            "Enforce login duplication policy",
            "Multiple login prevention tool",
            "Auto logout previous sessions",
            "Warn on duplicate login",
            "System login policy settings",
            "Restrict same ID login",
            "Disallow same account access",
            "Login session override settings",
            "Force logout old session",
            "Stop user from multiple access",
            "Prevent simultaneous logins"
        ]
    },

        "My_page" : {
        "advanced_dashboard": [
              "Open my advanced dashboard",
              "Show me the detailed dashboard",
              "I want to view the advanced dashboard",
              "Launch personal dashboard",
              "Access my dashboard overview",
              "Show full dashboard view",
              "Where is my performance dashboard ",
              "Detailed stats view please",
              "Open analytics dashboard",
              "Show me my activity dashboard",
              "View my advanced metrics",
              "Dashboard with all insights",
              "Open user performance screen",
              "Launch the detailed portal",
              "Dashboard summary please",
              "Where’s the advanced view ",
              "Access smart dashboard",
              "View detailed user data",
              "Give me the complete dashboard",
              "Show user-level insights",
              "My homepage analytics",
              "My dashboard overview screen",
              "Detailed home dashboard",
              "Display my progress dashboard",
              "Open user smart panel",
              "Advanced data visualization view",
              "Full dashboard access",
              "Comprehensive user dashboard",
              "Launch extended dashboard view",
              "Open advanced stats portal",
              "Dashboard with user metrics",
              "Performance board for me",
              "Detailed dashboard panel open",
              "Insights screen please",
              "User-specific dashboard",
              "Access smart analytics screen",
              "Extended view of dashboard",
              "Show all widgets on dashboard",
              "User dashboard with metrics",
              "Stats view for my profile",
              "Track my dashboard stats",
              "Graphical dashboard view",
              "Real-time dashboard please",
              "Display full dashboard",
              "Launch all-in-one dashboard",
              "Open personal data board",
              "My data summary dashboard",
              "Show detailed graphs dashboard",
              "Advanced control panel",
              "Show real-time dashboard analytics"
         ],
         "Messenger": [
              "Open my messenger",
              "Show all messages",
              "Check new messages",
              "View messenger screen",
              "Launch the messaging app",
              "Start a new conversation",
              "Send a message to staff",
              "Access school messenger",
              "Chat with someone",
              "I want to send a message",
              "Reply to recent messages",
              "Show unread messages",
              "Messenger notifications please",
              "Where is my messenger ",
              "Open chat box",
              "Message dashboard access",
              "Start messaging now",
              "Messenger history please",
              "Open latest conversations",
              "Private messaging panel",
              "Open group chats",
              "Open communication tab",
              "Message someone in school",
              "Check last chat",
              "Review my messenger",
              "Send internal message",
              "Show all conversations",
              "Quick chat screen open",
              "Staff messaging portal",
              "Student chat access",
              "Messenger inbox please",
              "Send note via messenger",
              "Messenger tool open",
              "Chat history access",
              "Recent chats view",
              "Open teacher messenger",
              "Admin communication tab",
              "Parent-teacher messenger",
              "Message tracker screen",
              "View sent messages",
              "Internal communication screen",
              "Messenger for students",
              "Messenger for teachers",
              "Send message now",
              "Where can I chat ",
              "Show message replies",
              "Inbox message list",
              "Messenger activity log",
              "Chat screen please",
              "Contact someone via messenger"
        ],
        "My Mailbox": [
              "Open my mailbox",
              "Show me my emails",
              "Check new mails",
              "Go to inbox",
              "Access my mailbox now",
              "View unread emails",
              "Read my messages",
              "Open mail dashboard",
              "Mailbox access please",
              "Launch personal inbox",
              "Where are my mails ",
              "Check today's emails",
              "Open mail panel",
              "Show sent mails",
              "Check received mails",
              "Open email communication",
              "Look at my mailbox",
              "Mailbox screen please",
              "Inbox with my messages",
              "My communication box",
              "Emails received today",
              "Show all mailbox messages",
              "New message alert",
              "View archived mails",
              "Open personal message inbox",
              "List all incoming mails",
              "See all mail threads",
              "Open school email system",
              "Access student mailbox",
              "Access staff mailbox",
              "Mail inbox please",
              "Get my messages",
              "Where’s my inbox ",
              "Go to mail screen",
              "Mailbox notification list",
              "Messages I received",
              "All recent mails",
              "Email viewer screen",
              "Mailbox logs open",
              "Open private mailbox",
              "Sent email history",
              "Check mailbox updates",
              "Mail management screen",
              "Show emails from today",
              "Open personal communication",
              "Staff communication inbox",
              "Open the mail center",
              "I want to check my email",
              "Review my email messages",
              "Access school messages"
        ],
        "Calendar Event": [
              "Open calendar events",
              "Show me upcoming events",
              "What’s on my calendar today ",
              "View scheduled activities",
              "Show school calendar",
              "Calendar view please",
              "List all events",
              "Check today's event schedule",
              "What's planned this week ",
              "Open my calendar",
              "Event calendar screen",
              "View full academic calendar",
              "Open monthly calendar events",
              "Calendar with school events",
              "Open class calendar",
              "Any holidays this month ",
              "Look at the calendar details",
              "Event listing for this week",
              "School events today",
              "Mark my calendar events",
              "Check future events",
              "Calendar and activities panel",
              "I want to see the calendar",
              "Launch event calendar",
              "Calendar timeline view",
              "All event dates please",
              "Daily calendar access",
              "What’s on schedule ",
              "Event summary page",
              "Check calendar events log",
              "Upcoming tests or events ",
              "Open term calendar",
              "Holiday calendar view",
              "Full calendar overview",
              "Academic events this month",
              "Today’s events on calendar",
              "Monthly planner screen",
              "Schedule view access",
              "Weekly event planner",
              "Yearly event listing",
              "Open schedule and events",
              "Student calendar view",
              "Calendar dashboard open",
              "I want to check school events",
              "Event panel for today",
              "Next event on my calendar",
              "Timeline of events please",
              "View exam calendar",
              "School day events",
              "Academic event tracker"
        ]
    },
     "institution": {
        "about_institution": [
            "Tell me about the institution ",
            "Give me details about the school ",
            "What is the history of this college ",
            "Can you describe the institution ",
            "Share an overview of the institution ",
            "What’s the mission of this institution ",
            "What are the vision and goals of the institution ",
            "I want to know more about this school ",
            "What makes this institution special ",
            "Who founded this institution ",
            "Describe the background of this college ",
            "When was the institution established ",
            "What’s the core purpose of the school ",
            "Tell me the story behind the institution ",
            "What is the institution known for ",
            "Share the founding principles of this college ",
            "What’s the legacy of this institution ",
            "Give me a brief about the school ",
            "Tell me what this institution stands for ",
            "What kind of institution is this ",
            "Describe the profile of this college ",
            "Can you explain the philosophy of the institution ",
            "I need an introduction to the school ",
            "What is this institution all about ",
            "How do you describe the institution ",
            "Provide the institution’s summary ",
            "Tell me about the values of this school ",
            "What does the institution focus on ",
            "Give a background summary of the college ",
            "What are the goals of this institution ",
            "Explain the institution’s objectives ",
            "What kind of education does the institution promote ",
            "Why was the institution established ",
            "What’s the primary mission of this school ",
            "Who are the founders of the institution ",
            "Tell me about the founders' vision ",
            "Describe the key features of the school ",
            "Give me a summary of the college mission and vision ",
            "Tell me about the guiding principles of the institution ",
            "What is the institutional identity ",
            "What kind of culture does this institution promote ",
            "Share key facts about the institution ",
            "Tell me the core values of the institution ",
            "What is the driving philosophy of this college ",
            "What does the institution aspire to be ",
            "What educational values does the school uphold ",
            "Can you tell the origin of this college ",
            "Give a historical overview of the institution ",
            "What are the foundational beliefs of this school ",
            "What type of institution is this — public or private "
        ],
        "gallery": [
            "Show me pictures of the school ",
            "Can I see the campus photos ",
            "Display images of the institution ",
            "Where can I view institution pictures ",
            "Give me photos of the college ",
            "I want to see pictures of the classrooms ",
            "Show institution building photos ",
            "Do you have images of the school infrastructure ",
            "Can I get photos of the labs and library ",
            "Show the main entrance of the institution ",
            "Display images of the auditorium ",
            "Show pictures of the playground ",
            "Can I view the photo gallery of the school ",
            "Are there pictures of the institution events ",
            "Give me images from past annual functions ",
            "Can I see images of students at the campus ",
            "Show some indoor facility pictures ",
            "I want to view exterior shots of the campus ",
            "Are there photos of the staff rooms ",
            "Show pictures of the sports area ",
            "Display pictures of the hostel ",
            "Can I view cafeteria pictures ",
            "Show me some pictures of the classrooms ",
            "Display the picture gallery of the college ",
            "Any image gallery available for the institution ",
            "Give me a tour of the institution in pictures ",
            "Can I see labs and classrooms in photos ",
            "Show the library through pictures ",
            "How does the school look from the outside ",
            "Are there recent pictures of events ",
            "Can you share infrastructure images ",
            "Give me the latest pictures of the institution ",
            "Show me how the campus looks ",
            "Do you have virtual tour pictures ",
            "Show photographs of the computer labs ",
            "I want to see institution facility images ",
            "Provide the latest school pictures ",
            "Any gallery available for student activities ",
            "Display images of school functions ",
            "Show snapshots from previous ceremonies ",
            "Are there any group photos of staff ",
            "Can I view classroom interior images ",
            "Give images of the science labs ",
            "Do you have any panoramic shots of the institution ",
            "Provide photos showing the cleanliness of the campus ",
            "Where can I find photos of school infrastructure ",
            "Are there pictures of extracurricular activities ",
            "Share campus view images ",
            "I’d like to see a photo overview of the college "
        ],
        "institution_address": [
            "What’s the address of the institution ",
            "Where is the school located ",
            "Give me the location of the college ",
            "Can I get the full address of the institution ",
            "Where is the institution situated ",
            "Share the address of the campus ",
            "Provide the address with landmarks ",
            "Tell me the postal address of the institution ",
            "Where exactly is the college ",
            "How do I reach the school ",
            "Show me the map location of the institution ",
            "What’s the pin code of the school ",
            "Where is the head office located ",
            "Give me directions to the institution ",
            "Is the college inside the city ",
            "How far is the school from the station ",
            "Where is the main gate of the institution ",
            "Tell me the road name of the school’s address ",
            "Can I find the address on Google Maps ",
            "Is the address same for all branches ",
            "Where can I park near the institution ",
            "What’s the GPS location of the school ",
            "How to get to the institution by bus ",
            "Which area is the school located in ",
            "Send me the exact coordinates of the college ",
            "Is the address available on the website ",
            "What is the nearest landmark to the institution ",
            "How do I find the school entrance ",
            "Can I get driving directions to the college ",
            "Where is the front desk located in the building ",
            "Tell me the nearest metro station to the school ",
            "Which city is the institution in ",
            "Give me the full postal location ",
            "Provide contact info along with the address ",
            "Is the college in a rural or urban area ",
            "What’s the local address of the school ",
            "What is the nearby bus stop to the institution ",
            "Give me the location with street name ",
            "Is it easy to locate the institution ",
            "Where is the admin office on the campus ",
            "Which block is the main office in ",
            "Share address details with zip code ",
            "Is there a landmark near the institution ",
            "Show me how to reach the school gate ",
            "Can I visit the institution directly ",
            "Where is the school gate located ",
            "What road is the school on ",
            "Tell me how to find the institution using GPS ",
            "Give me the contact and location together "
        ],
        "institution_board_information": [
            "Which board is the institution affiliated to ",
            "Is this school under CBSE or ICSE ",
            "What board governs the school curriculum ",
            "Tell me the board affiliation of the college ",
            "Is this a state board institution ",
            "Under which board does the school operate ",
            "Is the institution affiliated with AICTE ",
            "What educational board is the school following ",
            "Does this college follow university norms ",
            "Is the board CBSE, ICSE, or state syllabus ",
            "What board certificate do students get ",
            "Is the school recognized by a central board ",
            "Which examination board conducts the final exams ",
            "What’s the name of the accrediting board ",
            "Who grants affiliation to the school ",
            "Which board does the curriculum align with ",
            "Is the school board-approved ",
            "Are classes taught as per CBSE pattern ",
            "Which board’s norms are followed here ",
            "Is the institution accredited by NAAC or AICTE ",
            "Do students receive CBSE certification ",
            "Is the syllabus from ICSE or state board ",
            "Who provides the school’s affiliation ",
            "Is the institution recognized by UGC ",
            "Under what board does the college grant degrees ",
            "What is the exam board used by this school ",
            "Is this a government or private board institution ",
            "Which academic board is followed ",
            "Is this school part of a national education board ",
            "Tell me the board registration details ",
            "Is there an affiliation certificate available ",
            "Does the institution have board approval ",
            "Is the college board-certified ",
            "Are school marks recognized by CBSE ",
            "What’s the education board name ",
            "Who is the affiliating authority ",
            "Which board sets the syllabus for this institution ",
            "Is this college autonomous or affiliated ",
            "Is the institution affiliated with a university ",
            "Does it follow national education policy boards ",
            "Are courses verified by the affiliated board ",
            "Is the institution recognized internationally ",
            "Which board’s exams are conducted ",
            "Which curriculum board is used ",
            "Tell me about the board details ",
            "What education board does this institution adhere to ",
            "Do you have the board’s approval letter ",
            "What is the affiliation number of the institution ",
            "Is the college part of any university board ",
            "Are courses standardized by a national board "
        ]
    },
        
        "product": {
        "about_product": [
            "Tell me about your ERP product ",
            "What is this ERP software used for ",
            "Can you explain what your product does ",
            "What features does your ERP have ",
            "Give me a quick overview of your ERP system ",
            "What’s included in your ERP solution ",
            "Describe your product in brief ",
            "How does your ERP benefit schools ",
            "Is this ERP meant for colleges too ",
            "Tell me the main highlights of your product ",
            "Who can use this ERP system ",
            "What modules are there in your ERP ",
            "Is your ERP cloud-based or offline ",
            "What makes your product different from others ",
            "How is your ERP helpful to teachers ",
            "Can students access the ERP too ",
            "What problem does your ERP solve ",
            "How does the ERP improve school management ",
            "Why should schools choose your ERP ",
            "Can you tell me about the ERP's functionality ",
            "How secure is your ERP system ",
            "What kind of institutions is this ERP built for ",
            "Is your product customizable ",
            "How easy is it to use your ERP ",
            "What’s the purpose of this ERP product ",
            "How is your ERP different from a basic MIS ",
            "What technologies are used in your ERP ",
            "Does your ERP support mobile apps ",
            "What is the core of your ERP platform ",
            "Is your ERP scalable for large campuses ",
            "How can your ERP help school admins ",
            "What tasks can be automated using your ERP ",
            "Do you have AI or chatbot features in your product ",
            "How is communication handled in your ERP ",
            "How many schools are currently using this ERP ",
            "What’s the main goal of your product ",
            "Give a summary of your ERP software ",
            "Does your ERP support fee collection ",
            "Is your ERP compatible with biometric systems ",
            "How long has your ERP been in the market ",
            "What kind of training do you provide for your ERP ",
            "Can parents use your ERP too ",
            "What is the ERP platform built on ",
            "How flexible is your product for different curricula ",
            "Is the product suitable for universities ",
            "Can you give a demo of your ERP ",
            "Does your ERP work on smartphones ",
            "What kind of reports can we generate with your ERP ",
            "What ERP services do you provide ",
            "Tell me the benefits of your school ERP system ",
            "What areas of school operations does your ERP cover "
        ]
    }
    
}

In [7]:
classifier = IntentClassifier()

2025-07-08 11:43:21,413 - __main__ - INFO - Using device: cuda


In [8]:
classifier.train(
        intent_samples,
        epochs=10,
        batch_size=16,
        gradient_accumulation_steps=4,
        fp16=False,
        num_workers=16
    )

2025-07-08 11:43:24,542 - __main__ - INFO - Starting training on cuda
2025-07-08 11:43:24,543 - __main__ - INFO - Dataset size: 11570 examples
2025-07-08 11:43:24,545 - __main__ - INFO - Data preparation took 0.00 seconds
2025-07-08 11:43:24,549 - __main__ - INFO - Creating train and eval datasets
2025-07-08 11:43:24,549 - __main__ - INFO - Pre-computing encodings for all texts (this will speed up training)
2025-07-08 11:43:24,550 - __main__ - INFO - Processing 41 batches of encodings
Encoding batches: 100%|██████████████████████████████████████████████████████████████████████| 41/41 [00:01<00:00, 29.77it/s]
2025-07-08 11:43:25,941 - __main__ - INFO - Completed encoding 10413 texts
2025-07-08 11:43:25,941 - __main__ - INFO - Pre-computing encodings for all texts (this will speed up training)
2025-07-08 11:43:25,942 - __main__ - INFO - Processing 5 batches of encodings
Encoding batches: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 33.5

In [9]:
classifier.save_model("models/optimized_model.pt")

2025-07-08 12:12:16,722 - __main__ - INFO - Model saved to models/optimized_model.pt


In [28]:
test_messages = ["How to check the student a"]
for message in test_messages:
    result = classifier.predict(message)
    logger.info(f"\nInput: {message}")
    logger.info(f"Result: {result}")

2025-07-08 12:27:45,066 - __main__ - INFO - 
Input: How to check the student attendance
2025-07-08 12:27:45,066 - __main__ - INFO - Result: {'status': 'llm_fallback', 'confidence': {'main': 0.5690412521362305, 'sub': 0.0}, 'original_query': 'How to check the student attendance'}
